In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import log_loss

In [2]:
ls ../input

LANL-Earthquake-Prediction/  andrews-new-stuff/


In [3]:
X = pd.read_csv('../input/andrews-new-stuff/train_features.csv')
X_test = pd.read_csv('../input/andrews-new-stuff/test_features.csv')
y = pd.read_csv('../input/andrews-new-stuff/y.csv')
submission = pd.read_csv('../input/andrews-new-stuff/submission.csv')


In [4]:
X.shape

(4195, 831)

In [5]:
X.head()

,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,abs_max_roll_mean_10000,abs_max_roll_mean_50,abs_max_roll_mean_500,abs_max_roll_std_10,abs_max_roll_std_100,abs_max_roll_std_1000,abs_max_roll_std_10000,abs_max_roll_std_50,abs_max_roll_std_500,abs_mean,abs_percentile_1,abs_percentile_10,abs_percentile_20,abs_percentile_25,abs_percentile_30,abs_percentile_40,abs_percentile_5,abs_percentile_50,abs_percentile_60,abs_percentile_70,abs_percentile_75,abs_percentile_80,abs_percentile_90,abs_percentile_95,abs_percentile_99,abs_std,abs_trend,autocorrelation_10,autocorrelation_100,autocorrelation_1000,autocorrelation_10000,...,range_-3000_-2000,range_-4000_-3000,range_0_1000,range_1000_2000,range_2000_3000,range_3000_4000,range_minf_m4000,range_p4000_pinf,skew,spkt_welch_density_1,spkt_welch_density_10,spkt_welch_density_100,spkt_welch_density_5,spkt_welch_density_50,std,std_first_1000,std_first_10000,std_first_50000,std_last_1000,std_last_10000,std_last_50000,std_roll_mean_10,std_roll_mean_100,std_roll_mean_1000,std_roll_mean_10000,std_roll_mean_50,std_roll_mean_500,std_roll_std_10,std_roll_std_100,std_roll_std_1000,std_roll_std_10000,std_roll_std_50,std_roll_std_500,sum,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_5,time_rev_asym_stat_50,trend
0,4.883327,4.876661,4.809076,4.883860,7.027028,104.0,68.5,10.04,5.629,5.2457,12.82,6.042,78.313047,52.335255,31.029445,11.207164,61.404926,38.430379,5.576567,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,6.0,7.0,7.0,8.0,10.0,12.0,20.0,4.333325,-0.000011,-0.469692,0.000897,-0.004982,0.006429,...,0,0,138430,0,0,0,0,0,-0.024061,10.274048,331.303127,7.080859,90.706500,15.608985,5.101106,4.737159,11.207151,6.488552,2.979629,4.361407,3.664663,2.801800,0.452294,0.295715,0.240132,0.606039,0.319749,2.809071,3.111524,2.769772,1.219932,3.176148,2.921016,732617.0,-0.177656,-14.764689,-1.703218,-0.958331,5.087825,-3.268300e-06
1,4.725049,4.718421,4.653152,4.725527,7.380383,181.0,145.1,16.67,5.667,4.9398,28.26,6.774,122.978273,87.972617,38.643217,13.235293,94.730087,50.915658,5.734167,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,6.0,7.0,7.0,8.0,10.0,12.0,24.0,5.732777,-0.000005,-0.450367,-0.079143,0.028924,-0.003001,...,0,0,136635,0,0,0,0,0,0.390561,12.343507,696.779429,6.463803,422.346989,16.771432,6.588824,2.706886,3.976750,7.305233,2.672471,3.667890,5.493071,3.924070,0.496220,0.231587,0.118394,0.764507,0.267450,4.120785,4.893431,4.492905,3.003710,4.953472,4.679236,708865.0,-0.284957,91.001093,-38.729232,71.814108,21.336684,9.090424e-07
2,4.905511,4.898921,4.832204,4.906040,8.016930,140.0,94.8,12.38,5.957,5.2859,17.36,6.462,96.247424,53.353832,35.326896,14.344677,65.431379,45.481970,6.152647,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,6.0,7.0,8.0,8.0,11.0,14.0,30.0,5.895945,0.000010,-0.480152,-0.004648,0.000682,-0.008548,...,0,0,135749,0,0,0,0,0,0.217391,12.595403,621.906904,6.853157,396.415823,16.479469,6.967397,2.683260,8.454717,6.104836,3.050037,9.493983,8.603696,4.179729,0.530151,0.267012,0.195749,0.811309,0.302261,4.227960,4.959233,4.402140,2.590564,5.035827,4.675514,735959.0,-0.693163,-105.719163,78.751722,-57.852130,41.037612,3.962182e-06
3,4.901428,4.894639,4.827671,4.901985,7.606850,199.0,136.6,18.26,5.858,5.2169,27.72,7.114,141.450580,103.996664,52.805653,17.521969,113.368462,69.610117,5.933960,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,6.0,7.0,8.0,8.0,10.0,13.0,26.0,6.061214,-0.000002,-0.374296,-0.182497,0.003321,-0.002565,...,0,0,137391,0,0,0,0,0,0.757278,12.050065,317.693579,7.041170,623.474094,16.029480,6.922305,3.440866,6.866177,6.238109,2.958870,4.364430,5.652442,4.299460,0.581874,0.266700,0.173348,0.959834,0.302468,4.284176,5.247130,4.873523,3.545253,5.285204,5.054525,735336.0,0.234850,11.626424,263.667817,22.071565,-85.625097,1.637207e-06
4,4.908115,4.901855,4.836297,4.908528,7.895403,145.0,107.6,14.41,6.078,5.2651,19.98,6.862,99.647657,74.460128,45.532439,16.551133,83.845592,5

In [6]:
X_test.head()

,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,abs_max_roll_mean_10000,abs_max_roll_mean_50,abs_max_roll_mean_500,abs_max_roll_std_10,abs_max_roll_std_100,abs_max_roll_std_1000,abs_max_roll_std_10000,abs_max_roll_std_50,abs_max_roll_std_500,abs_mean,abs_percentile_1,abs_percentile_10,abs_percentile_20,abs_percentile_25,abs_percentile_30,abs_percentile_40,abs_percentile_5,abs_percentile_50,abs_percentile_60,abs_percentile_70,abs_percentile_75,abs_percentile_80,abs_percentile_90,abs_percentile_95,abs_percentile_99,abs_std,abs_trend,autocorrelation_10,autocorrelation_100,autocorrelation_1000,autocorrelation_10000,...,range_-3000_-2000,range_-4000_-3000,range_0_1000,range_1000_2000,range_2000_3000,range_3000_4000,range_minf_m4000,range_p4000_pinf,skew,spkt_welch_density_1,spkt_welch_density_10,spkt_welch_density_100,spkt_welch_density_5,spkt_welch_density_50,std,std_first_1000,std_first_10000,std_first_50000,std_last_1000,std_last_10000,std_last_50000,std_roll_mean_10,std_roll_mean_100,std_roll_mean_1000,std_roll_mean_10000,std_roll_mean_50,std_roll_mean_500,std_roll_std_10,std_roll_std_100,std_roll_std_1000,std_roll_std_10000,std_roll_std_50,std_roll_std_500,sum,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_5,time_rev_asym_stat_50,trend
0,4.491140,4.484860,4.426717,4.491630,6.703153,115.0,91.6,10.42,5.495,4.8284,14.28,6.092,63.410129,50.489998,24.717247,8.759360,55.856946,33.202016,5.224607,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,6.0,6.0,7.0,8.0,9.0,11.0,19.0,4.102161,-5.446287e-07,-0.369008,-0.033550,-0.004443,-0.005776,...,0,0,136578,0,0,0,0,0,0.327908,10.481778,325.971693,6.721441,172.767546,16.415903,4.893690,9.138029,5.226846,5.350451,4.060994,6.019017,4.793876,2.737524,0.419927,0.231891,0.142278,0.595202,0.263762,2.499556,2.799232,2.439570,1.524127,2.862491,2.609314,673767.0,-0.052494,24.461415,-24.696669,2.838643,-2.437498,-2.476946e-07
1,4.170633,4.165608,4.111654,4.170983,6.833615,152.0,99.3,11.90,5.009,4.4692,18.42,5.422,111.724662,58.373312,29.680311,11.596170,71.140057,39.792732,5.198340,0.0,1.0,2.0,2.0,3.0,4.0,1.0,4.0,5.0,6.0,7.0,7.0,9.0,12.0,24.0,5.045369,-2.454491e-06,-0.418828,-0.073823,-0.009462,0.009578,...,0,0,134753,0,0,0,0,0,0.295708,11.074685,220.254145,6.746122,298.980714,16.948847,5.922839,3.762414,3.523253,6.249515,2.786137,3.605147,4.147562,3.492504,0.470431,0.230914,0.143844,0.720321,0.265776,3.496219,4.121631,3.753388,2.364856,4.175896,3.931232,625673.0,0.588375,185.380124,-12.056081,56.675625,53.214837,2.009021e-06
2,4.609608,4.603891,4.544624,4.610042,7.180588,248.0,209.1,19.11,6.234,4.9169,33.16,7.684,161.392000,106.296735,53.069292,20.353477,136.437844,70.587119,5.597193,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,6.0,7.0,7.0,8.0,10.0,12.0,25.0,6.179525,-7.631588e-06,-0.495188,-0.039442,-0.007765,-0.003983,...,0,0,137911,0,0,0,0,0,0.428688,12.426695,1497.274134,6.778129,502.485900,16.683570,6.946990,2.813150,3.950119,9.793473,2.915698,3.063057,5.225913,4.491206,0.520981,0.247219,0.154180,0.753897,0.281453,4.247625,5.477172,5.154268,4.033028,5.534443,5.308515,691539.0,1.517607,416.757088,-32.027690,143.248043,-2.538065,3.577592e-07
3,4.530821,4.524946,4.466192,4.531238,6.079078,93.0,62.1,9.18,5.446,4.8740,12.74,5.948,66.016917,45.895149,22.803241,8.101416,56.016747,30.278838,4.961487,0.0,1.0,2.0,3.0,3.0,4.0,1.0,5.0,5.0,6.0,7.0,7.0,9.0,10.0,16.0,3.583863,-1.867193e-06,-0.379595,-0.082484,0.013332,0.000582,...,0,0,141702,0,0,0,0,0,0.061889,11.630546,127.914279,6.867442,60.507444,16.523814,4.114147,2.698147,4.001275,3.664088,2.651688,2.771185,3.480840,2.142932,0.386529,0.224909,0.161388,0.529305,0.254105,2.182659,2.431101,2.156890,1.320629,2.477723,2.286835,679721.0,0.051814,0.705854,15.880020,3.613541,-8.094229,-1.825526e-07
4,4.127661,4.121530,4.061062,4.128137,6.618645,177.0,107.7,14.13,5.027,4.4925,20.80,5.724,113.678885,76.743811,34.000818,13.789245,81.8

In [7]:
y.head()

,target
0,1.430797
1,1.391499
2,1.353196
3,1.313798
4,1.274400


In [8]:
submission.head()

,seg_id,time_to_failure
0,seg_00030f,4.072624
1,seg_0012b5,4.318170
2,seg_00184e,5.366592
3,seg_003339,8.408390
4,seg_0042cc,7.159201


In [9]:
alldata = pd.concat([X,X_test])

In [10]:
ss = StandardScaler()
alldata[alldata.columns] = ss.fit_transform(alldata[alldata.columns].astype(float))

In [11]:
alldata.head()

,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,abs_max_roll_mean_10000,abs_max_roll_mean_50,abs_max_roll_mean_500,abs_max_roll_std_10,abs_max_roll_std_100,abs_max_roll_std_1000,abs_max_roll_std_10000,abs_max_roll_std_50,abs_max_roll_std_500,abs_mean,abs_percentile_1,abs_percentile_10,abs_percentile_20,abs_percentile_25,abs_percentile_30,abs_percentile_40,abs_percentile_5,abs_percentile_50,abs_percentile_60,abs_percentile_70,abs_percentile_75,abs_percentile_80,abs_percentile_90,abs_percentile_95,abs_percentile_99,abs_std,abs_trend,autocorrelation_10,autocorrelation_100,autocorrelation_1000,autocorrelation_10000,...,range_-3000_-2000,range_-4000_-3000,range_0_1000,range_1000_2000,range_2000_3000,range_3000_4000,range_minf_m4000,range_p4000_pinf,skew,spkt_welch_density_1,spkt_welch_density_10,spkt_welch_density_100,spkt_welch_density_5,spkt_welch_density_50,std,std_first_1000,std_first_10000,std_first_50000,std_last_1000,std_last_10000,std_last_50000,std_roll_mean_10,std_roll_mean_100,std_roll_mean_1000,std_roll_mean_10000,std_roll_mean_50,std_roll_mean_500,std_roll_std_10,std_roll_std_100,std_roll_std_1000,std_roll_std_10000,std_roll_std_50,std_roll_std_500,sum,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_5,time_rev_asym_stat_50,trend
0,1.625663,1.625200,1.617486,1.625816,-0.029427,-0.223008,-0.196185,-0.145002,0.032927,1.190651,-0.160750,-0.042027,-0.205451,-0.184440,-0.115091,-0.121680,-0.176349,-0.143152,0.078022,0.0,-0.100362,-0.208467,0.844195,-0.007367,0.366465,0.518338,0.690514,1.023179,0.978352,0.086607,0.470583,0.209981,-0.010740,-0.161338,-0.172201,-0.367530,-0.518960,0.784644,-0.510395,0.830647,...,-0.057147,-0.048866,0.483395,-0.064845,-0.05894,-0.052707,-0.03104,-0.033491,-0.317423,-0.053301,-0.070827,0.926988,-0.076510,-0.703462,-0.174395,0.042112,0.598595,0.016603,-0.347160,-0.208408,-0.333493,-0.172715,-0.100269,0.257819,1.935137,-0.132207,0.100672,-0.199641,-0.194747,-0.190500,-0.225929,-0.195183,-0.193898,1.616545,0.012251,0.018724,0.007777,0.018639,-0.012427,-1.757518
1,1.118361,1.117360,1.110480,1.118392,0.111785,0.032510,0.146108,0.104402,0.043990,0.474772,0.121397,0.073423,0.035977,0.077916,-0.025891,-0.060794,0.036960,-0.018303,0.179404,0.0,-0.100362,-0.208467,0.844195,-0.007367,0.366465,0.518338,0.690514,1.023179,0.978352,0.086607,0.470583,0.209981,-0.010740,-0.038155,-0.006199,-0.168303,-0.306497,-0.392986,2.555447,-0.488613,...,-0.057147,-0.048866,0.109440,-0.064845,-0.05894,-0.052707,-0.03104,-0.033491,0.577126,-0.052099,-0.059095,-1.157168,-0.061648,0.279082,-0.001331,-0.436046,-0.169935,0.092494,-0.415665,-0.336459,-0.084276,-0.000414,-0.049436,-0.134397,-0.987512,-0.038155,-0.100586,0.022152,0.015449,0.012940,-0.011020,0.016284,0.013528,1.112473,0.012118,0.019145,0.007548,0.019592,-0.012281,0.498811
2,1.696767,1.696639,1.692689,1.696900,0.366171,-0.103545,-0.078661,-0.056977,0.128420,1.284728,-0.077787,0.024215,-0.108511,-0.176941,-0.064744,-0.027489,-0.150577,-0.072638,0.448604,0.0,-0.100362,-0.208467,0.844195,-0.007367,0.366465,0.518338,0.690514,1.023179,0.978352,1.458434,0.470583,0.692252,0.449267,0.146618,0.013156,0.349755,-0.633960,0.703065,0.001764,-1.264548,...,-0.057147,-0.048866,-0.075142,-0.064845,-0.05894,-0.052707,-0.03104,-0.033491,0.203511,-0.051953,-0.061499,0.157905,-0.062810,0.032304,0.042708,-0.441610,0.306035,-0.019055,-0.331457,0.739273,0.339713,0.038837,-0.010169,0.082268,0.869593,-0.010377,0.033375,0.040274,0.023211,0.002224,-0.060795,0.026083,0.013089,1.687469,0.011610,0.018362,0.008275,0.017893,-0.012103,2.147919
3,1.683679,1.682898,1.677950,1.683904,0.202289,0.092242,0.108125,0.164214,0.099597,1.123252,0.111529,0.127047,0.135825,0.195882,0.140030,0.067895,0.156261,0.168635,0.307927,0.0,-0.100362,-0.208467,0.844195,-0.007367,0.366465,0.518338,0.690514,1.023179,0.978352,1.458434,0.470583,0.209981,0.219264,0.02343

In [12]:
a = np.concatenate([y.target.values,submission.time_to_failure.values])

In [13]:
alldata['target']  = a

In [14]:
a = pd.qcut(alldata.target,3,labels=[0,1,2])

In [15]:
alldata.target = a

In [16]:
alldata['target_0'] = (alldata.target==0).astype(int)
alldata['target_1'] = (alldata.target==1).astype(int)
alldata['target_2'] = (alldata.target==2).astype(int)

In [17]:
alldata.target.unique()

[0, 2, 1]
Categories (3, int64): [0 < 1 < 2]

In [18]:
del alldata['target']

In [19]:
class GP:
    def __init__(self):
        self.classes = 3
        self.class_names = [ 'class_0',
                             'class_1',
                             'class_2']


    def GrabPredictions(self, data,n):
        oof_preds = np.zeros((len(data), len(self.class_names)))
        oof_preds[:,0] = self.GP_class_0(data,n)
        oof_preds[:,1] = self.GP_class_1(data,n)
        oof_preds[:,2] = self.GP_class_2(data,n)
        oof_df = pd.DataFrame(np.exp(oof_preds), columns=self.class_names)
        oof_df =oof_df.div(oof_df.sum(axis=1), axis=0)
        return oof_df


    def GP_class_0(self,data,n):
        v = pd.DataFrame()
        v["i0"] = -0.689578 
        v["i1"] = 0.010000*np.tanh(((((((data["num_peaks_10"]) * 2.0)) + (((((((((data["range_-1000_0"]) + (((data["ffti_range_-4000_-3000"]) + (data["percentile_roll_std_5"]))))) + (data["num_peaks_10"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i2"] = 0.010000*np.tanh(((((((((data["min_roll_std_10000"]) + (((data["percentile_roll_std_20"]) + (((((data["ffti_range_2000_3000"]) + (((data["num_peaks_10"]) * 2.0)))) + (data["num_peaks_10"]))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i3"] = 0.010000*np.tanh(((((((data["percentile_roll_std_25"]) + (((((((data["range_-1000_0"]) * 2.0)) + (((data["iqr"]) * 2.0)))) + (data["percentile_roll_std_1"]))))) + (data["fftr_range_-3000_-2000"]))) * 2.0)) 
        v["i4"] = 0.010000*np.tanh(((((((((data["num_peaks_10"]) + (data["fftr_range_-4000_-3000"]))) + (((data["iqr"]) + (((((data["percentile_roll_std_1"]) - (data["ffti_range_-1000_0"]))) + (data["percentile_roll_std_10"]))))))) * 2.0)) * 2.0)) 
        v["i5"] = 0.010000*np.tanh(((data["percentile_roll_std_10"]) + (((((((((((data["ffti_range_2000_3000"]) + (((data["num_peaks_10"]) * 2.0)))) + (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)) + (data["iqr"]))))) 
        v["i6"] = 0.010000*np.tanh(((((data["num_peaks_10"]) + (((((((((data["num_peaks_10"]) * 2.0)) + (data["percentile_roll_std_25"]))) + (((((data["abs_percentile_80"]) + (data["num_peaks_10"]))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i7"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_10"]) + (data["iqr"]))) * 2.0)) + (((data["num_peaks_10"]) + (((((data["num_peaks_10"]) + (data["num_peaks_10"]))) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i8"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_1"]) + (((((data["fftr_range_-3000_-2000"]) + (data["percentile_roll_std_20"]))) * 2.0)))) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i9"] = 0.010000*np.tanh(((((((((((data["fftr_range_-3000_-2000"]) + (((((((data["min_roll_std_10000"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) + (data["num_peaks_10"]))) * 2.0)) * 2.0)) 
        v["i10"] = 0.010000*np.tanh(((((((((((data["iqr"]) + (((data["abs_percentile_80"]) + (((data["num_peaks_10"]) + (data["percentile_roll_std_30"]))))))) + (data["num_peaks_10"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i11"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_10"]) + (((((data["fftr_range_-3000_-2000"]) + (data["percentile_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) + (((data["ffti_range_2000_3000"]) + (data["ffti_percentile_80"]))))) 
        v["i12"] = 0.010000*np.tanh(((((((((data["abs_percentile_80"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (((data["percentile_roll_std_40"]) + (((data["num_peaks_10"]) + (data["fftr_range_3000_4000"]))))))) * 2.0)) 
        v["i13"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_1"]) + (data["fftr_range_-4000_-3000"]))) + (data["num_peaks_10"]))) * 2.0)) * 2.0)) + (data["percentile_roll_std_20"]))) + (data["num_peaks_10"]))) * 2.0)) 
        v["i14"] = 0.010000*np.tanh(((((((data["iqr"]) + (((((data["percentile_roll_std_10"]) * 2.0)) + (data["num_peaks_10"]))))) + (((data["abs_percentile_80"]) + (((data["num_peaks_10"]) * 2.0)))))) * 2.0)) 
        v["i15"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_30"]) + (data["iqr"]))) * 2.0)) + (((data["percentile_95"]) * 2.0)))) + (data["ffti_range_-3000_-2000"]))) * 2.0)) 
        v["i16"] = 0.010000*np.tanh(((((((data["percentile_roll_std_30"]) + (data["percentile_roll_std_1"]))) * 2.0)) + (((((data["iqr"]) + (data["ffti_range_-4000_-3000"]))) * 2.0)))) 
        v["i17"] = 0.010000*np.tanh(((((((data["percentile_95"]) + (((((((data["num_peaks_10"]) * 2.0)) + (data["abs_percentile_80"]))) * 2.0)))) + (data["ffti_range_-4000_-3000"]))) * 2.0)) 
        v["i18"] = 0.010000*np.tanh(((((((data["percentile_roll_std_20"]) * 2.0)) * 2.0)) + (((((((((data["ffti_count_big"]) + (data["ffti_range_3000_4000"]))) + (data["num_peaks_10"]))) + (data["num_peaks_10"]))) * 2.0)))) 
        v["i19"] = 0.010000*np.tanh(((((data["percentile_roll_std_10"]) + (((data["num_peaks_10"]) + (((((data["num_peaks_10"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i20"] = 0.010000*np.tanh(((((((((((((data["num_peaks_10"]) * 2.0)) + (data["abs_percentile_80"]))) * 2.0)) * 2.0)) + (((((data["num_peaks_10"]) * 2.0)) + (((data["num_peaks_10"]) * 2.0)))))) * 2.0)) 
        v["i21"] = 0.010000*np.tanh(((((((((((((data["ffti_range_3000_4000"]) + (((((data["iqr"]) + (data["percentile_roll_std_5"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["percentile_roll_std_40"]))) 
        v["i22"] = 0.010000*np.tanh(((((((((((data["iqr"]) + (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) + (((((((data["ffti_range_-3000_-2000"]) + (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)))) 
        v["i23"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((((((((data["iqr"]) + (data["num_peaks_10"]))) + (data["percentile_roll_std_10"]))) + (data["num_peaks_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i24"] = 0.010000*np.tanh(((((((data["min_roll_std_100"]) + (data["percentile_roll_std_1"]))) + (((((((((data["abs_percentile_80"]) + (data["num_peaks_10"]))) + (data["num_peaks_10"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i25"] = 0.010000*np.tanh(((((((((data["abs_percentile_80"]) + (((((data["min_roll_std_10000"]) * 2.0)) + (data["num_peaks_10"]))))) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i26"] = 0.010000*np.tanh(((data["num_peaks_10"]) + (((((((((((data["iqr"]) * 2.0)) + (data["num_peaks_10"]))) + (((data["num_peaks_10"]) + (data["percentile_roll_std_10"]))))) * 2.0)) * 2.0)))) 
        v["i27"] = 0.010000*np.tanh(((((((data["fftr_range_-3000_-2000"]) + (((((data["iqr1"]) + (data["iqr"]))) + (((((data["iqr1"]) * 2.0)) + (data["num_peaks_10"]))))))) * 2.0)) * 2.0)) 
        v["i28"] = 0.010000*np.tanh(((((((((((data["num_peaks_10"]) + (data["min_roll_std_10000"]))) * 2.0)) * 2.0)) + (((data["abs_percentile_80"]) * 2.0)))) * 2.0)) 
        v["i29"] = 0.010000*np.tanh(((((((data["min_roll_std_100"]) + (data["min_roll_std_500"]))) + (((((((data["ffti_range_3000_4000"]) + (data["min_roll_std_10000"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i30"] = 0.010000*np.tanh(((((((((((((data["fftr_range_-4000_-3000"]) + (data["num_peaks_10"]))) * 2.0)) * 2.0)) * 2.0)) + (((data["ffti_abs_percentile_30"]) * 2.0)))) + (data["percentile_roll_std_10"]))) 
        v["i31"] = 0.010000*np.tanh(((((((data["fftr_range_-3000_-2000"]) + (((data["percentile_75"]) - (((data["fftr_range_-3000_-2000"]) - (((((data["fftr_range_-3000_-2000"]) + (data["percentile_roll_std_10"]))) * 2.0)))))))) + (data["min_roll_std_1000"]))) * 2.0)) 
        v["i32"] = 0.010000*np.tanh(((((data["fftr_range_-3000_-2000"]) + (((data["percentile_roll_std_1"]) + (((((data["percentile_roll_std_99"]) + (((data["percentile_roll_std_1"]) + (data["iqr"]))))) + (data["ffti_range_-4000_-3000"]))))))) * 2.0)) 
        v["i33"] = 0.010000*np.tanh(((((((data["abs_percentile_80"]) + ((((((((data["num_peaks_100"]) + (((data["percentile_roll_std_1"]) + (data["iqr"]))))/2.0)) + (data["num_peaks_10"]))) * 2.0)))) + (data["min_roll_std_500"]))) * 2.0)) 
        v["i34"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((((((((data["fftr_range_-3000_-2000"]) + (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i35"] = 0.010000*np.tanh(((((((data["abs_percentile_80"]) * 2.0)) + (((((((((((data["num_peaks_10"]) * 2.0)) + (((data["range_-1000_0"]) + (data["iqr"]))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i36"] = 0.010000*np.tanh(((((((((data["num_peaks_10"]) * 2.0)) + (data["ffti_range_2000_3000"]))) + (((((((data["abs_percentile_80"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i37"] = 0.010000*np.tanh(((((((((((data["num_peaks_10"]) * 2.0)) * 2.0)) + (((data["abs_percentile_80"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i38"] = 0.010000*np.tanh(((((((((((data["abs_percentile_80"]) * 2.0)) + (data["percentile_roll_std_1"]))) + (data["min_roll_std_10000"]))) + (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) * 2.0)) 
        v["i39"] = 0.010000*np.tanh(((((((((data["ffti_range_2000_3000"]) + (data["percentile_roll_std_25"]))) * 2.0)) + (((data["ffti_range_2000_3000"]) + (data["ffti_percentile_70"]))))) + (data["min_roll_std_500"]))) 
        v["i40"] = 0.010000*np.tanh(((((((data["min_roll_std_100"]) + (data["num_peaks_10"]))) * 2.0)) + (((((data["min_roll_std_10000"]) * 2.0)) + (((data["num_peaks_10"]) * 2.0)))))) 
        v["i41"] = 0.010000*np.tanh(((((data["iqr"]) + (((((((((data["percentile_95"]) * 2.0)) * 2.0)) + (((data["iqr"]) * 2.0)))) * 2.0)))) + (((data["iqr"]) + (data["min_roll_std_10000"]))))) 
        v["i42"] = 0.010000*np.tanh(((((data["num_peaks_10"]) + (((data["percentile_95"]) + (((data["abs_percentile_80"]) + (((data["min_roll_std_100"]) + (((data["num_peaks_10"]) * 2.0)))))))))) * 2.0)) 
        v["i43"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((((((data["num_peaks_10"]) * 2.0)) + (data["abs_mean"]))) + (data["min_roll_std_500"]))))) * 2.0)) * 2.0)) + (((data["iqr"]) * 2.0)))) 
        v["i44"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((data["percentile_90"]) + (((data["fftr_range_-3000_-2000"]) + (data["min_roll_std_500"]))))))) + (data["percentile_roll_std_25"]))) * 2.0)) * 2.0)) 
        v["i45"] = 0.010000*np.tanh(((((data["min_roll_std_500"]) * 2.0)) + (((((((data["ffti_spkt_welch_density_1"]) + (data["min_roll_std_10000"]))) + (((((data["iqr"]) + (data["abs_percentile_80"]))) * 2.0)))) * 2.0)))) 
        v["i46"] = 0.010000*np.tanh(((((((((data["abs_percentile_80"]) + (data["abs_percentile_80"]))) + (data["percentile_roll_std_10"]))) + (((data["num_peaks_10"]) + (((data["num_peaks_10"]) + (data["num_peaks_10"]))))))) * 2.0)) 
        v["i47"] = 0.010000*np.tanh(((((((data["abs_percentile_50"]) + (((((((((data["num_peaks_10"]) * 2.0)) + (data["fftr_range_-3000_-2000"]))) + (data["num_peaks_10"]))) + (data["percentile_roll_std_1"]))))) * 2.0)) * 2.0)) 
        v["i48"] = 0.010000*np.tanh(((((((((((data["abs_percentile_80"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (((data["min_roll_std_100"]) + (data["abs_percentile_80"]))))) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) 
        v["i49"] = 0.010000*np.tanh(((((data["num_peaks_10"]) * 2.0)) + (((((((data["fftr_range_-3000_-2000"]) + (data["num_peaks_10"]))) + (((data["min_roll_std_500"]) + (data["fftr_range_-3000_-2000"]))))) * 2.0)))) 
        v["i50"] = 0.010000*np.tanh(((((((((data["iqr"]) + (data["num_peaks_10"]))) * 2.0)) + (data["min_roll_std_500"]))) + (data["num_peaks_100"]))) 
        v["i51"] = 0.010000*np.tanh(((((data["ffti_range_-4000_-3000"]) + (((data["percentile_roll_std_5"]) + (((((data["iqr"]) * 2.0)) + (((((((data["percentile_95"]) * 2.0)) * 2.0)) + (data["min_roll_std_500"]))))))))) * 2.0)) 
        v["i52"] = 0.010000*np.tanh(((((((((data["min_roll_std_100"]) + (data["num_peaks_10"]))) + (data["num_peaks_10"]))) + (data["min_roll_std_10000"]))) * 2.0)) 
        v["i53"] = 0.010000*np.tanh(((((((((((data["num_peaks_10"]) * 2.0)) + (data["percentile_30"]))) * 2.0)) + (((((data["num_peaks_10"]) * 2.0)) + (data["ffti_spkt_welch_density_5"]))))) * 2.0)) 
        v["i54"] = 0.010000*np.tanh(((((((data["abs_percentile_50"]) + (data["num_peaks_10"]))) * 2.0)) + (((((data["min_roll_std_500"]) + (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i55"] = 0.010000*np.tanh(((((data["percentile_roll_std_1"]) + (((((data["min_roll_std_100"]) + (data["num_peaks_10"]))) * 2.0)))) * 2.0)) 
        v["i56"] = 0.010000*np.tanh(((((((data["num_peaks_10"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (((((((data["min_roll_std_100"]) + (((data["ffti_percentile_75"]) * 2.0)))) + (data["min_roll_std_100"]))) * 2.0)))) 
        v["i57"] = 0.010000*np.tanh(((((((data["min_roll_std_500"]) * 2.0)) + (((((data["num_peaks_10"]) + (((((data["min_roll_std_10000"]) + (data["percentile_95"]))) + (data["min_roll_std_100"]))))) * 2.0)))) + (data["num_peaks_10"]))) 
        v["i58"] = 0.010000*np.tanh(((((((((((((data["abs_percentile_80"]) + (data["num_peaks_10"]))) * 2.0)) + (data["percentile_70"]))) * 2.0)) + (((data["num_peaks_10"]) + (data["num_peaks_10"]))))) + (data["abs_percentile_80"]))) 
        v["i59"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((data["abs_percentile_80"]) * 2.0)))) * 2.0)) + (data["range_-1000_0"]))) + (((((data["ffti_abs_max_roll_mean_100"]) * 2.0)) + (((data["min_roll_std_500"]) * 2.0)))))) 
        v["i60"] = 0.010000*np.tanh(((((((data["percentile_roll_std_1"]) + (((((data["min_roll_std_100"]) + (data["min_roll_std_100"]))) * 2.0)))) + (data["percentile_roll_std_1"]))) + (((((data["num_peaks_10"]) + (data["percentile_95"]))) * 2.0)))) 
        v["i61"] = 0.010000*np.tanh(((((data["num_peaks_10"]) * 2.0)) + (((((((data["abs_percentile_70"]) + (data["fftr_range_-3000_-2000"]))) * 2.0)) + (((data["min_roll_std_500"]) + (data["fftr_range_-3000_-2000"]))))))) 
        v["i62"] = 0.010000*np.tanh(((((data["abs_percentile_80"]) * 2.0)) + (((((((((((data["fftr_range_-3000_-2000"]) + (data["percentile_90"]))) * 2.0)) * 2.0)) * 2.0)) + (((data["abs_percentile_80"]) + (data["percentile_roll_std_1"]))))))) 
        v["i63"] = 0.010000*np.tanh(((data["min_roll_std_10000"]) + (((((data["fftr_av_change_rate_roll_mean_100"]) + (((((data["fftr_range_-3000_-2000"]) * 2.0)) + (((((data["fftr_range_-3000_-2000"]) + (data["min_roll_std_500"]))) * 2.0)))))) + (data["min_roll_std_100"]))))) 
        v["i64"] = 0.010000*np.tanh(((((data["min_roll_std_1000"]) + (data["percentile_roll_std_1"]))) + (((((((data["min_roll_std_100"]) + ((((data["ffti_max_roll_mean_100"]) + (data["min_roll_std_1000"]))/2.0)))) * 2.0)) * 2.0)))) 
        v["i65"] = 0.010000*np.tanh(((((((data["abs_percentile_80"]) * 2.0)) + (((data["min_roll_std_500"]) + (data["ffti_abs_max_roll_mean_100"]))))) * 2.0)) 
        v["i66"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_10"]) + (data["abs_percentile_50"]))) * 2.0)) * 2.0)) + (data["abs_percentile_50"]))) * 2.0)) * 2.0)) + (data["fftr_range_-3000_-2000"]))) 
        v["i67"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((((data["iqr"]) * 2.0)) * 2.0)))) + (((data["num_peaks_10"]) * 2.0)))) + (((data["num_peaks_10"]) + (data["num_peaks_10"]))))) * 2.0)) 
        v["i68"] = 0.010000*np.tanh(((((((((data["percentile_95"]) + (data["fftr_range_2000_3000"]))) + (data["ffti_spkt_welch_density_5"]))) * 2.0)) * 2.0)) 
        v["i69"] = 0.010000*np.tanh(((((((data["fftr_range_2000_3000"]) + (((data["min_roll_std_10000"]) + (((((data["ffti_spkt_welch_density_1"]) + (data["iqr"]))) * 2.0)))))) * 2.0)) + (((data["min_roll_std_100"]) * 2.0)))) 
        v["i70"] = 0.010000*np.tanh(((((data["ffti_abs_percentile_20"]) + (((data["fftr_range_-3000_-2000"]) + (((((((data["fftr_range_-3000_-2000"]) + (data["percentile_roll_std_1"]))) * 2.0)) + (((data["fftr_range_-3000_-2000"]) * 2.0)))))))) * 2.0)) 
        v["i71"] = 0.010000*np.tanh(((((data["ffti_spkt_welch_density_1"]) * 2.0)) + (((((((((((data["abs_percentile_25"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i72"] = 0.010000*np.tanh(((((((data["percentile_70"]) + (((data["num_peaks_20"]) + (data["num_peaks_10"]))))) + (((((data["min_roll_std_100"]) + (data["min_roll_std_10000"]))) + (data["min_roll_std_100"]))))) * 2.0)) 
        v["i73"] = 0.010000*np.tanh(((((((data["iqr"]) + (data["min_roll_std_10000"]))) + (data["fftr_range_-3000_-2000"]))) + (((((((data["min_roll_std_10000"]) + (data["fftr_range_-3000_-2000"]))) + (data["fftr_range_-3000_-2000"]))) * 2.0)))) 
        v["i74"] = 0.010000*np.tanh(((((((data["min_roll_std_500"]) + (((data["abs_percentile_60"]) - (data["fftr_min_roll_mean_100"]))))) + (data["num_peaks_10"]))) + (((data["num_peaks_100"]) + (data["min_roll_std_500"]))))) 
        v["i75"] = 0.010000*np.tanh(((((((((((data["fftr_range_-3000_-2000"]) * 2.0)) + (data["min_roll_std_500"]))) + (data["iqr"]))) + (data["percentile_roll_std_1"]))) * 2.0)) 
        v["i76"] = 0.010000*np.tanh(((((((((data["min_roll_std_500"]) + (((((((data["ffti_range_-3000_-2000"]) + (data["percentile_95"]))) * 2.0)) + (data["fftr_range_-3000_-2000"]))))) * 2.0)) + (data["min_roll_std_50"]))) + (data["min_roll_std_10000"]))) 
        v["i77"] = 0.010000*np.tanh(((((data["percentile_90"]) * 2.0)) + (((((((data["min_roll_std_100"]) + (((((((data["fftr_range_-3000_-2000"]) * 2.0)) + (data["ffti_max_roll_mean_10"]))) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i78"] = 0.010000*np.tanh(((((((((data["min_roll_std_1000"]) + (data["num_peaks_10"]))) * 2.0)) + (((((data["min_roll_std_50"]) + (data["min_roll_std_10000"]))) * 2.0)))) + (data["min_roll_std_100"]))) 
        v["i79"] = 0.010000*np.tanh(((((((((((data["ffti_max_roll_mean_100"]) + (data["min_roll_std_100"]))) * 2.0)) + (data["min_roll_std_500"]))) * 2.0)) * 2.0)) 
        v["i80"] = 0.010000*np.tanh(((((((((data["percentile_90"]) + (((((data["percentile_90"]) * 2.0)) * 2.0)))) + (data["classic_sta_lta7_mean"]))) * 2.0)) + (((data["num_peaks_10"]) * 2.0)))) 
        v["i81"] = 0.010000*np.tanh(((((data["num_peaks_10"]) + (data["min_roll_std_100"]))) + (((data["iqr"]) + (((((((data["min_roll_std_50"]) + (((data["min_roll_std_100"]) * 2.0)))) * 2.0)) * 2.0)))))) 
        v["i82"] = 0.010000*np.tanh(((((((((((data["percentile_90"]) + (data["fftr_range_2000_3000"]))) * 2.0)) * 2.0)) + (data["ffti_kstat_3"]))) + (((data["fftr_range_2000_3000"]) + (((data["percentile_80"]) + (data["fftr_abs_percentile_70"]))))))) 
        v["i83"] = 0.010000*np.tanh(((data["min_roll_std_50"]) + (((((((((data["abs_percentile_50"]) + (data["min_roll_std_100"]))) + (data["ffti_abs_max_roll_mean_100"]))) * 2.0)) * 2.0)))) 
        v["i84"] = 0.010000*np.tanh(((((((((data["fftr_range_2000_3000"]) + ((((((data["fftr_range_2000_3000"]) + (data["percentile_95"]))/2.0)) * 2.0)))) + (((data["percentile_95"]) + (data["percentile_roll_mean_1"]))))) * 2.0)) - (data["ffti_min_roll_mean_100"]))) 
        v["i85"] = 0.010000*np.tanh(((((data["percentile_90"]) + (((data["percentile_70"]) + (((((data["abs_max_roll_mean_1000"]) + (((((data["num_peaks_50"]) + (data["min_roll_std_100"]))) * 2.0)))) + (data["fftr_range_-3000_-2000"]))))))) * 2.0)) 
        v["i86"] = 0.010000*np.tanh(((((((data["min_roll_std_500"]) - (((data["exp_Moving_average_30000_std"]) - (data["percentile_90"]))))) * 2.0)) * 2.0)) 
        v["i87"] = 0.010000*np.tanh(((((((data["percentile_roll_std_1"]) * 2.0)) + (((data["fftr_range_-3000_-2000"]) * 2.0)))) + (((((data["percentile_roll_std_1"]) + (((data["iqr"]) * 2.0)))) * 2.0)))) 
        v["i88"] = 0.010000*np.tanh(((((((data["ffti_range_2000_3000"]) + (data["abs_percentile_70"]))) * 2.0)) + (((((data["fftr_range_2000_3000"]) + (data["percentile_70"]))) * 2.0)))) 
        v["i89"] = 0.010000*np.tanh(((((((((data["min_roll_std_100"]) * 2.0)) + (data["min_roll_std_100"]))) + (data["percentile_roll_std_10"]))) + (((((data["fftr_range_-3000_-2000"]) + (data["percentile_90"]))) * 2.0)))) 
        v["i90"] = 0.010000*np.tanh(((((((((data["min_roll_std_100"]) * 2.0)) * 2.0)) + (((((data["min_roll_std_10000"]) * 2.0)) - (data["autocorrelation_1000"]))))) + (((((data["min_roll_std_10000"]) * 2.0)) - (data["autocorrelation_1000"]))))) 
        v["i91"] = 0.010000*np.tanh(((((((((((data["min_roll_std_100"]) * 2.0)) * 2.0)) + (data["fftr_range_-3000_-2000"]))) + (((data["ffti_abs_percentile_5"]) * 2.0)))) + (((data["fftr_range_-3000_-2000"]) * 2.0)))) 
        v["i92"] = 0.010000*np.tanh(((((((((data["percentile_90"]) + (((((data["percentile_90"]) * 2.0)) + (((data["fftr_range_-3000_-2000"]) - (data["exp_Moving_average_30000_std"]))))))) * 2.0)) + (data["percentile_90"]))) * 2.0)) 
        v["i93"] = 0.010000*np.tanh(((((data["percentile_roll_mean_10"]) + (((((((data["iqr"]) * 2.0)) * 2.0)) + (((((data["ffti_range_2000_3000"]) + (((data["fftr_range_2000_3000"]) * 2.0)))) + (data["percentile_80"]))))))) * 2.0)) 
        v["i94"] = 0.010000*np.tanh(((((((((((data["num_peaks_10"]) * 2.0)) + (data["mean_last_10000"]))) + (((((data["mean_last_10000"]) + (data["num_peaks_10"]))) + (data["num_peaks_10"]))))) + (data["fftr_spkt_welch_density_1"]))) * 2.0)) 
        v["i95"] = 0.010000*np.tanh(((((((((data["min_roll_std_500"]) * 2.0)) + (((((((((data["percentile_90"]) + (data["num_peaks_100"]))) * 2.0)) * 2.0)) - (data["num_peaks_100"]))))) + (data["percentile_90"]))) * 2.0)) 
        v["i96"] = 0.010000*np.tanh(((((((((data["min_roll_std_100"]) * 2.0)) + (((data["percentile_roll_std_20"]) + (((((data["fftr_spkt_welch_density_1"]) * 2.0)) + (data["min_roll_std_50"]))))))) + (data["ffti_abs_percentile_1"]))) * 2.0)) 
        v["i97"] = 0.010000*np.tanh(((((data["percentile_90"]) + (((data["percentile_70"]) + (data["percentile_90"]))))) + (((((data["min_roll_std_100"]) + (data["ffti_spkt_welch_density_1"]))) * 2.0)))) 
        v["i98"] = 0.010000*np.tanh(((((((data["min_roll_std_100"]) + (((((((data["min_roll_std_100"]) * 2.0)) + (((data["ffti_max_roll_mean_100"]) * 2.0)))) * 2.0)))) + (data["ffti_range_-3000_-2000"]))) * 2.0)) 
        v["i99"] = 0.010000*np.tanh(((((((data["percentile_90"]) + (((data["percentile_roll_mean_5"]) + (data["percentile_90"]))))) + (((data["num_peaks_50"]) + (data["percentile_90"]))))) * 2.0)) 
        v["i100"] = 0.010000*np.tanh(((((((((((data["percentile_20"]) + (data["percentile_90"]))) + (data["num_peaks_10"]))) * 2.0)) * 2.0)) + (data["ffti_kstat_2"]))) 
        v["i101"] = 0.010000*np.tanh(((((((data["num_peaks_10"]) / 2.0)) - (data["mean_change_rate_last_10000"]))) + (((((((((((data["num_peaks_10"]) + (data["abs_percentile_25"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i102"] = 0.010000*np.tanh(((((data["fftr_range_-3000_-2000"]) - (data["mean_change_rate_last_10000"]))) + (((((((data["ffti_abs_percentile_1"]) * 2.0)) - (data["mean_change_rate_first_1000"]))) * 2.0)))) 
        v["i103"] = 0.010000*np.tanh(((data["num_peaks_10"]) + (((((((((data["min_roll_std_100"]) + (((data["num_peaks_10"]) - (data["fftr_classic_sta_lta5_mean"]))))) * 2.0)) + (data["num_peaks_10"]))) * 2.0)))) 
        v["i104"] = 0.010000*np.tanh(((data["fftr_exp_Moving_average_300_std"]) + (((((((data["num_peaks_10"]) + (data["percentile_90"]))) + (((((data["percentile_90"]) + (((data["ffti_abs_percentile_10"]) + (data["classic_sta_lta7_mean"]))))) * 2.0)))) * 2.0)))) 
        v["i105"] = 0.010000*np.tanh(((((((data["fftr_range_-3000_-2000"]) - (((data["exp_Moving_average_30000_std"]) - (((data["ffti_mean_change_rate"]) - (((data["exp_Moving_average_30000_std"]) - (((data["percentile_90"]) * 2.0)))))))))) * 2.0)) * 2.0)) 
        v["i106"] = 0.010000*np.tanh(((((data["min_roll_std_10000"]) + ((((((-1.0*((data["mean_change_rate_last_10000"])))) + (data["mean_change_rate_last_10000"]))) + ((-1.0*((data["mean_change_rate_last_10000"])))))))) * 2.0)) 
        v["i107"] = 0.010000*np.tanh(((((data["percentile_90"]) + (((((data["min_roll_std_500"]) + (((data["ave_roll_mean_100"]) + (data["num_peaks_20"]))))) + (data["percentile_90"]))))) * 2.0)) 
        v["i108"] = 0.010000*np.tanh(((((((((data["ffti_abs_percentile_1"]) + (((data["percentile_90"]) + (((data["num_peaks_10"]) + (((data["ffti_mean_first_50000"]) + (data["ffti_mean_first_50000"]))))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i109"] = 0.010000*np.tanh(((((((data["percentile_90"]) + (data["min_roll_std_1000"]))) + (((data["min_roll_mean_10000"]) + (((data["percentile_90"]) + (data["ffti_max_roll_mean_100"]))))))) * 2.0)) 
        v["i110"] = 0.010000*np.tanh(((((((((((data["ffti_mean_change_rate"]) + (((data["ffti_spkt_welch_density_1"]) + (data["percentile_90"]))))) + (((data["ffti_kstat_1"]) + (data["fftr_range_-3000_-2000"]))))) * 2.0)) * 2.0)) + (data["ffti_kstat_1"]))) 
        v["i111"] = 0.010000*np.tanh(((((data["percentile_90"]) + (((data["fftr_percentile_roll_mean_20"]) + (((((((data["fftr_percentile_roll_mean_1"]) + (data["percentile_90"]))) * 2.0)) * 2.0)))))) + (data["fftr_percentile_roll_mean_10"]))) 
        v["i112"] = 0.010000*np.tanh(((((((((((((data["min_roll_std_100"]) + (data["fftr_spkt_welch_density_1"]))) * 2.0)) + (((data["ffti_abs_percentile_1"]) + (data["ffti_abs_max_roll_mean_50"]))))) * 2.0)) + (((data["fftr_percentile_roll_std_99"]) / 2.0)))) * 2.0)) 
        v["i113"] = 0.010000*np.tanh(((((data["num_peaks_10"]) - (data["ffti_exp_Moving_average_3000_std"]))) + (((((((((data["num_peaks_10"]) - (data["ffti_exp_Moving_average_3000_std"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i114"] = 0.010000*np.tanh(((((data["percentile_95"]) * 2.0)) + (((((((((data["percentile_roll_std_20"]) - (data["percentile_roll_std_70"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i115"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_10"]) / 2.0)) + (((((data["percentile_90"]) - (data["exp_Moving_average_50000_std"]))) + (data["percentile_90"]))))) * 2.0)) 
        v["i116"] = 0.010000*np.tanh(((((((((data["ffti_abs_percentile_1"]) - (data["mean_change_rate_first_1000"]))) + (((data["ffti_abs_percentile_1"]) - (data["mean_change_rate_first_1000"]))))) * 2.0)) + (data["percentile_90"]))) 
        v["i117"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_mean_75"]) + (data["ffti_abs_max_roll_mean_10"]))) + (data["percentile_roll_mean_10"]))) + (((data["percentile_90"]) + (data["num_peaks_10"]))))) * 2.0)) + (data["percentile_roll_mean_10"]))) 
        v["i118"] = 0.010000*np.tanh(((((data["min_roll_std_10000"]) + (((((((((((data["min_roll_std_10000"]) - (data["classic_sta_lta6_mean"]))) * 2.0)) - (data["classic_sta_lta6_mean"]))) + (data["fftr_percentile_roll_mean_75"]))) + (data["num_peaks_10"]))))) * 2.0)) 
        v["i119"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_10"]) + (data["fftr_percentile_roll_mean_75"]))) * 2.0)) + ((((((-1.0*((data["mean_change_rate_first_10000"])))) + (data["ffti_range_-3000_-2000"]))) * 2.0)))) 
        v["i120"] = 0.010000*np.tanh(((data["percentile_90"]) + (((((((data["abs_percentile_25"]) + (data["num_peaks_10"]))) * 2.0)) * 2.0)))) 
        v["i121"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_1"]) * 2.0)) - (data["percentile_20"]))) + (((((((((data["percentile_20"]) * 2.0)) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i122"] = 0.010000*np.tanh(((((((((((((data["min_roll_std_100"]) * 2.0)) + (data["min_roll_std_50"]))) * 2.0)) * 2.0)) + (((data["ffti_abs_max_roll_mean_50"]) * 2.0)))) * 2.0)) 
        v["i123"] = 0.010000*np.tanh(((((((((data["percentile_90"]) + (((((((data["percentile_90"]) + (data["fftr_percentile_roll_mean_10"]))) * 2.0)) * 2.0)))) + (data["fftr_percentile_roll_mean_10"]))) * 2.0)) * 2.0)) 
        v["i124"] = 0.010000*np.tanh(((((((data["percentile_90"]) - (data["std_roll_mean_10000"]))) + (((((data["num_peaks_10"]) / 2.0)) + (data["percentile_90"]))))) * 2.0)) 
        v["i125"] = 0.010000*np.tanh(((((((((((data["percentile_90"]) + (((((data["percentile_90"]) + (data["ffti_std_roll_mean_500"]))) - (data["ffti_std_roll_std_10"]))))) - (data["exp_Moving_average_30000_std"]))) * 2.0)) + (data["fftr_percentile_roll_std_5"]))) * 2.0)) 
        v["i126"] = 0.010000*np.tanh(((((((((data["percentile_20"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (((((data["percentile_90"]) + (((data["fftr_ave_roll_std_100"]) + (data["percentile_20"]))))) * 2.0)))) * 2.0)) 
        v["i127"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_mean_5"]) + (((data["percentile_90"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) + (((data["ffti_abs_max_roll_mean_50"]) + (data["ffti_time_rev_asym_stat_1"]))))) * 2.0)) 
        v["i128"] = 0.010000*np.tanh(((((((data["min_roll_std_100"]) + (((data["fftr_percentile_roll_mean_1"]) * 2.0)))) * 2.0)) + ((((((((((data["ffti_abs_percentile_5"]) + (((data["fftr_percentile_roll_mean_10"]) * 2.0)))/2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i129"] = 0.010000*np.tanh(((((((((((((data["av_change_abs_roll_mean_100"]) + (data["fftr_percentile_roll_mean_75"]))) + (((((((data["percentile_90"]) + (data["fftr_percentile_roll_mean_10"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i130"] = 0.010000*np.tanh(((((((data["min_roll_std_1000"]) + (data["ffti_abs_percentile_10"]))) - (((data["mean_change_rate_last_10000"]) * 2.0)))) * 2.0)) 
        v["i131"] = 0.010000*np.tanh(((data["percentile_roll_std_70"]) - (((data["fftr_time_rev_asym_stat_10"]) - (((((((((data["num_peaks_10"]) + ((-1.0*((data["percentile_roll_std_70"])))))) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i132"] = 0.010000*np.tanh(((((data["percentile_90"]) * 2.0)) + (((((((((((data["percentile_90"]) * 2.0)) + (data["percentile_20"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i133"] = 0.010000*np.tanh(((((((((data["ffti_abs_percentile_10"]) + (data["num_peaks_50"]))) - (data["mean_change_rate_first_1000"]))) * 2.0)) + (data["ffti_abs_max_roll_mean_100"]))) 
        v["i134"] = 0.010000*np.tanh(((((((((((((((data["min_roll_std_100"]) + (((data["ffti_mean"]) - (data["ffti_min_roll_mean_100"]))))) * 2.0)) * 2.0)) * 2.0)) + (data["min_roll_std_50"]))) * 2.0)) * 2.0)) 
        v["i135"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_80"]) + (((((data["fftr_percentile_roll_mean_75"]) + (data["ffti_mean"]))) + (((data["ffti_mean"]) + (data["num_peaks_10"]))))))) * 2.0)) 
        v["i136"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_5"]) + (((((((data["fftr_percentile_roll_std_5"]) - (((data["mean_change_rate_last_10000"]) + (data["exp_Moving_average_50000_std"]))))) + (data["num_peaks_50"]))) * 2.0)))) + (data["min_roll_std_10000"]))) * 2.0)) 
        v["i137"] = 0.010000*np.tanh(((((((((data["ffti_range_-2000_-1000"]) + (data["num_peaks_10"]))) + (((data["ffti_range_-2000_-1000"]) + (data["num_peaks_10"]))))) * 2.0)) * 2.0)) 
        v["i138"] = 0.010000*np.tanh((((-1.0*((data["classic_sta_lta6_mean"])))) + (((((data["fftr_abs_percentile_50"]) + (((((((data["percentile_roll_std_20"]) - (data["classic_sta_lta6_mean"]))) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i139"] = 0.010000*np.tanh(((data["min_roll_std_100"]) + (((((((((((data["min_roll_std_100"]) + (((data["min_roll_std_100"]) * 2.0)))) * 2.0)) + (data["min_roll_std_50"]))) + (data["min_roll_std_100"]))) + (data["min_roll_std_50"]))))) 
        v["i140"] = 0.010000*np.tanh(((((((((((((((((((data["percentile_90"]) * 2.0)) - (data["ffti_gmean"]))) - (data["ffti_percentile_roll_std_1"]))) * 2.0)) * 2.0)) * 2.0)) - (data["ffti_percentile_roll_std_5"]))) * 2.0)) * 2.0)) 
        v["i141"] = 0.010000*np.tanh(((((((data["ffti_mean_first_50000"]) * 2.0)) + (((((data["fftr_percentile_roll_std_5"]) + (data["fftr_percentile_roll_std_5"]))) + (((data["ffti_max_roll_mean_50"]) + (data["num_peaks_10"]))))))) + (data["fftr_percentile_roll_std_5"]))) 
        v["i142"] = 0.010000*np.tanh(((((((data["percentile_90"]) + (((data["fftr_percentile_roll_mean_10"]) + (((((((((data["percentile_90"]) + (data["fftr_percentile_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i143"] = 0.010000*np.tanh(((((((((data["min_first_10000"]) + (((((data["min_first_10000"]) + (((data["percentile_90"]) * 2.0)))) * 2.0)))) - (data["exp_Moving_average_50000_std"]))) * 2.0)) * 2.0)) 
        v["i144"] = 0.010000*np.tanh(((((((data["abs_percentile_75"]) + (((((((data["ffti_abs_percentile_5"]) - (data["mean_change_rate_first_1000"]))) + (data["ffti_abs_percentile_5"]))) + (data["autocorrelation_10"]))))) * 2.0)) * 2.0)) 
        v["i145"] = 0.010000*np.tanh(((((((((((((data["ffti_range_-2000_-1000"]) + (((data["min_roll_std_10000"]) * 2.0)))) - (data["ffti_abs_percentile_10"]))) * 2.0)) + (data["ffti_abs_percentile_10"]))) * 2.0)) - (data["fftr_abs_percentile_1"]))) 
        v["i146"] = 0.010000*np.tanh(((((((data["percentile_90"]) + (((((((data["percentile_90"]) * 2.0)) + (((data["classic_sta_lta7_mean"]) + (((data["fftr_percentile_roll_mean_10"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i147"] = 0.010000*np.tanh(((((((((((((data["num_peaks_10"]) - (data["percentile_roll_std_80"]))) * 2.0)) + (data["num_peaks_10"]))) * 2.0)) * 2.0)) - (data["percentile_95"]))) 
        v["i148"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_75"]) + (((((((data["percentile_roll_mean_1"]) - (data["mean_change_rate_last_50000"]))) + (data["fftr_percentile_roll_mean_75"]))) + (((((data["fftr_range_2000_3000"]) + (data["percentile_roll_mean_1"]))) * 2.0)))))) * 2.0)) 
        v["i149"] = 0.010000*np.tanh(((((((((((data["ffti_abs_percentile_10"]) + (((data["ffti_abs_max_roll_mean_10"]) + (((data["percentile_90"]) - (data["classic_sta_lta5_mean"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i150"] = 0.010000*np.tanh(((((data["ffti_hmean"]) + (((data["num_peaks_50"]) * 2.0)))) + (((data["ffti_mean_first_50000"]) + (data["ffti_abs_percentile_10"]))))) 
        v["i151"] = 0.010000*np.tanh(((((((((((((((data["classic_sta_lta7_mean"]) + (((data["ffti_abs_percentile_10"]) * 2.0)))) * 2.0)) + (data["num_peaks_50"]))) * 2.0)) * 2.0)) + (data["num_peaks_50"]))) * 2.0)) 
        v["i152"] = 0.010000*np.tanh(((((((((data["abs_percentile_50"]) - (((((((data["ffti_mean_last_50000"]) + (data["mean_change_rate_first_1000"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) - (data["ffti_mean_last_50000"]))) 
        v["i153"] = 0.010000*np.tanh(((((((((((data["ffti_abs_percentile_5"]) + (((data["ffti_abs_percentile_25"]) + (((data["fftr_percentile_roll_std_5"]) * 2.0)))))) - (data["percentile_20"]))) + (data["autocorrelation_10000"]))) * 2.0)) * 2.0)) 
        v["i154"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_5"]) + (((data["ffti_hmean"]) + (((data["ffti_av_change_abs_roll_std_100"]) + (data["ffti_mean_change_rate"]))))))) * 2.0)) + (((((data["ffti_av_change_abs_roll_std_100"]) - (data["mean_change_rate_first_1000"]))) * 2.0)))) 
        v["i155"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_mean_80"]) + (data["fftr_min_roll_mean_500"]))) + (data["num_peaks_10"]))) * 2.0)) + (data["fftr_min_roll_mean_500"]))) * 2.0)) 
        v["i156"] = 0.010000*np.tanh(((((((data["percentile_30"]) + (((data["ffti_mean_first_50000"]) + (data["fftr_min_roll_mean_10000"]))))) + (((((((data["fftr_percentile_roll_mean_80"]) + (data["percentile_90"]))) * 2.0)) + (data["av_change_abs_roll_std_10000"]))))) * 2.0)) 
        v["i157"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_10"]) + (((((data["ffti_abs_percentile_10"]) + (data["classic_sta_lta7_mean"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i158"] = 0.010000*np.tanh(((((((data["min_roll_std_50"]) + (data["percentile_roll_mean_5"]))) + (((data["ffti_max_roll_mean_100"]) + (data["ffti_mean_first_50000"]))))) * 2.0)) 
        v["i159"] = 0.010000*np.tanh(((((((((((((data["ffti_min_roll_mean_1000"]) * (data["fftr_mean_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) - (((data["mean_change_rate_first_1000"]) - (data["ffti_av_change_abs_roll_std_100"]))))) * 2.0)) 
        v["i160"] = 0.010000*np.tanh(((data["fftr_ave_roll_mean_500"]) + (((((data["ffti_mean"]) + (((((((data["ffti_mean"]) + (data["fftr_percentile_roll_mean_75"]))) * 2.0)) - (data["mean_change_rate"]))))) * 2.0)))) 
        v["i161"] = 0.010000*np.tanh(((((((((((((((((data["fftr_percentile_roll_mean_10"]) * 2.0)) + (((data["fftr_max_last_10000"]) + (data["fftr_percentile_roll_mean_10"]))))) * 2.0)) + (data["autocorrelation_10000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i162"] = 0.010000*np.tanh(((((data["percentile_80"]) * (data["autocorrelation_100"]))) + (((((((data["percentile_80"]) * (((((data["autocorrelation_100"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i163"] = 0.010000*np.tanh(((((((((((data["range_-1000_0"]) - (data["exp_Moving_std_50000_std"]))) - (data["mean_change_rate_last_10000"]))) + (((data["ffti_hmean"]) + (((data["ffti_abs_percentile_1"]) - (data["ffti_min_roll_std_10000"]))))))) * 2.0)) * 2.0)) 
        v["i164"] = 0.010000*np.tanh(((((((data["ffti_mean_first_50000"]) * 2.0)) + (((((((data["percentile_90"]) * 2.0)) - (data["ffti_exp_Moving_average_3000_std"]))) + (((data["fftr_percentile_roll_mean_1"]) + (data["min_roll_std_1000"]))))))) * 2.0)) 
        v["i165"] = 0.010000*np.tanh(((((((((data["time_rev_asym_stat_1"]) + (((data["ffti_abs_max_roll_mean_10"]) * 2.0)))) + (((data["autocorrelation_10"]) + (data["percentile_90"]))))) * 2.0)) * 2.0)) 
        v["i166"] = 0.010000*np.tanh(((data["fftr_range_-3000_-2000"]) + (((((((data["percentile_1"]) + (data["min_roll_std_100"]))) * 2.0)) * 2.0)))) 
        v["i167"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_mean_75"]) * 2.0)) - (((((data["ffti_abs_max_roll_mean_10000"]) + (data["ffti_min_roll_std_10000"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i168"] = 0.010000*np.tanh(((((((((data["fftr_percentile_roll_std_5"]) * 2.0)) - (data["mean_change_rate_first_1000"]))) + (((((((data["fftr_percentile_roll_std_5"]) - (data["mean_change_rate_first_1000"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i169"] = 0.010000*np.tanh(((((((((((((((data["percentile_70"]) + (data["ffti_av_change_abs_roll_std_10000"]))) + (data["autocorrelation_10000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["autocorrelation_10"]))) 
        v["i170"] = 0.010000*np.tanh(((((data["fftr_mean_last_50000"]) * (((((((((data["ffti_skew"]) - (data["ffti_abs_max_roll_mean_100"]))) * 2.0)) * 2.0)) - (((data["percentile_roll_std_70"]) - (((data["fftr_kstat_1"]) * 2.0)))))))) * 2.0)) 
        v["i171"] = 0.010000*np.tanh(((((((((((((((data["autocorrelation_10000"]) - (data["autocorrelation_1000"]))) - (data["autocorrelation_1000"]))) - (data["ffti_mean_last_50000"]))) * 2.0)) - (data["ffti_mean_last_50000"]))) * 2.0)) * 2.0)) 
        v["i172"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_5"]) * 2.0)) + (((((((((((((data["num_peaks_50"]) - (data["av_change_abs_roll_mean_500"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i173"] = 0.010000*np.tanh(((((((((((data["ffti_mean_first_50000"]) + (data["ffti_mean_first_50000"]))) - (((((data["std_roll_mean_10000"]) - (data["ffti_autocorrelation_1000"]))) - (data["ffti_mean_first_50000"]))))) - (data["av_change_abs_roll_mean_500"]))) * 2.0)) * 2.0)) 
        v["i174"] = 0.010000*np.tanh(((((data["num_peaks_10"]) + (((data["ffti_autocorrelation_10"]) + (((((((data["fftr_percentile_roll_std_5"]) - (data["ffti_percentile_90"]))) * 2.0)) - ((((data["fftr_min_roll_std_500"]) + (data["fftr_min_roll_std_500"]))/2.0)))))))) * 2.0)) 
        v["i175"] = 0.010000*np.tanh(((((((((((data["fftr_time_rev_asym_stat_1"]) * (((data["mean_first_50000"]) + (data["ffti_av_change_abs_roll_std_10000"]))))) * 2.0)) * 2.0)) + (data["ffti_max_last_1000"]))) * 2.0)) 
        v["i176"] = 0.010000*np.tanh(((((((((((((data["fftr_time_rev_asym_stat_50"]) * 2.0)) * 2.0)) * (((((data["autocorrelation_10"]) - (data["fftr_av_change_abs_roll_mean_10000"]))) * 2.0)))) + (data["autocorrelation_10"]))) * 2.0)) * 2.0)) 
        v["i177"] = 0.010000*np.tanh(((((((((data["autocorrelation_100"]) * 2.0)) * (((data["ffti_abs_max_roll_mean_100"]) + (((((data["percentile_90"]) * 2.0)) + (data["percentile_roll_mean_75"]))))))) * 2.0)) * 2.0)) 
        v["i178"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_mean_5"]) + (((((data["min_roll_std_10000"]) + (((data["ffti_percentile_roll_mean_5"]) - (data["mean_change_rate_first_10000"]))))) + (data["fftr_range_-3000_-2000"]))))) * 2.0)) 
        v["i179"] = 0.010000*np.tanh(((((data["num_peaks_50"]) + (((((((data["av_change_abs_roll_std_1000"]) - (((((((data["classic_sta_lta5_mean"]) - (((data["percentile_95"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i180"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_10"]) + (((((data["fftr_percentile_roll_mean_10"]) + (((data["percentile_90"]) + (((((data["fftr_min_roll_std_500"]) * (data["fftr_range_p4000_pinf"]))) * 2.0)))))) * 2.0)))) * 2.0)) 
        v["i181"] = 0.010000*np.tanh(((((((((((((data["ffti_mean"]) - (data["fftr_min_roll_std_500"]))) * 2.0)) - (data["fftr_num_peaks_50"]))) * 2.0)) - (data["min_first_10000"]))) - (data["ffti_exp_Moving_average_50000_mean"]))) 
        v["i182"] = 0.010000*np.tanh(((((((data["percentile_70"]) + (data["std_last_1000"]))) + (((((((data["ffti_av_change_abs_roll_std_100"]) * 2.0)) + (((data["ffti_abs_percentile_10"]) + (data["percentile_70"]))))) + (data["percentile_90"]))))) * 2.0)) 
        v["i183"] = 0.010000*np.tanh(((((((data["percentile_95"]) * 2.0)) + (((((((((data["ffti_percentile_roll_std_20"]) * (((data["fftr_av_change_abs_roll_std_10"]) * 2.0)))) + (data["fftr_min_last_1000"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i184"] = 0.010000*np.tanh(((((data["ffti_abs_percentile_10"]) + (((((((data["ffti_abs_percentile_10"]) + (data["classic_sta_lta7_mean"]))) * 2.0)) + (data["autocorrelation_10000"]))))) * 2.0)) 
        v["i185"] = 0.010000*np.tanh(((((data["autocorrelation_5"]) + (((((data["num_peaks_50"]) + (((((data["autocorrelation_5"]) * (((data["ffti_classic_sta_lta8_mean"]) - (data["fftr_time_rev_asym_stat_5"]))))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i186"] = 0.010000*np.tanh(((((data["autocorrelation_10000"]) + (((((((data["iqr1"]) + (data["autocorrelation_10"]))) * 2.0)) + (((((data["range_-1000_0"]) + (data["ffti_abs_percentile_1"]))) * 2.0)))))) * 2.0)) 
        v["i187"] = 0.010000*np.tanh(((data["percentile_40"]) + (((((((((data["num_peaks_10"]) + (data["percentile_40"]))) + (data["ffti_mean_change_rate"]))) * 2.0)) + (((data["max_first_10000"]) + ((-1.0*((data["fftr_min_roll_std_100"])))))))))) 
        v["i188"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_100"]) - (data["fftr_min_roll_std_100"]))) + (((((data["autocorrelation_10000"]) - (((((data["fftr_ave_roll_mean_10000"]) * (data["min_roll_std_50"]))) - (data["ffti_autocorrelation_1000"]))))) * 2.0)))) 
        v["i189"] = 0.010000*np.tanh((((9.66034984588623047)) * (((((data["max_to_min"]) * (((((data["ffti_autocorrelation_5000"]) * 2.0)) * 2.0)))) + (((data["ffti_percentile_roll_mean_5"]) * 2.0)))))) 
        v["i190"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_mean_80"]) * 2.0)) + (((((((((((data["min_first_10000"]) * 2.0)) + (data["percentile_90"]))) + (data["abs_percentile_75"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i191"] = 0.010000*np.tanh(((((((((((((((((((data["num_peaks_100"]) * (((data["ffti_abs_percentile_1"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i192"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_5"]) + (((data["ffti_abs_percentile_5"]) + (((data["fftr_min_last_1000"]) + (((data["percentile_70"]) + (data["ffti_abs_percentile_5"]))))))))) - (data["std_roll_mean_10000"]))) * 2.0)) 
        v["i193"] = 0.010000*np.tanh(((((((((((data["ffti_av_change_abs_roll_std_100"]) * 2.0)) - (((((data["mean_change_rate_last_50000"]) * 2.0)) - (data["mean_change_rate_last_10000"]))))) - (data["mean_change_rate_last_10000"]))) - (((data["mean_change_rate_last_10000"]) * 2.0)))) * 2.0)) 
        v["i194"] = 0.010000*np.tanh((((((((-1.0*((((data["classic_sta_lta1_mean"]) * ((-1.0*((data["autocorrelation_5"]))))))))) * 2.0)) - (data["fftr_percentile_roll_mean_80"]))) * 2.0)) 
        v["i195"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_5"]) - (((data["fftr_time_rev_asym_stat_50"]) * (((((((((data["ffti_classic_sta_lta6_mean"]) + ((-1.0*((((data["fftr_max_roll_mean_10"]) + (data["min_first_10000"])))))))) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i196"] = 0.010000*np.tanh(((((data["ffti_classic_sta_lta5_mean"]) * (data["kurt"]))) + (((((((data["fftr_abs_percentile_60"]) * (data["fftr_abs_percentile_25"]))) * 2.0)) * 2.0)))) 
        v["i197"] = 0.010000*np.tanh(((((data["ffti_mean_first_50000"]) * (data["ffti_mean_change_abs"]))) - (((data["mean_change_rate_last_10000"]) - (((((data["ffti_percentile_99"]) * 2.0)) + (((((data["ffti_mean_first_10000"]) * (data["ffti_mean_change_abs"]))) * 2.0)))))))) 
        v["i198"] = 0.010000*np.tanh(((((((((((data["fftr_range_2000_3000"]) + (data["mean_first_50000"]))) * 2.0)) * 2.0)) + (data["mean_first_50000"]))) * 2.0)) 
        v["i199"] = 0.010000*np.tanh(((((((((data["ffti_mean_first_50000"]) + (data["fftr_percentile_roll_mean_75"]))) + (((data["ffti_av_change_abs_roll_mean_500"]) + (data["ffti_av_change_abs_roll_std_100"]))))) + (data["num_peaks_50"]))) * 2.0)) 
        v["i200"] = 0.010000*np.tanh(((((((((((data["ffti_autocorrelation_10"]) * 2.0)) - (data["classic_sta_lta1_mean"]))) * 2.0)) * 2.0)) * (data["mean_change_rate_last_1000"]))) 
        v["i201"] = 0.010000*np.tanh(((((((((((data["fftr_hmean"]) * 2.0)) * 2.0)) * (((((data["ffti_mean"]) - (((data["Hilbert_mean"]) + (data["ave10"]))))) - (data["ffti_autocorrelation_50"]))))) * 2.0)) * 2.0)) 
        v["i202"] = 0.010000*np.tanh(((data["ffti_skew"]) * (((((((((data["fftr_percentile_roll_mean_20"]) * 2.0)) * 2.0)) + (data["ffti_percentile_roll_mean_25"]))) + (((data["ffti_max_last_1000"]) + (data["fftr_percentile_roll_mean_20"]))))))) 
        v["i203"] = 0.010000*np.tanh(((data["abs_percentile_25"]) - (((data["fftr_abs_max"]) * (((((((((((((((((data["fftr_min_roll_std_500"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i204"] = 0.010000*np.tanh(((data["fftr_num_peaks_50"]) * (((((((data["fftr_percentile_roll_std_10"]) - (((((((data["abs_percentile_60"]) - (((data["fftr_percentile_roll_std_10"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i205"] = 0.010000*np.tanh((((((data["fftr_percentile_roll_mean_10"]) + (((((((((((data["percentile_90"]) + (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) + (((((data["fftr_percentile_roll_mean_10"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))/2.0)) * 2.0)) 
        v["i206"] = 0.010000*np.tanh(((((((data["fftr_mean_last_50000"]) - (((((((((((data["fftr_mean_last_50000"]) * (((((data["fftr_std_roll_mean_500"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i207"] = 0.010000*np.tanh(((((((data["ffti_max_roll_mean_10"]) + (((((((((data["std_last_1000"]) + (data["percentile_70"]))) * 2.0)) + (data["av_change_rate_roll_std_500"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i208"] = 0.010000*np.tanh(((((((data["percentile_70"]) + (data["autocorrelation_5000"]))) * 2.0)) * 2.0)) 
        v["i209"] = 0.010000*np.tanh(((((data["ffti_abs_percentile_10"]) + (((((((data["ffti_ave10"]) * (data["exp_Moving_average_50000_std"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i210"] = 0.010000*np.tanh(((((((((((((((((data["abs_percentile_25"]) - (data["fftr_min_roll_std_100"]))) * 2.0)) - (data["max_roll_mean_10000"]))) * 2.0)) * 2.0)) - (data["max_roll_mean_10000"]))) * 2.0)) * 2.0)) 
        v["i211"] = 0.010000*np.tanh(((((((((((data["ffti_av_change_abs_roll_std_10"]) * 2.0)) * (data["min_roll_std_10"]))) * 2.0)) + (data["fftr_av_change_abs_roll_std_10000"]))) + (((((data["ffti_av_change_abs_roll_std_10"]) * 2.0)) * (data["min_roll_std_10"]))))) 
        v["i212"] = 0.010000*np.tanh((((((((data["av_change_abs_roll_mean_100"]) + (data["ffti_mean_change_rate"]))/2.0)) * 2.0)) + (((((((data["fftr_min_roll_std_500"]) * (((data["ffti_abs_percentile_10"]) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i213"] = 0.010000*np.tanh(((((data["abs_percentile_80"]) * (((((((((data["kurt"]) + (data["min_first_1000"]))) - (data["fftr_percentile_roll_mean_60"]))) * 2.0)) + (((data["min_first_1000"]) + (data["abs_percentile_80"]))))))) * 2.0)) 
        v["i214"] = 0.010000*np.tanh(((((((((((data["mean_change_rate_last_1000"]) * (((((data["ffti_autocorrelation_10"]) * 2.0)) - (data["num_peaks_50"]))))) * 2.0)) * 2.0)) * 2.0)) + (((((data["ffti_autocorrelation_10"]) * 2.0)) * 2.0)))) 
        v["i215"] = 0.010000*np.tanh(((((((((((data["min_last_10000"]) * (data["ffti_Hann_window_mean_15000"]))) * 2.0)) - (data["ffti_min_roll_std_500"]))) * 2.0)) - (((((data["autocorrelation_5"]) * (data["autocorrelation_10"]))) * 2.0)))) 
        v["i216"] = 0.010000*np.tanh(((((((((((((((((data["min_roll_std_10000"]) - (data["ffti_range_p4000_pinf"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_std_5"]))) * 2.0)) 
        v["i217"] = 0.010000*np.tanh(((((((((((data["fftr_autocorrelation_50"]) + (((((data["classic_sta_lta7_mean"]) - (data["mean_change_rate"]))) * 2.0)))) * 2.0)) * 2.0)) + (data["classic_sta_lta7_mean"]))) * 2.0)) 
        v["i218"] = 0.010000*np.tanh(((data["ffti_skew"]) + (((((((((((((data["classic_sta_lta4_mean"]) + (data["ffti_abs_percentile_10"]))) * (((data["fftr_min_roll_std_500"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i219"] = 0.010000*np.tanh(((((((((data["fftr_av_change_abs_roll_std_100"]) + (((((((data["ffti_abs_percentile_5"]) * 2.0)) + (data["ffti_abs_percentile_5"]))) - (data["mean_change_rate_last_10000"]))))) * (data["fftr_classic_sta_lta4_mean"]))) * 2.0)) * 2.0)) 
        v["i220"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_50"]) * (data["ffti_autocorrelation_50"]))) - (((data["fftr_hmean"]) * 2.0)))) - (data["fftr_min_roll_std_500"]))) 
        v["i221"] = 0.010000*np.tanh(((((((data["kurt"]) + (((((((data["ffti_max_first_10000"]) + (((((data["kurt"]) * (data["ffti_iqr"]))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i222"] = 0.010000*np.tanh(((((((((((data["ffti_abs_percentile_1"]) - (((data["mean_change_rate_first_1000"]) - (data["ffti_mean_first_50000"]))))) * 2.0)) * 2.0)) + (data["mean_change_rate_first_1000"]))) * 2.0)) 
        v["i223"] = 0.010000*np.tanh(((data["percentile_20"]) + (((((((((((data["fftr_percentile_roll_std_5"]) * 2.0)) * 2.0)) - (((((data["percentile_20"]) + (((data["ffti_av_change_abs_roll_std_50"]) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i224"] = 0.010000*np.tanh(((((((((data["classic_sta_lta7_mean"]) - (data["mean_change_rate_first_1000"]))) * 2.0)) + (((((data["percentile_90"]) - (((data["ffti_trend"]) - (data["ffti_mean_change_rate"]))))) * 2.0)))) * 2.0)) 
        v["i225"] = 0.010000*np.tanh(((((((data["ffti_mean"]) + (data["fftr_percentile_roll_mean_70"]))) + (((((data["ffti_av_change_abs_roll_std_10000"]) + (((data["percentile_70"]) + (data["fftr_min_first_1000"]))))) + (data["ffti_max_roll_mean_10"]))))) * 2.0)) 
        v["i226"] = 0.010000*np.tanh(((((((((((((data["ffti_abs_percentile_10"]) - (data["ffti_abs_max_roll_std_100"]))) + ((((-1.0*((((data["fftr_min_roll_std_100"]) - (data["abs_percentile_25"])))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i227"] = 0.010000*np.tanh(((data["fftr_percentile_roll_mean_70"]) + (((((((data["percentile_roll_std_30"]) + (((((((((data["autocorrelation_5"]) * (data["classic_sta_lta1_mean"]))) * 2.0)) * 2.0)) * 2.0)))) - (data["ffti_autocorrelation_10000"]))) * 2.0)))) 
        v["i228"] = 0.010000*np.tanh(((((data["ffti_max_last_1000"]) + (((data["ffti_min_roll_std_50"]) + (((((data["ffti_max_last_1000"]) + (((data["ffti_max_roll_mean_10"]) + (data["fftr_num_peaks_10"]))))) * 2.0)))))) * 2.0)) 
        v["i229"] = 0.010000*np.tanh(((((((data["fftr_autocorrelation_50"]) + (((((((data["ffti_min_roll_std_10"]) * (data["fftr_av_change_abs_roll_mean_50"]))) * 2.0)) + (data["percentile_roll_std_20"]))))) - (data["ffti_min_first_1000"]))) + (data["fftr_autocorrelation_50"]))) 
        v["i230"] = 0.010000*np.tanh((((((((((((data["percentile_70"]) + (data["std_last_10000"]))/2.0)) + (((((data["fftr_min_last_1000"]) + (data["classic_sta_lta2_mean"]))) * 2.0)))) * 2.0)) + (data["classic_sta_lta7_mean"]))) * 2.0)) 
        v["i231"] = 0.010000*np.tanh(((((((((((((data["min_roll_std_500"]) - (data["percentile_roll_std_70"]))) * 2.0)) * 2.0)) * 2.0)) - (data["percentile_roll_std_70"]))) * 2.0)) 
        v["i232"] = 0.010000*np.tanh((((((data["fftr_sum"]) + (data["num_peaks_10"]))/2.0)) - (((((data["fftr_sum"]) * (((((data["abs_percentile_80"]) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i233"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_std_5"]) - (data["ffti_mean_change_rate_last_10000"]))) * 2.0)) - (data["ffti_mean_change_rate_last_10000"]))) * 2.0)) - (data["fftr_min_roll_std_500"]))) * 2.0)) * 2.0)) 
        v["i234"] = 0.010000*np.tanh(((data["abs_percentile_60"]) + (((((data["ffti_hmean"]) + (((data["autocorrelation_10000"]) + (((data["ffti_autocorrelation_1000"]) + (data["ffti_mean_first_50000"]))))))) * 2.0)))) 
        v["i235"] = 0.010000*np.tanh(((((((((((data["autocorrelation_100"]) + (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["autocorrelation_100"]))) + (((((((data["autocorrelation_100"]) * 2.0)) + (data["max_to_min"]))) * 2.0)))) 
        v["i236"] = 0.010000*np.tanh(((((((((((((((((data["ffti_abs_percentile_5"]) - (data["ffti_max_last_1000"]))) + (((data["fftr_percentile_roll_std_5"]) + (data["autocorrelation_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i237"] = 0.010000*np.tanh(((((data["classic_sta_lta4_mean"]) * (((((((data["fftr_min_roll_std_500"]) * 2.0)) + (((((data["std_first_1000"]) * 2.0)) + (data["abs_percentile_70"]))))) + (((data["fftr_min_roll_std_500"]) * 2.0)))))) * 2.0)) 
        v["i238"] = 0.010000*np.tanh(((((data["percentile_90"]) + (((data["percentile_roll_mean_60"]) + (((data["percentile_70"]) * (data["autocorrelation_100"]))))))) * 2.0)) 
        v["i239"] = 0.010000*np.tanh(((((((data["ffti_abs_max_roll_mean_10"]) + (((((data["ffti_abs_max_roll_mean_10"]) + (((((data["autocorrelation_5000"]) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i240"] = 0.010000*np.tanh(((((((data["fftr_min_roll_mean_10000"]) + (((((data["ffti_autocorrelation_5000"]) * (((((((data["ffti_max_roll_std_100"]) - (data["fftr_percentile_roll_std_25"]))) * 2.0)) + (data["max_to_min"]))))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i241"] = 0.010000*np.tanh(((((((((data["fftr_classic_sta_lta5_mean"]) - (data["percentile_70"]))) * ((-1.0*((data["fftr_time_rev_asym_stat_10"])))))) + (((data["ffti_abs_percentile_10"]) * 2.0)))) * 2.0)) 
        v["i242"] = 0.010000*np.tanh(((data["ffti_mean"]) + (((((((((((data["percentile_roll_mean_95"]) * ((-1.0*((data["ffti_percentile_roll_std_20"])))))) * 2.0)) * 2.0)) * 2.0)) + (((data["autocorrelation_10000"]) - (data["mean_change_rate_first_10000"]))))))) 
        v["i243"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_mean_80"]) + (data["ffti_mean"]))) * 2.0)) - (data["mean_change_rate_last_50000"]))) * 2.0)) * 2.0)) 
        v["i244"] = 0.010000*np.tanh(((((data["autocorrelation_10000"]) + (data["fftr_percentile_roll_mean_10"]))) - (((data["ffti_exp_Moving_average_30000_std"]) * (((data["autocorrelation_10000"]) * 2.0)))))) 
        v["i245"] = 0.010000*np.tanh(((data["ffti_max_roll_mean_500"]) + (((((((((((((data["ffti_min_roll_std_1000"]) * (data["fftr_abs_max_roll_mean_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["ffti_max_roll_mean_500"]))))) 
        v["i246"] = 0.010000*np.tanh(((((((data["fftr_exp_Moving_average_50000_std"]) * (((((((data["fftr_time_rev_asym_stat_50"]) * 2.0)) * 2.0)) * 2.0)))) + (((((((data["ffti_autocorrelation_10"]) - (data["ffti_spkt_welch_density_10"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i247"] = 0.010000*np.tanh(((data["percentile_90"]) + (((data["ffti_percentile_roll_std_60"]) + ((((((((-1.0*((((((((data["ffti_av_change_abs_roll_std_50"]) * 2.0)) - (data["percentile_90"]))) * 2.0))))) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i248"] = 0.010000*np.tanh(((((data["fftr_max_to_min"]) - (((data["fftr_max_to_min"]) * (((((((((data["fftr_min_roll_std_500"]) * 2.0)) * 2.0)) + (((data["fftr_min_roll_std_500"]) * 2.0)))) * 2.0)))))) * 2.0)) 
        v["i249"] = 0.010000*np.tanh(((data["ffti_hmean"]) + (((((((((((data["ffti_num_peaks_10"]) * (((data["ffti_min_roll_std_50"]) * 2.0)))) + (data["ffti_max_last_1000"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i250"] = 0.010000*np.tanh(((((((((data["fftr_num_peaks_100"]) * (((data["percentile_40"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i251"] = 0.010000*np.tanh(((((((((data["fftr_abs_percentile_10"]) - (((((data["ffti_min_roll_std_500"]) - (data["ffti_abs_percentile_20"]))) * 2.0)))) * 2.0)) - (((data["fftr_ave_roll_std_500"]) - (data["ffti_av_change_abs_roll_std_100"]))))) - (data["fftr_percentile_40"]))) 
        v["i252"] = 0.010000*np.tanh(((((((data["fftr_time_rev_asym_stat_50"]) * (((((((((((data["fftr_time_rev_asym_stat_50"]) + (data["percentile_20"]))) * 2.0)) * 2.0)) - (data["fftr_min_roll_std_500"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i253"] = 0.010000*np.tanh(((((data["ffti_abs_max_roll_mean_500"]) + ((((((-1.0*((((data["mean_change_rate_last_10000"]) - (data["ffti_mean_change_rate"])))))) * 2.0)) * 2.0)))) + (((data["ffti_mean_change_rate"]) - (data["ffti_percentile_roll_std_1"]))))) 
        v["i254"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_5"]) + (((((((((data["mean_change_rate_last_1000"]) * (((data["fftr_time_rev_asym_stat_5"]) - (data["num_peaks_50"]))))) * 2.0)) * 2.0)) + (data["autocorrelation_10"]))))) * 2.0)) 
        v["i255"] = 0.010000*np.tanh(((((((((((((data["min_first_10000"]) + (data["percentile_90"]))) - (data["ffti_percentile_roll_std_1"]))) * 2.0)) + (((data["min_first_10000"]) * 2.0)))) * 2.0)) + (data["ffti_percentile_roll_std_1"]))) 
        v["i256"] = 0.010000*np.tanh(((((((data["ffti_percentile_80"]) - (((data["fftr_min_roll_std_100"]) - (((((data["ffti_exp_Moving_average_3000_std"]) * (data["kurt"]))) * 2.0)))))) * 2.0)) + (data["ffti_mean_change_rate"]))) 
        v["i257"] = 0.010000*np.tanh(((((((((((((data["fftr_min_roll_std_500"]) * 2.0)) * (((data["fftr_min_roll_std_500"]) + ((-1.0*((data["percentile_20"])))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i258"] = 0.010000*np.tanh(((((((((((((data["percentile_90"]) * 2.0)) * (((((data["ffti_av_change_rate_roll_std_50"]) * (data["autocorrelation_100"]))) + (data["min_roll_std_10000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i259"] = 0.010000*np.tanh(((((((((((data["ffti_skew"]) * (((((data["fftr_percentile_roll_mean_20"]) + (data["av_change_abs_roll_mean_50"]))) + (data["fftr_percentile_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i260"] = 0.010000*np.tanh(((((((((((((((((data["percentile_roll_std_20"]) - (data["percentile_roll_std_70"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["num_peaks_100"]))) + (data["percentile_roll_std_70"]))) 
        v["i261"] = 0.010000*np.tanh(((((((((((((((data["ffti_sum"]) + (data["ffti_abs_percentile_10"]))) - (data["fftr_abs_percentile_1"]))) * 2.0)) * 2.0)) * (data["autocorrelation_10000"]))) * 2.0)) - (data["fftr_abs_percentile_1"]))) 
        v["i262"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_std_10"]) + (((data["fftr_min_roll_std_500"]) * 2.0)))) + (((((data["fftr_av_change_abs_roll_std_10000"]) * 2.0)) * 2.0)))) * 2.0)) * (data["fftr_min_roll_std_500"]))) * 2.0)) * 2.0)) 
        v["i263"] = 0.010000*np.tanh(((((data["autocorrelation_5"]) + (((data["autocorrelation_50"]) + (((data["fftr_min_first_1000"]) + (data["fftr_abs_mean"]))))))) * (((((data["fftr_min_roll_std_100"]) - (data["fftr_autocorrelation_10000"]))) - (data["fftr_autocorrelation_10000"]))))) 
        v["i264"] = 0.010000*np.tanh(((((((((((data["ffti_mean_first_50000"]) + (data["ffti_abs_percentile_10"]))) * 2.0)) + (((((data["ffti_max_roll_std_100"]) * (data["min_first_10000"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i265"] = 0.010000*np.tanh(((((((((((((((((((data["ffti_min_roll_std_1000"]) * (data["mean_last_1000"]))) * 2.0)) + (data["ffti_mean_first_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i266"] = 0.010000*np.tanh(((((((data["fftr_num_peaks_10"]) + (data["ffti_percentile_roll_mean_1"]))) + (((((data["fftr_num_peaks_10"]) + (((data["fftr_percentile_roll_mean_75"]) + (data["percentile_roll_mean_80"]))))) * 2.0)))) * 2.0)) 
        v["i267"] = 0.010000*np.tanh(((data["percentile_roll_std_10"]) - (((((((((data["classic_sta_lta1_mean"]) + (((data["ffti_percentile_50"]) * 2.0)))) * 2.0)) * 2.0)) + (((data["classic_sta_lta1_mean"]) / 2.0)))))) 
        v["i268"] = 0.010000*np.tanh(((((((data["classic_sta_lta4_mean"]) + (data["fftr_percentile_roll_std_5"]))) + (((((((((((data["fftr_percentile_roll_std_5"]) - (data["mean_change_rate_first_1000"]))) * 2.0)) * 2.0)) * (data["classic_sta_lta4_mean"]))) * 2.0)))) * 2.0)) 
        v["i269"] = 0.010000*np.tanh(((((((((((data["abs_percentile_25"]) + (data["min_roll_std_500"]))) + (data["ffti_av_change_abs_roll_std_10000"]))) * 2.0)) * 2.0)) + (((data["fftr_max_last_10000"]) * 2.0)))) 
        v["i270"] = 0.010000*np.tanh(((((((data["classic_sta_lta2_mean"]) - (((data["spkt_welch_density_100"]) * 2.0)))) * 2.0)) + ((((data["percentile_roll_mean_5"]) + ((-1.0*((data["ffti_max_last_1000"])))))/2.0)))) 
        v["i271"] = 0.010000*np.tanh(((data["classic_sta_lta4_mean"]) * (((((((((data["ffti_exp_Moving_average_30000_std"]) + (((data["fftr_percentile_roll_std_5"]) * 2.0)))) + (data["fftr_percentile_roll_std_5"]))) + (data["ffti_abs_max_roll_mean_500"]))) * 2.0)))) 
        v["i272"] = 0.010000*np.tanh(((((((((((((((data["fftr_min_last_1000"]) * 2.0)) - (data["ffti_min_roll_std_500"]))) * 2.0)) * 2.0)) * 2.0)) - (((((data["ffti_c3_500"]) * 2.0)) - (data["ffti_min_roll_std_500"]))))) * 2.0)) 
        v["i273"] = 0.010000*np.tanh(((((((data["percentile_70"]) + (data["fftr_time_rev_asym_stat_50"]))) + (((((((data["fftr_time_rev_asym_stat_50"]) * (data["autocorrelation_10"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i274"] = 0.010000*np.tanh(((((((((((((((((data["fftr_percentile_roll_std_5"]) - (((data["exp_Moving_average_30000_std"]) + (data["mean_change_rate_first_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i275"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_50"]) * (data["min_roll_std_100"]))) * 2.0)) + (((((((data["autocorrelation_10000"]) * (data["ffti_max_to_min"]))) + (((data["autocorrelation_10000"]) * (data["ffti_max_to_min"]))))) * 2.0)))) 
        v["i276"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_100"]) + ((((((((((((-1.0*((data["ffti_min_roll_std_50"])))) * 2.0)) * 2.0)) * 2.0)) * (data["fftr_mean_last_1000"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i277"] = 0.010000*np.tanh(((((data["kurt"]) * (data["abs_percentile_70"]))) * 2.0)) 
        v["i278"] = 0.010000*np.tanh(((((((((((data["percentile_40"]) * ((((((((-1.0*((data["fftr_min_roll_std_500"])))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i279"] = 0.010000*np.tanh((((((((((((-1.0*((((data["fftr_time_rev_asym_stat_50"]) * (((data["fftr_classic_sta_lta6_mean"]) * 2.0))))))) * 2.0)) * 2.0)) + (data["std_last_1000"]))) * 2.0)) * 2.0)) 
        v["i280"] = 0.010000*np.tanh(((((((((((((((data["classic_sta_lta7_mean"]) + (((data["fftr_percentile_roll_std_5"]) + (data["range_-1000_0"]))))) * 2.0)) * 2.0)) + (data["percentile_70"]))) + (data["fftr_percentile_roll_std_5"]))) * 2.0)) * 2.0)) 
        v["i281"] = 0.010000*np.tanh(((((data["max_last_10000"]) + (((((((((data["classic_sta_lta7_mean"]) + (data["classic_sta_lta7_mean"]))) * (data["ffti_autocorrelation_10"]))) * 2.0)) * 2.0)))) + (data["classic_sta_lta7_mean"]))) 
        v["i282"] = 0.010000*np.tanh(((((((data["fftr_num_peaks_10"]) * 2.0)) + (((((data["fftr_av_change_abs_roll_std_10000"]) * (((data["fftr_av_change_abs_roll_std_10000"]) + (((data["fftr_av_change_abs_roll_std_10000"]) - (data["min_roll_std_500"]))))))) * 2.0)))) * 2.0)) 
        v["i283"] = 0.010000*np.tanh(((((((data["percentile_40"]) - (data["mean_change_rate"]))) + (((data["min_roll_std_10"]) * (((data["ffti_av_change_abs_roll_mean_500"]) + (((data["mean_change_rate"]) + (((data["fftr_mean_last_50000"]) * 2.0)))))))))) * 2.0)) 
        v["i284"] = 0.010000*np.tanh(((data["ffti_autocorrelation_500"]) - (((((data["ffti_autocorrelation_10000"]) * (((data["ffti_autocorrelation_10000"]) + (data["fftr_range_-3000_-2000"]))))) * ((((14.82702636718750000)) - (data["fftr_exp_Moving_std_3000_std"]))))))) 
        v["i285"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_60"]) * (((data["num_peaks_50"]) * 2.0)))) + (((((data["ffti_av_change_abs_roll_mean_100"]) * (((((((data["num_peaks_50"]) - (data["ffti_av_change_abs_roll_std_10"]))) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i286"] = 0.010000*np.tanh(((((((data["fftr_mean_first_50000"]) * (((data["ffti_num_peaks_100"]) - (((((data["fftr_std_roll_mean_500"]) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i287"] = 0.010000*np.tanh(((data["fftr_percentile_roll_mean_75"]) - (((((((((((((data["ffti_num_peaks_100"]) * ((((-1.0*((data["fftr_percentile_roll_mean_70"])))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i288"] = 0.010000*np.tanh(((data["classic_sta_lta4_mean"]) * (((((((((((((((data["abs_percentile_70"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i289"] = 0.010000*np.tanh(((((data["kurt"]) * (data["mean_first_50000"]))) + (((((data["ffti_min_roll_std_1000"]) * (((data["mean_first_1000"]) * 2.0)))) * 2.0)))) 
        v["i290"] = 0.010000*np.tanh(((((((data["fftr_range_-3000_-2000"]) + (data["fftr_range_-3000_-2000"]))) + (((data["ffti_kstat_1"]) + (((data["ffti_min_roll_std_50"]) * (data["ffti_max_to_min"]))))))) * 2.0)) 
        v["i291"] = 0.010000*np.tanh(((((data["autocorrelation_100"]) * (((data["percentile_60"]) + (((((((data["fftr_classic_sta_lta6_mean"]) - (((data["ffti_av_change_abs_roll_std_10000"]) * 2.0)))) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i292"] = 0.010000*np.tanh(((((((data["ffti_max_roll_mean_10"]) + (((((((data["ffti_min_roll_std_1000"]) * (((data["Hann_window_mean_50"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i293"] = 0.010000*np.tanh((((-1.0*((((data["ffti_exp_Moving_average_3000_std"]) * (data["autocorrelation_10000"])))))) + (((((((data["ffti_autocorrelation_5000"]) * (data["max_to_min"]))) * 2.0)) * 2.0)))) 
        v["i294"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_1"]) + (data["percentile_40"]))) * 2.0)) * 2.0)) * 2.0)) + (((((((data["percentile_roll_std_1"]) + (data["percentile_40"]))) * 2.0)) * 2.0)))) 
        v["i295"] = 0.010000*np.tanh(((((((((data["ffti_max_last_1000"]) * 2.0)) * 2.0)) + (((data["av_change_abs_roll_std_1000"]) + (data["ffti_hmean"]))))) + (((((data["max_to_min"]) + (((data["max_to_min"]) * 2.0)))) * 2.0)))) 
        v["i296"] = 0.010000*np.tanh(((((((((data["classic_sta_lta3_mean"]) * (((((data["ffti_max_roll_mean_10000"]) * 2.0)) * 2.0)))) * 2.0)) - (((data["kurt"]) * 2.0)))) * 2.0)) 
        v["i297"] = 0.010000*np.tanh((((((-1.0*((((((data["ffti_min_roll_std_10"]) * (((data["med"]) * 2.0)))) * 2.0))))) * 2.0)) * 2.0)) 
        v["i298"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_10"]) + (((((((((data["ffti_abs_percentile_10"]) + (data["ffti_av_change_abs_roll_std_100"]))) + (data["std_last_1000"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i299"] = 0.010000*np.tanh((((((((((((data["ffti_abs_percentile_10"]) * 2.0)) * 2.0)) + (data["fftr_av_change_rate_roll_mean_10"]))/2.0)) * 2.0)) + (((data["ffti_sum"]) - ((((data["percentile_20"]) + (data["percentile_20"]))/2.0)))))) 
        v["i300"] = 0.010000*np.tanh(((((((((data["num_peaks_10"]) * 2.0)) * (data["fftr_min_roll_std_50"]))) + (((((((data["fftr_med"]) * ((-1.0*((data["max_to_min"])))))) * 2.0)) + (data["classic_sta_lta7_mean"]))))) * 2.0)) 
        v["i301"] = 0.010000*np.tanh((((((-1.0*((((data["fftr_min_roll_std_500"]) * (data["percentile_roll_mean_75"])))))) - (data["fftr_min_first_10000"]))) - (((data["ffti_autocorrelation_10000"]) * ((-1.0*(((((10.0)) * (data["ffti_min_roll_std_50"])))))))))) 
        v["i302"] = 0.010000*np.tanh(((data["fftr_autocorrelation_50"]) - (((((data["fftr_time_rev_asym_stat_5"]) * (data["fftr_min_roll_std_10"]))) - (((((((data["percentile_20"]) + (data["fftr_min_roll_std_10"]))) * 2.0)) * (data["autocorrelation_100"]))))))) 
        v["i303"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_1"]) * (data["percentile_70"]))) + (((((((((data["autocorrelation_5000"]) * (((data["fftr_time_rev_asym_stat_1"]) * (data["classic_sta_lta4_mean"]))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i304"] = 0.010000*np.tanh(((((((((((data["spkt_welch_density_100"]) * 2.0)) * 2.0)) * 2.0)) * (data["ffti_autocorrelation_1000"]))) - (data["spkt_welch_density_100"]))) 
        v["i305"] = 0.010000*np.tanh(((data["autocorrelation_10000"]) * (((((((((((((((data["ffti_percentile_50"]) - (data["ffti_exp_Moving_average_3000_std"]))) * 2.0)) * 2.0)) - (data["ffti_av_change_abs_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i306"] = 0.010000*np.tanh(((((data["fftr_max_roll_mean_100"]) + (((((((data["min_roll_std_100"]) * (data["abs_percentile_70"]))) + ((-1.0*((data["ffti_spkt_welch_density_10"])))))) * 2.0)))) * 2.0)) 
        v["i307"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_500"]) + (((((data["mean_change_rate_last_1000"]) * (((data["ffti_autocorrelation_10"]) * 2.0)))) * 2.0)))) * 2.0)) - (((data["spkt_welch_density_100"]) * 2.0)))) 
        v["i308"] = 0.010000*np.tanh(((data["fftr_autocorrelation_10000"]) * ((-1.0*((((data["classic_sta_lta4_mean"]) + (((((((((data["fftr_min_last_1000"]) + (data["autocorrelation_50"]))) + (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0))))))))) 
        v["i309"] = 0.010000*np.tanh(((((((((data["min_last_1000"]) * (data["ffti_mean_change_abs"]))) + (data["ffti_abs_percentile_10"]))) * 2.0)) * 2.0)) 
        v["i310"] = 0.010000*np.tanh(((((((((((((((((data["ffti_max_last_1000"]) - (((data["ffti_std_roll_mean_10000"]) * 2.0)))) + (data["classic_sta_lta2_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i311"] = 0.010000*np.tanh(((((((((((((((data["ffti_av_change_abs_roll_std_10000"]) - (((data["ffti_autocorrelation_10000"]) * 2.0)))) - (((data["mean_change_rate"]) * 2.0)))) * 2.0)) * 2.0)) / 2.0)) * 2.0)) - (data["min_roll_std_10000"]))) 
        v["i312"] = 0.010000*np.tanh(((((((((((data["ffti_ave10"]) * (((data["mean_change_rate_last_1000"]) - (data["kurt"]))))) * 2.0)) * 2.0)) + (data["percentile_90"]))) * 2.0)) 
        v["i313"] = 0.010000*np.tanh(((((data["percentile_20"]) * (((((((data["fftr_time_rev_asym_stat_1"]) - (data["fftr_abs_percentile_20"]))) - (((data["fftr_mean_last_1000"]) + (data["fftr_std_roll_mean_50"]))))) * 2.0)))) * 2.0)) 
        v["i314"] = 0.010000*np.tanh(((((((((data["percentile_80"]) * (((data["autocorrelation_100"]) * 2.0)))) * 2.0)) * 2.0)) + (((data["autocorrelation_100"]) + (((((data["fftr_percentile_roll_mean_80"]) * 2.0)) * 2.0)))))) 
        v["i315"] = 0.010000*np.tanh((((((-1.0*((((data["ffti_percentile_50"]) + (((data["ffti_min_roll_std_10"]) * (((data["percentile_80"]) + (((data["percentile_roll_mean_5"]) + ((-1.0*((data["ffti_autocorrelation_100"]))))))))))))))) * 2.0)) * 2.0)) 
        v["i316"] = 0.010000*np.tanh((-1.0*((((data["fftr_min_roll_std_50"]) * (((((((data["fftr_range_0_1000"]) + (data["fftr_range_0_1000"]))) + (((((data["fftr_percentile_40"]) + (data["trend"]))) * 2.0)))) * 2.0))))))) 
        v["i317"] = 0.010000*np.tanh(((((data["ffti_range_2000_3000"]) - (((((((data["fftr_exp_Moving_average_50000_mean"]) - (((data["kurt"]) - (data["min_roll_std_500"]))))) * 2.0)) * 2.0)))) * (data["ffti_range_2000_3000"]))) 
        v["i318"] = 0.010000*np.tanh(((((((((((((data["ffti_abs_percentile_10"]) * 2.0)) * 2.0)) - (((((data["ffti_min_roll_std_500"]) * 2.0)) * 2.0)))) * 2.0)) - (((data["fftr_percentile_roll_std_50"]) * 2.0)))) * 2.0)) 
        v["i319"] = 0.010000*np.tanh(((((data["max_last_10000"]) + (((data["ffti_abs_percentile_10"]) + (((data["percentile_70"]) + (((data["percentile_70"]) + (data["ffti_av_change_abs_roll_std_10000"]))))))))) * 2.0)) 
        v["i320"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_std_40"]) * (data["ffti_exp_Moving_average_30000_std"]))) * 2.0)) * 2.0)) + (data["ffti_exp_Moving_average_30000_std"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i321"] = 0.010000*np.tanh(((((data["fftr_exp_Moving_average_3000_std"]) - (((((((((data["fftr_exp_Moving_average_3000_std"]) * 2.0)) * 2.0)) * (((((((((data["fftr_min_roll_std_500"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i322"] = 0.010000*np.tanh((((((((((((((((data["ffti_exp_Moving_average_30000_std"]) * (((data["ffti_ave10"]) * 2.0)))) + (data["percentile_95"]))/2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((data["Hann_window_mean_50"]) * 2.0)))) 
        v["i323"] = 0.010000*np.tanh((((((8.0)) * (((data["fftr_autocorrelation_50"]) - (data["ffti_range_-3000_-2000"]))))) + (((data["percentile_70"]) * ((((8.0)) * (data["autocorrelation_100"]))))))) 
        v["i324"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_50"]) + (((((data["ffti_percentile_roll_std_20"]) * (((((data["min_roll_std_1000"]) - (((data["ffti_min_roll_std_50"]) * 2.0)))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i325"] = 0.010000*np.tanh(((((((data["fftr_min_first_1000"]) + (((((data["fftr_abs_percentile_5"]) * (((((data["autocorrelation_5"]) + (data["ffti_min_roll_std_50"]))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i326"] = 0.010000*np.tanh(((((data["ffti_percentile_5"]) - (((data["fftr_abs_percentile_25"]) + (data["percentile_20"]))))) + (((((((((data["ffti_abs_percentile_10"]) * 2.0)) * 2.0)) - (data["fftr_percentile_roll_std_50"]))) - (data["ffti_abs_percentile_25"]))))) 
        v["i327"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_50"]) * (((((((data["ffti_exp_Moving_average_30000_std"]) - (data["fftr_min_roll_std_1000"]))) - (((data["fftr_classic_sta_lta6_mean"]) - (((data["percentile_roll_mean_80"]) - (data["min_roll_std_10000"]))))))) * 2.0)))) * 2.0)) 
        v["i328"] = 0.010000*np.tanh(((((data["ffti_ave10"]) - (((data["ffti_autocorrelation_10"]) * (((((data["fftr_percentile_roll_std_20"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i329"] = 0.010000*np.tanh(((((((((((((((((((((((data["classic_sta_lta4_mean"]) * (data["std_first_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i330"] = 0.010000*np.tanh((((((((-1.0*((((data["mean_change_rate"]) + (data["ffti_classic_sta_lta6_mean"])))))) * 2.0)) * 2.0)) - (((((((data["ffti_exp_Moving_average_3000_std"]) * 2.0)) * 2.0)) + (data["mean_change_rate"]))))) 
        v["i331"] = 0.010000*np.tanh(((((((((((data["fftr_range_-3000_-2000"]) + (data["max_last_10000"]))) + (((data["mean_first_50000"]) + (((data["ffti_percentile_roll_std_30"]) * 2.0)))))) * 2.0)) + (((data["ffti_percentile_roll_std_30"]) * 2.0)))) * 2.0)) 
        v["i332"] = 0.010000*np.tanh(((((data["min_roll_std_100"]) * 2.0)) + (((((((data["min_roll_std_100"]) * 2.0)) + (data["min_roll_std_50"]))) + (((((((data["autocorrelation_5"]) * (data["classic_sta_lta1_mean"]))) * 2.0)) * 2.0)))))) 
        v["i333"] = 0.010000*np.tanh((((((data["classic_sta_lta7_mean"]) + (data["fftr_time_rev_asym_stat_5"]))/2.0)) + (((((((((((data["fftr_mean_last_10000"]) * (data["fftr_time_rev_asym_stat_100"]))) * 2.0)) * 2.0)) * 2.0)) + (data["min_roll_std_10000"]))))) 
        v["i334"] = 0.010000*np.tanh(((data["fftr_num_peaks_10"]) + (((((((data["classic_sta_lta7_mean"]) + (((data["ffti_autocorrelation_10"]) * (((data["fftr_percentile_roll_mean_80"]) + (data["ffti_sum"]))))))) + (data["percentile_roll_std_10"]))) * 2.0)))) 
        v["i335"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_50"]) + (((((((data["ffti_autocorrelation_10000"]) * (data["ffti_min_roll_std_50"]))) * 2.0)) + (((data["ffti_av_change_abs_roll_std_50"]) + (((data["ffti_autocorrelation_10000"]) * (data["ffti_min_roll_std_50"]))))))))) * 2.0)) 
        v["i336"] = 0.010000*np.tanh(((((((data["percentile_roll_std_20"]) + (((((((((((data["percentile_roll_std_20"]) + (data["ffti_range_1000_2000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i337"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_mean_80"]) * 2.0)) / 2.0)) + (((((data["fftr_percentile_roll_mean_80"]) + (data["ffti_mean"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i338"] = 0.010000*np.tanh(((((((data["percentile_40"]) - (((((((data["num_peaks_50"]) * ((-1.0*((((data["fftr_percentile_roll_mean_50"]) + (data["fftr_percentile_roll_std_25"])))))))) * 2.0)) * 2.0)))) + (data["num_peaks_50"]))) * 2.0)) 
        v["i339"] = 0.010000*np.tanh(((((((data["autocorrelation_5"]) * (data["abs_percentile_60"]))) * 2.0)) + (((((((((data["abs_percentile_60"]) - (data["ffti_abs_max_roll_mean_100"]))) * (((data["fftr_num_peaks_20"]) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i340"] = 0.010000*np.tanh(((((data["fftr_mean_last_50000"]) * (((((((((((data["ffti_Hann_window_mean_15000"]) - (data["fftr_abs_percentile_1"]))) - (data["ffti_autocorrelation_500"]))) * 2.0)) * 2.0)) + (data["fftr_mean_last_50000"]))))) * 2.0)) 
        v["i341"] = 0.010000*np.tanh(((((((((((data["ffti_skew"]) + (((data["ffti_min_roll_std_50"]) + (data["ffti_min_roll_std_50"]))))) * (((data["ffti_autocorrelation_10000"]) + (data["fftr_percentile_roll_mean_60"]))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i342"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_mean_10"]) + (data["ffti_max_roll_std_500"]))) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_mean_1"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i343"] = 0.010000*np.tanh(((((((((((((data["ffti_min_roll_std_100"]) * (((data["ffti_percentile_roll_mean_80"]) - (data["ffti_percentile_roll_std_5"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i344"] = 0.010000*np.tanh(((((((((data["av_change_abs_roll_mean_10"]) * (((((((data["fftr_av_change_abs_roll_std_10000"]) + (((data["max_roll_std_10"]) - (data["trend"]))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i345"] = 0.010000*np.tanh(((((((((((((((data["percentile_90"]) - (data["ffti_percentile_roll_mean_60"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i346"] = 0.010000*np.tanh((((((((((((((data["ffti_av_change_abs_roll_std_10000"]) + (data["max_last_10000"]))) + (data["percentile_70"]))) + (((data["fftr_std_roll_std_10"]) + (data["max_last_50000"]))))) + (data["percentile_70"]))/2.0)) * 2.0)) * 2.0)) 
        v["i347"] = 0.010000*np.tanh(((((((data["fftr_classic_sta_lta2_mean"]) + (((((data["percentile_70"]) * 2.0)) + (data["ffti_av_change_abs_roll_std_10000"]))))) + (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) 
        v["i348"] = 0.010000*np.tanh(((((((data["fftr_std_roll_std_50"]) + (((data["Hann_window_mean_1500"]) * (data["exp_Moving_average_50000_std"]))))) + (((((data["ffti_av_change_abs_roll_std_10000"]) * (data["ffti_av_change_abs_roll_mean_10"]))) * 2.0)))) * 2.0)) 
        v["i349"] = 0.010000*np.tanh(((((((((data["autocorrelation_100"]) * 2.0)) + ((((((-1.0*((data["percentile_80"])))) + (data["autocorrelation_100"]))) * 2.0)))) * (((data["percentile_80"]) * 2.0)))) - (data["percentile_80"]))) 
        v["i350"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_30"]) - (data["percentile_roll_std_70"]))) * 2.0)) * 2.0)) * 2.0)) + ((-1.0*((data["percentile_roll_std_30"])))))) * 2.0)) * 2.0)) 
        v["i351"] = 0.010000*np.tanh(((data["fftr_av_change_abs_roll_mean_10"]) * (((((((((((((((((((data["fftr_min_roll_std_500"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i352"] = 0.010000*np.tanh(((((((((((((((((data["fftr_hmean"]) * (data["ffti_mean"]))) * 2.0)) + (data["mean_first_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i353"] = 0.010000*np.tanh(((((((((((data["autocorrelation_10000"]) + (data["min_roll_std_100"]))) + (data["fftr_mean_last_10000"]))) - (((data["autocorrelation_100"]) * (((data["fftr_time_rev_asym_stat_10"]) * 2.0)))))) / 2.0)) + (data["moment_3"]))) 
        v["i354"] = 0.010000*np.tanh(((data["percentile_40"]) + (((data["min_first_1000"]) + (((((data["fftr_percentile_roll_mean_80"]) + (((((((data["min_roll_std_10000"]) + (data["percentile_40"]))) * 2.0)) * 2.0)))) * 2.0)))))) 
        v["i355"] = 0.010000*np.tanh(((((((data["percentile_50"]) * (data["fftr_num_peaks_20"]))) * 2.0)) + (((((data["autocorrelation_5"]) * (data["ffti_classic_sta_lta2_mean"]))) * 2.0)))) 
        v["i356"] = 0.010000*np.tanh(((data["exp_Moving_average_50000_std"]) + (((data["exp_Moving_average_50000_std"]) * (((((((((((((data["ffti_ave10"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i357"] = 0.010000*np.tanh(((((((((((data["autocorrelation_100"]) * (((((data["percentile_70"]) * 2.0)) - (data["mean_change_rate"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i358"] = 0.010000*np.tanh(((((((((((((((data["ffti_percentile_roll_mean_80"]) * (((data["ffti_min_roll_std_100"]) - (((data["fftr_exp_Moving_average_3000_mean"]) * (data["fftr_min_roll_std_50"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i359"] = 0.010000*np.tanh(((data["fftr_classic_sta_lta2_mean"]) * ((-1.0*((((((((data["percentile_20"]) * 2.0)) * 2.0)) + (((((((((data["percentile_20"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))))) 
        v["i360"] = 0.010000*np.tanh(((((data["exp_Moving_average_30000_std"]) * ((((-1.0*((data["autocorrelation_10000"])))) + (data["ffti_ave10"]))))) * 2.0)) 
        v["i361"] = 0.010000*np.tanh(((((((data["mean_first_1000"]) * (((((data["av_change_abs_roll_std_100"]) + (((data["fftr_time_rev_asym_stat_1"]) * 2.0)))) - (data["fftr_range_-3000_-2000"]))))) * 2.0)) * 2.0)) 
        v["i362"] = 0.010000*np.tanh(((((data["ffti_mean_change_rate_last_10000"]) + (data["ffti_percentile_roll_std_40"]))) * (((-2.0) * ((((((10.17423725128173828)) + ((10.17423725128173828)))) + (data["ffti_mean_change_rate_last_10000"]))))))) 
        v["i363"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((((((((data["fftr_abs_percentile_30"]) + (data["classic_sta_lta4_mean"]))) * 2.0)) * (((data["fftr_min_roll_std_500"]) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i364"] = 0.010000*np.tanh(((((((((data["ffti_mean_change_abs"]) * (data["ffti_percentile_roll_mean_50"]))) * 2.0)) + (((((data["fftr_mean_last_1000"]) * (data["ffti_autocorrelation_10000"]))) * 2.0)))) * 2.0)) 
        v["i365"] = 0.010000*np.tanh(((((((((data["av_change_abs_roll_mean_100"]) + (data["fftr_num_peaks_10"]))) * 2.0)) * 2.0)) + (((((((((data["av_change_abs_roll_mean_100"]) + (data["fftr_num_peaks_10"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i366"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + (((data["percentile_70"]) + (((data["ffti_kstat_1"]) + (data["fftr_range_-3000_-2000"]))))))) * 2.0)) + (data["ffti_max_roll_mean_100"]))) * 2.0)) 
        v["i367"] = 0.010000*np.tanh((-1.0*((((((((data["fftr_min_roll_std_500"]) * 2.0)) * 2.0)) * 2.0))))) 
        v["i368"] = 0.010000*np.tanh(((((data["fftr_mean_last_1000"]) + (((data["autocorrelation_100"]) * 2.0)))) * 2.0)) 
        v["i369"] = 0.010000*np.tanh(((((((((data["ffti_av_change_abs_roll_std_10"]) + (((data["ffti_min_roll_std_50"]) * (((data["ffti_min_roll_std_50"]) + (((((data["ffti_autocorrelation_10000"]) * 2.0)) + (data["fftr_percentile_roll_mean_80"]))))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i370"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_100"]) + (((((data["kurt"]) * (((((data["kurt"]) + (data["fftr_percentile_roll_mean_99"]))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i371"] = 0.010000*np.tanh(((data["av_change_abs_roll_mean_50"]) + (((((((data["fftr_time_rev_asym_stat_50"]) * (((data["std_roll_mean_10000"]) - (data["fftr_percentile_roll_std_1"]))))) * 2.0)) * 2.0)))) 
        v["i372"] = 0.010000*np.tanh(((((((data["autocorrelation_10000"]) - (((data["ffti_exp_Moving_average_50000_std"]) - (data["fftr_min_roll_std_10"]))))) * 2.0)) * ((((((data["ffti_exp_Moving_average_50000_std"]) + (data["ffti_exp_Moving_average_50000_std"]))/2.0)) - (((data["ffti_ave10"]) * 2.0)))))) 
        v["i373"] = 0.010000*np.tanh(((((((((data["ffti_mean_last_1000"]) * 2.0)) * 2.0)) * (((((((data["ffti_percentile_roll_std_40"]) * 2.0)) + (data["ffti_percentile_roll_std_40"]))) * 2.0)))) + (data["ffti_mean_last_1000"]))) 
        v["i374"] = 0.010000*np.tanh(((((((((((data["ffti_autocorrelation_5000"]) * (((((((((data["max_to_min"]) + (((data["abs_max_roll_std_500"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i375"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_1"]) * (((((((data["mean_first_1000"]) * 2.0)) + (data["percentile_70"]))) + (((data["percentile_70"]) + (data["mean_first_1000"]))))))) * 2.0)) 
        v["i376"] = 0.010000*np.tanh(((((((((((data["ffti_av_change_abs_roll_std_1000"]) * 2.0)) * 2.0)) - (data["fftr_percentile_roll_mean_70"]))) + (data["autocorrelation_100"]))) * (((((data["fftr_time_rev_asym_stat_1"]) * 2.0)) * 2.0)))) 
        v["i377"] = 0.010000*np.tanh(((((((((data["fftr_mean_last_10000"]) - (((data["ffti_exp_Moving_average_50000_std"]) * (((data["autocorrelation_10000"]) * 2.0)))))) + (((data["ffti_exp_Moving_average_50000_std"]) * (data["ffti_ave10"]))))) * 2.0)) * 2.0)) 
        v["i378"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_mean_75"]) - (data["ffti_percentile_50"]))) * 2.0)) + (data["fftr_percentile_roll_mean_80"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i379"] = 0.010000*np.tanh(((((((((((data["ffti_max_first_1000"]) * (((data["ffti_av_change_abs_roll_std_10"]) - (data["ffti_percentile_roll_mean_50"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i380"] = 0.010000*np.tanh(((((data["fftr_hmean"]) * ((-1.0*((((((data["fftr_exp_Moving_average_50000_mean"]) * 2.0)) * 2.0))))))) - (((data["fftr_num_peaks_50"]) * (((((data["abs_percentile_60"]) * 2.0)) * 2.0)))))) 
        v["i381"] = 0.010000*np.tanh((((((((((((-1.0*((data["fftr_skew"])))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i382"] = 0.010000*np.tanh(((data["percentile_roll_mean_60"]) + (((data["av_change_rate_roll_mean_1000"]) * ((((12.16886711120605469)) * ((((12.16886711120605469)) * (data["fftr_range_2000_3000"]))))))))) 
        v["i383"] = 0.010000*np.tanh(((((((data["ffti_hmean"]) + (((data["ffti_abs_max_roll_mean_10"]) + (((data["ffti_abs_max_roll_mean_10"]) + (data["ffti_av_change_abs_roll_std_10000"]))))))) * 2.0)) * 2.0)) 
        v["i384"] = 0.010000*np.tanh(((data["percentile_20"]) - (((data["fftr_range_-4000_-3000"]) * (((((data["mean_change_rate"]) * ((9.0)))) * 2.0)))))) 
        v["i385"] = 0.010000*np.tanh(((((((((((((data["min_roll_std_1000"]) - (data["percentile_roll_std_80"]))) * 2.0)) * 2.0)) - (((((data["min_roll_std_1000"]) - (data["percentile_roll_std_80"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i386"] = 0.010000*np.tanh((((((-1.0*((((((((((((data["ffti_spkt_welch_density_10"]) - (((data["fftr_min_last_1000"]) - (data["fftr_percentile_50"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) * 2.0)) * 2.0)) 
        v["i387"] = 0.010000*np.tanh(((((data["ffti_abs_percentile_5"]) - (((((((((data["fftr_skew"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (data["fftr_skew"]))) 
        v["i388"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_80"]) + (((((data["percentile_roll_mean_70"]) * 2.0)) + (((data["min_first_10000"]) + (((((data["fftr_percentile_roll_mean_80"]) + (data["min_first_10000"]))) * 2.0)))))))) * 2.0)) 
        v["i389"] = 0.010000*np.tanh(((((data["mean_last_10000"]) * (((data["av_change_abs_roll_mean_50"]) - (data["fftr_range_2000_3000"]))))) - (((((data["fftr_range_2000_3000"]) * 2.0)) * (((data["fftr_std_roll_mean_100"]) + (data["autocorrelation_1000"]))))))) 
        v["i390"] = 0.010000*np.tanh(((((((((data["fftr_min_last_1000"]) - (data["fftr_time_rev_asym_stat_100"]))) + (((data["fftr_min_last_1000"]) + (((data["fftr_min_last_1000"]) - (data["fftr_time_rev_asym_stat_100"]))))))) - (((data["autocorrelation_10"]) / 2.0)))) * 2.0)) 
        v["i391"] = 0.010000*np.tanh(((((((data["fftr_std_roll_mean_10000"]) * (((data["mean_change_rate_first_1000"]) * 2.0)))) - (((((((data["fftr_std_roll_mean_10000"]) * (data["fftr_ave_roll_mean_10000"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i392"] = 0.010000*np.tanh(((((((data["fftr_time_rev_asym_stat_1"]) + (data["range_-1000_0"]))) * 2.0)) * ((((-1.0*((data["fftr_time_rev_asym_stat_10"])))) + (data["ffti_min_roll_std_500"]))))) 
        v["i393"] = 0.010000*np.tanh(((((((data["autocorrelation_5"]) * (data["abs_percentile_60"]))) + (data["ffti_abs_percentile_5"]))) + (((data["ffti_Hann_window_mean_15000"]) * (((data["min_roll_std_100"]) - (((data["fftr_time_rev_asym_stat_5"]) * 2.0)))))))) 
        v["i394"] = 0.010000*np.tanh(((((data["abs_percentile_60"]) * ((((((-1.0*((data["fftr_num_peaks_50"])))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i395"] = 0.010000*np.tanh(((data["fftr_hmean"]) * (((data["ffti_percentile_roll_std_30"]) - (((data["classic_sta_lta4_mean"]) + (((((data["classic_sta_lta4_mean"]) + (data["min_roll_std_100"]))) + (data["fftr_kstat_1"]))))))))) 
        v["i396"] = 0.010000*np.tanh((((13.05328464508056641)) * ((((13.05328464508056641)) * (((((data["ffti_num_peaks_20"]) * (data["classic_sta_lta7_mean"]))) - (((3.0) * (((data["ffti_num_peaks_10"]) * (data["classic_sta_lta7_mean"]))))))))))) 
        v["i397"] = 0.010000*np.tanh(((((((((((data["min_roll_std_10"]) * (((data["fftr_num_peaks_10"]) + (data["ffti_av_change_abs_roll_std_10"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i398"] = 0.010000*np.tanh(((data["ffti_min_roll_mean_100"]) * (((((((data["ffti_av_change_abs_roll_std_50"]) + (((((data["fftr_percentile_roll_mean_95"]) + (data["ffti_av_change_abs_roll_std_10000"]))) * 2.0)))) + (((data["percentile_70"]) * 2.0)))) * 2.0)))) 
        v["i399"] = 0.010000*np.tanh(((data["ffti_mean_change_abs"]) * (((((((data["mean_change_rate_last_10000"]) + (data["ffti_av_change_abs_roll_std_100"]))) - (((((data["fftr_abs_percentile_1"]) * 2.0)) + ((((-1.0*((data["min_roll_std_1000"])))) * 2.0)))))) * 2.0)))) 
        v["i400"] = 0.010000*np.tanh(((((data["av_change_abs_roll_mean_50"]) + (((data["fftr_num_peaks_100"]) * (((data["ffti_max_roll_mean_1000"]) + (((((data["ffti_mean_first_50000"]) + (data["ffti_mean_change_rate"]))) * 2.0)))))))) * 2.0)) 
        v["i401"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_5"]) * (((((data["num_peaks_100"]) - (((data["fftr_time_rev_asym_stat_10"]) * 2.0)))) + (((data["mean_first_50000"]) - (((data["fftr_time_rev_asym_stat_5"]) + (data["fftr_abs_percentile_5"]))))))))) 
        v["i402"] = 0.010000*np.tanh((((-1.0*((((data["ffti_min_roll_std_10"]) * (((data["exp_Moving_average_30000_mean"]) + (((data["ffti_min_roll_std_10"]) + (((data["mean_last_50000"]) + (((data["abs_percentile_60"]) + (data["ffti_max_roll_mean_100"])))))))))))))) * 2.0)) 
        v["i403"] = 0.010000*np.tanh(((((((((data["kurt"]) * (((data["fftr_classic_sta_lta6_mean"]) + (((data["spkt_welch_density_50"]) + (((data["fftr_max_roll_mean_10000"]) + (data["ffti_classic_sta_lta5_mean"]))))))))) + (data["min_roll_std_10000"]))) * 2.0)) * 2.0)) 
        v["i404"] = 0.010000*np.tanh(((((data["ffti_ave10"]) * 2.0)) + (((((((data["av_change_abs_roll_std_1000"]) + (((data["min_roll_std_10000"]) + (((data["fftr_min_roll_mean_10"]) * 2.0)))))) * 2.0)) * 2.0)))) 
        v["i405"] = 0.010000*np.tanh(((((((((data["fftr_range_-3000_-2000"]) + ((-1.0*((((((data["fftr_time_rev_asym_stat_10"]) + (data["fftr_time_rev_asym_stat_10"]))) * (data["fftr_time_rev_asym_stat_5"])))))))) + (data["std_last_10000"]))) * 2.0)) * 2.0)) 
        v["i406"] = 0.010000*np.tanh(((data["ffti_max_roll_mean_10"]) + (((((data["med"]) - (((((((data["mean_change_rate_first_1000"]) * 2.0)) * (data["med"]))) * (data["mean_change_rate_first_1000"]))))) * 2.0)))) 
        v["i407"] = 0.010000*np.tanh(((data["fftr_min_roll_std_500"]) * (((((((data["ffti_min_roll_std_500"]) * 2.0)) + (((((data["classic_sta_lta5_mean"]) * 2.0)) * 2.0)))) + (data["ffti_min_roll_std_500"]))))) 
        v["i408"] = 0.010000*np.tanh(((((((data["fftr_mean_last_1000"]) - (((data["fftr_autocorrelation_5000"]) * (((((((data["fftr_mean_last_1000"]) + (((((data["ffti_mean"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i409"] = 0.010000*np.tanh((((((data["ffti_percentile_roll_std_1"]) + (((((data["ffti_Hann_window_mean_15000"]) + (((data["fftr_percentile_roll_mean_80"]) + (((((data["autocorrelation_5000"]) - (data["ffti_min_roll_mean_10"]))) * 2.0)))))) * 2.0)))/2.0)) * 2.0)) 
        v["i410"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_75"]) + (((((((((((((data["fftr_percentile_roll_mean_75"]) * (((data["ffti_num_peaks_100"]) - (data["abs_percentile_70"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i411"] = 0.010000*np.tanh(((((((((((data["classic_sta_lta2_mean"]) * (((((data["ffti_autocorrelation_5000"]) + ((((data["fftr_classic_sta_lta6_mean"]) + (data["fftr_av_change_abs_roll_std_10000"]))/2.0)))) + (data["fftr_av_change_abs_roll_std_10000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i412"] = 0.010000*np.tanh(((data["fftr_min_roll_std_500"]) * (((((((((data["fftr_av_change_abs_roll_std_10000"]) - (data["ffti_percentile_roll_mean_60"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i413"] = 0.010000*np.tanh(((((data["min_first_1000"]) + (((((data["abs_percentile_80"]) + (((((data["ffti_min_first_1000"]) * (data["autocorrelation_10000"]))) + (data["ffti_abs_percentile_5"]))))) * 2.0)))) * 2.0)) 
        v["i414"] = 0.010000*np.tanh(((((data["ffti_num_peaks_100"]) * (((((data["av_change_rate_roll_std_10"]) + (((((data["fftr_percentile_50"]) * 2.0)) * 2.0)))) + (((((data["ffti_num_peaks_100"]) + (data["skew"]))) * 2.0)))))) * 2.0)) 
        v["i415"] = 0.010000*np.tanh(((((((((data["num_peaks_50"]) * (((((((data["ffti_av_change_abs_roll_mean_500"]) + (data["fftr_ave_roll_mean_500"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) + (data["num_peaks_50"]))) 
        v["i416"] = 0.010000*np.tanh((((10.0)) * (((((((((data["mean_last_1000"]) * ((((-1.0*((data["fftr_range_2000_3000"])))) - (data["ffti_mean_last_10000"]))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i417"] = 0.010000*np.tanh(((data["fftr_abs_max_roll_mean_10000"]) + (((data["max_roll_std_100"]) * (((((((((((data["classic_sta_lta3_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((data["classic_sta_lta3_mean"]) * 2.0)))))))) 
        v["i418"] = 0.010000*np.tanh(((((((((((((data["classic_sta_lta7_mean"]) * (data["ffti_autocorrelation_10"]))) * 2.0)) - (data["fftr_percentile_roll_std_40"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i419"] = 0.010000*np.tanh(((((((data["autocorrelation_100"]) * (data["percentile_70"]))) * 2.0)) + (((((((data["ffti_min_roll_std_50"]) * (data["ffti_autocorrelation_10000"]))) * 2.0)) + (((data["min_roll_std_1000"]) * (data["ffti_autocorrelation_1000"]))))))) 
        v["i420"] = 0.010000*np.tanh(((data["ffti_num_peaks_100"]) * (((((data["fftr_percentile_roll_mean_75"]) + (((data["spkt_welch_density_100"]) - ((-1.0*((((data["fftr_percentile_roll_mean_75"]) + (((data["spkt_welch_density_100"]) - (data["fftr_abs_max_roll_mean_10000"])))))))))))) * 2.0)))) 
        v["i421"] = 0.010000*np.tanh(((((data["ffti_percentile_60"]) + (((((((data["fftr_mean_first_10000"]) - (((data["fftr_min_roll_std_100"]) * (((data["ffti_mean_change_rate"]) + (data["fftr_moment_3"]))))))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i422"] = 0.010000*np.tanh(((data["fftr_std_roll_mean_50"]) + (((((((((data["ffti_percentile_roll_mean_50"]) * (data["ffti_mean_change_abs"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i423"] = 0.010000*np.tanh(((((((((((((((((data["av_change_abs_roll_std_10000"]) * 2.0)) * 2.0)) + (data["percentile_roll_mean_70"]))) + (data["ffti_mean_change_rate_first_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i424"] = 0.010000*np.tanh(((((((data["abs_percentile_50"]) * (((data["abs_percentile_50"]) + (((data["fftr_time_rev_asym_stat_5"]) + (((data["autocorrelation_5"]) * 2.0)))))))) * 2.0)) - (data["ffti_min_roll_std_10"]))) 
        v["i425"] = 0.010000*np.tanh((((((((-1.0*((((((((data["std_last_1000"]) + (data["ffti_max_last_1000"]))) * 2.0)) * (((((data["ffti_max_last_1000"]) * 2.0)) * 2.0))))))) + (data["fftr_num_peaks_10"]))) * 2.0)) * 2.0)) 
        v["i426"] = 0.010000*np.tanh(((((((((data["abs_percentile_50"]) * (data["autocorrelation_5"]))) + (((data["num_peaks_50"]) * (data["autocorrelation_5"]))))) * 2.0)) + (((((data["num_peaks_50"]) * 2.0)) * (data["autocorrelation_5"]))))) 
        v["i427"] = 0.010000*np.tanh(((data["ffti_abs_max_roll_mean_10"]) + (((((data["fftr_time_rev_asym_stat_50"]) * (((((((((data["autocorrelation_10"]) - (data["ffti_kstat_1"]))) * 2.0)) * 2.0)) * 2.0)))) + (data["ffti_kstat_1"]))))) 
        v["i428"] = 0.010000*np.tanh(((((((((data["percentile_roll_mean_75"]) + (data["ffti_max_last_1000"]))) + (data["percentile_roll_mean_75"]))) + (((((data["ffti_abs_percentile_10"]) - (data["fftr_ave_roll_mean_100"]))) + (data["percentile_roll_mean_75"]))))) * 2.0)) 
        v["i429"] = 0.010000*np.tanh(((((data["ffti_autocorrelation_50"]) + (((data["fftr_time_rev_asym_stat_1"]) * (((data["percentile_roll_mean_25"]) + (((data["ffti_range_-3000_-2000"]) + (data["fftr_abs_max_roll_mean_1000"]))))))))) * 2.0)) 
        v["i430"] = 0.010000*np.tanh((((-1.0*((((((((((data["percentile_90"]) - (data["fftr_mean_first_10000"]))) + (data["fftr_abs_percentile_10"]))) * (data["ffti_min_roll_std_10"]))) * 2.0))))) * 2.0)) 
        v["i431"] = 0.010000*np.tanh(((((((((((data["ffti_abs_max_roll_mean_50"]) * (((data["ffti_percentile_roll_mean_30"]) * 2.0)))) * 2.0)) * 2.0)) - (data["fftr_min_roll_std_500"]))) * 2.0)) 
        v["i432"] = 0.010000*np.tanh(((((((((data["autocorrelation_100"]) * (((((data["percentile_70"]) - (data["fftr_range_-3000_-2000"]))) * 2.0)))) * 2.0)) + (data["ffti_max_roll_mean_100"]))) * 2.0)) 
        v["i433"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_5"]) + (((((((((((((((data["ffti_av_change_abs_roll_mean_10"]) * (data["min_roll_std_1000"]))) * 2.0)) - (data["ffti_spkt_welch_density_10"]))) + (data["ffti_num_peaks_50"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i434"] = 0.010000*np.tanh(((((data["percentile_40"]) - (((((data["ffti_min_roll_std_10"]) * (((((data["percentile_roll_mean_90"]) * 2.0)) + (((((data["percentile_roll_mean_90"]) * 2.0)) * 2.0)))))) * 2.0)))) * 2.0)) 
        v["i435"] = 0.010000*np.tanh(((((((data["fftr_min_roll_std_10"]) * ((((10.0)) - (data["ffti_min_roll_std_10"]))))) * (data["ffti_min_roll_std_10"]))) * 2.0)) 
        v["i436"] = 0.010000*np.tanh(((data["ffti_percentile_25"]) * (((((data["min_roll_std_500"]) - (data["fftr_percentile_70"]))) * ((((12.01259803771972656)) - (((((data["ffti_percentile_25"]) - (((data["max"]) * 2.0)))) * 2.0)))))))) 
        v["i437"] = 0.010000*np.tanh((((-1.0*((((data["min_last_1000"]) + (((((((((data["min_last_1000"]) + (((((data["av_change_abs_roll_std_50"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0))))))) * 2.0)) 
        v["i438"] = 0.010000*np.tanh(((((((((data["autocorrelation_100"]) + (((((((data["classic_sta_lta1_mean"]) * 2.0)) + (data["autocorrelation_100"]))) * 2.0)))) * (data["percentile_80"]))) * 2.0)) * 2.0)) 
        v["i439"] = 0.010000*np.tanh(((((((((((data["fftr_autocorrelation_100"]) + (((((data["fftr_percentile_roll_std_5"]) + (((data["autocorrelation_100"]) + (data["fftr_autocorrelation_100"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i440"] = 0.010000*np.tanh(((((((((data["percentile_70"]) + (((data["ffti_max_roll_mean_10"]) + (((data["percentile_70"]) + (((data["fftr_range_-3000_-2000"]) + (data["ffti_av_change_abs_roll_std_10000"]))))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i441"] = 0.010000*np.tanh(((((((data["fftr_percentile_40"]) * ((((((((((((((-1.0*((data["max_to_min"])))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) + (data["fftr_percentile_40"]))) * 2.0)) 
        v["i442"] = 0.010000*np.tanh(((data["ffti_av_change_abs_roll_std_10"]) - (((((((data["mean_first_1000"]) * (((data["ffti_autocorrelation_500"]) - (data["fftr_autocorrelation_10"]))))) * 2.0)) * 2.0)))) 
        v["i443"] = 0.010000*np.tanh(((data["ffti_min_roll_std_50"]) * (((((((((data["ffti_min_roll_std_50"]) - (data["ffti_min_roll_std_50"]))) * 2.0)) - (((((data["ffti_autocorrelation_10"]) + (data["ffti_percentile_roll_std_20"]))) + (data["ffti_autocorrelation_10"]))))) * 2.0)))) 
        v["i444"] = 0.010000*np.tanh(((((data["fftr_autocorrelation_10000"]) * 2.0)) * (((((((data["std_roll_mean_10000"]) + (data["fftr_percentile_roll_std_60"]))) * 2.0)) + (((((data["percentile_roll_mean_70"]) - (data["mean_change_rate_first_1000"]))) + (data["fftr_percentile_roll_std_60"]))))))) 
        v["i445"] = 0.010000*np.tanh((((((((((-1.0*((((data["fftr_hmean"]) - (data["min_roll_std_1000"])))))) * 2.0)) * (((((((data["av_change_abs_roll_std_10"]) * 2.0)) + (data["fftr_range_-3000_-2000"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i446"] = 0.010000*np.tanh(((((((((((data["ffti_autocorrelation_5000"]) * (((data["max_to_min"]) * 2.0)))) - (data["ffti_mean_first_10000"]))) + (((data["max_to_min"]) * (data["ffti_autocorrelation_5000"]))))) * 2.0)) - (data["ffti_autocorrelation_5"]))) 
        v["i447"] = 0.009996*np.tanh(((data["fftr_percentile_roll_std_20"]) * (((((((((((((data["classic_sta_lta5_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((((data["fftr_autocorrelation_1000"]) - (data["percentile_roll_mean_25"]))) * 2.0)))))) 
        v["i448"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_10"]) * (((((((data["percentile_30"]) - (data["autocorrelation_5"]))) - (data["autocorrelation_5"]))) - (data["autocorrelation_5"]))))) + (data["fftr_time_rev_asym_stat_10"]))) 
        v["i449"] = 0.010000*np.tanh(((((data["ffti_max_roll_std_100"]) + (((((((data["ffti_min_roll_std_50"]) * (data["max_to_min"]))) * 2.0)) * 2.0)))) + (((data["ffti_min_roll_std_50"]) * (data["max_to_min"]))))) 
        v["i450"] = 0.010000*np.tanh((((((((((((data["min_roll_std_50"]) + (data["autocorrelation_5"]))/2.0)) * 2.0)) * 2.0)) * (data["abs_percentile_50"]))) * 2.0)) 
        v["i451"] = 0.010000*np.tanh(((((((((((data["fftr_range_-3000_-2000"]) * (((data["fftr_range_-3000_-2000"]) * 2.0)))) + (data["ffti_av_change_abs_roll_std_100"]))) + (((data["fftr_time_rev_asym_stat_10"]) * (((data["ffti_mean_change_abs"]) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i452"] = 0.010000*np.tanh(((((((((data["percentile_40"]) + (((data["percentile_roll_std_5"]) + (((((((data["ffti_abs_max_roll_mean_100"]) * (data["fftr_hmean"]))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i453"] = 0.010000*np.tanh(((((((((((((((data["min_roll_std_10000"]) - (data["fftr_percentile_roll_std_50"]))) - (data["fftr_percentile_roll_std_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i454"] = 0.010000*np.tanh(((data["min_roll_std_10000"]) + (((data["fftr_max_first_10000"]) + (((((data["autocorrelation_10000"]) + (((data["min_last_10000"]) * (data["ffti_Hann_window_mean_15000"]))))) * 2.0)))))) 
        v["i455"] = 0.010000*np.tanh(((((((data["fftr_abs_percentile_1"]) + (((((data["ffti_ave10"]) * 2.0)) - (data["mean_change_rate"]))))) + (data["max_to_min_diff"]))) * 2.0)) 
        v["i456"] = 0.010000*np.tanh(((((data["ffti_autocorrelation_10"]) * (((data["mean_first_1000"]) - (((((data["ffti_min_roll_std_50"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i457"] = 0.010000*np.tanh(((((data["ffti_exp_Moving_average_30000_std"]) * (data["ffti_av_change_abs_roll_mean_100"]))) + (((((((((data["ffti_exp_Moving_average_30000_std"]) * (data["ffti_av_change_abs_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i458"] = 0.010000*np.tanh(((((data["percentile_roll_mean_75"]) - (((data["fftr_range_-3000_-2000"]) * (((data["mean_last_1000"]) * 2.0)))))) * 2.0)) 
        v["i459"] = 0.010000*np.tanh(((((data["exp_Moving_average_3000_std"]) + (((((((data["exp_Moving_average_3000_std"]) * 2.0)) * 2.0)) * 2.0)))) * (((((data["fftr_time_rev_asym_stat_50"]) + (data["classic_sta_lta4_mean"]))) * 2.0)))) 
        v["i460"] = 0.010000*np.tanh(((((data["classic_sta_lta4_mean"]) * 2.0)) + (((((((data["classic_sta_lta4_mean"]) * (((data["classic_sta_lta4_mean"]) + (((((((data["ffti_av_change_abs_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)))) 
        v["i461"] = 0.010000*np.tanh((((((-1.0*((((data["percentile_20"]) - (((data["ffti_av_change_abs_roll_std_50"]) * (((data["percentile_20"]) - (((data["fftr_classic_sta_lta5_mean"]) * 2.0))))))))))) * 2.0)) * 2.0)) 
        v["i462"] = 0.010000*np.tanh(((((((data["fftr_abs_percentile_10"]) + (((data["min_first_50000"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i463"] = 0.010000*np.tanh(((((data["fftr_mean_last_1000"]) + (((((data["classic_sta_lta7_mean"]) + (data["fftr_min_last_1000"]))) + (((((((data["fftr_mean_last_1000"]) * 2.0)) - (data["autocorrelation_500"]))) * 2.0)))))) * 2.0)) 
        v["i464"] = 0.010000*np.tanh(((((((((((data["min_roll_std_10"]) - (((((data["fftr_exp_Moving_average_50000_mean"]) - (data["mean_change_rate_last_1000"]))) * 2.0)))) * 2.0)) * 2.0)) + (data["fftr_range_-3000_-2000"]))) * 2.0)) 
        v["i465"] = 0.010000*np.tanh(((((((((((data["ffti_percentile_roll_std_30"]) * 2.0)) * 2.0)) * (data["fftr_abs_percentile_1"]))) * 2.0)) + (data["fftr_autocorrelation_100"]))) 
        v["i466"] = 0.009996*np.tanh(((((data["ffti_hmean"]) + (((data["min_roll_std_10000"]) * (data["ffti_num_peaks_20"]))))) + (data["fftr_range_-3000_-2000"]))) 
        v["i467"] = 0.010000*np.tanh(((((((((((data["min_roll_std_10"]) / 2.0)) * 2.0)) + (((((((((data["mean_change_rate_last_1000"]) * (data["ffti_autocorrelation_10"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i468"] = 0.010000*np.tanh(((((((((data["percentile_70"]) * (data["autocorrelation_100"]))) - (data["ffti_min_roll_std_10"]))) * 2.0)) + ((((data["abs_percentile_60"]) + (data["ffti_min_roll_std_10"]))/2.0)))) 
        v["i469"] = 0.010000*np.tanh(((((((((data["ffti_abs_max_roll_std_100"]) - (data["ffti_percentile_roll_std_40"]))) * 2.0)) + (((((data["av_change_abs_roll_mean_10000"]) * (((((data["ffti_abs_max_roll_std_100"]) - (data["ffti_percentile_roll_std_40"]))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i470"] = 0.010000*np.tanh(((((((data["min_first_10000"]) + (((((((data["autocorrelation_10000"]) * (data["ffti_min_first_1000"]))) * 2.0)) + (((data["fftr_percentile_roll_mean_80"]) + (((data["ffti_kstat_1"]) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i471"] = 0.010000*np.tanh(((((((((data["fftr_time_rev_asym_stat_1"]) * (((((data["ffti_percentile_roll_std_40"]) * 2.0)) - (data["min_roll_std_50"]))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i472"] = 0.010000*np.tanh(((((((((data["fftr_Hann_window_mean_1500"]) * (((data["min_roll_std_100"]) - (data["min_roll_std_50"]))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i473"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_mean_80"]) * 2.0)) * 2.0)) - ((-1.0*((((data["autocorrelation_50"]) * 2.0))))))) 
        v["i474"] = 0.010000*np.tanh((((((14.20382785797119141)) * (((data["percentile_90"]) - (data["ffti_av_change_abs_roll_std_50"]))))) - ((((data["ffti_av_change_abs_roll_std_50"]) + (((data["ffti_av_change_abs_roll_std_50"]) - (data["ffti_av_change_abs_roll_std_50"]))))/2.0)))) 
        v["i475"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_500"]) + (((data["fftr_autocorrelation_5000"]) + (data["fftr_abs_max_roll_mean_10000"]))))) * 2.0)) * 2.0)) 
        v["i476"] = 0.010000*np.tanh(((((((((((((((data["ffti_percentile_25"]) * (((data["min_roll_std_500"]) - (data["fftr_range_minf_m4000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["ffti_percentile_25"]))) * 2.0)) 
        v["i477"] = 0.009996*np.tanh(((((((((((data["ffti_av_change_abs_roll_std_10"]) * (((data["min_roll_std_10"]) + (data["min_roll_std_10"]))))) * 2.0)) - (data["mean_change_rate"]))) * 2.0)) - (((data["mean_first_1000"]) * (data["fftr_Hann_window_mean_15000"]))))) 
        v["i478"] = 0.010000*np.tanh(((((((((((((data["ffti_iqr"]) + (((data["kurt"]) - (((data["percentile_roll_mean_5"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * (((data["ffti_iqr"]) * 2.0)))) * 2.0)) 
        v["i479"] = 0.010000*np.tanh((((-1.0*((data["fftr_abs_percentile_1"])))) - (((((((((data["fftr_abs_percentile_1"]) - ((-1.0*((data["mean_change_rate_last_10000"])))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i480"] = 0.010000*np.tanh(((((data["min_roll_std_10"]) * (((((((data["ffti_av_change_abs_roll_std_10"]) + (((data["fftr_mean_first_50000"]) + (data["ffti_av_change_abs_roll_mean_1000"]))))) + (((data["fftr_mean_first_50000"]) + (data["mean_change_rate_last_10000"]))))) * 2.0)))) * 2.0)) 
        v["i481"] = 0.010000*np.tanh(((data["ffti_ave10"]) * (((data["fftr_min_first_1000"]) + (((((((data["exp_Moving_average_50000_std"]) + (data["exp_Moving_average_50000_std"]))) - (data["spkt_welch_density_100"]))) + (((data["fftr_min_first_1000"]) - (data["exp_Moving_average_50000_std"]))))))))) 
        v["i482"] = 0.010000*np.tanh(((data["min_first_10000"]) + (((((((((data["autocorrelation_10000"]) * (((data["fftr_min_roll_std_100"]) + (data["fftr_autocorrelation_10000"]))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i483"] = 0.010000*np.tanh(((((((((((((((((((((data["exp_Moving_average_50000_std"]) * 2.0)) * (data["ffti_ave10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["ffti_ave10"]))) * 2.0)) * 2.0)) 
        v["i484"] = 0.009996*np.tanh(((((data["percentile_40"]) + (((((((((data["ffti_num_peaks_100"]) - (data["fftr_range_-3000_-2000"]))) * (data["fftr_med"]))) * 2.0)) * 2.0)))) + (data["fftr_max_roll_std_10"]))) 
        v["i485"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_5"]) + (data["classic_sta_lta7_mean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i486"] = 0.010000*np.tanh(((((data["ffti_autocorrelation_10"]) * (((((data["ffti_av_change_abs_roll_std_10"]) - (((data["fftr_range_-3000_-2000"]) + (data["ffti_hmean"]))))) - (((data["fftr_percentile_roll_std_40"]) * (data["ffti_autocorrelation_10"]))))))) * ((10.31002140045166016)))) 
        v["i487"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_std_5"]) * 2.0)) * 2.0)) - (data["fftr_mean_last_50000"]))) * 2.0)) * 2.0)) + (((((data["range_-1000_0"]) * 2.0)) * 2.0)))) 
        v["i488"] = 0.010000*np.tanh(((((data["fftr_percentile_60"]) - (((((data["fftr_min_roll_std_100"]) + (((((((data["ffti_std_roll_mean_1000"]) + (data["fftr_min_roll_std_100"]))) + (((data["ffti_std_roll_mean_1000"]) * 2.0)))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i489"] = 0.010000*np.tanh(((((((((((((((((data["min_roll_std_100"]) * (data["fftr_classic_sta_lta5_mean"]))) - (data["ffti_percentile_80"]))) - (((data["av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i490"] = 0.010000*np.tanh(((((data["fftr_min_last_1000"]) - (data["fftr_percentile_roll_std_30"]))) + (((data["ffti_max_roll_std_1000"]) + (((((((((data["av_change_abs_roll_mean_100"]) * (data["ffti_mean_change_abs"]))) * 2.0)) + (data["ffti_mean_first_50000"]))) * 2.0)))))) 
        v["i491"] = 0.010000*np.tanh((((-1.0*((data["fftr_mean_last_50000"])))) - (((((((((((((data["fftr_mean_last_50000"]) * 2.0)) * 2.0)) * 2.0)) * (data["fftr_std_roll_mean_10000"]))) * 2.0)) * 2.0)))) 
        v["i492"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_50"]) * (data["ffti_num_peaks_10"]))) * 2.0)) + (((((data["fftr_num_peaks_10"]) + (((((data["ffti_min_roll_std_50"]) * (data["ffti_num_peaks_10"]))) * 2.0)))) * 2.0)))) 
        v["i493"] = 0.010000*np.tanh(((((((((((((((((((data["abs_percentile_50"]) * (data["ffti_ave_roll_mean_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["autocorrelation_50"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i494"] = 0.010000*np.tanh(((((((((((data["ffti_trend"]) * (data["trend"]))) * 2.0)) - (((data["ffti_percentile_50"]) - (((data["av_change_abs_roll_std_1000"]) + (data["av_change_abs_roll_std_1000"]))))))) * 2.0)) * 2.0)) 
        v["i495"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_std_25"]) * (((((data["fftr_mean_change_abs"]) - (((data["fftr_abs_max_roll_mean_500"]) - (((((((data["fftr_mean_change_abs"]) * 2.0)) * 2.0)) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i496"] = 0.010000*np.tanh(((((data["ffti_skew"]) - ((((8.85958862304687500)) * ((-1.0*((data["fftr_percentile_roll_mean_20"])))))))) - (((data["ffti_skew"]) * ((((8.85958862304687500)) * ((-1.0*((data["fftr_percentile_roll_mean_20"])))))))))) 
        v["i497"] = 0.010000*np.tanh(((((data["fftr_mean_first_1000"]) * (((((data["ffti_ave_roll_mean_100"]) + (((data["autocorrelation_5"]) + (data["ffti_autocorrelation_100"]))))) + (((data["fftr_std_roll_mean_10000"]) + (((data["autocorrelation_5"]) * 2.0)))))))) * 2.0)) 
        v["i498"] = 0.010000*np.tanh(((((((((((data["classic_sta_lta4_mean"]) * (((data["ffti_min_roll_std_10"]) + (((data["fftr_time_rev_asym_stat_50"]) + (data["abs_percentile_60"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i499"] = 0.010000*np.tanh(((((((((((((((((data["autocorrelation_1000"]) * (((data["ffti_abs_percentile_10"]) - ((-1.0*((data["min_roll_std_50"])))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i500"] = 0.010000*np.tanh(((((((((((((((((((data["fftr_time_rev_asym_stat_10"]) * (data["percentile_70"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i501"] = 0.010000*np.tanh(((((((((data["fftr_min_last_1000"]) * 2.0)) * (((((((data["av_change_abs_roll_std_100"]) - (data["fftr_autocorrelation_10000"]))) * 2.0)) - (data["fftr_autocorrelation_10000"]))))) * 2.0)) - (data["percentile_roll_std_70"]))) 
        v["i502"] = 0.010000*np.tanh(((((((((((data["fftr_mean_last_1000"]) * 2.0)) * 2.0)) * 2.0)) + (data["percentile_roll_mean_70"]))) * (data["fftr_time_rev_asym_stat_5"]))) 
        v["i503"] = 0.009996*np.tanh(((((((((((data["fftr_time_rev_asym_stat_50"]) * 2.0)) + (data["fftr_time_rev_asym_stat_50"]))) * (((((data["ave_roll_mean_1000"]) - (data["ffti_percentile_roll_mean_30"]))) * 2.0)))) * 2.0)) + (((data["percentile_roll_mean_75"]) * 2.0)))) 
        v["i504"] = 0.010000*np.tanh(((((data["ffti_num_peaks_100"]) * (((data["fftr_min_roll_std_10"]) * 2.0)))) + (((data["min_roll_std_10000"]) + (((((data["ffti_num_peaks_100"]) * (((data["fftr_min_roll_std_10"]) * 2.0)))) - (data["mean_change_rate"]))))))) 
        v["i505"] = 0.010000*np.tanh(((((((((((((((((((((((data["fftr_autocorrelation_1000"]) * (data["fftr_percentile_roll_std_25"]))) * 2.0)) * 2.0)) * 2.0)) / 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i506"] = 0.010000*np.tanh(((data["fftr_autocorrelation_10"]) * (((((((((data["fftr_autocorrelation_10"]) - (((((data["ffti_min_roll_std_50"]) * 2.0)) * 2.0)))) * 2.0)) - (data["ffti_min_roll_std_50"]))) * 2.0)))) 
        v["i507"] = 0.010000*np.tanh(((((((data["autocorrelation_5000"]) * 2.0)) + (((((data["percentile_70"]) + (((((data["ffti_range_2000_3000"]) * (((((data["autocorrelation_5000"]) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i508"] = 0.010000*np.tanh(((data["percentile_70"]) - (((((((((data["ffti_av_change_abs_roll_std_10000"]) * (((data["ffti_av_change_abs_roll_std_100"]) + (((data["kurt"]) * (data["fftr_classic_sta_lta6_mean"]))))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i509"] = 0.010000*np.tanh(((((((((((((((((data["ffti_max_to_min_diff"]) * (((data["ffti_min_roll_std_50"]) - (data["fftr_autocorrelation_5000"]))))) * 2.0)) * 2.0)) * 2.0)) / 2.0)) + (data["abs_percentile_25"]))) * 2.0)) * 2.0)) 
        v["i510"] = 0.010000*np.tanh(((((data["percentile_70"]) + (((((((((data["ffti_mean_change_rate"]) * 2.0)) - (((data["mean_change_rate_last_10000"]) - (((data["ffti_mean_change_rate"]) * 2.0)))))) - (data["mean_change_rate_last_10000"]))) * 2.0)))) * 2.0)) 
        v["i511"] = 0.010000*np.tanh(((((data["min_roll_std_10000"]) + (((((((data["ffti_num_peaks_100"]) * (data["fftr_percentile_50"]))) * 2.0)) + (((data["fftr_autocorrelation_5000"]) * (data["ffti_mean_change_abs"]))))))) * 2.0)) 
        v["i512"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_std_40"]) * 2.0)) * (((((((((data["ffti_kurt"]) * 2.0)) * 2.0)) * 2.0)) + (((((((data["ffti_av_change_abs_roll_mean_1000"]) * 2.0)) * 2.0)) * 2.0))))))
        return v[v.columns[:n]].sum(axis=1)
    
    def GP_class_1(self,data,n):
        v = pd.DataFrame()
        v["i0"] = -0.939196 
        v["i1"] = 0.010000*np.tanh(((((((((((data["range_0_1000"]) - (((data["num_peaks_10"]) * 2.0)))) - (data["ffti_range_-4000_-3000"]))) * 2.0)) - (data["num_peaks_10"]))) + ((-1.0*((data["percentile_roll_std_25"])))))) 
        v["i2"] = 0.010000*np.tanh(((((((data["fftr_kurt"]) - (data["percentile_roll_std_30"]))) - (((((((((data["percentile_roll_std_25"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (((((data["percentile_roll_std_25"]) * 2.0)) * 2.0)))) 
        v["i3"] = 0.010000*np.tanh(((((((((((((data["range_0_1000"]) - (data["percentile_roll_std_30"]))) * 2.0)) + (data["fftr_percentile_25"]))) * 2.0)) + (data["range_0_1000"]))) * 2.0)) 
        v["i4"] = 0.010000*np.tanh(((((((((((((((data["percentile_5"]) - (data["num_peaks_10"]))) - (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)) + (data["range_0_1000"]))) * 2.0)) * 2.0)) 
        v["i5"] = 0.010000*np.tanh(((((((((((data["mean_change_rate_last_50000"]) + ((-1.0*((((data["ffti_range_3000_4000"]) + (((data["percentile_roll_std_30"]) * 2.0))))))))) * 2.0)) * 2.0)) + ((-1.0*((data["kstat_4"])))))) * 2.0)) 
        v["i6"] = 0.010000*np.tanh((((((((-1.0*((data["percentile_roll_std_10"])))) * 2.0)) + (((((((data["range_0_1000"]) - (data["num_peaks_10"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i7"] = 0.010000*np.tanh(((((((((((((((data["range_0_1000"]) - (data["abs_percentile_80"]))) * 2.0)) - (((data["percentile_roll_std_20"]) * 2.0)))) * 2.0)) * 2.0)) - (data["percentile_roll_std_25"]))) * 2.0)) 
        v["i8"] = 0.010000*np.tanh(((((((((data["range_0_1000"]) + ((((((((((-1.0*((data["percentile_roll_std_20"])))) * 2.0)) * 2.0)) + (data["range_0_1000"]))) * 2.0)))) * 2.0)) + (data["fftr_range_0_1000"]))) * 2.0)) 
        v["i9"] = 0.010000*np.tanh(((((((data["range_0_1000"]) + ((((-1.0*((data["percentile_95"])))) + (((((data["range_0_1000"]) - (data["ffti_range_-3000_-2000"]))) - (data["percentile_roll_std_50"]))))))) * 2.0)) * 2.0)) 
        v["i10"] = 0.010000*np.tanh(((data["mean_change_rate"]) - (((((((data["percentile_roll_std_25"]) * 2.0)) + (((((((((((((data["percentile_roll_std_25"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i11"] = 0.010000*np.tanh(((((data["mean_change_rate"]) - (((((((data["percentile_roll_std_10"]) + (((data["percentile_roll_std_30"]) + (data["iqr"]))))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i12"] = 0.010000*np.tanh((((((((((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) * 2.0)) + (((((data["ffti_range_-1000_0"]) - (data["percentile_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i13"] = 0.010000*np.tanh(((((((((((data["range_0_1000"]) + (data["fftr_percentile_20"]))) - (data["num_peaks_10"]))) * 2.0)) - (((data["min_roll_std_10000"]) * 2.0)))) * 2.0)) 
        v["i14"] = 0.010000*np.tanh(((data["mean_change_rate"]) - (((((((((((((data["percentile_roll_std_25"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i15"] = 0.010000*np.tanh(((((data["range_0_1000"]) + (((((((((data["range_0_1000"]) - (data["percentile_roll_std_30"]))) - (data["percentile_roll_std_30"]))) * 2.0)) * 2.0)))) - (((((data["percentile_roll_std_10"]) * 2.0)) * 2.0)))) 
        v["i16"] = 0.010000*np.tanh(((((data["mean_change_rate"]) - (((data["ffti_range_-3000_-2000"]) * 2.0)))) - (((((((data["percentile_roll_std_20"]) * 2.0)) * 2.0)) * 2.0)))) 
        v["i17"] = 0.010000*np.tanh((((((((-1.0*((((data["num_peaks_10"]) * 2.0))))) + (data["range_0_1000"]))) * 2.0)) * 2.0)) 
        v["i18"] = 0.010000*np.tanh(((((((((data["range_0_1000"]) * 2.0)) - (((((data["percentile_roll_std_30"]) + (data["iqr"]))) * 2.0)))) + (((data["fftr_min_last_1000"]) - (data["percentile_roll_std_5"]))))) * 2.0)) 
        v["i19"] = 0.010000*np.tanh((((((-1.0*((((data["percentile_roll_std_20"]) + (((((data["percentile_roll_std_20"]) * 2.0)) * 2.0))))))) - (data["fftr_ave_roll_std_10000"]))) * 2.0)) 
        v["i20"] = 0.010000*np.tanh((((((((((((((-1.0*((data["min_roll_std_10000"])))) - (data["ffti_range_-4000_-3000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i21"] = 0.010000*np.tanh(((((((((data["range_0_1000"]) * 2.0)) * 2.0)) + (data["range_-1000_0"]))) + ((((((-1.0*((data["iqr1"])))) + (data["fftr_range_0_1000"]))) * 2.0)))) 
        v["i22"] = 0.010000*np.tanh((((((((((((-1.0*((data["percentile_roll_std_20"])))) * 2.0)) * 2.0)) - (data["fftr_range_-4000_-3000"]))) * 2.0)) * 2.0)) 
        v["i23"] = 0.010000*np.tanh(((((((((data["range_0_1000"]) - (((data["percentile_roll_std_25"]) * 2.0)))) * 2.0)) + ((-1.0*((((data["percentile_roll_std_20"]) * 2.0))))))) * 2.0)) 
        v["i24"] = 0.010000*np.tanh(((((((((((((((data["mean_change_rate"]) + (data["percentile_5"]))) - (((data["iqr"]) * 2.0)))) * 2.0)) * 2.0)) - (data["percentile_roll_std_20"]))) * 2.0)) * 2.0)) 
        v["i25"] = 0.010000*np.tanh(((data["range_0_1000"]) - (((data["fftr_range_0_1000"]) - (((((((((data["range_0_1000"]) * 2.0)) - (data["ffti_range_-3000_-2000"]))) * 2.0)) * 2.0)))))) 
        v["i26"] = 0.010000*np.tanh((((((((((((((-1.0*((data["percentile_roll_std_20"])))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["percentile_roll_std_20"]))) 
        v["i27"] = 0.010000*np.tanh(((((((((((((data["range_0_1000"]) - (data["ffti_range_-3000_-2000"]))) + (((((data["range_0_1000"]) * 2.0)) + (data["range_0_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i28"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_std_20"]) * 2.0)) * (-3.0))) * 2.0)) - (data["min_roll_std_100"]))) * 2.0)) * 2.0)) 
        v["i29"] = 0.010000*np.tanh(((((((((((data["mean_change_rate"]) - (((data["percentile_roll_std_20"]) + (data["fftr_range_-4000_-3000"]))))) - (data["percentile_roll_std_20"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i30"] = 0.010000*np.tanh(((((((((((((((data["percentile_5"]) * 2.0)) + (data["fftr_range_-1000_0"]))) - (data["min_roll_std_100"]))) - (data["percentile_roll_std_1"]))) - (data["min_roll_std_10000"]))) * 2.0)) - (data["percentile_roll_std_1"]))) 
        v["i31"] = 0.010000*np.tanh(((((((((((((((((((data["range_0_1000"]) - (data["min_roll_std_10000"]))) * 2.0)) * 2.0)) - (data["ffti_range_-3000_-2000"]))) * 2.0)) - (data["range_0_1000"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i32"] = 0.010000*np.tanh(((((((data["mean_change_rate"]) * 2.0)) - (((data["percentile_roll_std_25"]) * 2.0)))) - (((data["min_roll_std_100"]) * 2.0)))) 
        v["i33"] = 0.010000*np.tanh((((-1.0*((data["percentile_roll_std_10"])))) - (((((((((((((data["min_roll_std_1000"]) + (data["percentile_roll_std_1"]))) * 2.0)) + (data["min_roll_std_100"]))) * 2.0)) * 2.0)) + (data["ave_roll_std_500"]))))) 
        v["i34"] = 0.010000*np.tanh((((((((-1.0*((data["percentile_roll_std_25"])))) + (((((((data["range_0_1000"]) + (((data["ffti_percentile_40"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) + (data["ffti_range_-1000_0"]))) 
        v["i35"] = 0.010000*np.tanh(((data["range_0_1000"]) + (((((((((((data["range_0_1000"]) - (data["abs_percentile_80"]))) * 2.0)) - (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)))) 
        v["i36"] = 0.010000*np.tanh(((((((((((((((data["ffti_percentile_25"]) + (data["range_0_1000"]))) * 2.0)) + (((data["ffti_percentile_40"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i37"] = 0.010000*np.tanh((((-1.0*(((-1.0*(((-1.0*((((((data["num_peaks_10"]) + (((data["min_roll_std_10000"]) * 2.0)))) - ((((-1.0*((data["min_roll_std_100"])))) * 2.0))))))))))))) * 2.0)) 
        v["i38"] = 0.010000*np.tanh(((data["fftr_percentile_25"]) + (((((data["fftr_abs_percentile_70"]) + (((((((data["percentile_5"]) - (data["ffti_range_2000_3000"]))) * 2.0)) * 2.0)))) - (((data["percentile_roll_std_25"]) * 2.0)))))) 
        v["i39"] = 0.010000*np.tanh(((data["percentile_roll_std_20"]) - (((((((((data["abs_percentile_5"]) - (((data["mean_change_rate"]) - (data["ffti_abs_max_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i40"] = 0.010000*np.tanh(((((((((((((((data["range_0_1000"]) - (((data["percentile_roll_std_25"]) * 2.0)))) * 2.0)) - (data["percentile_roll_std_25"]))) * 2.0)) - (((data["min_roll_std_100"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i41"] = 0.010000*np.tanh(((((((((((((data["range_0_1000"]) + (((((data["range_0_1000"]) - (data["iqr"]))) * 2.0)))) + (data["fftr_percentile_25"]))) * 2.0)) * 2.0)) - (data["range_-1000_0"]))) * 2.0)) 
        v["i42"] = 0.010000*np.tanh(((((data["fftr_percentile_25"]) + (((((((((((data["fftr_percentile_25"]) * 2.0)) - (data["ffti_range_2000_3000"]))) - (data["min_roll_std_500"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i43"] = 0.010000*np.tanh(((((((((((((((data["mean_change_rate"]) + (((data["ffti_range_0_1000"]) * 2.0)))) * 2.0)) + (((data["ffti_range_0_1000"]) + (data["mean_change_rate"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i44"] = 0.010000*np.tanh(((data["range_0_1000"]) + (((((((data["range_0_1000"]) + ((((((-1.0*((data["min_roll_std_10000"])))) * 2.0)) * 2.0)))) + ((-1.0*((data["abs_percentile_80"])))))) * 2.0)))) 
        v["i45"] = 0.010000*np.tanh(((((((((data["ffti_min_roll_std_1000"]) - (data["percentile_90"]))) - (((data["min_roll_std_10000"]) * 2.0)))) * 2.0)) + (data["percentile_5"]))) 
        v["i46"] = 0.010000*np.tanh(((((((data["ffti_max_to_min"]) - (((((data["percentile_roll_std_20"]) * 2.0)) * 2.0)))) + (data["ffti_max_to_min"]))) - (((((data["percentile_roll_std_20"]) * 2.0)) * 2.0)))) 
        v["i47"] = 0.010000*np.tanh(((data["abs_percentile_50"]) * (((((((((((data["ffti_percentile_40"]) + (data["range_0_1000"]))) * 2.0)) * 2.0)) + (((data["mean_change_rate_first_50000"]) + (data["range_0_1000"]))))) * 2.0)))) 
        v["i48"] = 0.010000*np.tanh(((((data["range_0_1000"]) - (data["percentile_95"]))) - (((((((((((((((data["percentile_95"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i49"] = 0.010000*np.tanh(((data["mean_change_rate_first_10000"]) - (((((((((data["min_roll_std_50"]) + (data["percentile_roll_std_20"]))) * 2.0)) + ((-1.0*((((data["ffti_percentile_roll_std_5"]) * 2.0))))))) * 2.0)))) 
        v["i50"] = 0.010000*np.tanh((((-1.0*((((((((((data["percentile_roll_std_25"]) - (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)) * 2.0))))) - ((((((data["percentile_roll_std_20"]) * 2.0)) + ((-1.0*((data["percentile_roll_std_25"])))))/2.0)))) 
        v["i51"] = 0.010000*np.tanh(((data["range_0_1000"]) + (((((((((data["exp_Moving_average_30000_mean"]) * ((((data["mean_change_rate_first_10000"]) + (data["range_0_1000"]))/2.0)))) * 2.0)) - (data["percentile_80"]))) * 2.0)))) 
        v["i52"] = 0.010000*np.tanh(((((((data["ffti_max_to_min"]) - (data["min_roll_std_10000"]))) + (((((data["mean_change_rate_first_10000"]) - (((data["min_roll_std_100"]) + (data["ffti_range_2000_3000"]))))) * 2.0)))) - (data["ffti_abs_percentile_1"]))) 
        v["i53"] = 0.010000*np.tanh((((((((((((((((((((((-1.0*((data["percentile_90"])))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i54"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_mean_90"]) * (data["percentile_20"]))) * 2.0)) * 2.0)) - (((data["fftr_range_2000_3000"]) * 2.0)))) * 2.0)) - (data["num_peaks_10"]))) 
        v["i55"] = 0.010000*np.tanh(((data["mean_change_rate_last_50000"]) + (((data["abs_percentile_50"]) * (((((((((((data["mean_change_rate_last_50000"]) + (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)) + (((data["ffti_percentile_20"]) * 2.0)))) * 2.0)))))) 
        v["i56"] = 0.010000*np.tanh((((((10.0)) * ((((((data["fftr_mean_change_rate_last_10000"]) + ((((((10.0)) * (data["mean_change_rate"]))) * 2.0)))/2.0)) - (data["percentile_roll_std_10"]))))) * (data["hmean"]))) 
        v["i57"] = 0.010000*np.tanh(((((((((((data["mean_change_rate_first_10000"]) * 2.0)) - (((data["abs_percentile_5"]) * 2.0)))) * 2.0)) - (data["min_roll_std_100"]))) - (data["abs_percentile_5"]))) 
        v["i58"] = 0.010000*np.tanh(((((((data["percentile_5"]) + (((((((((((data["mean_change_rate_last_10000"]) * (data["percentile_75"]))) * 2.0)) + ((-1.0*((data["percentile_90"])))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i59"] = 0.010000*np.tanh((((-1.0*((((data["abs_max"]) + (((data["min_roll_std_10000"]) - ((-1.0*(((-1.0*((data["ffti_max_to_min"])))))))))))))) - (((data["abs_percentile_70"]) * 2.0)))) 
        v["i60"] = 0.010000*np.tanh(((((data["autocorrelation_50"]) - (((data["min_roll_std_100"]) + (((((data["percentile_90"]) * 2.0)) * 2.0)))))) - (((((data["percentile_90"]) * 2.0)) * 2.0)))) 
        v["i61"] = 0.010000*np.tanh(((((((((((((data["exp_Moving_average_50000_mean"]) * (((data["mean_change_rate"]) + (data["mean_change_rate_last_10000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i62"] = 0.010000*np.tanh(((((data["mean_change_rate_first_50000"]) + (((((((((data["mean_change_rate_first_50000"]) + (data["range_0_1000"]))) * 2.0)) * 2.0)) * 2.0)))) * (data["percentile_roll_mean_20"]))) 
        v["i63"] = 0.010000*np.tanh((((((-1.0*((((((((((data["percentile_90"]) * 2.0)) * 2.0)) * 2.0)) + (data["percentile_90"])))))) * 2.0)) * 2.0)) 
        v["i64"] = 0.010000*np.tanh(((((((((((((data["percentile_20"]) * 2.0)) * (data["percentile_roll_mean_80"]))) * 2.0)) * 2.0)) * 2.0)) - ((-1.0*((data["percentile_90"])))))) 
        v["i65"] = 0.010000*np.tanh(((((data["mean_change_rate"]) + (((((((((((data["abs_percentile_25"]) * (((((data["mean_change_rate"]) - (data["min_roll_std_100"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i66"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_mean_95"]) - (data["min_roll_std_100"]))) - (((data["min_roll_std_100"]) + (data["max_to_min"]))))) - (((((data["max_to_min"]) + (data["min_roll_std_100"]))) + (data["percentile_70"]))))) 
        v["i67"] = 0.010000*np.tanh(((((((((((data["hmean"]) * 2.0)) * 2.0)) + (data["exp_Moving_average_30000_mean"]))) * (((data["percentile_20"]) + (data["percentile_20"]))))) - (((data["hmean"]) + (data["max_last_50000"]))))) 
        v["i68"] = 0.010000*np.tanh(((((((data["ffti_max_to_min"]) + ((-1.0*((data["ffti_range_2000_3000"])))))) - (data["min_roll_std_10000"]))) + (((((data["ffti_max_to_min"]) + (data["fftr_min_first_50000"]))) + (data["fftr_min_roll_std_10"]))))) 
        v["i69"] = 0.010000*np.tanh(((((((((data["ffti_max_to_min"]) - (data["percentile_80"]))) + (data["ffti_max_to_min"]))) * 2.0)) - (data["percentile_80"]))) 
        v["i70"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_5"]) + (((data["percentile_20"]) + (((data["percentile_20"]) * 2.0)))))) * (((data["ffti_percentile_80"]) + (((data["percentile_roll_mean_70"]) * 2.0)))))) * 2.0)) 
        v["i71"] = 0.010000*np.tanh(((((((((((((data["percentile_20"]) * 2.0)) + (((data["percentile_20"]) + (data["fftr_moment_1"]))))) * (data["fftr_abs_max_roll_std_10"]))) * 2.0)) + (data["fftr_num_peaks_100"]))) * 2.0)) 
        v["i72"] = 0.010000*np.tanh(((((((((((((((((data["fftr_exp_Moving_average_50000_std"]) * 2.0)) * (data["range_0_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["range_0_1000"]))) 
        v["i73"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_50"]) + (((((((((data["fftr_min_roll_std_50"]) + (((data["ffti_range_-4000_-3000"]) * (data["percentile_20"]))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i74"] = 0.010000*np.tanh(((((((((data["fftr_av_change_abs_roll_mean_10"]) + ((((((-1.0*((data["range_-1000_0"])))) + ((-1.0*((data["autocorrelation_10"])))))) * 2.0)))) * 2.0)) + (data["ffti_percentile_roll_mean_99"]))) * 2.0)) 
        v["i75"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_1"]) + (((((((((data["ffti_percentile_roll_std_1"]) - (data["percentile_20"]))) - (data["percentile_20"]))) * (((data["fftr_av_change_abs_roll_mean_50"]) * 2.0)))) * 2.0)))) 
        v["i76"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_500"]) + (((((data["fftr_min_roll_std_100"]) + (((data["ffti_min_roll_std_10000"]) - (((data["max_to_min"]) + (data["ffti_iqr"]))))))) * 2.0)))) + (data["ffti_min_roll_std_10000"]))) 
        v["i77"] = 0.010000*np.tanh(((((((((((data["ffti_abs_percentile_40"]) * (((data["ffti_mean_change_rate"]) - (data["percentile_75"]))))) + (((data["ffti_percentile_roll_std_1"]) * 2.0)))) * 2.0)) * 2.0)) - (((data["ffti_mean_change_rate"]) * 2.0)))) 
        v["i78"] = 0.010000*np.tanh(((((((data["percentile_20"]) * (data["Hann_window_mean_1500"]))) + (((data["abs_percentile_50"]) * ((((((data["fftr_percentile_roll_std_25"]) + (((data["mean_change_rate"]) * 2.0)))/2.0)) * 2.0)))))) * 2.0)) 
        v["i79"] = 0.010000*np.tanh(((((((((((((data["kstat_1"]) * (((data["percentile_20"]) * 2.0)))) + (data["autocorrelation_5"]))) + (data["fftr_min_roll_std_100"]))) - (data["percentile_70"]))) * 2.0)) * 2.0)) 
        v["i80"] = 0.010000*np.tanh(((((((((data["ffti_trend"]) + (((data["fftr_min_roll_std_50"]) + (((data["fftr_av_change_abs_roll_std_10"]) + (data["fftr_num_peaks_100"]))))))) - (data["percentile_90"]))) * 2.0)) * 2.0)) 
        v["i81"] = 0.010000*np.tanh(((((((((((((((((data["percentile_20"]) * ((((data["abs_percentile_70"]) + (data["percentile_roll_mean_25"]))/2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i82"] = 0.010000*np.tanh(((((((((((data["percentile_75"]) * (((data["ffti_max_to_min"]) + (data["mean_change_rate_last_10000"]))))) - (data["max_to_min"]))) * 2.0)) * 2.0)) + (data["classic_sta_lta3_mean"]))) 
        v["i83"] = 0.010000*np.tanh(((((((data["classic_sta_lta6_mean"]) + ((((((data["autocorrelation_50"]) + (data["ffti_percentile_roll_mean_99"]))/2.0)) + (data["autocorrelation_50"]))))) + (data["fftr_min_roll_std_10"]))) * 2.0)) 
        v["i84"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_50"]) * 2.0)) - ((-1.0*((((((((((data["fftr_min_roll_std_500"]) + (data["ffti_max_to_min"]))) + (data["fftr_min_roll_std_10"]))) * 2.0)) * 2.0))))))) 
        v["i85"] = 0.010000*np.tanh(((((((((((data["autocorrelation_5"]) - (data["ffti_percentile_90"]))) + (data["ffti_std_roll_mean_500"]))) * 2.0)) + (((data["ffti_max_to_min"]) - (data["percentile_70"]))))) * 2.0)) 
        v["i86"] = 0.010000*np.tanh(((((((data["classic_sta_lta3_mean"]) + (((((data["fftr_min_roll_std_50"]) + (((data["ffti_percentile_roll_mean_95"]) * 2.0)))) + (((data["fftr_min_roll_std_500"]) + (data["ffti_max_to_min"]))))))) * 2.0)) * 2.0)) 
        v["i87"] = 0.010000*np.tanh(((((data["Hann_window_mean_50"]) - (data["fftr_mean_change_abs"]))) * (((((((data["mean_change_rate_first_50000"]) * 2.0)) * 2.0)) - ((-1.0*((((data["mean_change_rate_last_10000"]) * 2.0))))))))) 
        v["i88"] = 0.010000*np.tanh(((((((((((((data["classic_sta_lta3_mean"]) - (data["max_to_min"]))) * 2.0)) * 2.0)) * 2.0)) - (data["ffti_percentile_roll_std_50"]))) + (data["max_to_min"]))) 
        v["i89"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_10"]) + (((((data["classic_sta_lta6_mean"]) + (((((data["percentile_20"]) * (((data["mean_first_10000"]) + (data["ffti_classic_sta_lta5_mean"]))))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i90"] = 0.010000*np.tanh(((((((data["fftr_min_roll_std_50"]) + (((((((((data["fftr_min_roll_std_50"]) + (((data["autocorrelation_50"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))) + (data["fftr_num_peaks_100"]))) + (data["fftr_min_roll_std_50"]))) 
        v["i91"] = 0.010000*np.tanh(((data["exp_Moving_average_30000_std"]) + (((data["autocorrelation_1000"]) + (((((((data["exp_Moving_average_30000_std"]) - (data["percentile_70"]))) * 2.0)) - (((data["percentile_70"]) * 2.0)))))))) 
        v["i92"] = 0.010000*np.tanh(((((((data["fftr_min_roll_std_50"]) - (data["autocorrelation_10"]))) + (data["classic_sta_lta3_mean"]))) + (((((((((data["mean_change_rate_last_1000"]) * (data["percentile_roll_mean_10"]))) * 2.0)) + (data["classic_sta_lta3_mean"]))) * 2.0)))) 
        v["i93"] = 0.010000*np.tanh(((data["fftr_num_peaks_100"]) + (((((((((((data["fftr_min_roll_std_10"]) - (((data["percentile_70"]) - (((data["ffti_max_to_min"]) * 2.0)))))) * 2.0)) + (data["ffti_autocorrelation_10000"]))) * 2.0)) * 2.0)))) 
        v["i94"] = 0.010000*np.tanh(((((data["percentile_75"]) * 2.0)) + (((((((data["fftr_min_roll_std_500"]) - (data["max_to_min"]))) * 2.0)) - (data["abs_max_roll_std_500"]))))) 
        v["i95"] = 0.010000*np.tanh(((data["med"]) * (((((data["fftr_max_roll_mean_500"]) + (data["med"]))) + (((((((data["percentile_20"]) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i96"] = 0.010000*np.tanh(((((((data["fftr_num_peaks_100"]) + (((data["ffti_autocorrelation_10000"]) + (data["ffti_std_roll_mean_1000"]))))) + (((data["fftr_min_roll_std_10"]) + (data["autocorrelation_50"]))))) * 2.0)) 
        v["i97"] = 0.010000*np.tanh(((data["percentile_20"]) * (((data["percentile_20"]) + (((((((data["fftr_percentile_roll_std_25"]) + (data["percentile_80"]))) * 2.0)) * 2.0)))))) 
        v["i98"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_500"]) + (((data["fftr_min_roll_std_50"]) - (((data["classic_sta_lta2_mean"]) * 2.0)))))) * 2.0)) 
        v["i99"] = 0.010000*np.tanh(((((((((data["fftr_num_peaks_20"]) - (((data["autocorrelation_10"]) - (((data["fftr_min_roll_std_50"]) + (data["autocorrelation_5"]))))))) + ((-1.0*((data["ffti_range_p4000_pinf"])))))) - (data["autocorrelation_10"]))) * 2.0)) 
        v["i100"] = 0.010000*np.tanh((((((((-1.0*((data["max_to_min"])))) - (data["percentile_70"]))) + (((data["fftr_percentile_roll_std_25"]) - ((((-1.0*((data["ffti_min_roll_std_500"])))) * 2.0)))))) * 2.0)) 
        v["i101"] = 0.010000*np.tanh((((((((((-1.0*((data["ffti_min_roll_std_50"])))) + (data["fftr_max_first_10000"]))) * (data["Hann_window_mean_1500"]))) + (data["fftr_num_peaks_100"]))) * 2.0)) 
        v["i102"] = 0.010000*np.tanh(((((((data["autocorrelation_5"]) - (data["av_change_abs_roll_std_1000"]))) - (((((data["max_to_min"]) * 2.0)) * 2.0)))) + (data["max_to_min"]))) 
        v["i103"] = 0.010000*np.tanh(((((data["percentile_20"]) * 2.0)) * (((((((((data["fftr_count_big"]) + (data["percentile_50"]))) * 2.0)) * 2.0)) + (((((data["hmean"]) * 2.0)) + (data["fftr_count_big"]))))))) 
        v["i104"] = 0.010000*np.tanh(((data["autocorrelation_50"]) + (((data["fftr_range_p4000_pinf"]) + (((data["autocorrelation_50"]) - (data["max_to_min"]))))))) 
        v["i105"] = 0.010000*np.tanh((((((((((-1.0*((((data["ffti_min_roll_std_50"]) * 2.0))))) * 2.0)) + ((((3.0) + (data["ffti_min_roll_std_50"]))/2.0)))) * 2.0)) * 2.0)) 
        v["i106"] = 0.010000*np.tanh(((((data["ffti_max_to_min"]) + (((((data["ffti_max_to_min"]) + (((data["fftr_min_roll_std_10"]) + (data["av_change_abs_roll_mean_500"]))))) * 2.0)))) + (data["ffti_min_roll_std_10000"]))) 
        v["i107"] = 0.010000*np.tanh(((((((((((((((data["ffti_max_to_min"]) - (data["percentile_90"]))) * 2.0)) + (data["autocorrelation_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i108"] = 0.010000*np.tanh(((((((data["percentile_roll_std_70"]) * 2.0)) + (data["mean_change_rate_first_10000"]))) + (((((((((data["mean_change_rate_first_10000"]) * 2.0)) * (data["fftr_max"]))) * 2.0)) + (data["fftr_num_peaks_20"]))))) 
        v["i109"] = 0.010000*np.tanh(((data["percentile_20"]) * (((((((((data["hmean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i110"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_std_25"]) + (data["classic_sta_lta3_mean"]))) + (((((data["fftr_percentile_roll_std_25"]) + (((data["classic_sta_lta3_mean"]) + (data["fftr_min_roll_std_10"]))))) + (data["autocorrelation_5"]))))) 
        v["i111"] = 0.010000*np.tanh(((((((data["fftr_mean_first_1000"]) * (((((((data["autocorrelation_1000"]) - (((data["ffti_av_change_abs_roll_std_100"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) + (data["percentile_roll_std_80"]))) 
        v["i112"] = 0.010000*np.tanh(((((((((((data["ffti_percentile_roll_std_1"]) * 2.0)) + ((((-1.0*((((data["ffti_mean_change_rate"]) - (data["fftr_num_peaks_100"])))))) * 2.0)))) * 2.0)) * 2.0)) + (((data["ffti_percentile_roll_std_1"]) * 2.0)))) 
        v["i113"] = 0.010000*np.tanh(((data["percentile_roll_std_70"]) + (((((((data["percentile_roll_std_70"]) + (((data["percentile_roll_std_70"]) + (((data["ffti_autocorrelation_10000"]) + (data["ffti_exp_Moving_average_3000_std"]))))))) * 2.0)) * 2.0)))) 
        v["i114"] = 0.010000*np.tanh(((((((((((((data["fftr_min_roll_std_50"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((((((data["autocorrelation_50"]) * 2.0)) / 2.0)) * 2.0)))) * 2.0)) 
        v["i115"] = 0.010000*np.tanh(((((((((data["autocorrelation_50"]) + (((((((((((data["percentile_roll_mean_20"]) * (data["mean_change_rate_first_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i116"] = 0.010000*np.tanh(((((((((data["ffti_autocorrelation_10000"]) * 2.0)) - (data["ffti_abs_percentile_1"]))) * 2.0)) - (((((data["ffti_abs_percentile_1"]) + (data["max_to_min"]))) + ((-1.0*((data["ffti_percentile_roll_std_1"])))))))) 
        v["i117"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_70"]) + ((((((-1.0*((data["fftr_abs_percentile_10"])))) - (data["max_to_min"]))) * 2.0)))) * 2.0)) * 2.0)) + (data["percentile_roll_std_70"]))) * 2.0)) 
        v["i118"] = 0.010000*np.tanh(((((((((data["autocorrelation_50"]) + (((((((data["mean_first_10000"]) + (data["mean_change_rate_first_10000"]))) * (data["percentile_roll_mean_25"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i119"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_std_20"]) * ((((9.0)) * (data["percentile_40"]))))) * 2.0)) * (data["fftr_min_roll_std_50"]))) * 2.0)) * ((((9.0)) - (data["percentile_40"]))))) 
        v["i120"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_70"]) * 2.0)) * 2.0)) + (((((((data["mean_first_1000"]) * (data["ffti_range_3000_4000"]))) + (((data["fftr_abs_percentile_1"]) + (data["percentile_roll_std_80"]))))) * 2.0)))) * 2.0)) 
        v["i121"] = 0.010000*np.tanh(((((((data["ffti_trend"]) - (data["fftr_mean_first_1000"]))) * 2.0)) + (((((data["ffti_trend"]) * 2.0)) + (((data["autocorrelation_5"]) + (data["classic_sta_lta8_mean"]))))))) 
        v["i122"] = 0.010000*np.tanh(((((((data["fftr_exp_Moving_average_300_std"]) + (((data["fftr_min_roll_std_10"]) * 2.0)))) + (data["ffti_autocorrelation_10000"]))) + (((((data["fftr_exp_Moving_average_300_std"]) * (data["fftr_exp_Moving_average_300_std"]))) * 2.0)))) 
        v["i123"] = 0.010000*np.tanh(((((((((((data["Hann_window_mean_150"]) + (data["percentile_roll_std_70"]))) + (data["fftr_min_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i124"] = 0.010000*np.tanh(((((((((((((((data["autocorrelation_1000"]) - (data["ffti_min_roll_std_50"]))) * 2.0)) - (((data["ffti_min_roll_std_50"]) * 2.0)))) * 2.0)) * 2.0)) - (data["ffti_min_roll_std_50"]))) * 2.0)) 
        v["i125"] = 0.010000*np.tanh((((3.0) + ((((data["max_last_50000"]) + (((data["fftr_min_roll_std_50"]) * ((((13.56763935089111328)) + (3.0))))))/2.0)))/2.0)) 
        v["i126"] = 0.010000*np.tanh(((((((((((((data["ffti_max_to_min"]) + (data["fftr_min_roll_std_500"]))) * 2.0)) - (((((data["num_peaks_100"]) - (((data["ffti_percentile_roll_mean_99"]) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i127"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_25"]) + (((((data["fftr_percentile_roll_std_25"]) + (((((data["fftr_percentile_roll_std_25"]) * 2.0)) - (data["max_to_min"]))))) + (2.0))))) * 2.0)) - (data["max_to_min"]))) 
        v["i128"] = 0.010000*np.tanh(((data["abs_percentile_50"]) * (((((((((((((((((data["autocorrelation_1000"]) + (data["ffti_mean_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i129"] = 0.010000*np.tanh((((8.0)) + ((((((8.0)) * ((((((8.0)) * (data["percentile_roll_std_40"]))) * 2.0)))) + (((data["percentile_roll_std_70"]) + (data["ffti_min_roll_std_10000"]))))))) 
        v["i130"] = 0.010000*np.tanh((((((((((5.0)) * 2.0)) + ((9.30793571472167969)))) * 2.0)) * ((((5.0)) * (((data["mean_first_10000"]) * (data["abs_percentile_50"]))))))) 
        v["i131"] = 0.010000*np.tanh(((((((((data["ffti_skew"]) * 2.0)) * 2.0)) - (((((data["ffti_mean_change_rate"]) * 2.0)) * 2.0)))) + (data["ffti_skew"]))) 
        v["i132"] = 0.010000*np.tanh(((((data["percentile_roll_std_70"]) + (((((((((data["percentile_roll_mean_20"]) * (data["mean_change_rate_first_10000"]))) - (((((data["ffti_abs_max_roll_mean_10000"]) * (data["classic_sta_lta3_mean"]))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i133"] = 0.010000*np.tanh(((data["percentile_roll_std_70"]) * (((data["ffti_mean_change_rate"]) + ((((10.0)) + (((((data["percentile_roll_std_70"]) * ((((9.0)) + (data["fftr_min_roll_std_50"]))))) + (data["percentile_roll_std_70"]))))))))) 
        v["i134"] = 0.010000*np.tanh(((((data["percentile_roll_std_70"]) * 2.0)) + (((data["ffti_autocorrelation_10000"]) + (((data["autocorrelation_5"]) * (((data["fftr_percentile_25"]) * (((data["ffti_Hann_window_mean_1500"]) + ((11.20730876922607422)))))))))))) 
        v["i135"] = 0.010000*np.tanh(((((((((((((data["ffti_trend"]) - (((((data["classic_sta_lta2_mean"]) + (data["ffti_mean_change_rate"]))) * 2.0)))) * 2.0)) - (data["fftr_percentile_roll_mean_80"]))) * 2.0)) - (data["fftr_percentile_roll_mean_80"]))) * 2.0)) 
        v["i136"] = 0.010000*np.tanh(((data["autocorrelation_5"]) + (((((data["fftr_max_roll_mean_10"]) * (((((data["mean_change_rate_last_10000"]) * 2.0)) + (((data["fftr_max_roll_mean_10"]) * 2.0)))))) + (data["ffti_percentile_roll_std_1"]))))) 
        v["i137"] = 0.010000*np.tanh(((3.0) + (((((data["percentile_roll_std_40"]) * 2.0)) * ((((((13.12057590484619141)) * 2.0)) + (((data["percentile_roll_std_40"]) * ((((((10.0)) * 2.0)) * 2.0)))))))))) 
        v["i138"] = 0.010000*np.tanh(((((((((data["ffti_autocorrelation_10000"]) + (((((((((((data["ffti_autocorrelation_10000"]) * 2.0)) * (data["ffti_classic_sta_lta4_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i139"] = 0.010000*np.tanh(((((((((((data["fftr_min_roll_std_10000"]) * 2.0)) * 2.0)) - ((((12.28333282470703125)) * (((data["fftr_min_roll_std_10000"]) * 2.0)))))) + (data["percentile_roll_std_60"]))) * 2.0)) 
        v["i140"] = 0.010000*np.tanh(((((data["max_roll_mean_10000"]) + (((((((((((data["ffti_classic_sta_lta5_mean"]) * 2.0)) + (data["fftr_time_rev_asym_stat_50"]))) * 2.0)) + (data["fftr_max_roll_std_10"]))) * 2.0)))) * 2.0)) 
        v["i141"] = 0.010000*np.tanh(((((((((((((((data["fftr_time_rev_asym_stat_1"]) * (((data["fftr_percentile_20"]) * 2.0)))) * 2.0)) * 2.0)) - (data["ffti_min_roll_std_50"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i142"] = 0.010000*np.tanh(((((((((((((((((data["percentile_roll_std_60"]) / 2.0)) - (((data["fftr_percentile_roll_std_80"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i143"] = 0.010000*np.tanh(((((((data["percentile_roll_std_70"]) * (((((3.0) * 2.0)) * 2.0)))) * 2.0)) * (((((data["percentile_roll_std_70"]) * ((11.34355449676513672)))) + ((11.34355449676513672)))))) 
        v["i144"] = 0.010000*np.tanh(((((((((((data["fftr_range_2000_3000"]) * 2.0)) * (((((data["abs_percentile_70"]) * 2.0)) * 2.0)))) * 2.0)) - (data["percentile_50"]))) - (((data["abs_percentile_70"]) + (data["abs_percentile_70"]))))) 
        v["i145"] = 0.010000*np.tanh(((((((((data["percentile_10"]) * (data["abs_percentile_40"]))) * (((((((((data["percentile_roll_mean_60"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i146"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_70"]) + (((data["autocorrelation_50"]) * (data["fftr_range_2000_3000"]))))) * 2.0)) + (data["fftr_min_roll_std_500"]))) * 2.0)) * 2.0)) + (data["fftr_min_roll_std_500"]))) * 2.0)) 
        v["i147"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_std_25"]) - (data["fftr_percentile_roll_std_25"]))) * 2.0)) + ((((data["autocorrelation_5"]) + (data["ffti_percentile_roll_std_5"]))/2.0)))) * 2.0)) + (data["fftr_autocorrelation_5000"]))) * 2.0)) * 2.0)) 
        v["i148"] = 0.010000*np.tanh(((((((((((data["fftr_num_peaks_100"]) + (((((data["ffti_autocorrelation_10000"]) + (((data["fftr_percentile_40"]) * (((data["fftr_min_roll_std_50"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i149"] = 0.010000*np.tanh(((((((data["percentile_roll_std_70"]) + (((data["percentile_roll_std_70"]) + (((((((data["fftr_min_roll_std_10"]) + (data["av_change_abs_roll_mean_500"]))) * 2.0)) * 2.0)))))) + (data["av_change_abs_roll_mean_500"]))) * 2.0)) 
        v["i150"] = 0.010000*np.tanh(((((((data["fftr_autocorrelation_500"]) - (data["ffti_mean_change_rate"]))) + (((((((((data["ffti_trend"]) * 2.0)) * (((data["fftr_num_peaks_100"]) * 2.0)))) * 2.0)) + (data["percentile_20"]))))) * 2.0)) 
        v["i151"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_70"]) + ((((((-1.0*((data["fftr_percentile_roll_std_80"])))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i152"] = 0.010000*np.tanh(((((((data["ffti_std_roll_mean_10000"]) - (data["ffti_autocorrelation_10"]))) * 2.0)) + ((((((((((((data["abs_max_roll_mean_10000"]) * (data["mean_first_10000"]))) * 2.0)) * 2.0)) + (data["ffti_std_roll_mean_10000"]))/2.0)) * 2.0)))) 
        v["i153"] = 0.010000*np.tanh(((((data["percentile_20"]) * (((((((((data["fftr_percentile_roll_std_25"]) + (((data["mean_last_1000"]) * (data["percentile_roll_std_40"]))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i154"] = 0.010000*np.tanh(((((((data["autocorrelation_5"]) - (((((data["percentile_roll_std_25"]) * 2.0)) * (((data["autocorrelation_5"]) + (((data["fftr_percentile_roll_std_25"]) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i155"] = 0.010000*np.tanh(((((((((data["ffti_exp_Moving_average_50000_std"]) * (((data["percentile_roll_std_10"]) - (((((data["ffti_autocorrelation_500"]) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i156"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_70"]) + ((((((-1.0*((data["ffti_percentile_roll_std_80"])))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i157"] = 0.010000*np.tanh(((data["autocorrelation_5"]) * ((((-1.0*((((((((((data["percentile_roll_std_40"]) * 2.0)) * 2.0)) * 2.0)) * 2.0))))) * 2.0)))) 
        v["i158"] = 0.010000*np.tanh(((((((((data["fftr_abs_percentile_1"]) + (((((((data["ffti_range_2000_3000"]) * (data["abs_percentile_80"]))) * 2.0)) * 2.0)))) * 2.0)) + (data["fftr_abs_percentile_1"]))) * 2.0)) 
        v["i159"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_80"]) + (((((data["ffti_percentile_roll_std_1"]) * 2.0)) + (((((data["autocorrelation_50"]) + (data["percentile_roll_std_60"]))) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i160"] = 0.010000*np.tanh(((((((((((((((data["fftr_time_rev_asym_stat_10"]) - (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) - (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) + (((data["fftr_percentile_roll_std_25"]) - (data["ffti_av_change_abs_roll_std_100"]))))) 
        v["i161"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_std_5"]) * 2.0)) - (((data["fftr_time_rev_asym_stat_1"]) * (((((((data["mean_first_1000"]) + (((((data["ffti_percentile_roll_std_5"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))))) 
        v["i162"] = 0.010000*np.tanh(((((data["percentile_roll_std_70"]) + (((((data["mean_first_10000"]) * (((((((data["fftr_percentile_roll_std_25"]) * (((data["med"]) * 2.0)))) + (data["med"]))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i163"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_5"]) + (((((((((data["fftr_num_peaks_50"]) * (data["ffti_percentile_roll_std_25"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i164"] = 0.010000*np.tanh(((((((((data["percentile_80"]) * (((((data["mean_change_rate_last_10000"]) - (data["autocorrelation_100"]))) - (data["autocorrelation_100"]))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i165"] = 0.010000*np.tanh(((((((((((((((((data["mean_first_10000"]) * (((data["fftr_percentile_roll_std_30"]) + (data["fftr_max_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i166"] = 0.010000*np.tanh(((((((((((data["ffti_percentile_roll_std_5"]) + (((((data["percentile_roll_std_60"]) - (data["ffti_mean_change_rate"]))) * 2.0)))) * 2.0)) * 2.0)) - (data["fftr_range_-3000_-2000"]))) * 2.0)) 
        v["i167"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_70"]) + (((((data["fftr_min_roll_std_50"]) + (((data["ffti_range_-4000_-3000"]) * (((data["mean_first_1000"]) * 2.0)))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i168"] = 0.010000*np.tanh(((data["autocorrelation_1000"]) + (((((((((data["exp_Moving_average_300_mean"]) + (data["fftr_min_roll_std_1000"]))) * (((((data["mean_change_rate_last_10000"]) + (data["autocorrelation_1000"]))) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i169"] = 0.010000*np.tanh((((((9.0)) * ((((data["percentile_roll_std_5"]) + (((data["fftr_range_2000_3000"]) * (((data["autocorrelation_50"]) - ((-1.0*((data["percentile_60"])))))))))/2.0)))) * 2.0)) 
        v["i170"] = 0.010000*np.tanh(((((((((((((data["fftr_mean_first_1000"]) - (data["mean_change_rate"]))) * (data["mean_first_1000"]))) * 2.0)) + (((data["ffti_percentile_roll_mean_95"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i171"] = 0.010000*np.tanh(((((data["fftr_hmean"]) + (((((data["mean_first_1000"]) * (((data["fftr_hmean"]) + (((data["ffti_av_change_abs_roll_mean_10000"]) - (data["fftr_percentile_30"]))))))) * 2.0)))) - (data["fftr_range_-3000_-2000"]))) 
        v["i172"] = 0.010000*np.tanh(((((((data["autocorrelation_50"]) + (((data["fftr_percentile_roll_std_40"]) + (((data["fftr_num_peaks_100"]) * (((((((data["ffti_trend"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i173"] = 0.010000*np.tanh(((((data["percentile_roll_std_70"]) + (((((data["fftr_num_peaks_100"]) + (((((data["fftr_std_roll_mean_50"]) * (((data["exp_Moving_average_30000_std"]) - (data["classic_sta_lta3_mean"]))))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i174"] = 0.010000*np.tanh(((((((((data["fftr_time_rev_asym_stat_10"]) * 2.0)) + (((data["percentile_60"]) * (((data["fftr_time_rev_asym_stat_10"]) + (((((data["ffti_range_-3000_-2000"]) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i175"] = 0.010000*np.tanh(((((((data["autocorrelation_1000"]) + (((((data["ffti_std_roll_mean_50"]) * (((data["exp_Moving_average_30000_std"]) * 2.0)))) + (data["percentile_roll_std_80"]))))) + (data["fftr_ave_roll_mean_1000"]))) * 2.0)) 
        v["i176"] = 0.010000*np.tanh(((((((data["percentile_roll_std_5"]) * 2.0)) + (((((data["autocorrelation_50"]) * 2.0)) + (((((data["fftr_mean_first_1000"]) * 2.0)) + (((data["percentile_roll_std_5"]) * 2.0)))))))) * 2.0)) 
        v["i177"] = 0.010000*np.tanh(((((((((((data["mean_change_rate_last_10000"]) * (((((data["percentile_roll_mean_99"]) * 2.0)) * 2.0)))) + (((data["mean_change_rate_last_10000"]) * (((data["percentile_roll_mean_99"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i178"] = 0.010000*np.tanh(((((((((data["fftr_time_rev_asym_stat_5"]) * (((((data["fftr_av_change_abs_roll_mean_10"]) + (data["av_change_abs_roll_mean_500"]))) + (((data["fftr_trend"]) + (data["av_change_abs_roll_mean_500"]))))))) + (data["av_change_abs_roll_mean_500"]))) * 2.0)) * 2.0)) 
        v["i179"] = 0.010000*np.tanh(((((data["abs_percentile_80"]) + (((((data["ffti_percentile_roll_std_1"]) + (((((data["percentile_roll_std_5"]) * 2.0)) * 2.0)))) + (((data["percentile_roll_std_5"]) * 2.0)))))) * 2.0)) 
        v["i180"] = 0.010000*np.tanh(((((data["ffti_skew"]) + (data["fftr_min_roll_std_10"]))) + (((((((((((((data["av_change_abs_roll_mean_500"]) + (data["fftr_min_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_std_20"]))) * 2.0)))) 
        v["i181"] = 0.010000*np.tanh(((((((((((((((((((data["autocorrelation_5000"]) + (data["av_change_abs_roll_std_10"]))) * 2.0)) * (data["ffti_range_1000_2000"]))) * 2.0)) - (data["fftr_range_-3000_-2000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i182"] = 0.010000*np.tanh(((((((data["abs_percentile_20"]) + (((((((data["autocorrelation_5"]) + ((-1.0*((((data["fftr_mean_last_1000"]) * 2.0))))))) - (data["abs_percentile_20"]))) * 2.0)))) - (data["fftr_range_-3000_-2000"]))) * 2.0)) 
        v["i183"] = 0.010000*np.tanh(((data["percentile_roll_std_80"]) + (((((data["mean_change_rate_first_10000"]) - (((((data["fftr_classic_sta_lta5_mean"]) * (((data["fftr_percentile_roll_std_25"]) + (data["fftr_percentile_roll_std_20"]))))) * 2.0)))) * 2.0)))) 
        v["i184"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_40"]) * 2.0)) + (data["autocorrelation_50"]))) * 2.0)) + (((((data["autocorrelation_50"]) + (((data["ffti_percentile_roll_std_1"]) + (data["autocorrelation_50"]))))) * 2.0)))) * 2.0)) 
        v["i185"] = 0.010000*np.tanh(((((((data["mean_first_1000"]) + (((data["exp_Moving_average_30000_std"]) + (((((data["autocorrelation_50"]) + (data["percentile_roll_std_5"]))) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i186"] = 0.010000*np.tanh(((((((data["percentile_60"]) * (data["fftr_abs_percentile_20"]))) + (((((data["ffti_autocorrelation_10000"]) * 2.0)) + (((data["num_peaks_20"]) * (data["num_peaks_20"]))))))) * 2.0)) 
        v["i187"] = 0.010000*np.tanh(((((((((data["fftr_percentile_roll_std_25"]) + (((data["percentile_roll_std_70"]) - (((data["fftr_percentile_roll_std_25"]) * (((((((data["percentile_roll_std_5"]) * 2.0)) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i188"] = 0.010000*np.tanh(((((((data["classic_sta_lta3_mean"]) + (((((((data["fftr_hmean"]) + (data["ffti_percentile_roll_mean_95"]))) * 2.0)) * 2.0)))) - (data["fftr_range_-3000_-2000"]))) + (data["ffti_percentile_roll_mean_95"]))) 
        v["i189"] = 0.010000*np.tanh(((((((((((((((data["ffti_autocorrelation_10"]) * (((data["fftr_percentile_roll_std_25"]) + (((data["percentile_roll_std_80"]) * 2.0)))))) * 2.0)) + (data["fftr_max_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i190"] = 0.010000*np.tanh(((((((((((((((((((((((data["fftr_range_3000_4000"]) - (data["fftr_range_2000_3000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i191"] = 0.010000*np.tanh(((data["fftr_hmean"]) * (((((data["fftr_autocorrelation_50"]) + (((((data["ffti_min_roll_std_100"]) + (data["autocorrelation_100"]))) + (data["ffti_hmean"]))))) + (data["fftr_autocorrelation_50"]))))) 
        v["i192"] = 0.010000*np.tanh(((((((((((data["av_change_rate_roll_std_10"]) + (((data["autocorrelation_1000"]) * (((data["ffti_Hann_window_mean_50"]) * ((-1.0*((((data["fftr_abs_max_roll_mean_50"]) * 2.0))))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i193"] = 0.010000*np.tanh(((((((((((((data["autocorrelation_5"]) * (data["min_last_10000"]))) * 2.0)) * 2.0)) * 2.0)) + (((((data["ffti_av_change_abs_roll_std_10000"]) * (data["med"]))) * 2.0)))) * 2.0)) 
        v["i194"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_1"]) + (((data["mean_first_50000"]) * (((((((data["autocorrelation_1000"]) * 2.0)) + (data["mean_first_50000"]))) - (data["fftr_time_rev_asym_stat_1"]))))))) 
        v["i195"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_100"]) * ((-1.0*((data["ffti_mean_change_rate"])))))) + ((((((-1.0*((((data["ffti_mean_change_rate"]) * 2.0))))) + (data["percentile_75"]))) * 2.0)))) * 2.0)) 
        v["i196"] = 0.010000*np.tanh(((((((((((((data["fftr_abs_percentile_1"]) + (data["fftr_num_peaks_100"]))) + (((data["ffti_kstat_1"]) * (((((data["fftr_num_peaks_100"]) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i197"] = 0.010000*np.tanh(((((((((((((((((((data["ffti_autocorrelation_10000"]) * 2.0)) * 2.0)) + (((data["ffti_autocorrelation_10000"]) * 2.0)))) * 2.0)) * (data["ffti_range_3000_4000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i198"] = 0.010000*np.tanh(((((data["ffti_ave_roll_mean_10"]) + (((((((((((data["av_change_abs_roll_std_50"]) - (((data["ffti_ave_roll_mean_10"]) * (data["fftr_mean_first_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i199"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_std_40"]) * (((((((((data["fftr_time_rev_asym_stat_10"]) + (((data["ffti_min_roll_std_10"]) * 2.0)))) + (data["mean_change_rate_last_10000"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i200"] = 0.010000*np.tanh(((((((data["fftr_abs_percentile_1"]) * 2.0)) + (((((((data["fftr_range_2000_3000"]) * (((data["min_roll_std_500"]) + (data["mean_last_10000"]))))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i201"] = 0.010000*np.tanh(((((((((((((((((data["ffti_min_roll_std_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["ffti_min_roll_std_50"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i202"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_500"]) - (((data["mean_change_rate_last_10000"]) * ((-1.0*((((((((data["mean_first_10000"]) + (data["mean_first_10000"]))) + (data["mean_first_10000"]))) + (data["fftr_time_rev_asym_stat_1"])))))))))) * 2.0)) 
        v["i203"] = 0.010000*np.tanh(((((((((((((data["autocorrelation_10000"]) * (data["sum"]))) * 2.0)) + (data["ffti_skew"]))) * 2.0)) + (data["ffti_skew"]))) * 2.0)) 
        v["i204"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_40"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i205"] = 0.010000*np.tanh(((((((((((((((((((data["ffti_percentile_roll_std_30"]) * (data["kurt"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((data["fftr_mean_last_10000"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i206"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_5000"]) + (((data["av_change_abs_roll_mean_100"]) * (((((((data["ffti_percentile_roll_std_1"]) + (data["fftr_time_rev_asym_stat_5"]))) + (data["max_last_10000"]))) + (data["fftr_autocorrelation_5000"]))))))) * 2.0)) * 2.0)) 
        v["i207"] = 0.010000*np.tanh(((((((data["percentile_roll_std_70"]) + (((((data["fftr_std_roll_mean_10000"]) * (((data["num_peaks_50"]) + (((data["num_peaks_50"]) + (data["ffti_abs_max_roll_mean_10000"]))))))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i208"] = 0.010000*np.tanh(((((((data["percentile_roll_std_5"]) + (((((((((data["mean_change_rate_last_10000"]) * (((((data["percentile_75"]) - (data["ffti_mean_change_abs"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i209"] = 0.010000*np.tanh(((((((((((((data["fftr_mean_last_1000"]) * 2.0)) * 2.0)) * (((data["ffti_min_roll_std_50"]) + (data["ffti_autocorrelation_5000"]))))) - (data["ffti_percentile_roll_std_80"]))) * 2.0)) * 2.0)) 
        v["i210"] = 0.010000*np.tanh(((data["percentile_roll_std_80"]) + (((((((data["mean_change_rate_first_1000"]) + (((((((data["ffti_av_change_abs_roll_mean_50"]) * (((data["fftr_percentile_roll_std_30"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i211"] = 0.010000*np.tanh(((((((((((((((((data["fftr_abs_max_roll_mean_10000"]) * (((data["num_peaks_100"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["classic_sta_lta2_mean"]))) * 2.0)) * 2.0)) 
        v["i212"] = 0.010000*np.tanh(((((((((data["autocorrelation_50"]) * 2.0)) + (((data["percentile_roll_std_5"]) + (((data["percentile_roll_std_5"]) * (((((data["autocorrelation_50"]) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i213"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_30"]) * 2.0)) * ((((((-1.0*((((((data["ffti_exp_Moving_average_30000_mean"]) * 2.0)) * 2.0))))) - (((data["classic_sta_lta1_mean"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i214"] = 0.010000*np.tanh(((data["ffti_trend"]) - (((((((((data["fftr_abs_percentile_10"]) + (((data["fftr_mean_last_1000"]) + (((data["fftr_mean_last_1000"]) - (data["ffti_autocorrelation_10000"]))))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i215"] = 0.010000*np.tanh(((data["percentile_roll_std_70"]) + (((((((data["fftr_abs_max_roll_mean_50"]) + (((data["percentile_roll_std_5"]) * 2.0)))) + (((((data["percentile_roll_std_40"]) * 2.0)) * 2.0)))) + (data["fftr_exp_Moving_average_50000_mean"]))))) 
        v["i216"] = 0.010000*np.tanh(((((((((((((data["fftr_min_roll_std_10000"]) * (((((data["percentile_roll_std_40"]) * 2.0)) * 2.0)))) + (data["fftr_percentile_roll_std_25"]))) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_std_25"]))) 
        v["i217"] = 0.010000*np.tanh(((((((data["ffti_max_to_min"]) + (((data["ffti_num_peaks_20"]) * (data["classic_sta_lta3_mean"]))))) * 2.0)) + (((((data["ffti_kstatvar_1"]) - (data["fftr_abs_percentile_25"]))) + (((data["fftr_min_roll_std_500"]) * 2.0)))))) 
        v["i218"] = 0.010000*np.tanh(((((((((((((((data["ffti_skew"]) * (((data["ffti_ave_roll_mean_500"]) - (data["ffti_autocorrelation_5"]))))) * 2.0)) * 2.0)) + (data["ffti_skew"]))) * 2.0)) * 2.0)) + (data["ffti_ave_roll_mean_500"]))) 
        v["i219"] = 0.010000*np.tanh(((((((((((((data["fftr_av_change_abs_roll_mean_50"]) * (((((((data["ffti_mean_change_abs"]) * (data["fftr_time_rev_asym_stat_10"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) - (data["fftr_abs_percentile_25"]))) * 2.0)) * 2.0)) 
        v["i220"] = 0.010000*np.tanh(((((((((((((((((((data["fftr_percentile_50"]) * (data["max_to_min"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i221"] = 0.010000*np.tanh(((((((((((((((data["fftr_min_roll_std_50"]) - (((data["classic_sta_lta2_mean"]) + (data["fftr_min_roll_mean_10000"]))))) + (data["fftr_min_roll_std_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i222"] = 0.010000*np.tanh((((((((((((data["ffti_min_roll_mean_10000"]) * (((data["classic_sta_lta3_mean"]) * 2.0)))) * 2.0)) * 2.0)) - (data["fftr_range_-3000_-2000"]))) + (data["classic_sta_lta3_mean"]))/2.0)) 
        v["i223"] = 0.010000*np.tanh((((((12.54508113861083984)) + (((data["percentile_roll_std_70"]) * ((12.54508113861083984)))))) * (((data["mean_first_1000"]) + (((data["percentile_roll_std_70"]) * ((12.54508113861083984)))))))) 
        v["i224"] = 0.010000*np.tanh(((((data["classic_sta_lta5_mean"]) + (((((((((data["percentile_roll_std_70"]) + (((data["autocorrelation_50"]) * (((data["percentile_roll_std_40"]) + (data["ffti_mean_change_rate"]))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i225"] = 0.010000*np.tanh(((((((((data["mean_change_rate_last_1000"]) * (((data["num_peaks_20"]) * 2.0)))) * 2.0)) * 2.0)) - (((data["ffti_mean_change_rate"]) - (((data["num_peaks_20"]) * (data["num_peaks_20"]))))))) 
        v["i226"] = 0.010000*np.tanh(((data["fftr_abs_percentile_5"]) + (((((data["ffti_autocorrelation_10000"]) * (((((data["fftr_abs_percentile_5"]) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i227"] = 0.010000*np.tanh(((((((data["av_change_abs_roll_mean_500"]) - (((((data["ffti_autocorrelation_10"]) - (((((data["ffti_mean_last_10000"]) - (((data["fftr_mean_last_10000"]) * (data["av_change_abs_roll_mean_500"]))))) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i228"] = 0.010000*np.tanh(((((((((((((((data["ffti_skew"]) + (data["fftr_abs_max_roll_mean_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["ffti_mean_change_rate"]))) * 2.0)) 
        v["i229"] = 0.010000*np.tanh(((((((((data["fftr_min_roll_std_10"]) + (data["percentile_20"]))) * 2.0)) + (data["fftr_min_roll_std_10"]))) * 2.0)) 
        v["i230"] = 0.010000*np.tanh(((((((((data["exp_Moving_average_50000_std"]) * (((data["av_change_abs_roll_mean_10"]) - (((data["autocorrelation_1000"]) - (data["av_change_abs_roll_mean_10"]))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i231"] = 0.010000*np.tanh(((((((((((((((((((data["av_change_abs_roll_std_10"]) - (data["fftr_percentile_roll_mean_90"]))) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_mean_90"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i232"] = 0.010000*np.tanh(((((((data["fftr_time_rev_asym_stat_50"]) * (((((((((((data["fftr_av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (data["ffti_av_change_abs_roll_std_10000"]))) + (data["classic_sta_lta3_mean"]))) 
        v["i233"] = 0.010000*np.tanh(((((((((((((data["fftr_time_rev_asym_stat_1"]) - (data["ffti_min_roll_std_50"]))) * 2.0)) * 2.0)) * (((data["fftr_min_roll_std_10"]) + (((data["fftr_percentile_25"]) * 2.0)))))) + (data["fftr_percentile_25"]))) * 2.0)) 
        v["i234"] = 0.010000*np.tanh(((((((((((((((((data["ffti_ave_roll_mean_10"]) * (((data["fftr_percentile_roll_mean_40"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i235"] = 0.010000*np.tanh((((6.0)) * ((((((((((((data["ffti_min_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)) * ((-1.0*((data["fftr_mean_first_1000"])))))) + (data["min_roll_std_10"]))/2.0)))) 
        v["i236"] = 0.010000*np.tanh(((((((((((((((((data["av_change_abs_roll_std_10"]) + (data["ffti_trend"]))) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_std_30"]))) + (data["fftr_percentile_roll_std_30"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i237"] = 0.010000*np.tanh(((((((((data["ffti_av_change_abs_roll_std_10000"]) * (((((data["percentile_50"]) * 2.0)) * 2.0)))) - (data["ffti_autocorrelation_50"]))) * 2.0)) * 2.0)) 
        v["i238"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_mean_70"]) * (((data["fftr_num_peaks_10"]) * (((((data["av_change_abs_roll_mean_100"]) * 2.0)) * 2.0)))))) - (data["fftr_num_peaks_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i239"] = 0.010000*np.tanh(((((((data["ffti_std_roll_mean_1000"]) + (((((data["ffti_abs_percentile_10"]) - (data["autocorrelation_5"]))) * (((data["autocorrelation_500"]) + (data["fftr_exp_Moving_average_30000_std"]))))))) * 2.0)) * 2.0)) 
        v["i240"] = 0.010000*np.tanh(((((((data["percentile_roll_std_1"]) + (data["ffti_exp_Moving_average_3000_std"]))) + (((((((data["percentile_roll_std_5"]) * 2.0)) * 2.0)) + (data["fftr_abs_max_roll_mean_10"]))))) * 2.0)) 
        v["i241"] = 0.010000*np.tanh(((((((((data["percentile_20"]) * (((((((((data["ffti_num_peaks_10"]) * (data["fftr_percentile_30"]))) * 2.0)) * 2.0)) - (data["ffti_av_change_abs_roll_std_50"]))))) * 2.0)) * 2.0)) - (data["ffti_av_change_abs_roll_std_50"]))) 
        v["i242"] = 0.010000*np.tanh(((((((((((((((((data["ffti_percentile_roll_std_5"]) * ((-1.0*((((data["fftr_time_rev_asym_stat_1"]) + (data["ffti_av_change_abs_roll_std_100"])))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i243"] = 0.010000*np.tanh(((((((((((((data["ffti_kstat_1"]) * 2.0)) * (((((data["mean_first_10000"]) + (data["fftr_num_peaks_100"]))) - (data["fftr_ave_roll_mean_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i244"] = 0.010000*np.tanh(((data["percentile_75"]) + (((((data["fftr_hmean"]) * (((((((((((((data["autocorrelation_100"]) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_hmean"]))) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i245"] = 0.010000*np.tanh(((((((((data["classic_sta_lta3_mean"]) + (((((data["percentile_roll_std_70"]) + (((data["fftr_hmean"]) * (((data["ffti_min_roll_std_100"]) * 2.0)))))) * 2.0)))) + (data["ffti_min_roll_std_100"]))) * 2.0)) * 2.0)) 
        v["i246"] = 0.010000*np.tanh(((((((((((data["percentile_roll_mean_70"]) + ((((-1.0*(((((data["mean_first_1000"]) + (data["fftr_mean_last_1000"]))/2.0))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i247"] = 0.010000*np.tanh((((((((((-1.0*((data["percentile_80"])))) * (((((data["autocorrelation_100"]) * 2.0)) * 2.0)))) + (data["ffti_classic_sta_lta1_mean"]))) * 2.0)) * 2.0)) 
        v["i248"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_std_25"]) + (data["ffti_autocorrelation_10000"]))) - (((data["classic_sta_lta8_mean"]) * (((data["fftr_abs_percentile_1"]) + (((((((data["fftr_abs_percentile_1"]) * 2.0)) * 2.0)) * 2.0)))))))) 
        v["i249"] = 0.010000*np.tanh(((data["autocorrelation_5"]) + (((((((((((data["fftr_num_peaks_100"]) - (data["ffti_mean_change_rate"]))) * 2.0)) * 2.0)) + (((((data["exp_Moving_average_50000_std"]) * (data["ffti_max_roll_mean_500"]))) * 2.0)))) * 2.0)))) 
        v["i250"] = 0.010000*np.tanh(((((((((data["gmean"]) * (((data["percentile_roll_std_70"]) - (((((data["autocorrelation_5"]) + (data["fftr_time_rev_asym_stat_1"]))) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i251"] = 0.010000*np.tanh(((((data["fftr_range_3000_4000"]) * (((((data["mean_first_1000"]) + (((((((((data["min_roll_std_500"]) * 2.0)) + (data["mean_first_1000"]))) + (data["min_roll_std_10"]))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i252"] = 0.010000*np.tanh(((((data["ffti_num_peaks_10"]) * (((((((((((data["fftr_skew"]) * ((7.0)))) * ((7.0)))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i253"] = 0.010000*np.tanh((((((((-1.0*((((((((data["fftr_percentile_roll_std_60"]) * (data["percentile_roll_mean_80"]))) * 2.0)) * 2.0))))) - (((data["fftr_percentile_roll_mean_95"]) * 2.0)))) * 2.0)) + (data["fftr_percentile_roll_std_60"]))) 
        v["i254"] = 0.010000*np.tanh(((((data["fftr_min_last_1000"]) * (((((data["ffti_range_2000_3000"]) + (((data["med"]) * 2.0)))) + (((((data["med"]) * 2.0)) - (data["fftr_range_-3000_-2000"]))))))) * 2.0)) 
        v["i255"] = 0.010000*np.tanh(((((((data["av_change_abs_roll_mean_500"]) - (((((data["ffti_percentile_roll_mean_1"]) + (((data["autocorrelation_5"]) * 2.0)))) * (data["fftr_min_roll_std_10"]))))) * 2.0)) * 2.0)) 
        v["i256"] = 0.010000*np.tanh(((((data["ffti_std_roll_mean_1000"]) + (((((data["abs_percentile_60"]) - (((data["ffti_std_roll_mean_1000"]) * (((((((data["classic_sta_lta3_mean"]) * 2.0)) * 2.0)) * 2.0)))))) * 2.0)))) * 2.0)) 
        v["i257"] = 0.010000*np.tanh(((((((((((((((((data["ffti_classic_sta_lta1_mean"]) - (data["fftr_range_p4000_pinf"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i258"] = 0.010000*np.tanh(((((data["mean_change_rate_last_1000"]) * (((((((data["fftr_time_rev_asym_stat_100"]) + (data["num_peaks_50"]))) + (((data["num_peaks_20"]) * 2.0)))) * 2.0)))) + (((data["num_peaks_20"]) * (data["num_peaks_20"]))))) 
        v["i259"] = 0.010000*np.tanh(((((((((((((data["abs_percentile_40"]) * (data["percentile_roll_std_60"]))) + (((data["ffti_exp_Moving_average_3000_std"]) + (((data["percentile_roll_std_70"]) + (data["ffti_min_roll_std_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i260"] = 0.010000*np.tanh(((((((((((((data["ffti_ave10"]) * (((((data["spkt_welch_density_100"]) + (data["autocorrelation_10000"]))) + (((data["spkt_welch_density_100"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i261"] = 0.010000*np.tanh(((((((data["min_roll_mean_10000"]) * (((((((data["ffti_sum"]) - (data["fftr_mean"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i262"] = 0.010000*np.tanh(((((((((data["fftr_abs_percentile_1"]) + (((data["fftr_percentile_roll_std_50"]) + (((data["fftr_med"]) * (data["fftr_num_peaks_100"]))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i263"] = 0.010000*np.tanh(((((((((data["fftr_min_roll_std_100"]) * 2.0)) * (((data["fftr_autocorrelation_500"]) - (((data["ffti_kstat_4"]) - (data["fftr_autocorrelation_5000"]))))))) * 2.0)) * 2.0)) 
        v["i264"] = 0.010000*np.tanh((((10.0)) * (((((((data["fftr_min_roll_std_10000"]) * (data["fftr_abs_percentile_5"]))) * 2.0)) - (((data["ffti_abs_percentile_1"]) * (data["fftr_min_roll_std_500"]))))))) 
        v["i265"] = 0.010000*np.tanh(((((((data["fftr_std_roll_mean_10000"]) * (((data["fftr_std_roll_mean_10000"]) + (((data["num_peaks_100"]) + (((((data["fftr_mean_first_50000"]) * 2.0)) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i266"] = 0.010000*np.tanh(((((data["ffti_ave_roll_mean_10"]) * (data["fftr_time_rev_asym_stat_10"]))) + (((((data["fftr_Hann_window_mean_1500"]) + (((((((data["ffti_ave_roll_mean_500"]) - (data["ffti_ave_roll_mean_10"]))) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i267"] = 0.010000*np.tanh(((((((data["ffti_av_change_abs_roll_mean_10"]) * (((((((((data["fftr_abs_max_roll_std_10"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * (data["autocorrelation_1000"]))) * 2.0)) 
        v["i268"] = 0.010000*np.tanh(((((data["fftr_std_roll_mean_500"]) + (((data["classic_sta_lta8_mean"]) + (((data["fftr_time_rev_asym_stat_1"]) * (((((data["min_first_50000"]) - (((data["ave_roll_mean_10"]) - (data["classic_sta_lta8_mean"]))))) * 2.0)))))))) * 2.0)) 
        v["i269"] = 0.010000*np.tanh(((((((((((((((((data["autocorrelation_5"]) - (((data["num_peaks_20"]) * (((data["autocorrelation_5"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i270"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_25"]) + (data["fftr_percentile_roll_mean_40"]))) + (((((data["ffti_abs_percentile_1"]) * 2.0)) * (((data["ffti_abs_percentile_1"]) - (((((data["fftr_percentile_roll_std_25"]) * 2.0)) * 2.0)))))))) * 2.0)) 
        v["i271"] = 0.010000*np.tanh(((((((((((data["fftr_abs_max_roll_mean_10"]) * (data["percentile_roll_mean_90"]))) - (((data["iqr"]) * (((data["autocorrelation_5"]) + (data["percentile_roll_mean_30"]))))))) * 2.0)) + (data["percentile_roll_std_70"]))) * 2.0)) 
        v["i272"] = 0.010000*np.tanh(((((((((((((((data["fftr_num_peaks_100"]) - (((((data["mean_last_10000"]) * (((data["fftr_num_peaks_100"]) * 2.0)))) / 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i273"] = 0.010000*np.tanh(((((((((((((((((((data["ffti_percentile_roll_mean_75"]) * (((data["ffti_exp_Moving_average_50000_mean"]) - (data["ffti_percentile_roll_std_5"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i274"] = 0.010000*np.tanh(((((((((data["fftr_time_rev_asym_stat_10"]) * 2.0)) + (((data["autocorrelation_1000"]) - ((((-1.0*((data["percentile_70"])))) * (((data["fftr_range_2000_3000"]) * 2.0)))))))) * 2.0)) + (data["ffti_exp_Moving_average_30000_std"]))) 
        v["i275"] = 0.010000*np.tanh(((((((((data["fftr_num_peaks_50"]) * ((((((((data["abs_percentile_60"]) + (data["fftr_abs_percentile_1"]))/2.0)) * 2.0)) * 2.0)))) + (((data["fftr_mean_first_1000"]) * (data["mean_first_10000"]))))) * 2.0)) * 2.0)) 
        v["i276"] = 0.010000*np.tanh(((((data["ffti_abs_percentile_1"]) * 2.0)) * (((((data["ffti_abs_percentile_1"]) * 2.0)) - (((((((((data["ffti_Hann_window_mean_15000"]) - (data["fftr_time_rev_asym_stat_5"]))) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i277"] = 0.010000*np.tanh(((((((data["fftr_time_rev_asym_stat_5"]) * (((((data["ffti_av_change_abs_roll_std_50"]) - (((data["abs_percentile_60"]) - (((data["ffti_autocorrelation_1000"]) * 2.0)))))) - (data["ffti_min_roll_std_10"]))))) * 2.0)) * 2.0)) 
        v["i278"] = 0.010000*np.tanh(((data["ffti_av_change_abs_roll_std_10000"]) * (((((((((((data["percentile_60"]) * 2.0)) - (data["abs_percentile_50"]))) * 2.0)) - (data["percentile_roll_std_5"]))) * 2.0)))) 
        v["i279"] = 0.010000*np.tanh(((((((((((((data["ffti_percentile_roll_std_5"]) - (((data["ffti_av_change_abs_roll_mean_50"]) * (data["fftr_percentile_roll_mean_30"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i280"] = 0.010000*np.tanh(((data["mean_change_rate_last_50000"]) * (((((((((((data["num_peaks_20"]) + (((data["fftr_percentile_roll_std_20"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) + (data["max_first_10000"]))))) 
        v["i281"] = 0.010000*np.tanh(((((data["fftr_num_peaks_50"]) * (((((((((((data["ffti_percentile_roll_std_25"]) * 2.0)) * 2.0)) * 2.0)) + (((data["ffti_exp_Moving_average_3000_std"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i282"] = 0.010000*np.tanh(((data["fftr_mean_first_1000"]) - (((((((((((((((((((data["ffti_Hann_window_mean_150"]) * (data["fftr_abs_percentile_1"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i283"] = 0.010000*np.tanh(((((((data["fftr_ave_roll_mean_10000"]) * (((data["ffti_autocorrelation_100"]) * 2.0)))) + (((((data["ffti_kstat_1"]) * (data["percentile_20"]))) + (data["autocorrelation_1000"]))))) * 2.0)) 
        v["i284"] = 0.010000*np.tanh(((((((((((((data["fftr_range_-3000_-2000"]) + (data["fftr_range_-3000_-2000"]))) * (((((data["percentile_60"]) * 2.0)) + (data["fftr_hmean"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i285"] = 0.010000*np.tanh((((((((((((13.42621898651123047)) * (((data["fftr_min_roll_std_500"]) * (data["mean_first_1000"]))))) * (data["mean_change_rate_last_1000"]))) - (data["fftr_abs_percentile_20"]))) - (data["fftr_abs_percentile_20"]))) * 2.0)) 
        v["i286"] = 0.010000*np.tanh((((((9.23430442810058594)) + (data["ffti_mean_change_rate"]))) * (((data["fftr_time_rev_asym_stat_50"]) * ((((9.0)) * (((data["ffti_ave_roll_mean_10000"]) - (data["mean_change_rate_first_50000"]))))))))) 
        v["i287"] = 0.010000*np.tanh(((data["percentile_80"]) + (((((((((data["min_roll_std_10000"]) + (((data["fftr_time_rev_asym_stat_5"]) * (((((data["fftr_abs_percentile_10"]) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) + (data["min_roll_std_10000"]))))) 
        v["i288"] = 0.010000*np.tanh(((((((((((((((((data["ffti_autocorrelation_10000"]) * (((data["fftr_abs_percentile_10"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) + (data["ffti_exp_Moving_average_3000_std"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i289"] = 0.010000*np.tanh(((((((data["mean_first_10000"]) + (data["fftr_autocorrelation_5000"]))) + (((data["fftr_percentile_roll_std_20"]) + (((((data["fftr_percentile_roll_std_25"]) * 2.0)) + (data["fftr_autocorrelation_5000"]))))))) * (((data["percentile_10"]) * 2.0)))) 
        v["i290"] = 0.010000*np.tanh((((((((((((((-1.0*((((data["Hann_window_mean_15000"]) * (((data["ffti_min_roll_std_1000"]) * 2.0))))))) * 2.0)) + (data["fftr_abs_max_roll_mean_500"]))) * 2.0)) * 2.0)) + (data["fftr_min_roll_std_10"]))) * 2.0)) 
        v["i291"] = 0.010000*np.tanh(((((((((data["autocorrelation_5"]) * 2.0)) + (((((((((data["ffti_percentile_roll_mean_1"]) * (data["classic_sta_lta3_mean"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) + (data["percentile_roll_std_5"]))) 
        v["i292"] = 0.010000*np.tanh(((data["ffti_ave_roll_mean_10"]) + (((((((((data["fftr_percentile_roll_mean_30"]) + (((((((((data["fftr_percentile_roll_std_10"]) * 2.0)) * (data["fftr_percentile_roll_mean_30"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i293"] = 0.010000*np.tanh(((((((data["num_peaks_20"]) * (data["mean_change_rate_last_50000"]))) * 2.0)) + (((((((((((data["num_peaks_20"]) * (data["num_peaks_20"]))) - (data["ffti_abs_percentile_1"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i294"] = 0.010000*np.tanh((((((((data["ffti_autocorrelation_10000"]) + (((((((((((data["std_roll_mean_10000"]) * (((data["ffti_autocorrelation_1000"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))/2.0)) * 2.0)) * 2.0)) 
        v["i295"] = 0.010000*np.tanh(((((((data["fftr_time_rev_asym_stat_50"]) * (((data["fftr_abs_max"]) - ((((8.0)) * (((((data["Hann_window_mean_150"]) - (data["autocorrelation_50"]))) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i296"] = 0.010000*np.tanh(((((((((data["percentile_40"]) * (data["ffti_autocorrelation_100"]))) * 2.0)) * 2.0)) + (((data["ffti_percentile_roll_std_40"]) + ((((((data["ffti_percentile_roll_std_40"]) + (data["ffti_percentile_roll_std_40"]))/2.0)) * (data["ffti_autocorrelation_100"]))))))) 
        v["i297"] = 0.010000*np.tanh(((((((((((data["autocorrelation_5"]) + (((((((((((data["autocorrelation_5"]) + (data["fftr_min_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i298"] = 0.010000*np.tanh(((((((((data["fftr_min_roll_std_500"]) + (((data["fftr_percentile_25"]) - (((data["max_to_min"]) * 2.0)))))) + (data["ffti_autocorrelation_5"]))) * 2.0)) + (((data["ffti_autocorrelation_5"]) - (data["ffti_min_roll_std_100"]))))) 
        v["i299"] = 0.010000*np.tanh(((((((((((((((((((data["ffti_range_2000_3000"]) * (((data["min_roll_std_500"]) - (data["abs_max_roll_std_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i300"] = 0.010000*np.tanh(((((((((((((data["autocorrelation_50"]) + (data["ffti_percentile_roll_std_1"]))) + (data["percentile_roll_std_5"]))) * 2.0)) * 2.0)) + (((((data["percentile_roll_std_5"]) * 2.0)) * 2.0)))) * 2.0)) 
        v["i301"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_50"]) * (((((data["fftr_percentile_roll_mean_50"]) - (((((data["ffti_mean"]) / 2.0)) * 2.0)))) - (((((data["ffti_mean"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i302"] = 0.010000*np.tanh(((((data["mean_first_10000"]) + (((((data["fftr_ave_roll_mean_10"]) + (((((((data["mean_first_10000"]) * 2.0)) * (((data["mean_last_1000"]) * 2.0)))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i303"] = 0.010000*np.tanh(((((((((((data["mean_last_1000"]) * (((((((data["autocorrelation_10000"]) + (data["ffti_max_roll_mean_100"]))) * 2.0)) + (data["mean_last_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i304"] = 0.010000*np.tanh(((((data["autocorrelation_1000"]) + (((((((data["autocorrelation_500"]) * (((((data["percentile_roll_mean_40"]) + (((data["ffti_percentile_75"]) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)))) + (data["autocorrelation_5"]))) 
        v["i305"] = 0.010000*np.tanh(((((((((data["fftr_time_rev_asym_stat_50"]) + (((((((data["percentile_75"]) - (data["num_peaks_50"]))) * 2.0)) * 2.0)))) * 2.0)) + (data["fftr_time_rev_asym_stat_50"]))) * 2.0)) 
        v["i306"] = 0.010000*np.tanh(((((((((((data["ffti_exp_Moving_average_50000_mean"]) * (((((data["ffti_percentile_roll_mean_90"]) - (data["fftr_mean_last_50000"]))) - (((data["fftr_abs_percentile_1"]) + (data["autocorrelation_1000"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i307"] = 0.010000*np.tanh(((data["fftr_percentile_roll_std_20"]) * (((((((((((((((data["mean_change_rate"]) + (data["mean_change_rate_first_10000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - ((-1.0*((data["mean_change_rate_first_10000"])))))))) 
        v["i308"] = 0.010000*np.tanh(((((((data["exp_Moving_average_50000_std"]) * 2.0)) * 2.0)) * (((((data["ffti_ave_roll_mean_10000"]) - (data["autocorrelation_1000"]))) + (((data["fftr_std_roll_mean_50"]) - (((data["exp_Moving_average_50000_std"]) - (data["classic_sta_lta2_mean"]))))))))) 
        v["i309"] = 0.010000*np.tanh(((((((((((((((((data["fftr_percentile_roll_mean_20"]) * (((data["ffti_autocorrelation_100"]) + (data["percentile_roll_std_80"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i310"] = 0.010000*np.tanh(((((data["mean_first_1000"]) * (((data["percentile_roll_std_5"]) * 2.0)))) + (((((data["ffti_exp_Moving_average_3000_std"]) + (((data["fftr_mean_first_1000"]) * (((data["ffti_autocorrelation_50"]) + (data["fftr_mean_first_1000"]))))))) * 2.0)))) 
        v["i311"] = 0.010000*np.tanh(((data["mean_first_10000"]) * ((((((((((((data["fftr_autocorrelation_500"]) + (((((data["fftr_max_roll_mean_50"]) + (data["mean_change_rate_last_10000"]))) * 2.0)))) * 2.0)) * 2.0)) + (data["mean_change_rate_last_10000"]))/2.0)) * 2.0)))) 
        v["i312"] = 0.010000*np.tanh(((((data["fftr_num_peaks_100"]) * (((((((((((((data["fftr_mean_last_1000"]) * 2.0)) - (data["percentile_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i313"] = 0.010000*np.tanh(((((((data["fftr_ave10"]) - (((data["fftr_time_rev_asym_stat_5"]) * (((data["fftr_time_rev_asym_stat_50"]) + (((data["percentile_80"]) + (((data["percentile_80"]) + (data["hmean"]))))))))))) * 2.0)) * 2.0)) 
        v["i314"] = 0.010000*np.tanh(((((data["min_roll_std_1000"]) + (((data["ffti_std_roll_std_1000"]) + (((((((data["min_roll_std_1000"]) * (((data["autocorrelation_50"]) - (data["autocorrelation_5"]))))) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i315"] = 0.010000*np.tanh(((((((((((data["autocorrelation_5"]) - (data["ffti_percentile_roll_std_30"]))) * 2.0)) + (((data["fftr_min_roll_std_50"]) + (((data["fftr_num_peaks_50"]) * (data["fftr_min_roll_std_50"]))))))) * 2.0)) * 2.0)) 
        v["i316"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_100"]) + (data["classic_sta_lta3_mean"]))) * (((data["ffti_min_roll_mean_10000"]) + (((((data["ffti_num_peaks_20"]) + (data["ffti_min_roll_mean_10000"]))) * 2.0)))))) * 2.0)) 
        v["i317"] = 0.010000*np.tanh(((((((((((((((data["percentile_roll_std_1"]) + (data["ffti_percentile_roll_mean_50"]))) * 2.0)) * 2.0)) + (((data["ffti_mean_change_rate"]) * (data["percentile_roll_std_1"]))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i318"] = 0.010000*np.tanh(((((((((((((data["exp_Moving_average_50000_std"]) * ((((((data["av_change_abs_roll_mean_10"]) + (data["fftr_percentile_roll_mean_70"]))/2.0)) - (data["autocorrelation_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i319"] = 0.010000*np.tanh(((((((((((data["percentile_90"]) * (((data["fftr_range_2000_3000"]) - (data["autocorrelation_5"]))))) * 2.0)) + (data["percentile_roll_std_5"]))) * 2.0)) * 2.0)) 
        v["i320"] = 0.010000*np.tanh(((((((data["mean_last_10000"]) * 2.0)) * ((((((((((-1.0*((((data["fftr_min_roll_std_1000"]) * 2.0))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i321"] = 0.010000*np.tanh((((((((-1.0*((((data["autocorrelation_5"]) * 2.0))))) * 2.0)) * (((data["ffti_count_big"]) * 2.0)))) + (((data["ffti_autocorrelation_5000"]) + (data["ffti_std_roll_mean_10000"]))))) 
        v["i322"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_1000"]) / 2.0)) * ((-1.0*((((((((((((((((data["med"]) * 2.0)) - (data["ffti_min_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0))))))) 
        v["i323"] = 0.010000*np.tanh((((((((data["percentile_roll_std_70"]) + (((((data["ffti_autocorrelation_5000"]) * (((data["fftr_mean_last_1000"]) + ((((((data["fftr_mean_last_1000"]) * 2.0)) + (data["ffti_autocorrelation_5000"]))/2.0)))))) * 2.0)))/2.0)) * 2.0)) * 2.0)) 
        v["i324"] = 0.010000*np.tanh(((((data["fftr_abs_percentile_1"]) + (((((((((((((((((data["ffti_autocorrelation_50"]) * (data["ffti_num_peaks_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i325"] = 0.010000*np.tanh(((((((((((data["ffti_av_change_abs_roll_std_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (((data["av_change_abs_roll_mean_10000"]) + ((((((data["fftr_abs_max_roll_mean_10"]) + (data["av_change_rate_roll_mean_500"]))) + (data["fftr_abs_max_roll_mean_10"]))/2.0)))))) 
        v["i326"] = 0.010000*np.tanh(((data["ffti_autocorrelation_10"]) * (((((((data["ffti_exp_Moving_average_50000_std"]) + (data["fftr_mean_first_10000"]))) - (data["ffti_mean_change_abs"]))) - (((data["ffti_mean_change_abs"]) - (data["fftr_abs_percentile_10"]))))))) 
        v["i327"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_std_60"]) * (data["ffti_autocorrelation_5"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((((data["percentile_roll_std_60"]) * 2.0)) * 2.0)))) 
        v["i328"] = 0.010000*np.tanh((((((((((-1.0*((((data["ffti_abs_percentile_1"]) - ((-1.0*((((data["fftr_Hann_window_mean_1500"]) - (data["fftr_num_peaks_100"]))))))))))) * (((data["ffti_sum"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i329"] = 0.010000*np.tanh(((((((((((((((((((data["ffti_percentile_roll_std_30"]) * (((data["fftr_classic_sta_lta5_mean"]) + (data["ffti_percentile_roll_std_30"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i330"] = 0.010000*np.tanh(((((((((((((data["fftr_min_roll_std_10000"]) * (((data["ffti_av_change_abs_roll_mean_100"]) + (data["fftr_abs_percentile_1"]))))) * 2.0)) - (data["fftr_min_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i331"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_mean_20"]) * 2.0)) * 2.0)) * ((14.60076618194580078)))) * (data["fftr_min_roll_std_10000"]))) + (((((data["percentile_roll_std_5"]) * (data["percentile_roll_std_40"]))) * (data["percentile_roll_std_5"]))))) 
        v["i332"] = 0.010000*np.tanh(((((((data["min_roll_std_10"]) * 2.0)) + (((((((((((((data["min_roll_std_10"]) * 2.0)) * 2.0)) - (data["percentile_roll_std_5"]))) * 2.0)) - (data["percentile_roll_std_5"]))) * 2.0)))) * 2.0)) 
        v["i333"] = 0.010000*np.tanh(((((((((data["mean_change_rate_last_50000"]) + (((((((data["ffti_mean_change_abs"]) * (((data["fftr_mean_last_1000"]) + (data["fftr_abs_percentile_1"]))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i334"] = 0.010000*np.tanh(((((data["percentile_roll_std_70"]) + (((((((((data["ffti_percentile_roll_std_20"]) - (((data["min_roll_std_1000"]) * (((data["autocorrelation_5"]) + (data["ffti_percentile_roll_std_5"]))))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i335"] = 0.010000*np.tanh(((((((((((data["mean_first_10000"]) * (((((data["ffti_percentile_roll_std_20"]) + (((data["mean_change_rate_last_10000"]) + (data["abs_percentile_40"]))))) + (data["fftr_abs_percentile_99"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i336"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_5000"]) + (((((((((((data["exp_Moving_average_30000_std"]) * (data["ffti_ave_roll_mean_10000"]))) * 2.0)) * 2.0)) + (data["autocorrelation_5"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i337"] = 0.010000*np.tanh(((((data["ffti_ave_roll_mean_100"]) + (((((data["ffti_exp_Moving_average_30000_std"]) * (((((data["fftr_autocorrelation_10"]) * 2.0)) * 2.0)))) + (((data["fftr_av_change_abs_roll_mean_1000"]) * (data["fftr_time_rev_asym_stat_5"]))))))) * 2.0)) 
        v["i338"] = 0.010000*np.tanh(((((((data["ffti_Hann_window_mean_150"]) * (((data["fftr_percentile_roll_mean_40"]) * 2.0)))) * 2.0)) + (((((data["percentile_roll_mean_5"]) * (((data["fftr_mean_first_1000"]) * 2.0)))) + (data["fftr_percentile_roll_mean_40"]))))) 
        v["i339"] = 0.010000*np.tanh(((((((((data["autocorrelation_100"]) * (((data["fftr_hmean"]) - (data["percentile_80"]))))) * 2.0)) * 2.0)) - (((data["fftr_min_roll_std_10000"]) * ((11.48147773742675781)))))) 
        v["i340"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_60"]) * (((((((((data["ffti_exp_Moving_average_50000_mean"]) - (data["ffti_mean_last_10000"]))) * 2.0)) - (data["percentile_roll_mean_80"]))) - (data["percentile_roll_mean_80"]))))) * 2.0)) * 2.0)) 
        v["i341"] = 0.010000*np.tanh(((((((((((((data["ffti_autocorrelation_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["std_roll_mean_10000"]))) * (((data["fftr_range_-4000_-3000"]) + (data["ffti_percentile_roll_mean_5"]))))) 
        v["i342"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_std_25"]) + (((((data["fftr_percentile_50"]) + (data["ffti_autocorrelation_100"]))) * (data["ffti_autocorrelation_100"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i343"] = 0.010000*np.tanh(((((((data["fftr_abs_max_roll_mean_500"]) + (((data["percentile_roll_std_5"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i344"] = 0.010000*np.tanh((-1.0*((((((data["classic_sta_lta8_mean"]) * (((((data["ffti_percentile_roll_mean_90"]) - (((data["classic_sta_lta4_mean"]) + (((data["fftr_time_rev_asym_stat_1"]) + (data["classic_sta_lta8_mean"]))))))) * 2.0)))) * 2.0))))) 
        v["i345"] = 0.010000*np.tanh(((((((((((((((data["fftr_range_3000_4000"]) * (data["mean_first_1000"]))) - (data["ffti_autocorrelation_10"]))) * 2.0)) + (data["fftr_time_rev_asym_stat_5"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i346"] = 0.010000*np.tanh(((data["percentile_60"]) + (((((((((data["std_roll_mean_10000"]) * (((data["ffti_spkt_welch_density_1"]) - (data["fftr_autocorrelation_5000"]))))) * 2.0)) + (((data["autocorrelation_5"]) * 2.0)))) * 2.0)))) 
        v["i347"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_std_5"]) * 2.0)) * 2.0)) + (data["fftr_mean_first_1000"]))) * (((data["fftr_abs_max_roll_mean_1000"]) - (((data["ffti_av_change_abs_roll_std_10000"]) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i348"] = 0.010000*np.tanh(((data["percentile_roll_mean_99"]) + (((((data["med"]) * (((data["fftr_min_first_1000"]) - (((((((data["fftr_min_roll_std_1000"]) * (data["percentile_roll_mean_99"]))) * 2.0)) * 2.0)))))) * 2.0)))) 
        v["i349"] = 0.010000*np.tanh(((((((((((data["fftr_abs_percentile_1"]) * 2.0)) * 2.0)) + (((((data["fftr_abs_percentile_1"]) - (data["ffti_autocorrelation_10000"]))) + (data["fftr_exp_Moving_average_300_std"]))))) + (data["mean_change_rate_last_10000"]))) * 2.0)) 
        v["i350"] = 0.010000*np.tanh(((((data["ffti_autocorrelation_10000"]) + (((((((data["mean_change_rate_first_10000"]) + (((data["mean_change_rate_first_10000"]) + (data["mean_change_rate_first_10000"]))))) + (data["ffti_autocorrelation_10000"]))) - (((data["ffti_num_peaks_100"]) * 2.0)))))) * 2.0)) 
        v["i351"] = 0.010000*np.tanh(((((((((((data["ffti_Hann_window_mean_15000"]) + (((((((data["fftr_time_rev_asym_stat_5"]) * (((data["ffti_Hann_window_mean_15000"]) - (data["fftr_time_rev_asym_stat_100"]))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i352"] = 0.010000*np.tanh(((((((((data["fftr_percentile_50"]) * (data["fftr_num_peaks_100"]))) - (data["ffti_av_change_abs_roll_std_100"]))) + (((((((data["ffti_percentile_roll_std_25"]) * (data["fftr_num_peaks_50"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i353"] = 0.010000*np.tanh(((((((((((data["av_change_abs_roll_mean_500"]) - (((((data["av_change_abs_roll_mean_500"]) * (data["fftr_min_roll_std_10"]))) * 2.0)))) * 2.0)) * 2.0)) - (data["fftr_min_roll_std_10"]))) * 2.0)) 
        v["i354"] = 0.010000*np.tanh(((((((((((((((((data["av_change_abs_roll_mean_100"]) * ((-1.0*((data["fftr_mean_last_1000"])))))) + (data["abs_max_roll_mean_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i355"] = 0.010000*np.tanh(((((((((((((((((data["av_change_abs_roll_std_10"]) * 2.0)) + (((((data["fftr_autocorrelation_100"]) * (data["fftr_min_roll_std_50"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i356"] = 0.010000*np.tanh(((data["fftr_abs_percentile_1"]) + (((((((((data["percentile_20"]) + (data["fftr_min_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i357"] = 0.010000*np.tanh(((data["fftr_autocorrelation_10000"]) * (((((((((((((data["fftr_min_roll_mean_500"]) * 2.0)) + (((((data["ffti_mean_change_abs"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i358"] = 0.010000*np.tanh(((((((((data["fftr_num_peaks_100"]) - (data["ffti_abs_percentile_1"]))) * (data["ffti_sum"]))) * 2.0)) * 2.0)) 
        v["i359"] = 0.010000*np.tanh(((((((((((data["ffti_exp_Moving_average_3000_std"]) + (((((((((data["percentile_90"]) * (data["ffti_autocorrelation_5"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i360"] = 0.010000*np.tanh(((((data["autocorrelation_50"]) * (((((((((data["percentile_roll_std_25"]) - (data["percentile_50"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i361"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_50"]) * 2.0)) + (((data["ffti_autocorrelation_50"]) * (((((((data["fftr_max_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)))))) + (data["ffti_autocorrelation_50"]))) 
        v["i362"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_std_20"]) * 2.0)) - (((data["ffti_med"]) * (((((((((((data["autocorrelation_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i363"] = 0.010000*np.tanh(((((((data["autocorrelation_1000"]) - (((((data["fftr_time_rev_asym_stat_50"]) * (((((data["ave_roll_mean_100"]) + (data["fftr_time_rev_asym_stat_50"]))) + ((-1.0*((data["ffti_abs_percentile_1"])))))))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i364"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_mean_40"]) - (((((((data["fftr_percentile_roll_mean_40"]) * 2.0)) * (((data["ffti_av_change_abs_roll_mean_10"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i365"] = 0.010000*np.tanh((((((data["fftr_mean_first_1000"]) + (data["abs_percentile_50"]))/2.0)) + ((((-1.0*((data["fftr_mean_first_1000"])))) * (((((data["ffti_min_roll_std_1000"]) * 2.0)) * 2.0)))))) 
        v["i366"] = 0.010000*np.tanh(((((((((((((((((data["av_change_abs_roll_mean_500"]) + (((data["ffti_skew"]) * 2.0)))) - (data["fftr_min_roll_std_500"]))) - (data["ffti_mean_change_rate"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i367"] = 0.010000*np.tanh(((((((data["fftr_min_roll_std_500"]) * 2.0)) + (data["fftr_percentile_roll_mean_40"]))) - (((((((((data["ffti_autocorrelation_50"]) * (((data["percentile_75"]) + (data["ffti_exp_Moving_average_3000_std"]))))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i368"] = 0.010000*np.tanh(((((((((((data["num_peaks_100"]) * (data["min_roll_std_10"]))) * 2.0)) + (((((((data["num_peaks_100"]) * (data["fftr_abs_max_roll_mean_10000"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i369"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_std_10"]) * 2.0)) * 2.0)) + (((data["ffti_min_roll_mean_10000"]) * (((((((((((data["classic_sta_lta3_mean"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i370"] = 0.010000*np.tanh(((((((((((((((data["ffti_percentile_roll_mean_25"]) * 2.0)) - (data["ffti_percentile_roll_std_5"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (((((data["ffti_percentile_roll_std_40"]) * 2.0)) * 2.0)))) 
        v["i371"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_mean_50"]) + ((-1.0*((((data["autocorrelation_50"]) * (((((((data["Hann_window_mean_1500"]) + (((data["autocorrelation_10000"]) * (data["percentile_80"]))))) * 2.0)) * 2.0))))))))) * 2.0)) 
        v["i372"] = 0.010000*np.tanh(((((((((data["fftr_percentile_roll_std_50"]) + (((((((data["autocorrelation_10000"]) * (((data["fftr_time_rev_asym_stat_50"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i373"] = 0.010000*np.tanh(((((((data["av_change_abs_roll_std_10"]) + (((((((data["ffti_kstat_1"]) * (((((data["percentile_60"]) + (data["ffti_max_to_min"]))) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i374"] = 0.010000*np.tanh(((data["fftr_range_-3000_-2000"]) * (((((((((((data["percentile_60"]) + (data["fftr_hmean"]))) + (data["autocorrelation_50"]))) * 2.0)) * 2.0)) + (((((data["percentile_60"]) * 2.0)) * 2.0)))))) 
        v["i375"] = 0.010000*np.tanh(((((((((data["fftr_percentile_roll_std_5"]) * 2.0)) * (((data["ffti_num_peaks_100"]) + (((((data["ffti_av_change_abs_roll_std_500"]) + (((data["ffti_av_change_abs_roll_std_500"]) + (data["percentile_roll_std_40"]))))) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i376"] = 0.010000*np.tanh(((((data["std_roll_mean_10000"]) + (((((((((((data["fftr_autocorrelation_10000"]) * 2.0)) * (data["ffti_min_roll_std_10"]))) - (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i377"] = 0.010000*np.tanh(((((data["ffti_abs_percentile_1"]) * 2.0)) * (((((data["ffti_av_change_abs_roll_mean_100"]) - (((data["num_peaks_100"]) - (data["ffti_av_change_abs_roll_mean_100"]))))) - (data["num_peaks_100"]))))) 
        v["i378"] = 0.010000*np.tanh(((((((((((((data["max_to_min"]) * (data["fftr_percentile_roll_std_20"]))) * 2.0)) + (data["ffti_percentile_roll_std_5"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i379"] = 0.010000*np.tanh(((((data["min_roll_std_100"]) + (((((((data["min_roll_std_100"]) * (data["fftr_time_rev_asym_stat_10"]))) * 2.0)) * 2.0)))) + (((data["classic_sta_lta4_mean"]) * (((data["fftr_min_first_1000"]) - (data["fftr_min_first_1000"]))))))) 
        v["i380"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_70"]) * 2.0)) + (((data["mean_change_rate_first_10000"]) * 2.0)))) * 2.0)) * 2.0)) + (((data["fftr_min_roll_std_500"]) * 2.0)))) 
        v["i381"] = 0.010000*np.tanh(((((data["fftr_num_peaks_100"]) + (data["fftr_num_peaks_100"]))) + (((data["fftr_num_peaks_100"]) - (((data["exp_Moving_average_300_mean"]) * ((((11.06118297576904297)) * (((data["ffti_med"]) + (data["fftr_num_peaks_100"]))))))))))) 
        v["i382"] = 0.010000*np.tanh(((((((((((((((data["ffti_percentile_80"]) * (((((data["autocorrelation_500"]) * 2.0)) * 2.0)))) + (data["mean_last_1000"]))) * 2.0)) + (data["fftr_max_roll_mean_1000"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i383"] = 0.010000*np.tanh(((data["fftr_percentile_roll_mean_50"]) + (((((((((((((data["fftr_autocorrelation_10"]) * 2.0)) * 2.0)) * (((data["ffti_exp_Moving_average_300_std"]) + (data["skew"]))))) * 2.0)) + (data["ffti_av_change_abs_roll_std_500"]))) * 2.0)))) 
        v["i384"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_std_20"]) - (((data["ffti_autocorrelation_500"]) * (data["ffti_exp_Moving_average_50000_std"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["ffti_exp_Moving_average_50000_std"]))) 
        v["i385"] = 0.010000*np.tanh(((((data["percentile_roll_mean_99"]) + (((((data["fftr_abs_percentile_1"]) * (((((((data["exp_Moving_average_50000_std"]) * 2.0)) * 2.0)) * 2.0)))) * (((data["ffti_num_peaks_100"]) * 2.0)))))) + (data["fftr_time_rev_asym_stat_5"]))) 
        v["i386"] = 0.010000*np.tanh(((((((((data["exp_Moving_average_30000_std"]) + (((data["percentile_roll_std_40"]) * 2.0)))) + (((data["percentile_roll_std_30"]) + (data["exp_Moving_average_30000_std"]))))) + (((data["percentile_roll_std_30"]) * 2.0)))) * 2.0)) 
        v["i387"] = 0.010000*np.tanh(((((((data["fftr_percentile_25"]) * 2.0)) + (((((data["std_roll_std_10"]) - ((((((-1.0*((((data["percentile_25"]) * 2.0))))) * 2.0)) * 2.0)))) * (data["ffti_num_peaks_100"]))))) * 2.0)) 
        v["i388"] = 0.010000*np.tanh(((data["ffti_autocorrelation_10000"]) + (((((((((((((((data["autocorrelation_10000"]) * 2.0)) * (((data["ffti_av_change_abs_roll_std_10"]) - (data["ffti_med"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i389"] = 0.010000*np.tanh(((((((((data["classic_sta_lta8_mean"]) + (data["fftr_c3_1000"]))) + (((((data["fftr_time_rev_asym_stat_50"]) * ((((-1.0*((data["max_to_min"])))) - (data["fftr_time_rev_asym_stat_5"]))))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i390"] = 0.010000*np.tanh(((((((((((data["ffti_av_change_abs_roll_std_10"]) * (((data["fftr_percentile_roll_mean_80"]) * 2.0)))) * 2.0)) * 2.0)) + (((data["ffti_max_to_min"]) * (((((data["fftr_autocorrelation_5000"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i391"] = 0.010000*np.tanh(((((((((((((data["autocorrelation_50"]) * (data["percentile_roll_std_5"]))) * 2.0)) * 2.0)) * 2.0)) + (((((data["percentile_roll_std_5"]) + (((data["autocorrelation_50"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i392"] = 0.010000*np.tanh((((((((((((((-1.0*((((((((data["fftr_percentile_roll_mean_40"]) * (data["ffti_av_change_abs_roll_mean_10"]))) * 2.0)) * 2.0))))) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_percentile_roll_mean_40"]))) * 2.0)) * 2.0)) 
        v["i393"] = 0.010000*np.tanh(((((((((((((data["abs_percentile_40"]) - (data["fftr_percentile_roll_mean_50"]))) * 2.0)) - (data["fftr_autocorrelation_100"]))) * (data["percentile_roll_std_70"]))) * 2.0)) * 2.0)) 
        v["i394"] = 0.010000*np.tanh(((((((data["abs_percentile_80"]) * (((data["max_first_1000"]) + (((((data["autocorrelation_500"]) + (data["ffti_mean_change_rate"]))) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i395"] = 0.010000*np.tanh(((((((data["ffti_exp_Moving_average_3000_std"]) + (((((((data["percentile_roll_std_5"]) + (data["ffti_exp_Moving_average_3000_std"]))) + (((data["percentile_roll_std_80"]) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i396"] = 0.010000*np.tanh(((((((((((((((data["ffti_autocorrelation_50"]) * 2.0)) * 2.0)) * (data["fftr_mean_last_10000"]))) * 2.0)) * 2.0)) * 2.0)) + (((data["ffti_mean_last_1000"]) + (data["fftr_mean_last_10000"]))))) 
        v["i397"] = 0.010000*np.tanh(((((((((((data["iqr"]) + (data["fftr_percentile_25"]))) + (((((data["abs_percentile_75"]) * (data["mean_last_1000"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i398"] = 0.010000*np.tanh(((((((((data["classic_sta_lta4_mean"]) * (((data["ffti_min_first_1000"]) + (data["ffti_min_first_1000"]))))) * 2.0)) + (((data["fftr_time_rev_asym_stat_5"]) * ((((-1.0*((data["fftr_exp_Moving_average_50000_mean"])))) * 2.0)))))) * 2.0)) 
        v["i399"] = 0.010000*np.tanh(((((data["ffti_std_first_1000"]) - (((((((((data["fftr_num_peaks_100"]) * (((((((data["ffti_av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) + (data["sum"]))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i400"] = 0.010000*np.tanh(((((((data["trend"]) + (((((((data["ffti_min_roll_std_100"]) * (((data["ffti_percentile_roll_mean_20"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i401"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_10"]) * (((((data["percentile_roll_mean_99"]) + (data["fftr_autocorrelation_50"]))) + (data["autocorrelation_500"]))))) + (data["ffti_av_change_abs_roll_std_500"]))) * 2.0)) 
        v["i402"] = 0.010000*np.tanh(((((((((data["fftr_min_roll_mean_50"]) * 2.0)) + (((((data["min_roll_std_50"]) * 2.0)) * (((((data["autocorrelation_50"]) - (data["fftr_range_2000_3000"]))) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i403"] = 0.010000*np.tanh(((((data["fftr_percentile_30"]) - (((data["fftr_time_rev_asym_stat_10"]) * (((((data["ffti_mean_change_abs"]) * 2.0)) * 2.0)))))) - (((data["fftr_min_roll_mean_50"]) * (((data["ffti_mean_change_abs"]) * 2.0)))))) 
        v["i404"] = 0.010000*np.tanh(((((((((data["ffti_av_change_abs_roll_mean_50"]) + (((((((data["fftr_min_roll_std_500"]) * (((data["autocorrelation_1000"]) + (((data["ffti_av_change_abs_roll_mean_50"]) * 2.0)))))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i405"] = 0.010000*np.tanh(((data["max_to_min"]) + (((((((((((data["ffti_max_roll_mean_50"]) - (((data["ffti_percentile_roll_std_50"]) * 2.0)))) * 2.0)) * 2.0)) * (((data["Hann_window_mean_1500"]) * 2.0)))) * 2.0)))) 
        v["i406"] = 0.010000*np.tanh(((((((((((((((((((data["fftr_kurt"]) * (data["ffti_num_peaks_10"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i407"] = 0.010000*np.tanh(((((((((((((data["ffti_autocorrelation_50"]) + (data["ffti_autocorrelation_50"]))) + (((data["fftr_range_2000_3000"]) + (data["fftr_range_2000_3000"]))))) + (data["mean_last_1000"]))) * (data["mean_last_1000"]))) * 2.0)) * 2.0)) 
        v["i408"] = 0.010000*np.tanh(((((((((((((data["kurt"]) - (((data["ffti_autocorrelation_10000"]) * 2.0)))) * (data["fftr_percentile_40"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i409"] = 0.010000*np.tanh(((((((data["fftr_percentile_50"]) * (((((((((((((data["abs_percentile_80"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i410"] = 0.010000*np.tanh(((((((((((((((((data["ffti_av_change_abs_roll_std_10000"]) * ((-1.0*((data["percentile_roll_std_1"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i411"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_5"]) * 2.0)) * (((((((data["ffti_med"]) + (((data["ffti_autocorrelation_1000"]) * 2.0)))) - (data["ffti_min_roll_std_50"]))) - (((data["abs_percentile_60"]) - (data["ffti_max_to_min"]))))))) 
        v["i412"] = 0.010000*np.tanh(((((((((data["exp_Moving_average_30000_std"]) * 2.0)) * ((-1.0*((((((data["exp_Moving_average_30000_std"]) * 2.0)) + (((((((data["autocorrelation_1000"]) * 2.0)) * 2.0)) * 2.0))))))))) * 2.0)) * 2.0)) 
        v["i413"] = 0.010000*np.tanh(((((((data["classic_sta_lta4_mean"]) * 2.0)) + (data["ffti_exp_Moving_average_30000_std"]))) * (((((data["fftr_autocorrelation_10000"]) - (data["min_roll_std_10"]))) - (((((data["ffti_exp_Moving_average_30000_std"]) - (data["fftr_autocorrelation_10000"]))) * 2.0)))))) 
        v["i414"] = 0.010000*np.tanh((((((((5.30953788757324219)) * (((((data["mean_first_10000"]) * (((data["Hann_window_mean_150"]) + (data["fftr_min_roll_std_1000"]))))) * ((5.30953788757324219)))))) * 2.0)) * 2.0)) 
        v["i415"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_1"]) + (((((((data["fftr_exp_Moving_average_3000_mean"]) * (data["ffti_autocorrelation_1000"]))) + (((data["ffti_min_roll_std_1000"]) * (((((data["fftr_av_change_abs_roll_mean_500"]) * 2.0)) * 2.0)))))) * 2.0)))) 
        v["i416"] = 0.010000*np.tanh(((((data["ffti_max_to_min"]) + (((((((((((data["autocorrelation_500"]) * (data["fftr_hmean"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i417"] = 0.010000*np.tanh(((((data["classic_sta_lta7_mean"]) * (((((((((((((((data["ffti_av_change_abs_roll_std_500"]) * 2.0)) * 2.0)) + (data["autocorrelation_5000"]))) * 2.0)) + (data["fftr_min_roll_std_500"]))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i418"] = 0.010000*np.tanh(((((((((((((data["ffti_min_roll_std_50"]) * (data["ffti_autocorrelation_10"]))) - ((-1.0*((data["fftr_percentile_roll_std_20"])))))) * 2.0)) * 2.0)) + (data["fftr_max_roll_mean_1000"]))) * 2.0)) 
        v["i419"] = 0.010000*np.tanh(((data["percentile_roll_std_5"]) + (((((((((((((((((data["ffti_num_peaks_50"]) * (data["ffti_autocorrelation_50"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i420"] = 0.010000*np.tanh(((((((((((((data["percentile_roll_std_80"]) + (((data["fftr_mean_first_1000"]) * (((data["ffti_autocorrelation_50"]) - (data["fftr_time_rev_asym_stat_1"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i421"] = 0.010000*np.tanh(((data["fftr_min_roll_std_1000"]) + (((((((data["mean_change_rate_first_10000"]) + (((data["fftr_range_3000_4000"]) + (((data["percentile_60"]) * (((data["fftr_range_2000_3000"]) - (data["fftr_min_roll_std_1000"]))))))))) * 2.0)) * 2.0)))) 
        v["i422"] = 0.010000*np.tanh(((data["ffti_autocorrelation_100"]) + (((((((data["ffti_exp_Moving_average_300_std"]) + (((((((data["ffti_autocorrelation_100"]) * (data["fftr_min_roll_std_50"]))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i423"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_std_60"]) * 2.0)) * (((data["av_change_abs_roll_mean_10"]) * 2.0)))) + (((data["exp_Moving_average_50000_std"]) * (data["av_change_abs_roll_mean_10"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i424"] = 0.010000*np.tanh(((((data["mean_change_rate_last_1000"]) * (((data["num_peaks_50"]) + ((((((((data["num_peaks_50"]) + (((((data["fftr_time_rev_asym_stat_100"]) * 2.0)) * 2.0)))/2.0)) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i425"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_100"]) * 2.0)) + (((((((((((data["fftr_time_rev_asym_stat_100"]) * 2.0)) * 2.0)) + (((data["autocorrelation_10000"]) * 2.0)))) * 2.0)) * (data["std_roll_mean_10000"]))))) 
        v["i426"] = 0.010000*np.tanh(((((((((((((data["autocorrelation_10000"]) - (data["percentile_25"]))) * (((data["exp_Moving_average_3000_mean"]) - (data["autocorrelation_10000"]))))) - (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i427"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_10"]) * ((-1.0*((data["ffti_skew"])))))) + (((data["classic_sta_lta3_mean"]) * (((((data["fftr_min_roll_mean_10000"]) - (((data["fftr_autocorrelation_10"]) - (data["classic_sta_lta3_mean"]))))) * 2.0)))))) 
        v["i428"] = 0.010000*np.tanh(((((((((data["ffti_trend"]) + (((data["ffti_percentile_roll_mean_20"]) * 2.0)))) * (((((((data["fftr_autocorrelation_5000"]) + (data["ffti_trend"]))) * 2.0)) + (data["fftr_time_rev_asym_stat_50"]))))) * 2.0)) * 2.0)) 
        v["i429"] = 0.010000*np.tanh(((((((((data["fftr_min_roll_std_500"]) - (data["fftr_autocorrelation_10000"]))) + (((((((((data["ffti_av_change_abs_roll_std_10000"]) - (data["fftr_autocorrelation_10000"]))) * 2.0)) * (data["fftr_min_roll_std_500"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i430"] = 0.010000*np.tanh(((((((((((((((((data["fftr_percentile_roll_mean_5"]) * (data["ffti_autocorrelation_100"]))) + (((data["max_to_min"]) * (data["ffti_max_to_min_diff"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i431"] = 0.010000*np.tanh(((((((((((((((((data["ffti_min_roll_mean_50"]) * (((((data["av_change_rate_roll_mean_10000"]) + (data["autocorrelation_5"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i432"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_mean_10"]) + (((((((((data["fftr_min_roll_std_10"]) * (((data["fftr_percentile_roll_mean_40"]) - (((data["fftr_min_first_1000"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i433"] = 0.010000*np.tanh(((data["ffti_mean_change_abs"]) * (((((((((((((data["min_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i434"] = 0.010000*np.tanh(((((((((data["abs_max_roll_mean_10000"]) - (data["ffti_num_peaks_50"]))) + (((data["fftr_num_peaks_50"]) + (((data["abs_max_roll_mean_10000"]) - (data["ffti_num_peaks_50"]))))))) * 2.0)) * 2.0)) 
        v["i435"] = 0.010000*np.tanh(((((((((((((data["fftr_autocorrelation_1000"]) * (((data["std_roll_mean_10000"]) + (data["av_change_abs_roll_mean_50"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i436"] = 0.010000*np.tanh(((((((((((((((data["min_roll_std_10"]) - (((((data["ffti_autocorrelation_10000"]) * 2.0)) * (data["ffti_std_roll_mean_1000"]))))) * 2.0)) * 2.0)) * 2.0)) + (data["ffti_std_roll_mean_1000"]))) * 2.0)) * 2.0)) 
        v["i437"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_20"]) - (data["spkt_welch_density_100"]))) * 2.0)) * (data["fftr_time_rev_asym_stat_1"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i438"] = 0.010000*np.tanh(((((((((((data["fftr_min"]) - (((data["autocorrelation_10000"]) * (((data["fftr_av_change_abs_roll_std_50"]) * 2.0)))))) - (data["ffti_av_change_abs_roll_std_100"]))) * 2.0)) - (data["ffti_percentile_roll_std_70"]))) * 2.0)) 
        v["i439"] = 0.010000*np.tanh(((((data["fftr_hmean"]) * (((((((((((((data["autocorrelation_500"]) * 2.0)) * (((((data["mean_first_50000"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i440"] = 0.010000*np.tanh(((((data["fftr_time_rev_asym_stat_10"]) + (((((data["percentile_roll_mean_5"]) * (data["fftr_mean_first_1000"]))) + (((data["fftr_time_rev_asym_stat_10"]) + (((((data["ffti_av_change_abs_roll_std_500"]) * 2.0)) * 2.0)))))))) * 2.0)) 
        v["i441"] = 0.010000*np.tanh(((data["autocorrelation_500"]) * (((((((((((((data["fftr_autocorrelation_1000"]) + (((data["percentile_80"]) + (data["fftr_autocorrelation_1000"]))))) * 2.0)) + (data["abs_max_roll_mean_10000"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i442"] = 0.010000*np.tanh(((((((((((((data["min_roll_std_500"]) + (data["fftr_abs_percentile_5"]))) + (data["fftr_min_roll_std_10000"]))) * (data["fftr_min_roll_std_10000"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i443"] = 0.010000*np.tanh(((((((((data["ffti_autocorrelation_10"]) * (((((data["max_to_min"]) + (data["ffti_max_first_1000"]))) * 2.0)))) * 2.0)) * 2.0)) + (data["max_to_min"]))) 
        v["i444"] = 0.010000*np.tanh(((data["fftr_min_roll_std_10"]) * (((((((((((((data["ffti_std_roll_mean_10000"]) - (data["percentile_roll_std_40"]))) - (data["fftr_num_peaks_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i445"] = 0.010000*np.tanh(((data["ffti_autocorrelation_10000"]) - (((((((((data["ffti_skew"]) * 2.0)) * (data["fftr_min_roll_std_10"]))) * 2.0)) * 2.0)))) 
        v["i446"] = 0.010000*np.tanh(((((((((((((((((data["ffti_sum"]) * ((-1.0*((data["fftr_mean_last_10000"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["fftr_mean_last_10000"]))) * 2.0)) * 2.0)) 
        v["i447"] = 0.009996*np.tanh(((((((((data["fftr_kstat_1"]) + (((data["ffti_classic_sta_lta6_mean"]) * ((-1.0*((((((((data["kurt"]) * 2.0)) * 2.0)) * 2.0))))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i448"] = 0.010000*np.tanh(((((((((((((((data["ffti_percentile_roll_mean_30"]) * (((((((data["ffti_trend"]) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i449"] = 0.010000*np.tanh(((((((((data["min_roll_std_10"]) * (((((((((((data["ffti_min_roll_std_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) - (data["max_to_min"]))) * 2.0)) * 2.0)) 
        v["i450"] = 0.010000*np.tanh(((((data["percentile_50"]) * (((data["ffti_av_change_abs_roll_std_10000"]) + (((((data["fftr_autocorrelation_5"]) + (data["fftr_percentile_roll_std_25"]))) - (((((data["fftr_percentile_roll_std_60"]) * 2.0)) * 2.0)))))))) * 2.0)) 
        v["i451"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_std_20"]) - ((((((((((((((-1.0*((((data["fftr_percentile_roll_std_20"]) * (data["max_to_min"])))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i452"] = 0.010000*np.tanh(((((((((((((((((((((((data["fftr_percentile_roll_std_5"]) * (data["fftr_percentile_roll_mean_40"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i453"] = 0.010000*np.tanh(((((data["mean_change_rate_last_1000"]) + (((((((((((((data["mean_change_rate_last_1000"]) * (((data["num_peaks_50"]) + (data["max_roll_mean_10000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i454"] = 0.010000*np.tanh(((((data["mean_change_rate_last_10000"]) - (((((((data["fftr_min_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)))) * (((((((data["fftr_min_roll_std_1000"]) * 2.0)) + (data["mean_change_rate_last_10000"]))) + (data["ave_roll_mean_10000"]))))) 
        v["i455"] = 0.010000*np.tanh(((((((((((data["ffti_av_change_abs_roll_mean_50"]) * (data["ffti_abs_percentile_1"]))) + (((((data["ffti_av_change_abs_roll_mean_50"]) * (((data["autocorrelation_1000"]) * (data["percentile_roll_mean_60"]))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i456"] = 0.010000*np.tanh(((((((((data["std_roll_mean_10000"]) * (((data["autocorrelation_10000"]) * 2.0)))) * 2.0)) * 2.0)) + (((data["fftr_time_rev_asym_stat_100"]) + (data["percentile_75"]))))) 
        v["i457"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_std_30"]) * 2.0)) + (((data["ffti_kstat_1"]) * ((((-1.0*((((((data["classic_sta_lta1_mean"]) - ((-1.0*((data["fftr_mean_last_50000"])))))) * 2.0))))) * 2.0)))))) * 2.0)) 
        v["i458"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_5"]) * 2.0)) * 2.0)) + (((data["ffti_range_2000_3000"]) * (((data["fftr_hmean"]) + (data["fftr_hmean"]))))))) * 2.0)) 
        v["i459"] = 0.010000*np.tanh(((data["ffti_ave10"]) * (((((((((((((((data["spkt_welch_density_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((data["spkt_welch_density_100"]) - (data["ffti_ave10"]))))) * 2.0)) * 2.0)))) 
        v["i460"] = 0.010000*np.tanh(((((data["ffti_autocorrelation_5000"]) + (((((((data["min_roll_std_100"]) * (data["ffti_av_change_abs_roll_mean_10000"]))) * 2.0)) * 2.0)))) - (((((data["classic_sta_lta3_mean"]) * (((data["ffti_std_roll_mean_10000"]) * 2.0)))) * 2.0)))) 
        v["i461"] = 0.010000*np.tanh(((((((((((((((((data["autocorrelation_1000"]) * 2.0)) * 2.0)) * (data["ffti_mean_change_abs"]))) * 2.0)) * 2.0)) * (data["ave_roll_mean_10000"]))) * 2.0)) * 2.0)) 
        v["i462"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_std_20"]) * 2.0)) * 2.0)) * (((((data["fftr_max_first_50000"]) - (((data["fftr_percentile_roll_mean_60"]) * 2.0)))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i463"] = 0.010000*np.tanh(((((((data["med"]) * (((((data["autocorrelation_10"]) * (((((data["spkt_welch_density_50"]) * 2.0)) * 2.0)))) - (((data["ffti_mean_first_50000"]) + (data["fftr_min_roll_mean_10000"]))))))) * 2.0)) * 2.0)) 
        v["i464"] = 0.010000*np.tanh(((((data["fftr_autocorrelation_500"]) * 2.0)) - (((((((((((data["fftr_percentile_roll_std_25"]) * (((data["fftr_autocorrelation_500"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i465"] = 0.010000*np.tanh(((((data["autocorrelation_5"]) + (((((data["autocorrelation_5"]) + (((((((((((data["fftr_percentile_roll_std_10"]) * (data["fftr_percentile_roll_mean_25"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i466"] = 0.009996*np.tanh(((data["std_roll_mean_10000"]) * (((((((((((((((data["autocorrelation_10000"]) - (data["classic_sta_lta4_mean"]))) * 2.0)) * 2.0)) - (data["autocorrelation_10000"]))) * 2.0)) * 2.0)) - (data["classic_sta_lta4_mean"]))))) 
        v["i467"] = 0.010000*np.tanh(((((((((data["fftr_min_roll_std_100"]) * 2.0)) - (((data["ffti_min_roll_mean_500"]) - (((data["autocorrelation_5"]) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i468"] = 0.010000*np.tanh(((((data["percentile_60"]) + (((((((((data["fftr_min_roll_std_10"]) * (data["spkt_welch_density_100"]))) * 2.0)) * 2.0)) * 2.0)))) + (((((data["fftr_min_roll_std_10"]) * (data["spkt_welch_density_100"]))) * 2.0)))) 
        v["i469"] = 0.010000*np.tanh(((((data["fftr_mean_first_50000"]) + (((data["ffti_av_change_abs_roll_std_10"]) * (((((((data["ffti_min_roll_std_10"]) + (((data["ffti_min_roll_std_10"]) - (data["fftr_std_roll_std_10000"]))))) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i470"] = 0.010000*np.tanh(((((data["ffti_std_roll_mean_500"]) - (((((((((((((((data["fftr_time_rev_asym_stat_50"]) * (data["min_last_50000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i471"] = 0.010000*np.tanh(((((((data["autocorrelation_1000"]) + (((((data["fftr_mean_first_1000"]) - (data["fftr_ave_roll_mean_10000"]))) * 2.0)))) * 2.0)) * (((data["fftr_mean_first_1000"]) + (((data["percentile_roll_mean_10"]) + (data["percentile_roll_mean_10"]))))))) 
        v["i472"] = 0.010000*np.tanh(((((((((((((((((data["autocorrelation_500"]) + (data["autocorrelation_50"]))) * 2.0)) * (data["fftr_hmean"]))) + (data["percentile_roll_std_5"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i473"] = 0.010000*np.tanh(((((((data["ffti_Hann_window_mean_150"]) * (((data["ffti_Hann_window_mean_150"]) - (((((((((data["fftr_abs_percentile_1"]) * 2.0)) - (data["fftr_range_-3000_-2000"]))) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) 
        v["i474"] = 0.010000*np.tanh(((((((((((data["av_change_abs_roll_std_10"]) - (((data["percentile_60"]) * (((((data["fftr_min_roll_std_1000"]) - (data["fftr_abs_percentile_25"]))) - (data["classic_sta_lta2_mean"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i475"] = 0.010000*np.tanh(((((data["ffti_exp_Moving_average_3000_std"]) + (((((((((data["ffti_exp_Moving_average_3000_std"]) + (((((data["ffti_skew"]) + ((-1.0*((data["ffti_mean_change_abs"])))))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i476"] = 0.010000*np.tanh(((((((((((data["ffti_mean_change_abs"]) * (((data["fftr_abs_percentile_1"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i477"] = 0.009996*np.tanh(((((data["fftr_mean_first_1000"]) * (((data["ffti_mean_change_abs"]) - (((data["ffti_min_roll_std_500"]) - (data["autocorrelation_1000"]))))))) * 2.0)) 
        v["i478"] = 0.010000*np.tanh(((((data["max_first_10000"]) + (data["fftr_percentile_roll_std_50"]))) + ((-1.0*((((((data["ffti_mean_change_rate"]) - (((((((data["fftr_num_peaks_100"]) + (data["fftr_percentile_roll_std_50"]))) * 2.0)) * 2.0)))) * 2.0))))))) 
        v["i479"] = 0.010000*np.tanh(((((((((data["ffti_max_to_min"]) + (data["fftr_time_rev_asym_stat_5"]))) * (((((-1.0*((data["ffti_min_roll_std_50"])))) + (data["ffti_ave10"]))/2.0)))) * 2.0)) * 2.0)) 
        v["i480"] = 0.010000*np.tanh(((((((data["fftr_percentile_40"]) + (((data["ffti_min_roll_std_500"]) + (data["ffti_min_roll_std_500"]))))) + (data["ffti_min_roll_std_500"]))) * (((((data["ffti_min_roll_std_500"]) - (data["ffti_autocorrelation_10000"]))) - (data["ffti_autocorrelation_10000"]))))) 
        v["i481"] = 0.010000*np.tanh(((((((((data["num_peaks_20"]) + (((data["ffti_autocorrelation_10000"]) - (data["ffti_mean_change_rate"]))))) + (((((data["fftr_abs_percentile_10"]) * (data["ffti_autocorrelation_10000"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i482"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_std_5"]) + (data["ffti_percentile_50"]))) + (((((((((((data["fftr_min_roll_std_100"]) * (data["fftr_autocorrelation_5000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i483"] = 0.010000*np.tanh(((((data["fftr_min_roll_std_100"]) * ((((((((-1.0*((data["ffti_kstat_1"])))) + (data["fftr_autocorrelation_500"]))) * 2.0)) * 2.0)))) + (data["fftr_min_roll_std_100"]))) 
        v["i484"] = 0.009996*np.tanh(((((((((((((((((((data["ffti_percentile_roll_mean_50"]) + (data["fftr_time_rev_asym_stat_50"]))) * 2.0)) - (data["ffti_av_change_abs_roll_std_10"]))) * 2.0)) * 2.0)) - (data["fftr_num_peaks_10"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i485"] = 0.010000*np.tanh((((((-1.0*((((data["ffti_num_peaks_20"]) * 2.0))))) * (((((data["ffti_max_last_1000"]) + (data["ffti_num_peaks_10"]))) + (data["fftr_min_roll_std_1000"]))))) * 2.0)) 
        v["i486"] = 0.010000*np.tanh(((((data["abs_percentile_60"]) * (((((((data["ffti_num_peaks_50"]) * 2.0)) * 2.0)) * 2.0)))) - (((data["ffti_num_peaks_50"]) * 2.0)))) 
        v["i487"] = 0.010000*np.tanh(((((((data["fftr_time_rev_asym_stat_100"]) + (((((((((((((data["fftr_hmean"]) * (data["fftr_abs_max_roll_mean_500"]))) * 2.0)) + (data["av_change_rate_roll_mean_10"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i488"] = 0.010000*np.tanh(((((data["fftr_mean_first_1000"]) * (data["abs_percentile_25"]))) - (((((((((((data["ffti_percentile_50"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["exp_Moving_average_30000_std"]))))) 
        v["i489"] = 0.010000*np.tanh(((data["classic_sta_lta6_mean"]) * (((data["fftr_mean_first_50000"]) - (((((((((((((((data["fftr_percentile_roll_mean_40"]) - (data["fftr_num_peaks_100"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i490"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_mean_30"]) * 2.0)) + (((((((data["ffti_autocorrelation_500"]) + (((data["exp_Moving_average_50000_std"]) * 2.0)))) * (((data["ffti_percentile_roll_mean_30"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i491"] = 0.010000*np.tanh((((((data["ffti_exp_Moving_average_3000_std"]) * 2.0)) + (((((data["autocorrelation_5000"]) - (((((data["ffti_exp_Moving_average_3000_std"]) * 2.0)) * (((((data["ffti_autocorrelation_10000"]) - (data["std_roll_mean_10000"]))) * 2.0)))))) * 2.0)))/2.0)) 
        v["i492"] = 0.010000*np.tanh(((((((((((((data["ffti_percentile_roll_std_30"]) * (((data["ffti_num_peaks_100"]) * 2.0)))) * 2.0)) + (((data["ffti_percentile_roll_std_30"]) * (((data["ffti_kurt"]) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i493"] = 0.010000*np.tanh(((((((((((((((((data["fftr_percentile_50"]) * (((data["abs_percentile_80"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (data["fftr_sum"]))) * 2.0)) * 2.0)) 
        v["i494"] = 0.010000*np.tanh(((((data["percentile_roll_mean_10"]) * (((data["fftr_min_roll_std_50"]) + (((((((data["mean_change_rate_first_1000"]) * 2.0)) + (((data["fftr_min_roll_std_50"]) + (data["mean_change_rate_first_1000"]))))) * 2.0)))))) * 2.0)) 
        v["i495"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_500"]) - (data["ffti_max_roll_mean_50"]))) - (((((((data["autocorrelation_1000"]) - (((data["av_change_abs_roll_mean_10"]) * 2.0)))) * (((data["exp_Moving_average_30000_std"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i496"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_mean_10"]) * 2.0)) * (((((((((((((data["ave_roll_mean_50"]) * (((data["ffti_num_peaks_50"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i497"] = 0.010000*np.tanh(((data["ffti_autocorrelation_5"]) + (((data["ffti_mean"]) * (((((((data["ffti_ave_roll_mean_1000"]) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i498"] = 0.010000*np.tanh(((data["fftr_classic_sta_lta8_mean"]) - (((data["percentile_80"]) * (((data["autocorrelation_100"]) - (((((data["num_peaks_20"]) - (data["percentile_80"]))) - (((data["fftr_autocorrelation_5000"]) - (data["percentile_80"]))))))))))) 
        v["i499"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_5"]) - (((data["fftr_time_rev_asym_stat_1"]) * (((data["ffti_min_roll_std_100"]) + (((((data["ffti_percentile_roll_std_5"]) * ((3.0)))) * ((3.0)))))))))) 
        v["i500"] = 0.010000*np.tanh((((-1.0*((data["kurt"])))) + (((((((data["ffti_percentile_roll_std_30"]) + (((data["ffti_percentile_roll_std_30"]) * 2.0)))) * 2.0)) * (((((data["kurt"]) + (data["fftr_max_roll_mean_100"]))) * 2.0)))))) 
        v["i501"] = 0.010000*np.tanh((((((((-1.0*((((((data["ffti_autocorrelation_10000"]) * (data["ffti_min_roll_std_500"]))) + (((data["ffti_kstat_1"]) * (((data["ffti_min_roll_std_500"]) * 2.0))))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i502"] = 0.010000*np.tanh((-1.0*((((data["fftr_time_rev_asym_stat_1"]) * (((((((data["ffti_min_roll_std_10000"]) * 2.0)) + (((((data["gmean"]) + (((data["ffti_min_roll_std_10000"]) * 2.0)))) * 2.0)))) * 2.0))))))) 
        v["i503"] = 0.009996*np.tanh(((((((((((data["fftr_time_rev_asym_stat_50"]) * (((((((data["ffti_abs_percentile_10"]) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i504"] = 0.010000*np.tanh(((((((((((((((((data["max_to_min"]) * (((data["fftr_autocorrelation_100"]) + (data["fftr_min_roll_std_1000"]))))) * 2.0)) * 2.0)) - (data["fftr_range_2000_3000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i505"] = 0.010000*np.tanh(((((((((((((data["ffti_classic_sta_lta5_mean"]) * 2.0)) * (((((((data["autocorrelation_50"]) * 2.0)) - (data["ffti_classic_sta_lta5_mean"]))) - (data["ffti_classic_sta_lta5_mean"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i506"] = 0.010000*np.tanh((((((((((((((-1.0*((((data["max_to_min"]) * (((data["ffti_exp_Moving_average_30000_std"]) - (data["classic_sta_lta8_mean"])))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i507"] = 0.010000*np.tanh(((((((((((((data["ffti_sum"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * ((((((-1.0*((data["exp_Moving_average_50000_std"])))) + (data["mean_first_10000"]))) * 2.0)))) * 2.0)) 
        v["i508"] = 0.010000*np.tanh(((((((((((((((data["ffti_exp_Moving_average_3000_std"]) * (((data["autocorrelation_10000"]) - (data["mean_change_rate_first_10000"]))))) * 2.0)) * 2.0)) + (data["min_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i509"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_5000"]) + (((data["ffti_abs_max_roll_mean_10"]) + (((data["fftr_classic_sta_lta4_mean"]) + (((((data["autocorrelation_5000"]) + (data["ffti_autocorrelation_5"]))) * 2.0)))))))) * 2.0)) * 2.0)) 
        v["i510"] = 0.010000*np.tanh((((data["fftr_percentile_roll_mean_30"]) + (((((((((data["mean_change_rate_last_1000"]) - (data["ffti_autocorrelation_1000"]))) * (((data["num_peaks_20"]) * 2.0)))) * 2.0)) * 2.0)))/2.0)) 
        v["i511"] = 0.010000*np.tanh(((((data["autocorrelation_5"]) - (((((((data["autocorrelation_5"]) * (((data["autocorrelation_5000"]) * (((data["fftr_exp_Moving_average_50000_std"]) * 2.0)))))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i512"] = 0.010000*np.tanh(((((data["ffti_num_peaks_50"]) + (((((data["ffti_autocorrelation_50"]) * (((((((((((data["ffti_num_peaks_50"]) * 2.0)) + (data["av_change_rate_roll_mean_50"]))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) * 2.0))
        return v[v.columns[:n]].sum(axis=1)
    
    def GP_class_2(self,data,n):
        v = pd.DataFrame()
        v["i0"] = -2.232397 
        v["i1"] = 0.010000*np.tanh(((((((((((((data["ffti_range_0_1000"]) / 2.0)) - (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) - (((data["kstat_1"]) * 2.0)))) 
        v["i2"] = 0.010000*np.tanh(((((((((((((((data["ffti_range_0_1000"]) - (data["abs_percentile_80"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((data["ffti_range_0_1000"]) * 2.0)))) 
        v["i3"] = 0.010000*np.tanh((((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) + (((((data["fftr_av_change_rate_roll_mean_1000"]) - (data["abs_percentile_80"]))) * 2.0)))) * 2.0)) - (((data["fftr_range_-3000_-2000"]) + (data["percentile_75"]))))) 
        v["i4"] = 0.010000*np.tanh(((((((((data["ffti_range_0_1000"]) + (((((data["mean_change_rate"]) - (data["Hann_window_mean_1500"]))) * 2.0)))) * 2.0)) * 2.0)) + (((((data["mean_change_rate"]) - (data["Hann_window_mean_1500"]))) * 2.0)))) 
        v["i5"] = 0.010000*np.tanh((((-1.0*((((((data["num_peaks_10"]) * 2.0)) + (((((((data["num_peaks_10"]) + (data["fftr_range_-4000_-3000"]))) * 2.0)) - (((data["num_peaks_10"]) * 2.0))))))))) * 2.0)) 
        v["i6"] = 0.010000*np.tanh(((data["num_peaks_10"]) - (((((((((((data["percentile_roll_std_10"]) + (data["iqr"]))) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)))) 
        v["i7"] = 0.010000*np.tanh(((((((((((data["ffti_range_0_1000"]) - (data["abs_percentile_80"]))) * 2.0)) * 2.0)) * 2.0)) - (((((((data["percentile_roll_std_25"]) - (data["ffti_range_0_1000"]))) * 2.0)) * 2.0)))) 
        v["i8"] = 0.010000*np.tanh((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) - (((data["ffti_range_-3000_-2000"]) - (((((((data["ffti_percentile_30"]) - (data["num_peaks_10"]))) - (data["exp_Moving_average_30000_mean"]))) * 2.0)))))) * 2.0)) 
        v["i9"] = 0.010000*np.tanh((((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) + ((-1.0*((data["num_peaks_10"])))))) * 2.0)) + ((-1.0*((data["num_peaks_10"])))))) 
        v["i10"] = 0.010000*np.tanh(((((((((((-1.0*((((data["ffti_range_-4000_-3000"]) * 2.0))))) * 2.0)) - (data["num_peaks_10"]))) + (((((data["mean_change_rate"]) - (data["abs_max_roll_mean_10000"]))) * 2.0)))/2.0)) - (data["num_peaks_10"]))) 
        v["i11"] = 0.010000*np.tanh(((((data["ffti_range_0_1000"]) - (((((data["abs_percentile_80"]) * 2.0)) + (((((data["abs_percentile_80"]) + (data["abs_percentile_80"]))) / 2.0)))))) - (((data["fftr_range_-3000_-2000"]) * 2.0)))) 
        v["i12"] = 0.010000*np.tanh((((((-1.0*((data["abs_percentile_80"])))) * 2.0)) + (((((((((data["fftr_range_0_1000"]) - (data["abs_percentile_80"]))) * 2.0)) * 2.0)) - (data["abs_percentile_80"]))))) 
        v["i13"] = 0.010000*np.tanh((((((((((((-1.0*((data["iqr"])))) - (data["num_peaks_10"]))) * 2.0)) - (data["num_peaks_10"]))) * 2.0)) * 2.0)) 
        v["i14"] = 0.010000*np.tanh((((((((((-1.0*((data["abs_percentile_80"])))) * 2.0)) + (data["fftr_range_-1000_0"]))) * 2.0)) * 2.0)) 
        v["i15"] = 0.010000*np.tanh(((((((((((((data["ffti_percentile_30"]) - (data["ffti_range_3000_4000"]))) - (data["iqr"]))) * 2.0)) * 2.0)) - (data["fftr_range_0_1000"]))) * 2.0)) 
        v["i16"] = 0.010000*np.tanh(((((((((((((((data["fftr_av_change_abs_roll_mean_100"]) + (data["mean_change_rate"]))) * 2.0)) - (data["ffti_range_-4000_-3000"]))) * 2.0)) * 2.0)) * 2.0)) + ((-1.0*((data["classic_sta_lta5_mean"])))))) 
        v["i17"] = 0.010000*np.tanh(((((((((((((data["range_0_1000"]) - (data["exp_Moving_average_50000_mean"]))) - (data["abs_percentile_60"]))) + (data["ffti_range_-1000_0"]))) + (((data["range_0_1000"]) * 2.0)))) - (data["ffti_range_2000_3000"]))) * 2.0)) 
        v["i18"] = 0.010000*np.tanh(((((((((data["fftr_av_change_abs_roll_mean_50"]) + (((((data["fftr_av_change_abs_roll_mean_50"]) + (((data["range_0_1000"]) * 2.0)))) + (data["ffti_range_-1000_0"]))))) + (data["range_0_1000"]))) - (data["fftr_range_-4000_-3000"]))) * 2.0)) 
        v["i19"] = 0.010000*np.tanh(((((((((((((data["fftr_abs_trend"]) - (data["fftr_range_-3000_-2000"]))) - (data["iqr"]))) - (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i20"] = 0.010000*np.tanh((((-1.0*((data["num_peaks_10"])))) + ((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) - (data["num_peaks_10"]))) * 2.0)))) 
        v["i21"] = 0.010000*np.tanh(((((((((((data["fftr_range_0_1000"]) - (data["percentile_roll_std_30"]))) - (data["percentile_75"]))) - (((data["percentile_roll_std_20"]) - (data["fftr_max_to_min"]))))) * 2.0)) * 2.0)) 
        v["i22"] = 0.010000*np.tanh((((((((((((data["mean_change_rate"]) + (data["fftr_mean_change_abs"]))) * 2.0)) * 2.0)) + (data["range_0_1000"]))/2.0)) * 2.0)) 
        v["i23"] = 0.010000*np.tanh(((((((((data["ffti_range_-1000_0"]) + ((-1.0*((data["num_peaks_10"])))))) * 2.0)) * 2.0)) + (((data["ffti_range_0_1000"]) - (((data["num_peaks_10"]) * 2.0)))))) 
        v["i24"] = 0.010000*np.tanh(((((data["ffti_range_0_1000"]) + (((((data["percentile_roll_mean_80"]) + (((((((data["mean_change_rate"]) - (data["percentile_roll_mean_80"]))) * 2.0)) * 2.0)))) * 2.0)))) - (data["ffti_range_0_1000"]))) 
        v["i25"] = 0.010000*np.tanh(((((((((data["ffti_range_0_1000"]) - (data["ffti_range_-4000_-3000"]))) - (((data["abs_percentile_80"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i26"] = 0.010000*np.tanh(((((((data["mean_change_rate"]) + (((((((((data["mean_change_rate"]) + (data["fftr_av_change_abs_roll_std_1000"]))) * 2.0)) * 2.0)) * 2.0)))) - (data["num_peaks_10"]))) * 2.0)) 
        v["i27"] = 0.010000*np.tanh(((((((((data["ffti_range_-1000_0"]) * 2.0)) - (data["fftr_range_-4000_-3000"]))) + ((((-1.0*((data["percentile_80"])))) - (((data["fftr_range_-4000_-3000"]) + (data["av_change_rate_roll_std_10"]))))))) * 2.0)) 
        v["i28"] = 0.010000*np.tanh(((((((data["fftr_range_0_1000"]) - (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) * 2.0)) - (((data["num_peaks_10"]) * 2.0)))) 
        v["i29"] = 0.010000*np.tanh((((((((((((((-1.0*((data["iqr"])))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i30"] = 0.010000*np.tanh(((data["autocorrelation_10"]) - (((((((data["percentile_roll_std_1"]) + (data["fftr_range_-4000_-3000"]))) * 2.0)) + (((((data["num_peaks_10"]) * 2.0)) + (((data["fftr_range_3000_4000"]) * 2.0)))))))) 
        v["i31"] = 0.010000*np.tanh(((((((data["fftr_max_to_min"]) + (((((data["range_0_1000"]) - (data["abs_percentile_60"]))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i32"] = 0.010000*np.tanh(((((data["iqr"]) * 2.0)) - (((((((data["iqr"]) - (data["fftr_iqr1"]))) + (((((data["percentile_75"]) + (data["percentile_roll_std_30"]))) * 2.0)))) * 2.0)))) 
        v["i33"] = 0.010000*np.tanh(((((((data["num_peaks_10"]) - (((((data["ffti_range_-4000_-3000"]) * 2.0)) * 2.0)))) - (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) * 2.0)) 
        v["i34"] = 0.010000*np.tanh((((((-1.0*((data["percentile_roll_std_50"])))) - (((((((data["ffti_range_3000_4000"]) + (data["percentile_roll_std_50"]))) * 2.0)) * 2.0)))) - ((((-1.0*((data["percentile_roll_std_50"])))) * 2.0)))) 
        v["i35"] = 0.010000*np.tanh(((((((((-1.0) - (data["percentile_roll_std_1"]))) + (((((((data["num_peaks_10"]) - (data["abs_percentile_80"]))) - (data["num_peaks_10"]))) - (data["num_peaks_10"]))))) * 2.0)) * 2.0)) 
        v["i36"] = 0.010000*np.tanh(((((data["fftr_range_-1000_0"]) + ((((((((((-1.0*((data["abs_percentile_80"])))) + ((-1.0*((data["abs_percentile_80"])))))) * 2.0)) * 2.0)) * 2.0)))) - (data["percentile_roll_std_5"]))) 
        v["i37"] = 0.010000*np.tanh(((((data["num_peaks_10"]) - (((((((data["ffti_range_-4000_-3000"]) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)))) - (data["num_peaks_10"]))) 
        v["i38"] = 0.010000*np.tanh(((((((((data["fftr_av_change_abs_roll_mean_1000"]) + ((((((-1.0*((data["ffti_range_-3000_-2000"])))) - (data["percentile_roll_std_40"]))) * 2.0)))) + (data["percentile_10"]))) * 2.0)) + (data["fftr_max_to_min"]))) 
        v["i39"] = 0.010000*np.tanh((((((data["abs_percentile_80"]) + (data["abs_percentile_80"]))/2.0)) + ((((((-1.0*((data["ffti_count_big"])))) - (((((((data["min_roll_std_10000"]) + (data["abs_percentile_80"]))) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i40"] = 0.010000*np.tanh((((-1.0*((((((((data["percentile_75"]) + (data["num_peaks_10"]))) * 2.0)) + ((((-1.0*(((((data["ffti_range_0_1000"]) + ((-1.0*((data["fftr_kstatvar_1"])))))/2.0))))) * 2.0))))))) * 2.0)) 
        v["i41"] = 0.010000*np.tanh(((((((((((((data["ffti_av_change_rate_roll_std_1000"]) * ((-1.0*((data["classic_sta_lta2_mean"])))))) - (((data["iqr"]) * 2.0)))) * 2.0)) - (((data["percentile_roll_std_1"]) * 2.0)))) * 2.0)) * 2.0)) 
        v["i42"] = 0.010000*np.tanh(((((((((data["fftr_abs_percentile_30"]) - (data["percentile_roll_std_1"]))) - (((((data["ffti_range_3000_4000"]) + (data["percentile_roll_std_20"]))) * 2.0)))) - (data["percentile_80"]))) - (data["ffti_range_2000_3000"]))) 
        v["i43"] = 0.010000*np.tanh((((((-1.0*((((data["abs_percentile_80"]) * 2.0))))) - (((((data["abs_percentile_80"]) * 2.0)) * 2.0)))) - (((((data["min_roll_std_10000"]) * 2.0)) * 2.0)))) 
        v["i44"] = 0.010000*np.tanh((-1.0*((((((((((((data["fftr_classic_sta_lta4_mean"]) + (data["abs_percentile_80"]))) * 2.0)) * 2.0)) * 2.0)) + (((data["abs_percentile_80"]) * 2.0))))))) 
        v["i45"] = 0.010000*np.tanh((((((((-1.0*((((((data["num_peaks_10"]) * 2.0)) * 2.0))))) - (data["mean_first_50000"]))) * 2.0)) - (((((((data["percentile_roll_mean_30"]) - (data["range_0_1000"]))) * 2.0)) * 2.0)))) 
        v["i46"] = 0.010000*np.tanh((((((((-1.0*((((data["iqr"]) * 2.0))))) * 2.0)) + (((((data["iqr"]) * 2.0)) / 2.0)))) - (((((((data["iqr"]) * 2.0)) * 2.0)) * 2.0)))) 
        v["i47"] = 0.010000*np.tanh(((((data["ffti_range_0_1000"]) + (((((((data["range_0_1000"]) - (data["abs_percentile_70"]))) * 2.0)) + (((data["range_0_1000"]) * 2.0)))))) * 2.0)) 
        v["i48"] = 0.010000*np.tanh((((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) - (data["num_peaks_10"]))) + ((-1.0*((((data["iqr"]) * 2.0))))))) * 2.0)) 
        v["i49"] = 0.010000*np.tanh(((((((((((data["num_peaks_10"]) - (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i50"] = 0.010000*np.tanh((((((((((-1.0*((((((data["abs_percentile_80"]) * 2.0)) * 2.0))))) - (data["num_peaks_10"]))) - (data["num_peaks_10"]))) * 2.0)) * 2.0)) 
        v["i51"] = 0.010000*np.tanh(((((data["fftr_range_-4000_-3000"]) - (((((((data["percentile_75"]) / 2.0)) + (((((((data["percentile_75"]) + (data["percentile_roll_std_10"]))) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i52"] = 0.010000*np.tanh(((((((((data["fftr_range_-1000_0"]) - (data["num_peaks_10"]))) * 2.0)) * 2.0)) - (((data["num_peaks_10"]) - ((-1.0*((((((data["num_peaks_10"]) * 2.0)) * 2.0))))))))) 
        v["i53"] = 0.010000*np.tanh(((((((((data["ffti_range_0_1000"]) - (data["num_peaks_10"]))) - (data["num_peaks_10"]))) + (((data["fftr_range_-1000_0"]) - (data["exp_Moving_average_30000_mean"]))))) * 2.0)) 
        v["i54"] = 0.010000*np.tanh(((data["ffti_num_peaks_100"]) - (((((((data["ffti_range_-4000_-3000"]) * 2.0)) * 2.0)) + (((data["percentile_75"]) + (((data["percentile_roll_std_10"]) * 2.0)))))))) 
        v["i55"] = 0.010000*np.tanh((((((((((((((-1.0*((data["abs_percentile_80"])))) * 2.0)) + ((-1.0*((data["num_peaks_10"])))))) * 2.0)) * 2.0)) * 2.0)) + (((data["fftr_min_roll_mean_10"]) * 2.0)))) 
        v["i56"] = 0.010000*np.tanh((((-1.0*((((data["abs_percentile_80"]) + (((data["abs_percentile_80"]) * 2.0))))))) - (((data["min_roll_std_1000"]) * 2.0)))) 
        v["i57"] = 0.010000*np.tanh((((-1.0*((((((((data["iqr"]) + (((data["iqr"]) * 2.0)))) + (data["iqr"]))) * 2.0))))) - (data["num_peaks_10"]))) 
        v["i58"] = 0.010000*np.tanh(((data["autocorrelation_10"]) - (((data["num_peaks_10"]) + (((((data["percentile_75"]) + (data["min_roll_std_10000"]))) * 2.0)))))) 
        v["i59"] = 0.010000*np.tanh(((((((((((((data["mean_change_rate"]) - ((((-1.0*(((-1.0*((data["ffti_kstat_3"]))))))) * 2.0)))) + (data["fftr_av_change_abs_roll_mean_500"]))) * 2.0)) * 2.0)) * 2.0)) - (data["fftr_av_change_abs_roll_mean_500"]))) 
        v["i60"] = 0.010000*np.tanh(((((((data["autocorrelation_10"]) - (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) - (((data["ffti_range_-4000_-3000"]) + (((data["percentile_roll_std_1"]) + (data["ffti_range_-1000_0"]))))))) * 2.0)) 
        v["i61"] = 0.010000*np.tanh((((((((-1.0*((data["num_peaks_10"])))) - (((((((((data["num_peaks_10"]) * 2.0)) * 2.0)) + (data["num_peaks_10"]))) * 2.0)))) - ((-1.0*((data["num_peaks_10"])))))) * 2.0)) 
        v["i62"] = 0.010000*np.tanh((((((((-1.0*((data["iqr"])))) * 2.0)) * 2.0)) * 2.0)) 
        v["i63"] = 0.010000*np.tanh((((((((((((-1.0*((data["abs_percentile_80"])))) - (data["ffti_range_-4000_-3000"]))) * 2.0)) - (data["ffti_range_-4000_-3000"]))) * 2.0)) * 2.0)) 
        v["i64"] = 0.010000*np.tanh(((((data["autocorrelation_10"]) - (((data["ffti_range_3000_4000"]) + (data["min_roll_std_10000"]))))) * 2.0)) 
        v["i65"] = 0.010000*np.tanh(((((((data["fftr_max_to_min"]) - (((((data["num_peaks_10"]) + (data["num_peaks_10"]))) * 2.0)))) - (((((((((data["num_peaks_10"]) * 2.0)) * 2.0)) / 2.0)) * 2.0)))) * 2.0)) 
        v["i66"] = 0.010000*np.tanh((((-1.0*((((((((data["iqr"]) * 2.0)) + (data["iqr"]))) * 2.0))))) - (((data["percentile_95"]) * 2.0)))) 
        v["i67"] = 0.010000*np.tanh((((((((-1.0*((((((data["ffti_range_3000_4000"]) * 2.0)) + (data["mean_first_50000"])))))) * 2.0)) - (((((data["ffti_range_3000_4000"]) + (data["percentile_roll_std_5"]))) * 2.0)))) * 2.0)) 
        v["i68"] = 0.010000*np.tanh(((((((data["num_peaks_10"]) + ((-1.0*((data["abs_percentile_80"])))))) - (((data["ffti_range_3000_4000"]) + (data["min_roll_std_10000"]))))) - (((((data["fftr_range_-3000_-2000"]) + (data["abs_percentile_80"]))) * 2.0)))) 
        v["i69"] = 0.010000*np.tanh(((((((((((data["fftr_av_change_abs_roll_mean_500"]) - (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)) - (data["num_peaks_10"]))) * 2.0)) 
        v["i70"] = 0.010000*np.tanh(((((data["num_peaks_10"]) - (((((data["percentile_roll_std_20"]) + (((data["ffti_range_3000_4000"]) + (data["num_peaks_10"]))))) * 2.0)))) - (((data["percentile_roll_std_30"]) * 2.0)))) 
        v["i71"] = 0.010000*np.tanh(((((((((data["range_0_1000"]) - (data["abs_percentile_80"]))) * 2.0)) + ((((-1.0*((data["num_peaks_10"])))) * 2.0)))) * 2.0)) 
        v["i72"] = 0.010000*np.tanh(((((data["autocorrelation_10"]) - (data["num_peaks_10"]))) - (((data["abs_percentile_80"]) + (((((((((((data["num_peaks_10"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i73"] = 0.010000*np.tanh((((-1.0*((data["num_peaks_10"])))) - (((((((data["ffti_classic_sta_lta2_mean"]) + (data["percentile_roll_mean_40"]))) + (data["percentile_roll_std_1"]))) + (((data["percentile_roll_mean_5"]) + (((data["num_peaks_10"]) * 2.0)))))))) 
        v["i74"] = 0.010000*np.tanh((((((((-1.0*((((data["percentile_75"]) + (data["num_peaks_10"])))))) - (data["fftr_max"]))) - (data["ffti_range_3000_4000"]))) - (data["num_peaks_10"]))) 
        v["i75"] = 0.010000*np.tanh((((((((-1.0*((data["min_roll_std_10000"])))) + ((((-1.0*((data["min_roll_std_10000"])))) + ((((-1.0*((data["num_peaks_10"])))) * 2.0)))))) * 2.0)) - (((data["num_peaks_10"]) * 2.0)))) 
        v["i76"] = 0.010000*np.tanh(((data["fftr_av_change_abs_roll_mean_1000"]) + (((((data["range_0_1000"]) - (((data["num_peaks_10"]) * 2.0)))) + (((data["fftr_classic_sta_lta7_mean"]) + (data["mean_change_rate_last_50000"]))))))) 
        v["i77"] = 0.010000*np.tanh((((((((-1.0*((data["abs_percentile_80"])))) - (data["abs_percentile_80"]))) * 2.0)) + ((((-1.0*((data["abs_percentile_80"])))) + ((((((-1.0*((data["abs_percentile_80"])))) * 2.0)) * 2.0)))))) 
        v["i78"] = 0.010000*np.tanh((((((((-1.0*((data["num_peaks_10"])))) * 2.0)) - (((data["iqr"]) + (((data["num_peaks_10"]) * 2.0)))))) - (((data["num_peaks_10"]) - ((-1.0*((data["gmean"])))))))) 
        v["i79"] = 0.010000*np.tanh(((((data["ffti_percentile_30"]) - (((((((data["ffti_range_3000_4000"]) + (data["ffti_iqr"]))) + (data["percentile_roll_std_10"]))) + (((data["percentile_roll_std_10"]) + (data["ffti_range_3000_4000"]))))))) * 2.0)) 
        v["i80"] = 0.010000*np.tanh(((((((data["percentile_10"]) - (data["percentile_roll_mean_20"]))) * 2.0)) - (((data["min_roll_std_1000"]) + (data["range_-1000_0"]))))) 
        v["i81"] = 0.010000*np.tanh((((-1.0*((((((data["num_peaks_10"]) - ((-1.0*((((data["num_peaks_10"]) * 2.0))))))) * 2.0))))) - (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) 
        v["i82"] = 0.010000*np.tanh(((((((data["percentile_roll_std_25"]) - (data["percentile_roll_std_25"]))) * 2.0)) + (((((((data["autocorrelation_10"]) - (data["abs_percentile_80"]))) * 2.0)) - (((data["num_peaks_10"]) + (data["abs_percentile_80"]))))))) 
        v["i83"] = 0.010000*np.tanh(((((((((((-1.0) - (data["num_peaks_10"]))) - (((data["num_peaks_10"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i84"] = 0.010000*np.tanh((((((((((data["ffti_percentile_20"]) - (data["ffti_av_change_rate_roll_std_10000"]))) + ((-1.0*((((data["ffti_range_3000_4000"]) * 2.0))))))/2.0)) - (((data["abs_percentile_80"]) * 2.0)))) * 2.0)) 
        v["i85"] = 0.010000*np.tanh(((((((((((data["mean_change_rate"]) + (data["fftr_av_change_abs_roll_std_100"]))) * 2.0)) + (((data["fftr_av_change_abs_roll_std_100"]) + (data["mean_change_rate"]))))) * 2.0)) * 2.0)) 
        v["i86"] = 0.010000*np.tanh((((((((((-1.0*(((((data["abs_percentile_80"]) + (((data["abs_percentile_80"]) * 2.0)))/2.0))))) * 2.0)) - (data["percentile_roll_std_40"]))) * 2.0)) - (((((data["abs_percentile_80"]) * 2.0)) * 2.0)))) 
        v["i87"] = 0.010000*np.tanh(((((((data["mean_change_rate"]) - (data["ave_roll_mean_500"]))) - (data["ffti_range_-4000_-3000"]))) * 2.0)) 
        v["i88"] = 0.010000*np.tanh(((((((((data["ffti_percentile_30"]) - (((data["num_peaks_10"]) + (data["num_peaks_10"]))))) - (data["num_peaks_10"]))) - ((((((3.0) + (data["percentile_80"]))/2.0)) * 2.0)))) * 2.0)) 
        v["i89"] = 0.010000*np.tanh(((data["autocorrelation_10"]) + ((((((((((-1.0*((((data["abs_percentile_80"]) * 2.0))))) * 2.0)) * 2.0)) + (((data["autocorrelation_10"]) * 2.0)))) * 2.0)))) 
        v["i90"] = 0.010000*np.tanh((((((((((-1.0*((data["percentile_75"])))) - (((data["num_peaks_10"]) + (data["percentile_75"]))))) * 2.0)) + ((((-1.0*((data["fftr_exp_Moving_average_300_mean"])))) - (data["num_peaks_10"]))))) * 2.0)) 
        v["i91"] = 0.010000*np.tanh(((((((((((((-1.0) - (((data["min_roll_std_100"]) + (((data["num_peaks_10"]) * 2.0)))))) * 2.0)) * 2.0)) + (data["ffti_std_roll_std_10"]))) - (data["num_peaks_10"]))) * 2.0)) 
        v["i92"] = 0.010000*np.tanh(((data["autocorrelation_10"]) - (((((data["percentile_roll_std_30"]) + (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) + (((data["num_peaks_10"]) - (((data["autocorrelation_10"]) * 2.0)))))))) 
        v["i93"] = 0.010000*np.tanh(((((((data["iqr"]) * 2.0)) - (((((((((((data["iqr"]) * 2.0)) * 2.0)) + (data["percentile_roll_std_5"]))) * 2.0)) * 2.0)))) - (data["iqr"]))) 
        v["i94"] = 0.010000*np.tanh(((((((((data["ffti_range_-1000_0"]) * 2.0)) - (((((data["abs_percentile_80"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i95"] = 0.010000*np.tanh(((((((((data["autocorrelation_10"]) - (((((data["percentile_roll_std_1"]) + (data["min_roll_std_1000"]))) * 2.0)))) - (data["percentile_roll_std_50"]))) - (((data["fftr_exp_Moving_average_300_std"]) * 2.0)))) - (data["autocorrelation_50"]))) 
        v["i96"] = 0.010000*np.tanh((((((-1.0*((((((data["num_peaks_10"]) * 2.0)) * 2.0))))) - (((((((data["num_peaks_10"]) * 2.0)) + (((data["iqr"]) * 2.0)))) * 2.0)))) - (data["percentile_roll_mean_90"]))) 
        v["i97"] = 0.010000*np.tanh(((data["ffti_percentile_10"]) - (((data["num_peaks_10"]) + (((((((((data["num_peaks_10"]) + (((data["percentile_75"]) * 2.0)))) * 2.0)) * 2.0)) + (data["percentile_75"]))))))) 
        v["i98"] = 0.010000*np.tanh(((((data["percentile_roll_std_20"]) - (((data["percentile_roll_std_20"]) * 2.0)))) + (((((((((-1.0) - (data["percentile_roll_std_20"]))) * 2.0)) - (data["ffti_abs_max_roll_mean_10"]))) * 2.0)))) 
        v["i99"] = 0.010000*np.tanh(((((data["mean_change_rate"]) + (data["fftr_mean_change_abs"]))) + (((((data["range_0_1000"]) + (data["fftr_mean_change_abs"]))) + (data["autocorrelation_10"]))))) 
        v["i100"] = 0.010000*np.tanh((-1.0*((((((((((data["ffti_std_roll_mean_10"]) + (1.0))) * 2.0)) + (((((data["num_peaks_10"]) + (((data["num_peaks_10"]) + (data["min_roll_std_500"]))))) * 2.0)))) * 2.0))))) 
        v["i101"] = 0.010000*np.tanh(((((data["mean_change_rate_first_50000"]) + (((data["mean_change_rate_first_50000"]) - (((((((data["min_roll_std_1000"]) * 2.0)) * 2.0)) * 2.0)))))) - (data["time_rev_asym_stat_50"]))) 
        v["i102"] = 0.010000*np.tanh((((-1.0*((1.0)))) - (((((data["iqr"]) - (data["mean_change_rate_first_1000"]))) + (data["min_roll_std_10000"]))))) 
        v["i103"] = 0.010000*np.tanh(((((((data["av_change_rate_roll_std_1000"]) - (((((data["ffti_percentile_roll_std_95"]) + (((data["percentile_roll_std_5"]) * 2.0)))) + (data["percentile_roll_std_1"]))))) * 2.0)) + (-3.0))) 
        v["i104"] = 0.010000*np.tanh(((((((-2.0) - (data["iqr"]))) - (data["iqr"]))) - (((data["num_peaks_10"]) - (((data["ffti_abs_max_roll_std_1000"]) - (data["num_peaks_10"]))))))) 
        v["i105"] = 0.010000*np.tanh(((((((((((((data["mean_change_rate"]) - (data["ave_roll_mean_1000"]))) * 2.0)) * 2.0)) * 2.0)) - (((data["iqr"]) + (data["percentile_75"]))))) + (data["mean_change_rate"]))) 
        v["i106"] = 0.010000*np.tanh(((((((data["mean_change_rate_first_50000"]) * 2.0)) - (data["percentile_75"]))) - (((((3.0)) + (((data["percentile_75"]) * 2.0)))/2.0)))) 
        v["i107"] = 0.010000*np.tanh(((((((-2.0) - (data["num_peaks_50"]))) - (data["abs_percentile_80"]))) - (((data["num_peaks_10"]) * 2.0)))) 
        v["i108"] = 0.010000*np.tanh(((((((((data["percentile_75"]) * 2.0)) - (data["percentile_75"]))) - (((((data["percentile_75"]) * 2.0)) * 2.0)))) + (data["fftr_max_to_min"]))) 
        v["i109"] = 0.010000*np.tanh((((((-1.0*((((data["percentile_roll_mean_5"]) + (data["percentile_roll_std_1"])))))) - (((data["std_first_1000"]) + (((data["percentile_roll_std_1"]) + (((data["min_roll_std_1000"]) * 2.0)))))))) + (-2.0))) 
        v["i110"] = 0.010000*np.tanh(((data["ffti_min_roll_mean_100"]) + (((((((((((data["autocorrelation_10"]) - (data["min_roll_std_1000"]))) * 2.0)) + (data["autocorrelation_10"]))) * 2.0)) - (data["fftr_min_roll_mean_100"]))))) 
        v["i111"] = 0.010000*np.tanh(((((((((data["mean_change_rate"]) - (data["percentile_75"]))) * 2.0)) - (data["autocorrelation_10"]))) - (2.0))) 
        v["i112"] = 0.010000*np.tanh(((((((((((((data["mean_change_rate"]) - (data["med"]))) * 2.0)) - (((1.0) + (data["fftr_std_roll_mean_100"]))))) - (data["percentile_roll_std_1"]))) * 2.0)) * 2.0)) 
        v["i113"] = 0.010000*np.tanh((((((((((data["exp_Moving_average_300_std"]) + (((data["mean_change_rate"]) - (data["percentile_75"]))))/2.0)) + (-2.0))) + (((data["mean_change_rate"]) - (data["percentile_75"]))))) * 2.0)) 
        v["i114"] = 0.010000*np.tanh(((((((((((data["mean_change_rate_first_1000"]) - (data["num_peaks_10"]))) * 2.0)) * 2.0)) / 2.0)) - (2.0))) 
        v["i115"] = 0.010000*np.tanh(((((((-2.0) - (((((data["min_roll_std_10000"]) * 2.0)) * 2.0)))) * 2.0)) - (((data["autocorrelation_10"]) - (-2.0))))) 
        v["i116"] = 0.010000*np.tanh(((((((((-2.0) - (((data["num_peaks_10"]) * 2.0)))) * 2.0)) + (data["ffti_percentile_roll_std_99"]))) + (((((data["num_peaks_10"]) - (data["num_peaks_10"]))) * 2.0)))) 
        v["i117"] = 0.010000*np.tanh(((((data["mean_change_rate"]) - ((((data["exp_Moving_average_3000_mean"]) + ((((data["num_peaks_10"]) + (((((3.0) + (((data["num_peaks_10"]) * 2.0)))) * 2.0)))/2.0)))/2.0)))) * 2.0)) 
        v["i118"] = 0.010000*np.tanh(((((((((((data["ffti_min_roll_mean_10"]) + (((data["autocorrelation_10"]) + (data["fftr_min_roll_mean_500"]))))) * 2.0)) * 2.0)) + (data["percentile_roll_std_1"]))) + (((((data["mean_change_rate_last_10000"]) * 2.0)) / 2.0)))) 
        v["i119"] = 0.010000*np.tanh(((((((((((data["mean_change_rate"]) - (data["fftr_exp_Moving_average_30000_std"]))) * 2.0)) - (data["mean_change_rate"]))) * 2.0)) * 2.0)) 
        v["i120"] = 0.010000*np.tanh((((((((((((((-1.0) + (-1.0))/2.0)) + (((data["range_0_1000"]) + (data["mean_change_rate_first_1000"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i121"] = 0.010000*np.tanh(((((((((-3.0) - (((data["num_peaks_10"]) * 2.0)))) * 2.0)) - (((((data["num_peaks_10"]) * 2.0)) * 2.0)))) + (((-3.0) - (data["percentile_20"]))))) 
        v["i122"] = 0.010000*np.tanh(((((((((-2.0) - (((data["percentile_roll_std_1"]) * 2.0)))) * 2.0)) + (((data["mean_change_rate_first_1000"]) + (data["mean_change_rate_first_1000"]))))) * 2.0)) 
        v["i123"] = 0.010000*np.tanh(((((data["range_0_1000"]) - (((((data["min_roll_std_1000"]) - (((data["mean_change_rate_first_1000"]) + (-2.0))))) - (((data["percentile_75"]) - (data["percentile_75"]))))))) + (data["mean_change_rate_first_1000"]))) 
        v["i124"] = 0.010000*np.tanh(((data["min_roll_std_1000"]) - (((((data["num_peaks_10"]) + (((((((((data["num_peaks_10"]) + (((((3.0)) + (data["num_peaks_10"]))/2.0)))) * 2.0)) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i125"] = 0.010000*np.tanh(((((data["mean_change_rate_last_50000"]) - (2.0))) + (((((data["mean_change_rate_last_50000"]) - (data["percentile_75"]))) - (data["percentile_75"]))))) 
        v["i126"] = 0.010000*np.tanh(((-2.0) + (((((data["autocorrelation_10"]) + (((data["mean_change_rate_last_10000"]) + (data["mean_change_rate_last_10000"]))))) + (data["ffti_min_first_10000"]))))) 
        v["i127"] = 0.010000*np.tanh(((-3.0) + ((((-1.0*((((data["num_peaks_10"]) * 2.0))))) - (((((((data["num_peaks_10"]) + (data["num_peaks_10"]))) * 2.0)) * 2.0)))))) 
        v["i128"] = 0.010000*np.tanh(((-2.0) + (((((data["mean_change_rate"]) * 2.0)) - (data["percentile_75"]))))) 
        v["i129"] = 0.010000*np.tanh(((-3.0) + (((((((((((((data["mean_change_rate_first_1000"]) * 2.0)) * 2.0)) * 2.0)) - (data["skew"]))) - (data["percentile_75"]))) - (data["percentile_roll_std_30"]))))) 
        v["i130"] = 0.010000*np.tanh(((((((((((-2.0) - (((data["percentile_75"]) * 2.0)))) + (data["mean_change_rate_first_1000"]))) + (data["mean_change_rate"]))) * 2.0)) * 2.0)) 
        v["i131"] = 0.010000*np.tanh(((((data["range_0_1000"]) * 2.0)) + (((((-3.0) + (-3.0))) + (((((data["range_0_1000"]) - (data["min_roll_std_500"]))) * 2.0)))))) 
        v["i132"] = 0.010000*np.tanh(((((((((data["mean_change_rate_first_1000"]) - (data["min_roll_std_100"]))) * 2.0)) + (((data["skew"]) * 2.0)))) * 2.0)) 
        v["i133"] = 0.010000*np.tanh(((((((((data["range_0_1000"]) + (-1.0))) * 2.0)) + (((data["skew"]) * 2.0)))) * 2.0)) 
        v["i134"] = 0.010000*np.tanh(((((data["mean_change_rate_first_1000"]) + (((((data["range_0_1000"]) + (((data["percentile_5"]) + (data["skew"]))))) * 2.0)))) + (-3.0))) 
        v["i135"] = 0.010000*np.tanh(((data["mean_change_rate_last_10000"]) + (((-3.0) + (((data["percentile_10"]) + (data["skew"]))))))) 
        v["i136"] = 0.010000*np.tanh((((((((-1.0*((data["percentile_75"])))) + (((-2.0) + (data["mean_change_rate_first_50000"]))))) - (-1.0))) * 2.0)) 
        v["i137"] = 0.010000*np.tanh(((data["mean_change_rate_first_1000"]) + (((((-1.0) - (((data["abs_percentile_90"]) * 2.0)))) * 2.0)))) 
        v["i138"] = 0.010000*np.tanh(((((((data["mean_change_rate_first_1000"]) + (((((data["skew"]) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i139"] = 0.010000*np.tanh(((data["mean_change_rate"]) + (((((((data["skew"]) * 2.0)) - (data["percentile_roll_std_20"]))) + (((-3.0) + (((data["skew"]) - (data["percentile_roll_std_5"]))))))))) 
        v["i140"] = 0.010000*np.tanh(((((data["mean_change_rate_first_1000"]) * 2.0)) + (((((data["skew"]) * 2.0)) * 2.0)))) 
        v["i141"] = 0.010000*np.tanh((((((((data["max_to_min"]) * 2.0)) + (data["mean_change_rate_first_1000"]))/2.0)) + (((((data["mean_change_rate_last_10000"]) + (-2.0))) + (data["fftr_abs_percentile_10"]))))) 
        v["i142"] = 0.010000*np.tanh(((((((((-2.0) + (data["mean_change_rate_first_50000"]))) + (data["fftr_av_change_abs_roll_std_10"]))) * 2.0)) + (((data["mean_change_rate"]) + (data["mean_change_rate_first_50000"]))))) 
        v["i143"] = 0.010000*np.tanh(((((-3.0) - (((data["min_roll_std_10000"]) * 2.0)))) - (((((data["min_roll_std_1000"]) * 2.0)) - (((data["autocorrelation_5000"]) - (data["autocorrelation_5000"]))))))) 
        v["i144"] = 0.010000*np.tanh(((((-1.0) - (data["num_peaks_10"]))) + (((((((-1.0) - (data["num_peaks_10"]))) + (data["exp_Moving_average_50000_std"]))) * 2.0)))) 
        v["i145"] = 0.010000*np.tanh((((((((data["skew"]) + (-3.0))) + (((((data["abs_max_roll_std_100"]) - (1.0))) - (1.0))))/2.0)) - (((data["num_peaks_10"]) * 2.0)))) 
        v["i146"] = 0.010000*np.tanh(((((((((data["skew"]) - (data["min_roll_std_500"]))) * 2.0)) - (data["percentile_75"]))) * 2.0)) 
        v["i147"] = 0.010000*np.tanh((((((((-3.0) + (data["mean_change_rate"]))/2.0)) * 2.0)) + (((data["mean_change_rate_first_50000"]) + (((((data["mean_change_rate"]) - (data["percentile_roll_mean_10"]))) * 2.0)))))) 
        v["i148"] = 0.010000*np.tanh(((data["ffti_min_roll_std_50"]) + (((((data["fftr_mean_first_1000"]) + (((((-2.0) + (data["mean_change_rate_first_10000"]))) - (data["percentile_75"]))))) - (data["percentile_75"]))))) 
        v["i149"] = 0.010000*np.tanh(((((((((data["mean_change_rate"]) * 2.0)) * 2.0)) - (((data["fftr_Hann_window_mean_1500"]) - (((((((-3.0) - (data["percentile_75"]))) - (data["percentile_75"]))) * 2.0)))))) * 2.0)) 
        v["i150"] = 0.010000*np.tanh(((((((data["skew"]) * 2.0)) - ((((2.37314748764038086)) - ((-1.0*((data["min_roll_std_1000"])))))))) + (((((data["skew"]) * 2.0)) + (data["range_p4000_pinf"]))))) 
        v["i151"] = 0.010000*np.tanh(((data["range_0_1000"]) + (((((((((((data["fftr_mean_last_1000"]) + (data["range_0_1000"]))) + (-1.0))) + (data["abs_percentile_20"]))) * 2.0)) + (((data["ffti_percentile_roll_std_50"]) * 2.0)))))) 
        v["i152"] = 0.010000*np.tanh(((data["ffti_exp_Moving_average_30000_std"]) + ((((((((((((((-1.0) - (data["num_peaks_10"]))) * 2.0)) * 2.0)) + (-3.0))/2.0)) + (data["percentile_10"]))) * 2.0)))) 
        v["i153"] = 0.010000*np.tanh((((((-1.0*((data["min_roll_std_100"])))) + (((-2.0) + (((data["mean_change_rate_last_10000"]) + (((data["skew"]) * 2.0)))))))) + (data["ffti_mean_change_rate"]))) 
        v["i154"] = 0.010000*np.tanh(((((((data["fftr_mean_last_1000"]) + ((((data["mean_change_rate"]) + (data["ffti_min_roll_std_100"]))/2.0)))) + ((((-2.0) + (((((data["ffti_min_roll_mean_10"]) + (data["fftr_min_roll_mean_100"]))) * 2.0)))/2.0)))) * 2.0)) 
        v["i155"] = 0.010000*np.tanh(((-3.0) + (((((((((data["skew"]) + (data["ffti_exp_Moving_average_30000_std"]))) * 2.0)) + (((data["mean_change_rate_first_1000"]) * 2.0)))) * 2.0)))) 
        v["i156"] = 0.010000*np.tanh((((((((((((((data["max_to_min"]) + (-2.0))/2.0)) - (data["min_roll_std_1000"]))) * 2.0)) + (data["exp_Moving_average_50000_std"]))) + (data["ffti_min_roll_std_100"]))) * 2.0)) 
        v["i157"] = 0.010000*np.tanh(((-2.0) + (((((((data["abs_percentile_30"]) + ((-1.0*((((data["percentile_75"]) * 2.0))))))) * 2.0)) + (((-3.0) - (-3.0))))))) 
        v["i158"] = 0.010000*np.tanh(((((((((data["mean_change_rate_first_10000"]) + ((((data["mean_change_rate"]) + (((((-2.0) * 2.0)) - (((data["fftr_autocorrelation_100"]) - (data["mean_change_rate"]))))))/2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i159"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_std_50"]) - (data["percentile_roll_std_25"]))) * 2.0)) + (-2.0))) 
        v["i160"] = 0.010000*np.tanh(((((((-2.0) + (-2.0))) + (((((data["mean_change_rate_last_10000"]) + (((data["ffti_min_roll_std_50"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i161"] = 0.010000*np.tanh(((((((((((((data["ffti_percentile_roll_std_50"]) - (data["percentile_95"]))) * 2.0)) * 2.0)) + (data["ffti_percentile_roll_std_50"]))) - (((data["ffti_min_roll_mean_500"]) - (-1.0))))) * 2.0)) 
        v["i162"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_100"]) + (((data["mean_change_rate_first_1000"]) * 2.0)))) + (((((data["skew"]) * 2.0)) + (((data["mean_change_rate_first_1000"]) * 2.0)))))) 
        v["i163"] = 0.010000*np.tanh(((data["fftr_abs_max_roll_mean_10000"]) * ((((((13.49804401397705078)) + ((((data["fftr_abs_max_roll_mean_10000"]) + ((13.49804115295410156)))/2.0)))) + (((data["ffti_min_roll_std_100"]) * 2.0)))))) 
        v["i164"] = 0.010000*np.tanh((-1.0*((((((data["min_roll_std_100"]) - (((((((((data["ffti_min_roll_std_100"]) - (data["min_roll_std_100"]))) + (data["exp_Moving_average_50000_std"]))) + (data["exp_Moving_average_50000_std"]))) * 2.0)))) * 2.0))))) 
        v["i165"] = 0.010000*np.tanh(((-2.0) * 2.0)) 
        v["i166"] = 0.010000*np.tanh(((data["mean_change_rate"]) + ((((-1.0*((((data["percentile_75"]) * 2.0))))) + (-2.0))))) 
        v["i167"] = 0.010000*np.tanh((((((((((((data["ffti_min_roll_std_10"]) + (data["ffti_mean"]))/2.0)) * 2.0)) + (-3.0))) + (((((data["mean_change_rate"]) + (data["ffti_min_first_10000"]))) + (data["mean_change_rate"]))))) * 2.0)) 
        v["i168"] = 0.010000*np.tanh(((((data["abs_percentile_20"]) + (((((((((data["skew"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) 
        v["i169"] = 0.010000*np.tanh(((((((((((data["fftr_mean_last_1000"]) * 2.0)) + (data["fftr_time_rev_asym_stat_100"]))) - (data["ffti_autocorrelation_100"]))) + (data["kurt"]))) * 2.0)) 
        v["i170"] = 0.010000*np.tanh(((-2.0) + (((((((((data["ffti_min_roll_std_50"]) + (((((data["ffti_min_roll_mean_500"]) * 2.0)) * 2.0)))) * 2.0)) + (data["ffti_exp_Moving_average_30000_std"]))) + (data["range_0_1000"]))))) 
        v["i171"] = 0.010000*np.tanh(-2.0) 
        v["i172"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_std_30"]) * 2.0)) + (((((data["skew"]) * 2.0)) + (((data["ffti_percentile_roll_std_30"]) * 2.0)))))) 
        v["i173"] = 0.010000*np.tanh(((((data["fftr_mean_last_1000"]) - (data["percentile_roll_std_70"]))) - (((data["min_roll_std_1000"]) - (((((((((data["ffti_percentile_roll_std_50"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) 
        v["i174"] = 0.010000*np.tanh(((data["ffti_min_roll_std_10"]) + (((data["exp_Moving_average_30000_std"]) + (((((data["ffti_min_roll_std_10"]) * 2.0)) * 2.0)))))) 
        v["i175"] = 0.010000*np.tanh(((((data["fftr_exp_Moving_average_50000_std"]) * (data["ffti_min_roll_mean_10"]))) + (((((data["mean_last_1000"]) + (((((data["ffti_percentile_roll_std_20"]) + (((((data["mean_last_1000"]) * 2.0)) * 2.0)))) * 2.0)))) * 2.0)))) 
        v["i176"] = 0.010000*np.tanh(((((((((data["ffti_min_roll_mean_500"]) + (data["ffti_exp_Moving_average_50000_std"]))) - (data["ffti_exp_Moving_average_50000_std"]))) + (((data["ffti_percentile_roll_std_20"]) * 2.0)))) - (((data["ffti_autocorrelation_500"]) - (data["ffti_percentile_roll_std_50"]))))) 
        v["i177"] = 0.010000*np.tanh(((((((((((((((data["mean_last_1000"]) - (data["percentile_roll_mean_30"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((data["fftr_mean_change_abs"]) * 2.0)))) * 2.0)) 
        v["i178"] = 0.010000*np.tanh(((((((data["min_roll_std_1000"]) + (data["ffti_min_roll_std_50"]))) * 2.0)) + ((((((((data["ffti_min_roll_std_50"]) * 2.0)) * 2.0)) + (-3.0))/2.0)))) 
        v["i179"] = 0.010000*np.tanh(((((-3.0) * 2.0)) + (-3.0))) 
        v["i180"] = 0.010000*np.tanh((((((((((data["abs_percentile_40"]) * 2.0)) + ((((data["fftr_av_change_abs_roll_std_1000"]) + (((((data["abs_percentile_60"]) / 2.0)) * 2.0)))/2.0)))/2.0)) - (data["ave_roll_mean_10"]))) * 2.0)) 
        v["i181"] = 0.010000*np.tanh((((data["av_change_abs_roll_mean_10000"]) + (((data["max_to_min"]) + (data["ffti_num_peaks_10"]))))/2.0)) 
        v["i182"] = 0.010000*np.tanh(((data["fftr_mean_last_1000"]) + (((((((((((((data["mean_change_rate"]) - (data["ave_roll_mean_1000"]))) + (data["fftr_range_2000_3000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i183"] = 0.010000*np.tanh(((((data["max_first_10000"]) - (data["percentile_roll_std_5"]))) - ((((((data["mean_change_rate"]) + (data["autocorrelation_5000"]))) + (data["kurt"]))/2.0)))) 
        v["i184"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_std_50"]) + (data["ffti_exp_Moving_average_50000_std"]))) * 2.0)) + (((((0.0) + (data["ffti_exp_Moving_average_50000_std"]))) - (data["ffti_max_to_min"]))))) 
        v["i185"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_50"]) * 2.0)) + (((data["ffti_min_roll_std_50"]) + (-2.0))))) 
        v["i186"] = 0.010000*np.tanh(((((((((((data["ffti_percentile_roll_std_50"]) * 2.0)) + (((((data["fftr_abs_percentile_10"]) + (data["range_0_1000"]))) * 2.0)))) - (data["ffti_autocorrelation_50"]))) * 2.0)) + (data["ffti_percentile_roll_std_50"]))) 
        v["i187"] = 0.010000*np.tanh(((((((data["abs_percentile_40"]) - (data["fftr_autocorrelation_500"]))) + (data["ffti_ave_roll_mean_10"]))) + (((data["ffti_percentile_roll_std_50"]) * 2.0)))) 
        v["i188"] = 0.010000*np.tanh((((((data["skew"]) + (-2.0))) + (((data["abs_percentile_20"]) + (((-3.0) + (((data["range_-2000_-1000"]) + (data["percentile_10"]))))))))/2.0)) 
        v["i189"] = 0.010000*np.tanh(((((((data["skew"]) * 2.0)) + (data["percentile_5"]))) * 2.0)) 
        v["i190"] = 0.010000*np.tanh(((((-3.0) + (((((((((data["skew"]) * 2.0)) - (((data["ffti_autocorrelation_100"]) - (data["ffti_mean_change_rate"]))))) + (data["exp_Moving_average_30000_std"]))) * 2.0)))) * 2.0)) 
        v["i191"] = 0.010000*np.tanh(((((data["ffti_Hann_window_mean_150"]) + (((((((data["max_to_min"]) + ((((-2.0) + (-1.0))/2.0)))) + (data["exp_Moving_average_30000_std"]))) * 2.0)))) * 2.0)) 
        v["i192"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_20"]) * 2.0)) 
        v["i193"] = 0.010000*np.tanh(((((data["skew"]) - (data["min_roll_std_10000"]))) * 2.0)) 
        v["i194"] = 0.010000*np.tanh(((((data["time_rev_asym_stat_10"]) * 2.0)) - (((data["min_roll_std_100"]) * 2.0)))) 
        v["i195"] = 0.010000*np.tanh(((data["fftr_min_roll_mean_1000"]) + (((data["exp_Moving_average_50000_std"]) + (((((data["fftr_min_roll_mean_1000"]) + (data["fftr_abs_percentile_10"]))) * 2.0)))))) 
        v["i196"] = 0.010000*np.tanh(((((data["mean_change_rate"]) + (((((((data["med"]) * 2.0)) * (data["fftr_num_peaks_100"]))) * 2.0)))) * 2.0)) 
        v["i197"] = 0.010000*np.tanh(((-3.0) + (((((-3.0) * 2.0)) * (data["fftr_percentile_roll_std_25"]))))) 
        v["i198"] = 0.010000*np.tanh(((((data["ffti_mean_change_rate"]) + (((((data["ffti_min_roll_std_50"]) + (((data["ffti_percentile_roll_std_50"]) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i199"] = 0.010000*np.tanh(((((((((((((((data["mean_change_rate_first_1000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((data["percentile_10"]) * 2.0)))) 
        v["i200"] = 0.010000*np.tanh(((((((data["percentile_20"]) - ((-1.0*((((((data["fftr_percentile_roll_std_40"]) + (((data["ffti_av_change_abs_roll_std_50"]) * 2.0)))) - (((data["percentile_roll_mean_25"]) * 2.0))))))))) * 2.0)) * 2.0)) 
        v["i201"] = 0.010000*np.tanh(((((((((((((((data["fftr_percentile_roll_std_70"]) * 2.0)) + (((((data["abs_percentile_40"]) / 2.0)) + (data["skew"]))))) * 2.0)) - (data["skew"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i202"] = 0.010000*np.tanh(((data["ffti_mad"]) - (((data["ffti_autocorrelation_100"]) - (((data["ffti_min_roll_std_50"]) / 2.0)))))) 
        v["i203"] = 0.010000*np.tanh((((((((-1.0*((data["ffti_autocorrelation_100"])))) - (((data["ffti_autocorrelation_100"]) * (((data["ffti_min_roll_std_10"]) + (data["min_last_1000"]))))))) - (data["ffti_autocorrelation_500"]))) - (data["ffti_autocorrelation_100"]))) 
        v["i204"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_10"]) + (((((data["fftr_min_roll_mean_10"]) + (((data["ffti_min_roll_std_10"]) + (data["fftr_min_roll_mean_10"]))))) + (data["ffti_min_roll_std_10"]))))) * 2.0)) 
        v["i205"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_mean_25"]) * (-2.0))) * 2.0)) * 2.0)) / 2.0)) + (-2.0))) 
        v["i206"] = 0.010000*np.tanh(-2.0) 
        v["i207"] = 0.010000*np.tanh(((data["ffti_ave_roll_mean_10"]) + (data["ffti_mean_change_rate"]))) 
        v["i208"] = 0.010000*np.tanh(((((((((((data["ffti_min_roll_std_10"]) * 2.0)) * 2.0)) + (((data["ffti_av_change_abs_roll_std_100"]) + ((-1.0*((data["ffti_range_p4000_pinf"])))))))) * 2.0)) * 2.0)) 
        v["i209"] = 0.010000*np.tanh(((data["mean_change_rate_first_1000"]) + (((-2.0) - (((data["num_peaks_10"]) + ((((data["ffti_autocorrelation_50"]) + (data["percentile_90"]))/2.0)))))))) 
        v["i210"] = 0.010000*np.tanh((((-1.0*((((data["min_roll_std_10"]) * 2.0))))) * 2.0)) 
        v["i211"] = 0.010000*np.tanh(((((((((data["mean_last_1000"]) * 2.0)) - (data["percentile_75"]))) * 2.0)) * 2.0)) 
        v["i212"] = 0.010000*np.tanh(((((data["percentile_30"]) + ((((((data["exp_Moving_average_30000_std"]) / 2.0)) + (((data["percentile_10"]) / 2.0)))/2.0)))) + (((data["abs_percentile_40"]) + (data["kurt"]))))) 
        v["i213"] = 0.010000*np.tanh((((((-3.0) - (((((data["ffti_max_roll_std_10000"]) - (((-3.0) + (data["ffti_num_peaks_100"]))))) + (data["range_3000_4000"]))))) + (-3.0))/2.0)) 
        v["i214"] = 0.010000*np.tanh(((data["autocorrelation_500"]) + ((((((-1.0*((((data["ffti_percentile_roll_mean_25"]) + (data["min_last_50000"])))))) * 2.0)) * 2.0)))) 
        v["i215"] = 0.010000*np.tanh(((((data["mean_change_rate_first_1000"]) * 2.0)) * 2.0)) 
        v["i216"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_100"]) * ((-1.0*((data["ffti_min_roll_std_100"])))))) + (data["ffti_Hann_window_mean_150"]))) 
        v["i217"] = 0.010000*np.tanh(data["ffti_min_roll_std_50"]) 
        v["i218"] = 0.010000*np.tanh((((((((((data["ffti_autocorrelation_100"]) * (((data["fftr_av_change_abs_roll_std_1000"]) * 2.0)))) + ((-1.0*((data["ffti_autocorrelation_500"])))))/2.0)) + ((-1.0*((data["fftr_spkt_welch_density_1"])))))) * 2.0)) 
        v["i219"] = 0.010000*np.tanh(((((((((data["ffti_av_change_abs_roll_std_50"]) * 2.0)) + (data["ffti_percentile_roll_std_50"]))) + (((data["percentile_10"]) / 2.0)))) + ((((((data["fftr_mean_change_abs"]) + (data["percentile_10"]))/2.0)) * 2.0)))) 
        v["i220"] = 0.010000*np.tanh((((-1.0*(((((-1.0*((((data["percentile_30"]) + (((data["fftr_av_change_abs_roll_mean_50"]) * 2.0))))))) + (((data["min_roll_std_100"]) - (((data["ffti_percentile_roll_std_40"]) + (data["fftr_min_roll_std_100"])))))))))) * 2.0)) 
        v["i221"] = 0.010000*np.tanh(((((((((((data["percentile_10"]) + ((((-1.0*((data["ffti_autocorrelation_50"])))) * 2.0)))) + (data["fftr_max_roll_mean_10000"]))) + (data["exp_Moving_average_30000_std"]))) - (data["fftr_num_peaks_20"]))) + (data["abs_percentile_20"]))) 
        v["i222"] = 0.010000*np.tanh(((((((((((((data["ffti_percentile_roll_std_50"]) * 2.0)) + (((data["exp_Moving_average_50000_std"]) * 2.0)))) - (data["min_roll_std_100"]))) * 2.0)) * 2.0)) - (((data["exp_Moving_average_50000_std"]) * 2.0)))) 
        v["i223"] = 0.010000*np.tanh(((data["abs_percentile_40"]) + (((((((((((data["fftr_range_2000_3000"]) * 2.0)) + (data["ffti_min_roll_std_10"]))) + (data["ffti_min_roll_std_10"]))) + (data["ffti_num_peaks_100"]))) / 2.0)))) 
        v["i224"] = 0.010000*np.tanh(((((((data["exp_Moving_average_30000_std"]) - (data["fftr_num_peaks_100"]))) * 2.0)) * 2.0)) 
        v["i225"] = 0.010000*np.tanh(((((data["fftr_exp_Moving_average_30000_std"]) * (((((((data["ffti_percentile_roll_std_20"]) * 2.0)) * 2.0)) - (data["fftr_percentile_25"]))))) * 2.0)) 
        v["i226"] = 0.010000*np.tanh(((-2.0) + (((data["percentile_10"]) * 2.0)))) 
        v["i227"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_std_50"]) + (data["range_0_1000"]))) * 2.0)) 
        v["i228"] = 0.010000*np.tanh(((((((((data["abs_percentile_40"]) - (((((data["ffti_min_roll_std_50"]) + (data["fftr_range_2000_3000"]))) * (data["fftr_min_roll_std_500"]))))) - (data["autocorrelation_5000"]))) - (data["autocorrelation_5000"]))) - (data["percentile_75"]))) 
        v["i229"] = 0.010000*np.tanh(((((((data["fftr_max_roll_mean_10000"]) * 2.0)) + ((((((-1.0*((data["percentile_roll_std_20"])))) * 2.0)) + (data["percentile_10"]))))) * ((10.0)))) 
        v["i230"] = 0.010000*np.tanh(((((-3.0) + (-2.0))) + (((-2.0) * 2.0)))) 
        v["i231"] = 0.010000*np.tanh(((((((data["percentile_roll_mean_95"]) * (((-3.0) + (data["ave_roll_std_10000"]))))) * 2.0)) - (((data["max_last_50000"]) + (((data["mean_last_1000"]) * 2.0)))))) 
        v["i232"] = 0.010000*np.tanh(((((((((data["fftr_time_rev_asym_stat_100"]) * 2.0)) * 2.0)) + (data["mean_change_rate_last_10000"]))) * 2.0)) 
        v["i233"] = 0.010000*np.tanh(((((((((((((((data["ffti_min_roll_std_100"]) * 2.0)) + (data["abs_percentile_40"]))) / 2.0)) * 2.0)) * 2.0)) - (data["abs_percentile_40"]))) - (data["autocorrelation_5000"]))) 
        v["i234"] = 0.010000*np.tanh(((((data["fftr_range_2000_3000"]) * 2.0)) + (((((((data["ffti_percentile_roll_std_20"]) * (data["fftr_range_-2000_-1000"]))) * 2.0)) * 2.0)))) 
        v["i235"] = 0.010000*np.tanh((((((((((-1.0*((((data["fftr_percentile_95"]) * 2.0))))) - (data["min_roll_std_100"]))) + (-2.0))) - (data["fftr_autocorrelation_100"]))) * 2.0)) 
        v["i236"] = 0.010000*np.tanh(((data["mean_change_rate_last_10000"]) + (((data["abs_percentile_20"]) + (((data["fftr_mean_last_1000"]) + (((data["fftr_mean_last_1000"]) + (data["mean_change_rate_last_50000"]))))))))) 
        v["i237"] = 0.010000*np.tanh(((((data["fftr_max_roll_mean_10000"]) * 2.0)) * 2.0)) 
        v["i238"] = 0.010000*np.tanh(((((((data["mean_last_1000"]) + ((((((((-1.0*((data["fftr_range_2000_3000"])))) * 2.0)) * (data["percentile_roll_std_1"]))) + ((((data["ffti_num_peaks_100"]) + (data["fftr_av_change_abs_roll_std_1000"]))/2.0)))))) * 2.0)) * 2.0)) 
        v["i239"] = 0.010000*np.tanh((((-1.0*((((data["ffti_autocorrelation_500"]) - ((((-1.0*((data["min_roll_std_100"])))) - (((data["autocorrelation_10000"]) * 2.0))))))))) - (data["min_roll_std_100"]))) 
        v["i240"] = 0.010000*np.tanh((((((((((-1.0*((data["ffti_autocorrelation_500"])))) + (data["ffti_mean"]))) + (data["min_last_10000"]))) * 2.0)) * 2.0)) 
        v["i241"] = 0.010000*np.tanh(((data["min_roll_std_10000"]) - (((((data["min_roll_std_10000"]) * (((((data["ffti_range_2000_3000"]) * 2.0)) * 2.0)))) * 2.0)))) 
        v["i242"] = 0.010000*np.tanh(((((data["mean_last_1000"]) + (data["autocorrelation_500"]))) + (data["mean_change_rate_first_1000"]))) 
        v["i243"] = 0.010000*np.tanh(((((((data["ffti_mean_change_rate_last_10000"]) * 2.0)) - (data["max_first_50000"]))) - (((((((data["fftr_autocorrelation_50"]) - (-3.0))) * 2.0)) + (data["fftr_autocorrelation_50"]))))) 
        v["i244"] = 0.010000*np.tanh(((((((((data["ffti_min_roll_std_100"]) - (data["ffti_sum"]))) * 2.0)) * 2.0)) + (((data["ffti_min_roll_std_100"]) - (data["autocorrelation_50"]))))) 
        v["i245"] = 0.010000*np.tanh(((((data["kurt"]) + (((((data["fftr_abs_percentile_5"]) - (data["num_peaks_10"]))) * 2.0)))) + (data["fftr_av_change_abs_roll_mean_500"]))) 
        v["i246"] = 0.010000*np.tanh(((((((((data["abs_percentile_20"]) + (((((((data["fftr_range_2000_3000"]) / 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) - (data["ffti_abs_max_roll_mean_10"]))) 
        v["i247"] = 0.010000*np.tanh(((((((((((data["fftr_av_change_abs_roll_std_10"]) * 2.0)) * 2.0)) + (((((data["fftr_min_roll_mean_10"]) * 2.0)) / 2.0)))) * 2.0)) + (data["fftr_min_roll_mean_10"]))) 
        v["i248"] = 0.010000*np.tanh((((((((-1.0*((((((((data["mean_last_1000"]) * (((data["percentile_50"]) * 2.0)))) * 2.0)) * 2.0))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i249"] = 0.010000*np.tanh(((data["min_last_10000"]) - (((data["ffti_autocorrelation_500"]) - (((data["classic_sta_lta6_mean"]) - (((((((((data["min_roll_std_10000"]) + (data["ffti_autocorrelation_500"]))) * 2.0)) - (data["ffti_ave_roll_mean_100"]))) / 2.0)))))))) 
        v["i250"] = 0.010000*np.tanh(((((data["mean_change_rate"]) - (((data["fftr_std_roll_mean_500"]) * 2.0)))) + (((((((data["mean_change_rate_first_1000"]) * 2.0)) * 2.0)) * 2.0)))) 
        v["i251"] = 0.010000*np.tanh(((((((data["ffti_exp_Moving_std_300_mean"]) * 2.0)) * 2.0)) + (((((((data["ffti_num_peaks_100"]) + (((((data["skew"]) * 2.0)) * 2.0)))) + (data["ffti_num_peaks_100"]))) * 2.0)))) 
        v["i252"] = 0.010000*np.tanh(((((((((data["fftr_mean_last_10000"]) * 2.0)) + (((data["ffti_min_roll_mean_10"]) * 2.0)))) * 2.0)) + (((data["percentile_10"]) / 2.0)))) 
        v["i253"] = 0.010000*np.tanh(((data["abs_percentile_40"]) + (((((((((((data["fftr_percentile_roll_std_40"]) + (((((data["ffti_min_roll_mean_10"]) * 2.0)) * 2.0)))) + (data["ffti_av_change_abs_roll_std_50"]))) * 2.0)) * 2.0)) * 2.0)))) 
        v["i254"] = 0.010000*np.tanh(((data["ffti_percentile_roll_mean_70"]) * (((((((data["abs_percentile_50"]) - (data["ffti_percentile_roll_std_20"]))) - (data["ffti_percentile_roll_std_20"]))) * 2.0)))) 
        v["i255"] = 0.010000*np.tanh((((((((data["fftr_exp_Moving_std_30000_mean"]) * 2.0)) + (((data["fftr_time_rev_asym_stat_100"]) + (data["fftr_mean_last_1000"]))))/2.0)) * 2.0)) 
        v["i256"] = 0.010000*np.tanh(((((((((data["fftr_range_2000_3000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i257"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_20"]) + (((((-2.0) + (data["mean_change_rate"]))) - (data["av_change_abs_roll_std_10000"]))))) 
        v["i258"] = 0.010000*np.tanh(data["exp_Moving_average_50000_std"]) 
        v["i259"] = 0.010000*np.tanh(data["range_0_1000"]) 
        v["i260"] = 0.010000*np.tanh(((((data["ffti_range_-3000_-2000"]) - (data["ffti_percentile_roll_mean_30"]))) * 2.0)) 
        v["i261"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_10"]) / 2.0)) + (((data["mean_last_10000"]) + (data["ffti_percentile_20"]))))) + (((((data["ffti_min_roll_std_10"]) + (data["fftr_min_roll_mean_100"]))) * 2.0)))) 
        v["i262"] = 0.010000*np.tanh(((((((((((data["percentile_roll_std_75"]) - (data["percentile_roll_std_5"]))) * 2.0)) + (data["ffti_percentile_roll_mean_80"]))) * 2.0)) - (data["percentile_roll_std_5"]))) 
        v["i263"] = 0.010000*np.tanh((((((((data["std_roll_mean_10000"]) + (data["std_roll_mean_10000"]))/2.0)) + (-1.0))) * 2.0)) 
        v["i264"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_std_50"]) + (((((data["ffti_abs_percentile_1"]) - (data["autocorrelation_100"]))) * 2.0)))) * 2.0)) 
        v["i265"] = 0.010000*np.tanh(data["fftr_av_change_abs_roll_mean_1000"]) 
        v["i266"] = 0.010000*np.tanh(((((data["fftr_c3_10"]) * 2.0)) + (((data["moment_4"]) - (data["ffti_skew"]))))) 
        v["i267"] = 0.010000*np.tanh((((-1.0*((((data["autocorrelation_5000"]) + (((((((((data["abs_percentile_40"]) - (data["percentile_roll_std_5"]))) + (data["percentile_roll_std_50"]))) * (data["percentile_75"]))) + (data["percentile_roll_std_5"])))))))) * 2.0)) 
        v["i268"] = 0.010000*np.tanh((((-1.0*((((data["percentile_roll_std_5"]) + (data["autocorrelation_5000"])))))) + (data["percentile_roll_std_5"]))) 
        v["i269"] = 0.010000*np.tanh(((((((((((((((data["skew"]) - (data["fftr_ave_roll_mean_500"]))) * 2.0)) * 2.0)) * 2.0)) - (data["fftr_ave_roll_mean_500"]))) * 2.0)) + (data["fftr_ave_roll_mean_50"]))) 
        v["i270"] = 0.010000*np.tanh(((data["ffti_Hann_window_mean_1500"]) - (((data["fftr_percentile_80"]) * (data["fftr_min_roll_std_50"]))))) 
        v["i271"] = 0.010000*np.tanh(((data["abs_percentile_20"]) + (((((((((((((((data["ffti_percentile_roll_std_20"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (((data["fftr_abs_max"]) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i272"] = 0.010000*np.tanh(((((-2.0) * 2.0)) + (((((-2.0) + (data["mean_change_rate"]))) * 2.0)))) 
        v["i273"] = 0.010000*np.tanh(((((data["ffti_min_roll_mean_10"]) * 2.0)) + (((((((data["ffti_min_roll_mean_10"]) * 2.0)) * 2.0)) * 2.0)))) 
        v["i274"] = 0.010000*np.tanh(((((((data["mean_change_rate_first_50000"]) + (((data["skew"]) * 2.0)))) * 2.0)) + (((((((data["fftr_range_-2000_-1000"]) * 2.0)) * 2.0)) + (data["fftr_min_roll_mean_100"]))))) 
        v["i275"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_50"]) + (data["ffti_min_roll_mean_10"]))) 
        v["i276"] = 0.010000*np.tanh(((((-3.0) / 2.0)) + (data["ffti_min_roll_std_10"]))) 
        v["i277"] = 0.010000*np.tanh(((((data["fftr_abs_percentile_20"]) * 2.0)) + (((((((data["percentile_10"]) + (((data["ffti_percentile_roll_std_50"]) * 2.0)))) * 2.0)) * 2.0)))) 
        v["i278"] = 0.010000*np.tanh(((-1.0) + ((((-1.0*((data["ffti_time_rev_asym_stat_50"])))) - (((data["ffti_percentile_roll_std_30"]) * (((((data["ffti_classic_sta_lta6_mean"]) * 2.0)) * 2.0)))))))) 
        v["i279"] = 0.010000*np.tanh(((((((data["skew"]) + (data["ffti_autocorrelation_5"]))) * 2.0)) + ((((((data["ffti_classic_sta_lta8_mean"]) + (data["ffti_autocorrelation_5"]))/2.0)) + (((((data["abs_percentile_40"]) * 2.0)) * 2.0)))))) 
        v["i280"] = 0.010000*np.tanh(((data["ffti_num_peaks_100"]) + (((data["ffti_Hann_window_mean_150"]) + (((data["autocorrelation_50"]) * (((data["mean_change_rate_first_10000"]) - (data["autocorrelation_50"]))))))))) 
        v["i281"] = 0.010000*np.tanh((((((data["percentile_75"]) + (((((((data["fftr_percentile_roll_std_70"]) * 2.0)) * 2.0)) - (data["min_roll_std_10"]))))/2.0)) - (((data["classic_sta_lta1_mean"]) - (data["min_roll_std_10"]))))) 
        v["i282"] = 0.010000*np.tanh(((((data["ffti_av_change_abs_roll_std_50"]) + (((data["ffti_range_2000_3000"]) + (((((((data["ffti_av_change_abs_roll_std_50"]) + (((data["exp_Moving_average_30000_std"]) + (data["ffti_range_2000_3000"]))))) * 2.0)) * 2.0)))))) + (data["mean_change_rate"]))) 
        v["i283"] = 0.010000*np.tanh((-1.0*((((((data["min_roll_std_10000"]) + (((((((data["fftr_classic_sta_lta1_mean"]) + (((data["autocorrelation_5000"]) + (data["autocorrelation_5000"]))))) * 2.0)) + (data["min_roll_std_10000"]))))) * 2.0))))) 
        v["i284"] = 0.010000*np.tanh(((((((((data["mean_change_rate"]) * (((((data["ffti_mean"]) + (data["ffti_mean"]))) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) 
        v["i285"] = 0.010000*np.tanh(((-2.0) + (((((data["skew"]) * 2.0)) - (((((data["fftr_abs_percentile_5"]) * 2.0)) * (data["fftr_abs_percentile_5"]))))))) 
        v["i286"] = 0.010000*np.tanh(((((data["percentile_5"]) + (((((data["ffti_std_roll_mean_1000"]) - (data["abs_percentile_5"]))) + (data["min_last_10000"]))))) + (data["abs_percentile_40"]))) 
        v["i287"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_20"]) * 2.0)) 
        v["i288"] = 0.010000*np.tanh(data["ffti_range_2000_3000"]) 
        v["i289"] = 0.010000*np.tanh(((data["abs_percentile_40"]) - (((((data["autocorrelation_5000"]) + (data["abs_percentile_75"]))) + (((data["min_roll_std_10000"]) + (data["autocorrelation_5000"]))))))) 
        v["i290"] = 0.010000*np.tanh(((data["ffti_min_roll_mean_10"]) - (((((data["ffti_autocorrelation_100"]) - (((((((data["ffti_exp_Moving_average_30000_std"]) * 2.0)) * 2.0)) - (data["percentile_roll_std_5"]))))) * 2.0)))) 
        v["i291"] = 0.010000*np.tanh(((data["percentile_10"]) - (((data["min_roll_std_500"]) * 2.0)))) 
        v["i292"] = 0.010000*np.tanh(((((((((data["ffti_num_peaks_100"]) * 2.0)) * 2.0)) * 2.0)) + ((((-1.0*(((((-1.0*((data["fftr_range_-3000_-2000"])))) / 2.0))))) * 2.0)))) 
        v["i293"] = 0.010000*np.tanh(((((data["ffti_num_peaks_100"]) - (data["autocorrelation_5000"]))) - (((((data["autocorrelation_50"]) + (data["autocorrelation_50"]))) * (data["autocorrelation_500"]))))) 
        v["i294"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_5000"]) * 2.0)) * (((data["fftr_abs_percentile_5"]) + (data["mean_last_1000"]))))) * 2.0)) 
        v["i295"] = 0.010000*np.tanh(((((data["fftr_range_2000_3000"]) + (data["fftr_range_2000_3000"]))) + (((((((((data["fftr_range_2000_3000"]) * 2.0)) * 2.0)) + (((data["mean_first_1000"]) * 2.0)))) * 2.0)))) 
        v["i296"] = 0.010000*np.tanh(((((((data["ffti_num_peaks_100"]) * 2.0)) * (data["ffti_ave_roll_mean_10"]))) + (((data["ffti_num_peaks_100"]) + (((((data["ffti_ave_roll_mean_10"]) * 2.0)) * (data["ffti_num_peaks_100"]))))))) 
        v["i297"] = 0.010000*np.tanh((((((((((((data["fftr_av_change_abs_roll_mean_500"]) * 2.0)) * 2.0)) + (data["fftr_abs_percentile_5"]))) + (((data["fftr_abs_percentile_5"]) * 2.0)))) + (((((data["percentile_20"]) * 2.0)) * 2.0)))/2.0)) 
        v["i298"] = 0.010000*np.tanh((((((((data["abs_percentile_20"]) + (((data["ffti_abs_percentile_1"]) + (data["ffti_min_roll_mean_10"]))))) + (((data["percentile_10"]) * 2.0)))) + (data["fftr_abs_percentile_10"]))/2.0)) 
        v["i299"] = 0.010000*np.tanh(((data["percentile_40"]) * (((((data["classic_sta_lta2_mean"]) + (data["ffti_min_roll_mean_1000"]))) + (((data["num_peaks_50"]) + (data["ffti_mean_change_rate"]))))))) 
        v["i300"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_10"]) + (((data["fftr_time_rev_asym_stat_100"]) * 2.0)))) * 2.0)) 
        v["i301"] = 0.010000*np.tanh(((((((data["fftr_abs_percentile_5"]) * 2.0)) + (((((data["ffti_mean_change_rate_last_10000"]) * 2.0)) * 2.0)))) * 2.0)) 
        v["i302"] = 0.010000*np.tanh((((((-1.0*((data["ffti_percentile_roll_std_50"])))) - (((data["abs_percentile_20"]) * 2.0)))) + (((data["abs_percentile_20"]) - (((data["ffti_skew"]) * 2.0)))))) 
        v["i303"] = 0.010000*np.tanh(((data["moment_4"]) + (((((((data["fftr_range_2000_3000"]) - (data["num_peaks_10"]))) * 2.0)) - (((data["num_peaks_10"]) * 2.0)))))) 
        v["i304"] = 0.010000*np.tanh(((((((((((data["ffti_num_peaks_100"]) - (data["fftr_autocorrelation_50"]))) * 2.0)) * 2.0)) - (data["ffti_av_change_abs_roll_mean_1000"]))) + (((((((((data["ffti_av_change_abs_roll_std_50"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i305"] = 0.010000*np.tanh(((((((data["autocorrelation_500"]) + (data["autocorrelation_500"]))) - (data["av_change_abs_roll_mean_100"]))) * 2.0)) 
        v["i306"] = 0.010000*np.tanh(data["fftr_std_roll_std_10000"]) 
        v["i307"] = 0.010000*np.tanh(((-2.0) + (((((((((((((data["ffti_ave_roll_mean_50"]) + (data["fftr_abs_percentile_10"]))) * 2.0)) * 2.0)) + (data["mean_first_1000"]))) - (data["fftr_percentile_roll_mean_90"]))) - (data["fftr_percentile_roll_mean_50"]))))) 
        v["i308"] = 0.010000*np.tanh(((((data["percentile_10"]) * 2.0)) - (((data["abs_percentile_40"]) * (data["ffti_mean_last_1000"]))))) 
        v["i309"] = 0.010000*np.tanh(((data["percentile_10"]) + (((((data["abs_percentile_20"]) / 2.0)) + (((data["percentile_10"]) + (((data["fftr_percentile_50"]) - (data["ffti_autocorrelation_500"]))))))))) 
        v["i310"] = 0.010000*np.tanh(((((data["min_roll_std_10"]) - ((((((-1.0*((((-1.0) - (((data["ffti_min_first_50000"]) * 2.0))))))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i311"] = 0.010000*np.tanh(((data["fftr_percentile_roll_mean_80"]) * (((data["abs_percentile_25"]) + (data["gmean"]))))) 
        v["i312"] = 0.010000*np.tanh(((((data["ffti_percentile_roll_std_50"]) * 2.0)) + (((((data["ffti_min_roll_mean_500"]) - (data["fftr_percentile_roll_std_70"]))) * (data["ffti_percentile_roll_std_70"]))))) 
        v["i313"] = 0.010000*np.tanh(((data["skew"]) + (((((data["fftr_sum"]) + (((data["fftr_abs_percentile_10"]) - (data["ffti_exp_Moving_std_300_mean"]))))) + (((data["abs_percentile_20"]) * 2.0)))))) 
        v["i314"] = 0.010000*np.tanh(((((data["autocorrelation_50"]) * (((data["fftr_kurt"]) * 2.0)))) + (data["fftr_kurt"]))) 
        v["i315"] = 0.010000*np.tanh(data["abs_percentile_20"]) 
        v["i316"] = 0.010000*np.tanh((((data["av_change_abs_roll_mean_500"]) + (((-3.0) - (data["fftr_classic_sta_lta8_mean"]))))/2.0)) 
        v["i317"] = 0.010000*np.tanh(((((((data["skew"]) - (((data["autocorrelation_10000"]) + (data["ffti_autocorrelation_50"]))))) * 2.0)) * 2.0)) 
        v["i318"] = 0.010000*np.tanh(((((data["med"]) * (((((data["fftr_num_peaks_100"]) - (data["min"]))) + (((data["med"]) + (data["fftr_num_peaks_100"]))))))) * 2.0)) 
        v["i319"] = 0.010000*np.tanh((((((((-1.0*((((((data["mean_last_50000"]) * 2.0)) * 2.0))))) + ((((-1.0*((((((data["percentile_roll_mean_99"]) * 2.0)) * 2.0))))) * 2.0)))) * 2.0)) + (data["fftr_spkt_welch_density_10"]))) 
        v["i320"] = 0.010000*np.tanh(((((((-2.0) + (((data["ffti_num_peaks_100"]) * 2.0)))) + (((-2.0) + (((data["ffti_num_peaks_100"]) * 2.0)))))) * 2.0)) 
        v["i321"] = 0.010000*np.tanh((((((-1.0*((data["num_peaks_10"])))) * (data["ffti_min_roll_std_100"]))) + (((data["num_peaks_100"]) + (((((data["num_peaks_100"]) + (data["percentile_30"]))) / 2.0)))))) 
        v["i322"] = 0.010000*np.tanh(((((data["fftr_min"]) - (((data["count_big"]) + (data["fftr_percentile_roll_mean_50"]))))) - (data["num_peaks_10"]))) 
        v["i323"] = 0.010000*np.tanh(((((((data["fftr_abs_max_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)) 
        v["i324"] = 0.010000*np.tanh((-1.0*((((data["ffti_autocorrelation_500"]) - (data["ffti_percentile_roll_mean_25"])))))) 
        v["i325"] = 0.010000*np.tanh(((((((data["fftr_percentile_roll_mean_1"]) * (data["fftr_range_3000_4000"]))) * 2.0)) * 2.0)) 
        v["i326"] = 0.010000*np.tanh(((((((data["ffti_percentile_roll_std_20"]) + (data["ffti_min_roll_mean_10"]))) * 2.0)) - (((((data["exp_Moving_average_50000_std"]) * 2.0)) * (((((((data["fftr_time_rev_asym_stat_50"]) * 2.0)) * 2.0)) - (data["fftr_range_2000_3000"]))))))) 
        v["i327"] = 0.010000*np.tanh(((((((data["fftr_mean_first_10000"]) + (data["spkt_welch_density_100"]))) * 2.0)) + (((((((data["ffti_abs_percentile_1"]) - (data["percentile_roll_std_5"]))) * 2.0)) * 2.0)))) 
        v["i328"] = 0.010000*np.tanh(-2.0) 
        v["i329"] = 0.010000*np.tanh(((((((data["fftr_range_2000_3000"]) * 2.0)) * 2.0)) + (data["fftr_range_2000_3000"]))) 
        v["i330"] = 0.010000*np.tanh((((((-1.0*((data["ffti_percentile_roll_mean_50"])))) - (data["autocorrelation_10000"]))) * 2.0)) 
        v["i331"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_100"]) + (((((data["skew"]) + (data["skew"]))) * 2.0)))) * 2.0)) 
        v["i332"] = 0.010000*np.tanh(((((((data["ffti_num_peaks_100"]) * (((data["ffti_Hann_window_mean_50"]) - (data["ffti_mean_change_rate"]))))) * 2.0)) * 2.0)) 
        v["i333"] = 0.010000*np.tanh(((data["ffti_min_roll_std_100"]) + (((data["ffti_Hann_window_mean_150"]) - (data["autocorrelation_100"]))))) 
        v["i334"] = 0.010000*np.tanh(((((((data["percentile_roll_std_75"]) - (data["ffti_min_roll_mean_10"]))) + (data["autocorrelation_500"]))) - (((((data["min_roll_std_100"]) * 2.0)) - (data["mean_change_rate"]))))) 
        v["i335"] = 0.010000*np.tanh((-1.0*((data["fftr_autocorrelation_500"])))) 
        v["i336"] = 0.010000*np.tanh(((((((data["fftr_abs_percentile_10"]) + ((((((data["ffti_percentile_roll_std_50"]) * 2.0)) + (data["ffti_min_roll_std_1000"]))/2.0)))) - (data["num_peaks_10"]))) * 2.0)) 
        v["i337"] = 0.010000*np.tanh((((((data["av_change_abs_roll_mean_10000"]) + (((((((((data["mean_change_rate"]) - (data["autocorrelation_5000"]))) - (data["fftr_min_roll_mean_10"]))) * 2.0)) * 2.0)))/2.0)) - (data["ffti_percentile_roll_mean_50"]))) 
        v["i338"] = 0.010000*np.tanh(((((data["ffti_std_roll_std_10"]) + ((((data["ffti_ave_roll_mean_1000"]) + (((data["fftr_abs_percentile_30"]) - (((data["min_roll_std_10000"]) * 2.0)))))/2.0)))) * 2.0)) 
        v["i339"] = 0.010000*np.tanh(((((((((((data["fftr_max_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)) + (((((data["exp_Moving_average_50000_std"]) * 2.0)) + (data["ffti_percentile_roll_std_40"]))))) * 2.0)) 
        v["i340"] = 0.010000*np.tanh(((((((data["fftr_abs_percentile_5"]) + (data["abs_percentile_40"]))) - (((data["classic_sta_lta1_mean"]) - (((data["abs_percentile_40"]) * 2.0)))))) / 2.0)) 
        v["i341"] = 0.010000*np.tanh(((((((((((data["fftr_av_change_abs_roll_std_100"]) * 2.0)) - (data["Hann_window_mean_150"]))) - (data["iqr1"]))) * 2.0)) * 2.0)) 
        v["i342"] = 0.010000*np.tanh(((data["abs_percentile_40"]) + (data["fftr_range_2000_3000"]))) 
        v["i343"] = 0.010000*np.tanh((((3.0)) * (((((data["autocorrelation_50"]) * (((data["ffti_percentile_25"]) - (((data["ffti_kurt"]) * (((data["percentile_10"]) * (data["percentile_10"]))))))))) - (data["ffti_max_roll_mean_10"]))))) 
        v["i344"] = 0.010000*np.tanh(((((((data["exp_Moving_average_30000_std"]) + (data["ffti_mean"]))) - (data["percentile_90"]))) + (data["ffti_percentile_40"]))) 
        v["i345"] = 0.010000*np.tanh(((((((((((((((((data["fftr_mean_first_10000"]) + (data["fftr_abs_percentile_5"]))) * 2.0)) * 2.0)) * 2.0)) + (((data["fftr_abs_percentile_5"]) * 2.0)))/2.0)) * 2.0)) + (((data["ffti_min_roll_std_10"]) * 2.0)))/2.0)) 
        v["i346"] = 0.010000*np.tanh(((((((((data["fftr_num_peaks_100"]) * (((((data["med"]) * 2.0)) * 2.0)))) + (((data["fftr_num_peaks_100"]) * (data["med"]))))) * 2.0)) * 2.0)) 
        v["i347"] = 0.010000*np.tanh(((data["trend"]) * (((((data["ffti_num_peaks_100"]) * 2.0)) * 2.0)))) 
        v["i348"] = 0.010000*np.tanh(((data["fftr_av_change_abs_roll_mean_10"]) + (((data["mean_last_1000"]) * ((((-1.0*((((data["ffti_autocorrelation_100"]) * 2.0))))) + (data["fftr_abs_percentile_5"]))))))) 
        v["i349"] = 0.010000*np.tanh(((((((((((data["ffti_percentile_roll_std_40"]) + (data["ffti_av_change_abs_roll_mean_1000"]))) * 2.0)) * 2.0)) + (data["ffti_max_to_min"]))) + (((((data["abs_percentile_60"]) * 2.0)) * (data["ffti_max_to_min"]))))) 
        v["i350"] = 0.010000*np.tanh(((((((data["mean_change_rate_first_1000"]) * 2.0)) * 2.0)) + (((((-1.0) - (((-1.0) / 2.0)))) * 2.0)))) 
        v["i351"] = 0.010000*np.tanh((((((data["fftr_abs_percentile_10"]) + (data["fftr_abs_percentile_10"]))/2.0)) + (data["av_change_rate_roll_mean_10000"]))) 
        v["i352"] = 0.010000*np.tanh(((((((((((data["ffti_Hann_window_mean_50"]) - (((data["fftr_autocorrelation_500"]) / 2.0)))) * 2.0)) * 2.0)) - (data["fftr_autocorrelation_500"]))) * 2.0)) 
        v["i353"] = 0.010000*np.tanh(((((((((((8.84529399871826172)) * ((((((8.84529399871826172)) * (data["fftr_range_2000_3000"]))) * 2.0)))) * 2.0)) + (data["ffti_mean_last_1000"]))/2.0)) - (data["fftr_num_peaks_10"]))) 
        v["i354"] = 0.010000*np.tanh(((data["mean_change_rate_first_1000"]) - (data["ffti_percentile_roll_mean_50"]))) 
        v["i355"] = 0.010000*np.tanh(((((data["ffti_autocorrelation_100"]) + (((data["mean_change_rate"]) + (((((-2.0) - (data["ffti_autocorrelation_100"]))) + (data["ffti_min_roll_mean_10"]))))))) - ((((data["ffti_av_change_abs_roll_mean_1000"]) + (data["ffti_max_roll_mean_100"]))/2.0)))) 
        v["i356"] = 0.010000*np.tanh(((((((((data["spkt_welch_density_100"]) - (((((data["ffti_av_change_abs_roll_std_10"]) * 2.0)) - (data["min_roll_std_10"]))))) * 2.0)) * 2.0)) * (((data["spkt_welch_density_100"]) * 2.0)))) 
        v["i357"] = 0.010000*np.tanh(((data["fftr_abs_percentile_5"]) * 2.0)) 
        v["i358"] = 0.010000*np.tanh(((data["ffti_num_peaks_50"]) + (data["ffti_min_roll_std_50"]))) 
        v["i359"] = 0.010000*np.tanh(data["fftr_abs_percentile_10"]) 
        v["i360"] = 0.010000*np.tanh(((((((data["ffti_mean"]) * (data["fftr_av_change_rate_roll_mean_500"]))) - (((data["fftr_c3_50"]) - (data["fftr_range_1000_2000"]))))) * 2.0)) 
        v["i361"] = 0.010000*np.tanh(((((((((data["ffti_min_roll_mean_10"]) - (data["ffti_percentile_roll_std_40"]))) * 2.0)) - (((data["ffti_percentile_roll_mean_60"]) - (data["ffti_mean_change_abs"]))))) + (data["mean_change_rate"]))) 
        v["i362"] = 0.010000*np.tanh(((data["ffti_mean_last_1000"]) - (data["ffti_mean_change_rate_first_50000"]))) 
        v["i363"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_100"]) / 2.0)) * (((data["ffti_ave_roll_mean_100"]) + (((data["ffti_ave_roll_mean_100"]) + (data["ffti_Hann_window_mean_50"]))))))) + (((data["mean_change_rate_last_1000"]) + (data["fftr_av_change_abs_roll_std_1000"]))))) 
        v["i364"] = 0.010000*np.tanh(((data["abs_percentile_20"]) + (((((data["fftr_min_roll_std_500"]) * (((data["min_roll_std_10"]) * 2.0)))) * (data["fftr_range_2000_3000"]))))) 
        v["i365"] = 0.010000*np.tanh(((((data["exp_Moving_average_30000_std"]) + (((((((data["ffti_skew"]) + (data["fftr_min_roll_std_50"]))) * (((data["ffti_Hann_window_mean_50"]) * 2.0)))) * 2.0)))) + (data["ffti_percentile_roll_std_50"]))) 
        v["i366"] = 0.010000*np.tanh((((data["fftr_std_roll_mean_100"]) + (data["autocorrelation_50"]))/2.0)) 
        v["i367"] = 0.010000*np.tanh(((((((((data["abs_percentile_20"]) * 2.0)) + (((data["fftr_min_roll_mean_10"]) * 2.0)))) - (((data["mean"]) - (data["fftr_autocorrelation_5000"]))))) * 2.0)) 
        v["i368"] = 0.010000*np.tanh(((((((((((((((((data["ffti_autocorrelation_100"]) * (data["fftr_percentile_roll_std_40"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i369"] = 0.010000*np.tanh(data["ffti_percentile_roll_std_40"]) 
        v["i370"] = 0.010000*np.tanh(((((((((((data["fftr_percentile_roll_std_25"]) * (((((data["fftr_percentile_roll_std_25"]) - (data["fftr_percentile_roll_std_25"]))) - (((((data["ffti_mean_change_abs"]) * 2.0)) * 2.0)))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i371"] = 0.010000*np.tanh(data["percentile_10"]) 
        v["i372"] = 0.010000*np.tanh(((data["percentile_10"]) + (((((data["percentile_10"]) * 2.0)) + ((((((data["percentile_10"]) + (data["percentile_10"]))) + (((data["fftr_num_peaks_50"]) * 2.0)))/2.0)))))) 
        v["i373"] = 0.010000*np.tanh(((((((data["fftr_range_2000_3000"]) * 2.0)) * (((3.0) - (((((data["min_roll_std_10"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i374"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_100"]) - (((data["ffti_autocorrelation_100"]) * (((data["ffti_percentile_roll_std_50"]) + (((((data["ffti_percentile_roll_std_50"]) + (data["max_to_min"]))) * 2.0)))))))) 
        v["i375"] = 0.010000*np.tanh((((data["ffti_std_first_1000"]) + ((((((((((((data["classic_sta_lta2_mean"]) * 2.0)) + (data["spkt_welch_density_100"]))) + (data["ffti_min_roll_mean_1000"]))/2.0)) - (data["min_roll_mean_10000"]))) * 2.0)))/2.0)) 
        v["i376"] = 0.010000*np.tanh(((((((data["ffti_range_-3000_-2000"]) + (data["min_roll_mean_50"]))) + (((((data["percentile_10"]) + ((((-1.0*((data["fftr_percentile_roll_std_60"])))) * 2.0)))) * 2.0)))) * 2.0)) 
        v["i377"] = 0.010000*np.tanh(((data["spkt_welch_density_100"]) + (((data["ffti_abs_percentile_1"]) * (((data["fftr_min_roll_std_1000"]) * (((((data["abs_percentile_40"]) + (((data["ffti_min_first_1000"]) / 2.0)))) * 2.0)))))))) 
        v["i378"] = 0.010000*np.tanh(((((((((data["ffti_percentile_roll_std_30"]) * 2.0)) * (((((((data["ffti_min"]) * 2.0)) * 2.0)) + (data["fftr_Hann_window_mean_1500"]))))) * 2.0)) * 2.0)) 
        v["i379"] = 0.010000*np.tanh(((((data["num_peaks_100"]) + (((data["ffti_ave_roll_mean_10"]) + (((((((data["ffti_mean_change_rate"]) + (data["ffti_mean_change_rate"]))) + (data["ffti_av_change_abs_roll_std_50"]))) * 2.0)))))) + (data["ffti_mean_change_rate"]))) 
        v["i380"] = 0.010000*np.tanh(((data["ffti_kstat_1"]) + (((-2.0) + (((data["ffti_num_peaks_100"]) + (-3.0))))))) 
        v["i381"] = 0.010000*np.tanh(((((((((((((((((data["fftr_abs_max_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i382"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_100"]) + (data["mean_change_rate_first_10000"]))) 
        v["i383"] = 0.010000*np.tanh(((((((((((((((((data["mean_first_50000"]) * (((data["ffti_percentile_roll_std_20"]) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i384"] = 0.010000*np.tanh(((((data["mean_change_rate"]) + (((((data["num_peaks_100"]) - (data["ffti_Hilbert_mean"]))) + (((((data["ffti_percentile_roll_std_50"]) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i385"] = 0.010000*np.tanh(data["ffti_min_roll_mean_10"]) 
        v["i386"] = 0.010000*np.tanh(((data["ffti_autocorrelation_500"]) * (((((data["ffti_exp_Moving_average_30000_std"]) * 2.0)) * 2.0)))) 
        v["i387"] = 0.010000*np.tanh(((data["fftr_classic_sta_lta6_mean"]) * (((((data["fftr_classic_sta_lta6_mean"]) + ((((((((data["fftr_min_roll_std_10"]) * 2.0)) * 2.0)) + (data["exp_Moving_average_30000_std"]))/2.0)))) - ((-1.0*((data["fftr_min_roll_std_10"])))))))) 
        v["i388"] = 0.010000*np.tanh((((((((((data["classic_sta_lta2_mean"]) * 2.0)) * 2.0)) + (((((data["ffti_percentile_roll_mean_25"]) * 2.0)) * 2.0)))/2.0)) + (data["spkt_welch_density_100"]))) 
        v["i389"] = 0.010000*np.tanh(((-3.0) - ((((((((((-1.0*((data["spkt_welch_density_100"])))) * (data["ffti_ave_roll_mean_500"]))) + (((-2.0) / 2.0)))) * 2.0)) - (data["max_to_min_diff"]))))) 
        v["i390"] = 0.010000*np.tanh(((((data["moment_4"]) * 2.0)) - (((((((((((((((data["mean_last_1000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["max_roll_mean_10000"]))))) 
        v["i391"] = 0.010000*np.tanh(((data["ffti_sum"]) * (((((data["fftr_av_change_abs_roll_mean_10000"]) + (data["ffti_sum"]))) + (((((data["ffti_iqr1"]) * (data["fftr_classic_sta_lta2_mean"]))) + (data["fftr_av_change_abs_roll_mean_10000"]))))))) 
        v["i392"] = 0.010000*np.tanh(data["abs_percentile_20"]) 
        v["i393"] = 0.010000*np.tanh(((data["fftr_autocorrelation_10000"]) + (((data["ffti_percentile_roll_std_20"]) + (((((((((((((data["fftr_percentile_roll_std_40"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * (data["fftr_autocorrelation_100"]))))))) 
        v["i394"] = 0.010000*np.tanh(((data["classic_sta_lta3_mean"]) - (((data["fftr_percentile_roll_mean_20"]) + (data["fftr_autocorrelation_100"]))))) 
        v["i395"] = 0.010000*np.tanh(((((data["fftr_abs_percentile_5"]) * 2.0)) + (((data["std_roll_mean_10000"]) - (data["fftr_percentile_roll_mean_20"]))))) 
        v["i396"] = 0.010000*np.tanh(data["mean_last_1000"]) 
        v["i397"] = 0.010000*np.tanh(((((data["ffti_min_first_1000"]) + (data["ffti_range_-3000_-2000"]))) - (data["ffti_range_-3000_-2000"]))) 
        v["i398"] = 0.010000*np.tanh((((((data["ffti_mean_change_rate_last_10000"]) + (((((data["ffti_av_change_abs_roll_mean_1000"]) * 2.0)) * 2.0)))) + (data["fftr_abs_percentile_20"]))/2.0)) 
        v["i399"] = 0.010000*np.tanh(((data["fftr_min_roll_std_50"]) * (data["ffti_ave_roll_mean_10"]))) 
        v["i400"] = 0.010000*np.tanh(((data["ffti_exp_Moving_average_50000_std"]) + (((((((data["fftr_min_roll_std_50"]) * (((data["fftr_min_roll_std_50"]) / 2.0)))) * 2.0)) + ((((((data["ffti_av_change_rate_roll_mean_10000"]) + (-3.0))) + (1.0))/2.0)))))) 
        v["i401"] = 0.010000*np.tanh((((((((-1.0*((data["ave_roll_mean_10000"])))) * 2.0)) - (data["fftr_time_rev_asym_stat_5"]))) * 2.0)) 
        v["i402"] = 0.010000*np.tanh(((((((data["fftr_mean_first_10000"]) * 2.0)) + (data["fftr_min_last_1000"]))) + (((data["min_roll_std_50"]) * (((((data["fftr_abs_percentile_10"]) * 2.0)) - (data["ffti_min_roll_std_10000"]))))))) 
        v["i403"] = 0.010000*np.tanh(((((((data["abs_percentile_10"]) * 2.0)) - (((data["ffti_abs_percentile_25"]) * (((data["fftr_range_2000_3000"]) * 2.0)))))) + (data["ffti_num_peaks_10"]))) 
        v["i404"] = 0.010000*np.tanh(((((((((data["abs_percentile_40"]) / 2.0)) * (((data["ffti_min_roll_mean_10"]) * 2.0)))) + (data["ffti_min_first_1000"]))) + (data["ffti_min_first_1000"]))) 
        v["i405"] = 0.010000*np.tanh(((data["fftr_time_rev_asym_stat_5"]) + (((((((data["fftr_time_rev_asym_stat_5"]) * 2.0)) + (data["fftr_time_rev_asym_stat_5"]))) * (((data["med"]) * 2.0)))))) 
        v["i406"] = 0.010000*np.tanh(((((data["fftr_av_change_abs_roll_mean_500"]) + (data["time_rev_asym_stat_1"]))) * 2.0)) 
        v["i407"] = 0.010000*np.tanh((((((((data["mean_change_rate_last_50000"]) + (((data["ffti_min_roll_std_10"]) - (data["fftr_time_rev_asym_stat_5"]))))/2.0)) + (((((((data["ffti_percentile_roll_mean_40"]) + (data["percentile_25"]))/2.0)) + (data["ffti_min_roll_std_10"]))/2.0)))) * 2.0)) 
        v["i408"] = 0.010000*np.tanh(((((data["ffti_percentile_5"]) / 2.0)) + (((data["ffti_abs_percentile_20"]) + (((data["ffti_range_-3000_-2000"]) + (data["percentile_10"]))))))) 
        v["i409"] = 0.010000*np.tanh(((data["ffti_min_roll_mean_10"]) + (((((((((data["fftr_range_-3000_-2000"]) + (data["std_roll_mean_10000"]))) * 2.0)) + (data["fftr_range_-3000_-2000"]))) * 2.0)))) 
        v["i410"] = 0.010000*np.tanh(((((((((data["num_peaks_100"]) + (data["min_roll_mean_50"]))) * (data["autocorrelation_100"]))) + (data["ffti_percentile_40"]))) * 2.0)) 
        v["i411"] = 0.010000*np.tanh(data["ffti_gmean"]) 
        v["i412"] = 0.010000*np.tanh(((data["ffti_min_roll_std_10000"]) + (((((((((((data["ffti_Hann_window_mean_15000"]) * 2.0)) * 2.0)) * 2.0)) * (((data["ffti_min_roll_std_50"]) * 2.0)))) - (((data["fftr_percentile_roll_mean_30"]) + (data["ffti_Hann_window_mean_15000"]))))))) 
        v["i413"] = 0.010000*np.tanh((((-1.0*((data["ffti_percentile_60"])))) + (data["ave_roll_std_500"]))) 
        v["i414"] = 0.010000*np.tanh(((((((((((data["ffti_min_roll_std_100"]) * 2.0)) * 2.0)) / 2.0)) - (data["ffti_num_peaks_20"]))) * 2.0)) 
        v["i415"] = 0.010000*np.tanh(((((((data["ffti_autocorrelation_50"]) + (((((data["fftr_abs_max_roll_mean_100"]) * 2.0)) * 2.0)))) * (((((data["fftr_percentile_roll_std_25"]) * 2.0)) * 2.0)))) - (data["fftr_num_peaks_10"]))) 
        v["i416"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_100"]) * (data["ffti_percentile_roll_mean_60"]))) + (((((data["ffti_av_change_abs_roll_mean_10"]) * (data["ffti_percentile_roll_mean_60"]))) * 2.0)))) 
        v["i417"] = 0.010000*np.tanh(((((data["ffti_range_2000_3000"]) + (((((((data["fftr_range_2000_3000"]) - (((data["min_roll_std_10"]) - (data["ffti_abs_percentile_1"]))))) * 2.0)) * 2.0)))) * 2.0)) 
        v["i418"] = 0.010000*np.tanh(((((((((((data["fftr_av_change_abs_roll_mean_50"]) * 2.0)) * 2.0)) + (((data["ffti_autocorrelation_1000"]) / 2.0)))) + (data["fftr_autocorrelation_5"]))) + (data["ffti_min_roll_mean_10"]))) 
        v["i419"] = 0.010000*np.tanh(((data["ffti_num_peaks_100"]) * (((((((data["ffti_num_peaks_100"]) * (data["percentile_10"]))) * 2.0)) - (data["ffti_autocorrelation_500"]))))) 
        v["i420"] = 0.010000*np.tanh(((data["ffti_num_peaks_10"]) + (((((((data["abs_percentile_20"]) * 2.0)) * 2.0)) + (((data["spkt_welch_density_100"]) * 2.0)))))) 
        v["i421"] = 0.010000*np.tanh(((((data["mean_change_rate_first_50000"]) - (((data["range_-4000_-3000"]) - (((((((((data["fftr_percentile_roll_std_70"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)))))) / 2.0)) 
        v["i422"] = 0.010000*np.tanh((((((-1.0*((((data["abs_percentile_50"]) * (data["ffti_kstat_1"])))))) * 2.0)) + (((data["ffti_abs_percentile_10"]) + (data["fftr_percentile_roll_mean_80"]))))) 
        v["i423"] = 0.010000*np.tanh(((data["fftr_min_roll_std_50"]) * (((((((data["ffti_exp_Moving_average_3000_mean"]) * 2.0)) * 2.0)) * 2.0)))) 
        v["i424"] = 0.010000*np.tanh(((((data["ffti_mean_change_rate"]) * 2.0)) + (((((data["percentile_20"]) + (data["ffti_min_roll_std_10"]))) + (data["ffti_percentile_roll_std_50"]))))) 
        v["i425"] = 0.010000*np.tanh(data["ffti_min_first_10000"]) 
        v["i426"] = 0.010000*np.tanh(((((data["mean_last_1000"]) * 2.0)) * 2.0)) 
        v["i427"] = 0.010000*np.tanh(((((data["ffti_mean_change_rate_last_10000"]) + (data["ffti_mean_change_rate_last_10000"]))) * 2.0)) 
        v["i428"] = 0.010000*np.tanh(((((((-3.0) * (-3.0))) * 2.0)) * ((-1.0*((data["ffti_ave_roll_mean_1000"])))))) 
        v["i429"] = 0.010000*np.tanh(((((data["percentile_20"]) * (data["autocorrelation_50"]))) + (((data["autocorrelation_50"]) + (data["fftr_min_last_50000"]))))) 
        v["i430"] = 0.010000*np.tanh(((data["ffti_ave_roll_mean_10000"]) + (((data["ffti_percentile_roll_mean_20"]) * 2.0)))) 
        v["i431"] = 0.010000*np.tanh(((data["fftr_percentile_90"]) - (((((((((((data["fftr_min_roll_std_500"]) * (((data["fftr_percentile_roll_mean_25"]) + (((data["fftr_percentile_90"]) + (data["fftr_percentile_roll_mean_30"]))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i432"] = 0.010000*np.tanh(((-3.0) + (((-3.0) * 2.0)))) 
        v["i433"] = 0.010000*np.tanh(((data["fftr_Hann_window_mean_1500"]) * ((((-1.0*(((((data["fftr_abs_max_roll_mean_1000"]) + (((data["fftr_min_roll_std_10"]) - ((-1.0*((data["autocorrelation_10"])))))))/2.0))))) - (data["ffti_num_peaks_50"]))))) 
        v["i434"] = 0.010000*np.tanh(((((((data["fftr_av_change_abs_roll_std_10"]) + (data["fftr_av_change_abs_roll_std_10"]))) / 2.0)) + (((data["abs_percentile_40"]) * ((-1.0*((data["percentile_75"])))))))) 
        v["i435"] = 0.010000*np.tanh(((((((data["fftr_num_peaks_10"]) * 2.0)) - (data["fftr_autocorrelation_50"]))) + (data["mean_change_rate"]))) 
        v["i436"] = 0.010000*np.tanh(((((((data["percentile_10"]) + (((data["percentile_10"]) + (data["percentile_10"]))))) * 2.0)) * 2.0)) 
        v["i437"] = 0.010000*np.tanh(((((data["fftr_moment_4"]) - (((data["ffti_iqr1"]) - (data["min_roll_std_100"]))))) * (-3.0))) 
        v["i438"] = 0.010000*np.tanh(((((((((data["percentile_roll_std_90"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i439"] = 0.010000*np.tanh(((((((data["fftr_kurt"]) * (((data["ffti_av_change_abs_roll_std_10"]) * 2.0)))) * 2.0)) - ((((((data["fftr_kurt"]) + (((data["ffti_autocorrelation_50"]) * 2.0)))/2.0)) * 2.0)))) 
        v["i440"] = 0.010000*np.tanh(((data["ffti_av_change_abs_roll_mean_100"]) * (((((((((((((((((data["ffti_num_peaks_20"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i441"] = 0.010000*np.tanh(((((((((data["ffti_abs_max_roll_std_10000"]) * 2.0)) - ((((-1.0*((data["fftr_range_-2000_-1000"])))) * 2.0)))) * 2.0)) * 2.0)) 
        v["i442"] = 0.010000*np.tanh(((((((data["ffti_num_peaks_50"]) - (data["min_roll_mean_10000"]))) - (data["ffti_max_to_min_diff"]))) * 2.0)) 
        v["i443"] = 0.010000*np.tanh(((data["fftr_min_roll_std_10"]) * (((((data["ffti_classic_sta_lta6_mean"]) * 2.0)) + (data["fftr_max_first_1000"]))))) 
        v["i444"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_80"]) + (data["av_change_rate_roll_mean_500"]))) 
        v["i445"] = 0.010000*np.tanh(((((((((((data["ffti_num_peaks_50"]) * (((((((data["classic_sta_lta1_mean"]) * (data["exp_Moving_average_300_mean"]))) / 2.0)) + (data["fftr_percentile_50"]))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i446"] = 0.010000*np.tanh(((data["ffti_av_change_abs_roll_mean_500"]) - (((data["abs_percentile_50"]) * (((((data["ffti_ave_roll_mean_10"]) * 2.0)) + (((data["ffti_ave_roll_mean_50"]) + (data["ffti_med"]))))))))) 
        v["i447"] = 0.009996*np.tanh(((((data["abs_percentile_40"]) / 2.0)) - (data["percentile_70"]))) 
        v["i448"] = 0.010000*np.tanh(((data["fftr_abs_percentile_10"]) * 2.0)) 
        v["i449"] = 0.010000*np.tanh(((-2.0) + (((data["ffti_ave_roll_mean_10"]) + (data["ffti_num_peaks_100"]))))) 
        v["i450"] = 0.010000*np.tanh(((((data["exp_Moving_std_50000_mean"]) / 2.0)) + (((((data["kurt"]) * 2.0)) + (data["ffti_percentile_roll_std_1"]))))) 
        v["i451"] = 0.010000*np.tanh(((((data["ffti_mean_change_abs"]) * (data["ffti_mean_change_rate"]))) + (((((data["ffti_mean_change_rate_last_10000"]) * 2.0)) + (data["ffti_ave_roll_mean_50"]))))) 
        v["i452"] = 0.010000*np.tanh(((((((((((((data["exp_Moving_average_3000_std"]) * (data["ffti_percentile_roll_mean_25"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (((((((data["av_change_abs_roll_mean_500"]) * (data["std_roll_mean_10000"]))) * 2.0)) * 2.0)))) 
        v["i453"] = 0.010000*np.tanh(((data["percentile_20"]) + (((((data["fftr_percentile_roll_std_10"]) * (((data["min_roll_std_100"]) - (data["max_to_min"]))))) + (data["percentile_10"]))))) 
        v["i454"] = 0.010000*np.tanh(((((((((data["mean_first_50000"]) - (data["fftr_exp_Moving_average_300_std"]))) + (((data["fftr_percentile_roll_mean_90"]) * 2.0)))) * 2.0)) * ((-1.0*((data["fftr_time_rev_asym_stat_100"])))))) 
        v["i455"] = 0.010000*np.tanh(((((((((((((((data["ffti_exp_Moving_average_50000_std"]) / 2.0)) * 2.0)) - (data["ffti_min_roll_std_100"]))) * (data["ffti_autocorrelation_500"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i456"] = 0.010000*np.tanh(((((((((((data["percentile_75"]) * (data["autocorrelation_5"]))) * 2.0)) - ((-1.0*((data["percentile_roll_std_90"])))))) + (((((data["mean_change_rate_first_10000"]) * 2.0)) * (data["autocorrelation_50"]))))) * 2.0)) 
        v["i457"] = 0.010000*np.tanh(((((((data["ffti_abs_percentile_1"]) - (data["autocorrelation_100"]))) * 2.0)) + (data["mean_change_rate_first_50000"]))) 
        v["i458"] = 0.010000*np.tanh(((((((((data["med"]) * (((((((data["fftr_num_peaks_100"]) - (data["autocorrelation_10000"]))) - (data["ffti_percentile_roll_mean_30"]))) * 2.0)))) - (data["std_first_1000"]))) * 2.0)) * 2.0)) 
        v["i459"] = 0.010000*np.tanh(((((((((((data["ffti_percentile_roll_std_30"]) * (((((((data["fftr_classic_sta_lta5_mean"]) - (((data["fftr_classic_sta_lta5_mean"]) * 2.0)))) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i460"] = 0.010000*np.tanh(((((data["exp_Moving_average_50000_std"]) + (data["fftr_range_2000_3000"]))) * 2.0)) 
        v["i461"] = 0.010000*np.tanh(((((((((((((data["ffti_percentile_roll_mean_25"]) - (data["fftr_time_rev_asym_stat_5"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((data["ffti_percentile_roll_mean_25"]) - (data["fftr_time_rev_asym_stat_5"]))))) 
        v["i462"] = 0.010000*np.tanh(((((((data["ffti_av_change_abs_roll_mean_1000"]) * (data["ffti_percentile_40"]))) + (data["fftr_av_change_abs_roll_std_100"]))) + (((data["abs_percentile_20"]) + (data["ffti_min_roll_mean_10"]))))) 
        v["i463"] = 0.010000*np.tanh(((((((((data["abs_percentile_20"]) * 2.0)) / 2.0)) * (data["ffti_autocorrelation_5"]))) + ((-1.0*((data["ave_roll_std_10000"])))))) 
        v["i464"] = 0.010000*np.tanh(((((((((((((data["abs_percentile_20"]) + (-3.0))) / 2.0)) + (data["ffti_min_last_10000"]))/2.0)) + (((((data["abs_percentile_20"]) * 2.0)) + (data["abs_percentile_20"]))))) + (-2.0))/2.0)) 
        v["i465"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_50"]) + (((((data["ffti_trend"]) * 2.0)) + (((data["min_last_10000"]) * 2.0)))))) 
        v["i466"] = 0.009996*np.tanh(((((((data["fftr_abs_percentile_10"]) * 2.0)) + ((((((((((-2.0) + (data["mean_change_rate"]))/2.0)) + (data["ffti_min_roll_mean_1000"]))) * 2.0)) * 2.0)))) / 2.0)) 
        v["i467"] = 0.010000*np.tanh(((((((((data["ffti_range_-3000_-2000"]) - (data["autocorrelation_10000"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i468"] = 0.010000*np.tanh(((((((data["min_last_50000"]) * (((((((((((((((data["ffti_autocorrelation_5"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) * 2.0)) * 2.0)) 
        v["i469"] = 0.010000*np.tanh(((((((data["abs_max_roll_mean_100"]) - (((data["ffti_min_roll_std_500"]) * (data["ffti_av_change_abs_roll_mean_100"]))))) * 2.0)) * 2.0)) 
        v["i470"] = 0.010000*np.tanh(((((((((data["ffti_min_roll_mean_10"]) + (data["mean_change_rate"]))) * 2.0)) * 2.0)) * 2.0)) 
        v["i471"] = 0.010000*np.tanh(((((((((((((((data["fftr_min_roll_std_10"]) * (data["fftr_mean_first_10000"]))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (data["fftr_min_roll_std_10"]))) 
        v["i472"] = 0.010000*np.tanh(((data["percentile_50"]) * (((data["ffti_min_roll_std_50"]) + ((((data["ffti_std_last_10000"]) + (((((((data["skew"]) * 2.0)) * 2.0)) * 2.0)))/2.0)))))) 
        v["i473"] = 0.010000*np.tanh(((data["ffti_min_roll_std_10"]) * (data["percentile_50"]))) 
        v["i474"] = 0.010000*np.tanh((((((((((((data["ffti_ave_roll_std_10"]) + (((data["fftr_min"]) + (data["ffti_mean_change_rate_last_10000"]))))/2.0)) + (data["ffti_mean_change_rate_last_10000"]))) + (data["ffti_mean_change_rate_last_10000"]))) * 2.0)) + (data["fftr_range_2000_3000"]))) 
        v["i475"] = 0.010000*np.tanh(((((((((data["ffti_percentile_roll_std_50"]) + ((-1.0*((data["ffti_kurt"])))))) * 2.0)) * 2.0)) + (data["percentile_5"]))) 
        v["i476"] = 0.010000*np.tanh(((data["percentile_75"]) * ((((((8.74050235748291016)) * ((((((8.74050235748291016)) * ((((((8.74050235748291016)) * (data["fftr_percentile_roll_std_70"]))) / 2.0)))) * 2.0)))) * 2.0)))) 
        v["i477"] = 0.009996*np.tanh((((((((((((-3.0) * 2.0)) * 2.0)) * 2.0)) * 2.0)) + (-2.0))/2.0)) 
        v["i478"] = 0.010000*np.tanh(((data["ffti_exp_Moving_average_30000_std"]) * (((data["ffti_autocorrelation_500"]) * 2.0)))) 
        v["i479"] = 0.010000*np.tanh(data["mean_change_rate_first_50000"]) 
        v["i480"] = 0.010000*np.tanh(((data["ffti_percentile_roll_std_1"]) + (data["autocorrelation_500"]))) 
        v["i481"] = 0.010000*np.tanh(((data["ffti_mean_change_rate_last_10000"]) * 2.0)) 
        v["i482"] = 0.010000*np.tanh(((((((data["av_change_abs_roll_mean_500"]) + ((9.0)))) * (((data["spkt_welch_density_100"]) * (data["av_change_abs_roll_mean_500"]))))) / 2.0)) 
        v["i483"] = 0.010000*np.tanh((((((data["ffti_min_roll_std_100"]) * (data["sum"]))) + (((data["ffti_min_roll_std_100"]) * (((data["ffti_ave_roll_mean_50"]) + (((((data["ffti_ave_roll_mean_50"]) + (data["ffti_range_-1000_0"]))) + (data["ffti_ave_roll_mean_50"]))))))))/2.0)) 
        v["i484"] = 0.009996*np.tanh((((((-3.0) + (-2.0))/2.0)) - (data["mean_change_rate_first_50000"]))) 
        v["i485"] = 0.010000*np.tanh(((((data["ffti_min_roll_mean_10"]) - (((data["ffti_sum"]) * (((((((data["min_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)))))) * 2.0)) 
        v["i486"] = 0.010000*np.tanh(((((((((((data["ffti_min_roll_std_100"]) * (data["classic_sta_lta8_mean"]))) * 2.0)) * 2.0)) + (((data["ffti_min_roll_std_100"]) * 2.0)))) * 2.0)) 
        v["i487"] = 0.010000*np.tanh(((((data["fftr_percentile_roll_std_40"]) * 2.0)) + (((((data["percentile_10"]) * 2.0)) + (((((((((data["fftr_time_rev_asym_stat_100"]) / 2.0)) * 2.0)) + (data["skew"]))) + (data["ffti_min_roll_mean_10"]))))))) 
        v["i488"] = 0.010000*np.tanh(((data["mean_change_rate"]) + (-2.0))) 
        v["i489"] = 0.010000*np.tanh(((((data["ffti_min_roll_mean_10"]) + (((((((data["fftr_abs_percentile_5"]) * 2.0)) * 2.0)) * 2.0)))) + (data["exp_Moving_average_50000_std"]))) 
        v["i490"] = 0.010000*np.tanh(((data["ffti_min_first_50000"]) * 2.0)) 
        v["i491"] = 0.010000*np.tanh((((((((-1.0*((((((data["fftr_min_roll_std_10"]) * 2.0)) * (((data["ffti_Hann_window_mean_15000"]) * 2.0))))))) * 2.0)) * 2.0)) * 2.0)) 
        v["i492"] = 0.010000*np.tanh(((data["mean_change_rate_first_50000"]) + (((data["mean_change_rate"]) + (((((data["min_roll_mean_10"]) - (data["mean_change_rate_first_10000"]))) + (data["fftr_av_change_abs_roll_std_1000"]))))))) 
        v["i493"] = 0.010000*np.tanh(((((data["fftr_abs_percentile_1"]) * 2.0)) - (((data["fftr_av_change_rate_roll_std_50"]) + (((((data["ffti_max_roll_mean_1000"]) * (data["fftr_abs_max_roll_mean_500"]))) * 2.0)))))) 
        v["i494"] = 0.010000*np.tanh((((((((((((((data["fftr_percentile_roll_mean_95"]) * 2.0)) + (data["ffti_Hann_window_mean_50"]))/2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i495"] = 0.010000*np.tanh(((data["mean_change_rate_first_50000"]) * (data["mean_change_rate"]))) 
        v["i496"] = 0.010000*np.tanh(((data["ffti_iqr1"]) + (((((((((((data["av_change_abs_roll_std_100"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) * 2.0)))) 
        v["i497"] = 0.010000*np.tanh((((((((((-1.0*(((-1.0*((((data["mean_last_1000"]) * (data["ffti_autocorrelation_5000"]))))))))) * 2.0)) * 2.0)) * 2.0)) * 2.0)) 
        v["i498"] = 0.010000*np.tanh(((((((data["ffti_range_-3000_-2000"]) + (data["std_roll_mean_10000"]))) * 2.0)) * 2.0)) 
        v["i499"] = 0.010000*np.tanh(((((((data["percentile_10"]) + (((((((data["c3_10"]) * 2.0)) * 2.0)) * 2.0)))) - (((data["percentile_1"]) - (data["ffti_av_change_rate_roll_mean_100"]))))) * 2.0)) 
        v["i500"] = 0.010000*np.tanh(((((((((data["fftr_av_change_abs_roll_mean_50"]) * (((data["ffti_exp_Moving_average_3000_mean"]) + (data["fftr_autocorrelation_100"]))))) / 2.0)) + (((data["fftr_av_change_abs_roll_mean_50"]) + (data["fftr_num_peaks_10"]))))) * 2.0)) 
        v["i501"] = 0.010000*np.tanh(((((((data["ffti_min_roll_std_500"]) * (data["percentile_10"]))) + (((((data["ffti_ave_roll_mean_100"]) * (((data["ffti_min_roll_std_100"]) + (data["ffti_ave_roll_mean_100"]))))) * 2.0)))) + (data["ffti_min_roll_mean_10"]))) 
        v["i502"] = 0.010000*np.tanh(((((data["ffti_min_roll_std_50"]) * 2.0)) + (((((data["fftr_time_rev_asym_stat_1"]) + (((((data["percentile_10"]) + (((-1.0) * 2.0)))) / 2.0)))) - (((data["abs_percentile_40"]) * 2.0)))))) 
        v["i503"] = 0.009996*np.tanh((((((((((((data["ffti_min_roll_mean_10"]) + (data["mean_change_rate"]))/2.0)) * 2.0)) - (data["ffti_percentile_25"]))) * 2.0)) - (((((data["ffti_min_roll_mean_10"]) + (data["ffti_moment_3"]))) - (data["ffti_percentile_roll_std_60"]))))) 
        v["i504"] = 0.010000*np.tanh(((((data["abs_max_roll_mean_50"]) + (((((data["mean_last_1000"]) * 2.0)) * (((data["fftr_min_roll_std_50"]) * 2.0)))))) * 2.0)) 
        v["i505"] = 0.010000*np.tanh(((((((((((((data["fftr_percentile_roll_std_40"]) * 2.0)) * 2.0)) * 2.0)) * (data["ffti_autocorrelation_100"]))) * 2.0)) + (data["classic_sta_lta2_mean"]))) 
        v["i506"] = 0.010000*np.tanh(((((((data["min_roll_std_10"]) * 2.0)) - ((((((-1.0*(((-1.0*((data["num_peaks_100"]))))))) + (-3.0))) * 2.0)))) * (((data["ffti_percentile_roll_std_60"]) + (-3.0))))) 
        v["i507"] = 0.010000*np.tanh(((((((((((((data["fftr_max_roll_mean_10000"]) * 2.0)) * 2.0)) * 2.0)) * 2.0)) - (((((data["fftr_max_roll_mean_10000"]) * 2.0)) * 2.0)))) * (data["ffti_sum"]))) 
        v["i508"] = 0.010000*np.tanh((((-1.0*((((data["exp_Moving_average_50000_std"]) * (data["max_roll_mean_10"])))))) - (data["autocorrelation_50"]))) 
        v["i509"] = 0.010000*np.tanh(((((((((((data["fftr_range_2000_3000"]) * 2.0)) * 2.0)) + (data["fftr_range_2000_3000"]))) * 2.0)) + (((data["fftr_range_2000_3000"]) * 2.0)))) 
        v["i510"] = 0.010000*np.tanh(data["autocorrelation_10000"]) 
        v["i511"] = 0.010000*np.tanh((-1.0*((((data["fftr_min_roll_std_50"]) * 2.0))))) 
        v["i512"] = 0.010000*np.tanh(((((((data["fftr_spkt_welch_density_10"]) + (data["mean_change_rate"]))) + (((data["ffti_min_roll_std_10"]) * 2.0)))) - (3.0)))
        return v[v.columns[:n]].sum(axis=1)

In [20]:
gp = GP()
o = gp.GrabPredictions(alldata[:y.shape[0]],513)

In [21]:
log_loss(alldata[:y.shape[0]].target_0,o.class_0)

0.40300236577974125

In [22]:
log_loss(alldata[:y.shape[0]].target_1,o.class_1)

0.6031674166386606

In [23]:
log_loss(alldata[:y.shape[0]].target_2,o.class_2)

0.5147977108896208

In [24]:
oof_preds = gp.GrabPredictions(alldata[:y.shape[0]],513).values
sub_preds = gp.GrabPredictions(alldata[y.shape[0]:],513).values

In [25]:
oof_preds[0]

array([0.75539511, 0.2190648 , 0.02554009])

In [26]:
params = np.linalg.lstsq(oof_preds, y.target,rcond=-1)[0]

In [27]:
params

array([ 1.57712574,  7.91315986, 10.26991291])

In [28]:
trainpreds = np.dot(oof_preds,params)
print(mean_absolute_error(y.target,trainpreds))

1.8830865750553414


In [29]:
testpreds = np.dot(sub_preds,params)
sub = pd.DataFrame({'seg_id':submission.seg_id, 'time_to_failure':testpreds})
sub.to_csv('gpcut.csv',index=False)

